In [ ]:
# %pip install diffusers["torch"] transformers
# %pip install torchmetrics

In [ ]:
# This mounts your Google Drive to the Colab VM.
# from google.colab import drive
# drive.mount('/content/drive')
# FOLDERNAME = 'stanford/cs231n/cs231n_project/'
# assert FOLDERNAME is not None, "[!] Enter the foldername."
# import sys
# sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import matplotlib.pyplot as plt


import torchvision.datasets as dset
from torchvision.datasets import MNIST, CIFAR10
import torchvision.transforms as T
import torchvision.transforms as transforms
import torchvision.transforms.functional
from torchvision.transforms import v2
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim import AdamW
from tqdm import tqdm
import math
import torch.nn.functional as F

import numpy as np
from torchmetrics.image.inception import InceptionScore

import ddpm
import utils

/Users/lihenan/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# select device
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
elif torch.backends.mps.is_available():
    DEVICE = torch.device('mps')
else:
    DEVICE = torch.device('cpu')
# DEVICE = torch.device('cpu')
print(f'{DEVICE=}')

# hyper param

DATASET = 'CIFAR10'  # CIFAR10 or MNIST
print(f'{DATASET=}')
if DATASET == 'MNIST':
    IMG_SIZE = (32, 32, 1) # original images are padded to make size 32 x 32
    DATASET = MNIST
elif DATASET == 'CIFAR10':
    IMG_SIZE = (32, 32, 3)
    DATASET = CIFAR10
print(f'{IMG_SIZE=}')

BATCH_SIZE = 64
N_LAYERS = 8
HIDDEN_DIM = 256
HIDDEN_DIMS = [HIDDEN_DIM for _ in range(N_LAYERS)]
N_TIMESTAMPS = 1000
TIMESTAMP_EMBED_DIM = 256
# BETA_MIN_MAX = [1e-4, 2e-2]
LEARNING_RATE = 5e-5
N_EPOCH = 2

# prepare data

dataset_path = './datasets'
transform = transforms.Compose([
    transforms.Resize(32), transforms.ToTensor()
])  # TODO: try subtract mean & div by std
train_dataset = DATASET(dataset_path, transform=transform,
                        train=True, download=True)
eval_dataset = DATASET(dataset_path, transform=transform,
                       train=True, download=True)
test_dataset = DATASET(dataset_path, transform=transform,
                       train=False, download=True)
NUM_DATASET = len(train_dataset)
NUM_DATASET = 1000  # use a smaller dataset
NUM_TRAIN = int(NUM_DATASET * 0.8)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))
# train_sample = list(train_loader)[0][0]

# print(train_dataset.data.type())
# print(train_dataset.data.float().mean(), train_dataset.data.min(), train_dataset.data.max(), train_dataset.data.shape)
# print(train_sample.mean(), train_sample.min(), train_sample.max(), train_sample.shape)

eval_loader = DataLoader(dataset=eval_dataset, batch_size=BATCH_SIZE,
                         sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, NUM_DATASET)))
test_loader = DataLoader(dataset=test_dataset,  batch_size=BATCH_SIZE)

N_CLASS = len(train_dataset.classes)
print(f'{N_CLASS=} {NUM_TRAIN=}')

DEVICE=device(type='mps')
DATASET='CIFAR10'
IMG_SIZE=(32, 32, 3)
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
N_CLASS=10 NUM_TRAIN=800


In [ ]:
IMAGE_CHANNEL = IMG_SIZE[2]
eps_model = ddpm.UNet(
    image_channels=IMAGE_CHANNEL,
    n_channels=64,
    ch_mults=[1, 2, 2, 4],
    is_attn=[False, False, False, True],
).to(DEVICE)

diffusion = ddpm.DenoiseDiffusion(
    eps_model=eps_model,
    n_diffusion_timestep=N_TIMESTAMPS,
    device=DEVICE,
)

optimizer = Adam(eps_model.parameters(), lr=LEARNING_RATE)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of model parameters: ", count_parameters(eps_model))
print("forwarding diffusion sampling...")
eps_model.eval()
with torch.no_grad():
    for x, class_ in train_loader:
        print(x.mean(), x.min(), x.max(), x.shape)
        x = x[:8].to(DEVICE)
        xt = torch.tensor([]).to(DEVICE)
        for t in range(0, N_TIMESTAMPS, 100):
            xt = torch.cat((xt, diffusion.q(
                x, torch.full((8, ), t).to(DEVICE))))
        print(xt.mean(), xt.min(), xt.max(), xt.shape)
        utils.draw_sample_image(xt, f"adding noise")
        break  # only sample 1 batch
eps_model.train()


In [ ]:
print("start training")

train_loss_arr = [0]
eval_loss_arr = [0]
iters = [0]

EVAL_EVERY_ITER = 10
EVAL_BATCH = 10

def evaluate():
    with torch.no_grad():
        eval_loss = 0.0
        train_loss = 0.0
        eps_model.eval()
        for train_batch_idx, (img_, class_) in enumerate(train_loader):
            train_loss += diffusion.loss(img_.to(DEVICE), class_.to(DEVICE)).item()
            if train_batch_idx > EVAL_BATCH:
                break
        for eval_batch_idx, (img_, class_) in enumerate(eval_loader):
            eval_loss += diffusion.loss(img_.to(DEVICE), class_.to(DEVICE)).item()
            if eval_batch_idx > EVAL_BATCH:
                break
        eval_loss /= eval_batch_idx
        train_loss /= train_batch_idx
        eval_loss_arr.append(eval_loss)
        train_loss_arr.append(train_loss)
        iters.append(iters[-1] + EVAL_EVERY_ITER)
        eps_model.train()

i = 0
for epoch in range(N_EPOCH):
    print(f'training {epoch=}')
    for batch_idx, (img_, class_) in tqdm(enumerate(train_loader), total=len(train_loader)):
        i += 1
        if i % EVAL_EVERY_ITER == 0:
            evaluate()
        
        optimizer.zero_grad()
        loss = diffusion.loss(img_.to(DEVICE), class_.to(DEVICE))
        loss.backward()
        optimizer.step()
        
            
plt.figure()
plt.plot(iters[1:], train_loss_arr[1:], label = 'train')
plt.plot(iters[1:], eval_loss_arr[1:], label = 'validation')
plt.xlabel('iters')
plt.ylabel('loss')
plt.legend()
plt.title('training loss vs validation loss')

In [ ]:
eps_model.eval()
print('sampling...')
class_ = torch.arange(8).to(DEVICE)
with torch.no_grad():
    x = torch.randn([8, IMAGE_CHANNEL, 32, 32], device=DEVICE)
    xt = torch.tensor([]).to(DEVICE)
    for t_ in tqdm(range(N_TIMESTAMPS)):
        t = N_TIMESTAMPS - t_ - 1
        t = x.new_full((1,), t, dtype=torch.long)
        x = diffusion.p_sample(x, t, class_)
        if t_ % 200 == 0 or t_ == N_TIMESTAMPS - 1:
            xt = torch.cat((xt, x))
    utils.draw_sample_image(xt, f"generating")

In [ ]:
# FID
import torch
from torchmetrics.image.fid import FrechetInceptionDistance

fid = FrechetInceptionDistance(feature=64, normalize=True)

for generated_img in xt:
    fid.update(generated_img, real=False)

# generated_imgs = v2.ToDtype(torch.uint8, scale=True)(xt) # scaled to [0, 255]

for test_img, test_class in enumerate(test_loader):
    for i in len(test_img):
        print(test_img)
        print(test_class)
        break
    break

# 
# _ = torch.manual_seed(123)
# 
# 
# generate two slightly overlapping image intensity distributions
# imgs_dist1 = torch.randint(0, 200, (100, 3, 299, 299), dtype=torch.uint8)
# imgs_dist2 = torch.randint(100, 255, (100, 3, 299, 299), dtype=torch.uint8)
# fid.update(imgs_dist1, real=True)
# fid.update(imgs_dist2, real=False)
# fid.compute()

In [ ]:

class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        device = x.device
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = x[:, None] * emb[None, :]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb


class ConvBlock(nn.Conv2d):
    """
        Conv2D Block
            Args:
                x: (N, C_in, H, W)
            Returns:
                y: (N, C_out, H, W)
    """

    def __init__(self, in_channels, out_channels, kernel_size, activation_fn=None, drop_rate=0.,
                 stride=1, padding='same', dilation=1, groups=1, bias=True, gn=False, gn_groups=8):

        if padding == 'same':
            padding = kernel_size // 2 * dilation

        super(ConvBlock, self).__init__(in_channels, out_channels, kernel_size,
                                        stride=stride, padding=padding, dilation=dilation,
                                        groups=groups, bias=bias)

        self.activation_fn = nn.SiLU() if activation_fn else None
        self.group_norm = nn.GroupNorm(gn_groups, out_channels) if gn else None

    def forward(self, x, time_embedding=None, residual=False):

        if residual:
            # in the paper, diffusion timestep embedding was only applied to residual blocks of U-Net
            x = x + time_embedding
            y = x
            x = super(ConvBlock, self).forward(x)
            y = y + x
        else:
            y = super(ConvBlock, self).forward(x)
        y = self.group_norm(y) if self.group_norm is not None else y
        y = self.activation_fn(y) if self.activation_fn is not None else y

        return y


class Denoiser(nn.Module):

    def __init__(self, image_resolution, hidden_dims=[256, 256], diffusion_time_embedding_dim=256, n_times=1000):
        """
        image_resolution: example array [32, 32, 3]

        """
        super(Denoiser, self).__init__()

        _, _, img_C = image_resolution

        self.time_embedding = SinusoidalPosEmb(diffusion_time_embedding_dim)

        self.in_project = ConvBlock(img_C, hidden_dims[0], kernel_size=7)

        self.time_project = nn.Sequential(
            ConvBlock(diffusion_time_embedding_dim,
                      hidden_dims[0], kernel_size=1, activation_fn=True),
            ConvBlock(hidden_dims[0], hidden_dims[0], kernel_size=1))

        self.convs = nn.ModuleList([ConvBlock(
            in_channels=hidden_dims[0], out_channels=hidden_dims[0], kernel_size=3)])

        for idx in range(1, len(hidden_dims)):
            self.convs.append(ConvBlock(hidden_dims[idx-1], hidden_dims[idx], kernel_size=3, dilation=3**((idx-1)//2),
                                        activation_fn=True, gn=True, gn_groups=8))

        self.out_project = ConvBlock(
            hidden_dims[-1], out_channels=img_C, kernel_size=3)

    def forward(self, perturbed_x, diffusion_t):
        y = perturbed_x

        diffusion_embedding = self.time_embedding(diffusion_t)
        diffusion_embedding = self.time_project(
            diffusion_embedding.unsqueeze(-1).unsqueeze(-2))

        y = self.in_project(y)

        for i in range(len(self.convs)):
            y = self.convs[i](y, diffusion_embedding, residual=True)

        y = self.out_project(y)

        return y


model = Denoiser(image_resolution=IMG_SIZE,
                 hidden_dims=HIDDEN_DIMS,
                 diffusion_time_embedding_dim=TIMESTAMP_EMBED_DIM,
                 n_times=N_TIMESTAMPS).to(DEVICE)

In [ ]:
class Diffusion(nn.Module):
    def __init__(self, model, image_resolution=[32, 32, 3], n_times=1000, beta_minmax=[1e-4, 2e-2], device='cuda'):

        super(Diffusion, self).__init__()

        self.n_times = n_times
        self.img_H, self.img_W, self.img_C = image_resolution

        self.model = model

        # define linear variance schedule(betas)
        beta_1, beta_T = beta_minmax
        betas = torch.linspace(start=beta_1, end=beta_T, steps=n_times).to(
            device)  # follows DDPM paper
        self.sqrt_betas = torch.sqrt(betas)

        # define alpha for forward diffusion kernel
        self.alphas = 1 - betas
        self.sqrt_alphas = torch.sqrt(self.alphas)
        alpha_bars = torch.cumprod(self.alphas, dim=0)
        self.sqrt_one_minus_alpha_bars = torch.sqrt(1-alpha_bars)
        self.sqrt_alpha_bars = torch.sqrt(alpha_bars)

        self.device = device

    def scale_to_minus_one_to_one(self, x):
        # according to the DDPMs paper, normalization seems to be crucial to train reverse process network
        return x * 2 - 1

    def reverse_scale_to_zero_to_one(self, x):
        return (x + 1) * 0.5

    def make_noisy(self, x_zeros, t):
        # perturb x_0 into x_t (i.e., take x_0 samples into forward diffusion kernels)
        epsilon = torch.randn_like(x_zeros).to(self.device)

        sqrt_alpha_bar = self.extract(self.sqrt_alpha_bars, t, x_zeros.shape)
        sqrt_one_minus_alpha_bar = self.extract(
            self.sqrt_one_minus_alpha_bars, t, x_zeros.shape)

        # Let's make noisy sample!: i.e., Forward process with fixed variance schedule
        #      i.e., sqrt(alpha_bar_t) * x_zero + sqrt(1-alpha_bar_t) * epsilon
        noisy_sample = x_zeros * sqrt_alpha_bar + epsilon * sqrt_one_minus_alpha_bar

        return noisy_sample.detach(), epsilon

    def forward(self, x_zeros):
        x_zeros = self.scale_to_minus_one_to_one(x_zeros)

        B, _, _, _ = x_zeros.shape

        # (1) randomly choose diffusion time-step
        t = torch.randint(low=0, high=self.n_times,
                          size=(B,)).long().to(self.device)

        # (2) forward diffusion process: perturb x_zeros with fixed variance schedule
        perturbed_images, epsilon = self.make_noisy(x_zeros, t)

        # (3) predict epsilon(noise) given perturbed data at diffusion-timestep t.
        pred_epsilon = self.model(perturbed_images, t)

        return perturbed_images, epsilon, pred_epsilon

    def denoise_at_t(self, x_t, timestep, t):
        B, _, _, _ = x_t.shape
        if t > 1:
            z = torch.randn_like(x_t).to(self.device)
        else:
            z = torch.zeros_like(x_t).to(self.device)

        # at inference, we use predicted noise(epsilon) to restore perturbed data sample.
        epsilon_pred = self.model(x_t, timestep)

        alpha = self.extract(self.alphas, timestep, x_t.shape)
        sqrt_alpha = self.extract(self.sqrt_alphas, timestep, x_t.shape)
        sqrt_one_minus_alpha_bar = self.extract(
            self.sqrt_one_minus_alpha_bars, timestep, x_t.shape)
        sqrt_beta = self.extract(self.sqrt_betas, timestep, x_t.shape)

        # denoise at time t, utilizing predicted noise
        x_t_minus_1 = 1 / sqrt_alpha * \
            (x_t - (1-alpha)/sqrt_one_minus_alpha_bar*epsilon_pred) + sqrt_beta*z

        return x_t_minus_1.clamp(-1., 1)

    def sample(self, N):
        # start from random noise vector, x_0 (for simplicity, x_T declared as x_t instead of x_T)
        x_t = torch.randn((N, self.img_C, self.img_H,
                          self.img_W)).to(self.device)

        # autoregressively denoise from x_T to x_0
        #     i.e., generate image from noise, x_T
        for t in range(self.n_times-1, -1, -1):
            timestep = torch.tensor([t]).repeat_interleave(
                N, dim=0).long().to(self.device)
            x_t = self.denoise_at_t(x_t, timestep, t)

        # denormalize x_0 into 0 ~ 1 ranged values.
        x_0 = self.reverse_scale_to_zero_to_one(x_t)

        return x_0


diffusion = Diffusion(model, image_resolution=IMG_SIZE, n_times=N_TIMESTAMPS,
                      beta_minmax=BETA_MIN_MAX, device=DEVICE).to(DEVICE)

optimizer = Adam(diffusion.parameters(), lr=LEARNING_RATE)
denoising_loss = nn.MSELoss()

In [ ]:

from torchsummary import summary

summary(diffusion, (1, 28, 28))



model.eval()
for batch_idx, (x, class_) in enumerate(train_loader):
    print(x.mean(), x.min(), x.max(), x.shape)
    x = x[:8].to(DEVICE)
    # perturbed_images, epsilon, pred_epsilon = diffusion(x)
    # perturbed_images = diffusion.reverse_scale_to_zero_to_one(perturbed_images)
    for t in range(0, N_TIMESTAMPS, 100):
        noisy_sample, _ = diffusion.make_noisy(
            x, torch.full((8, ), t).to(DEVICE))
        # print(f'{len(noisy_sample)}')
        # show_image(noisy_sample[0])
        draw_sample_image(noisy_sample, f"adding noise t={t}")
    break

# print(f'{len(perturbed_images)=}')
# show_image(perturbed_images[0])
# show_image(perturbed_images[1])
# show_image(perturbed_images[63])

In [ ]:
model.eval()

with torch.no_grad():
    generated_images = diffusion.sample(N=8)


In [ ]:
show_image(generated_images[0])

In [ ]:

# draw_sample_image(perturbed_images, "Perturbed Images")
draw_sample_image(generated_images, "Generated Images")
draw_sample_image(x[:8], "Ground-truth Images")


In [ ]:
from ignite.metrics import InceptionScore
from diffusers import DDPMPipeline

pipeline = DDPMPipeline.from_pretrained('1aurent/ddpm-mnist').to(DEVICE)
print(f'{pipeline.device}')

images = torch.tensor([])
for i in range(10):
    image_tensor = torchvision.transforms.functional.pil_to_tensor(pipeline().images[0])
    images = torch.cat((images, image_tensor))
    print(images.shape)

# metric = InceptionScore()
# metric.attach(default_evaluator, "is")
# y = torch.rand(10, 3, 299, 299)
# state = default_evaluator.run([y])
# print(state.metrics["is"])


In [ ]:
inception = InceptionScore()
# generate some images
# imgs = torch.randint(0, 255, (100, 3, 299, 299), dtype=torch.uint8)
# print(type(image))
for image in images:

    image_tensor = torchvision.transforms.functional.pil_to_tensor(image)
    print(f'{image_tensor.float().mean()=} {image_tensor.min()=} {image_tensor.max()=} {image_tensor.shape=}')
    # print(image_tensor)
    # print(image_tensor.expand(1, 3, 28, 28))
    expanded_image_tensor = image_tensor.expand(1, 3, 28, 28)
    # print(f'{expanded_image_tensor.shape}')
    inception.update(expanded_image_tensor)
    print(f'{inception.compute()=}')

In [8]:
# 8 generated CIFAR10 images from training for 60 epoch
generated_images = torch.tensor([[[[ 9.5553e-01,  9.5428e-01,  9.5793e-01,  9.5878e-01,  9.5339e-01,
            9.5412e-01,  9.5198e-01,  9.5299e-01,  9.5320e-01,  9.6023e-01,
            9.5846e-01,  9.5238e-01,  9.5655e-01,  9.6375e-01,  9.7168e-01,
            9.8092e-01,  9.7331e-01,  9.4962e-01,  9.4496e-01,  9.5329e-01,
            9.6543e-01,  9.6899e-01,  9.3161e-01,  9.3916e-01,  9.5001e-01,
            9.3765e-01,  9.6156e-01,  9.6578e-01,  9.4569e-01,  9.5509e-01,
            9.6044e-01,  9.6582e-01],
          [ 9.7404e-01,  9.4835e-01,  9.5204e-01,  9.5245e-01,  9.4144e-01,
            9.5352e-01,  9.5704e-01,  9.5547e-01,  9.6161e-01,  9.5827e-01,
            9.4177e-01,  9.7198e-01,  9.7624e-01,  9.6349e-01,  9.6493e-01,
            9.3893e-01,  9.7097e-01,  9.4296e-01,  9.6286e-01,  9.5892e-01,
            9.5374e-01,  9.5473e-01,  9.5032e-01,  9.3562e-01,  9.5106e-01,
            9.5843e-01,  9.5709e-01,  9.5237e-01,  9.6512e-01,  9.7800e-01,
            9.7552e-01,  9.9348e-01],
          [ 9.4148e-01,  9.3540e-01,  9.4988e-01,  9.4700e-01,  9.4460e-01,
            9.4247e-01,  9.4163e-01,  9.4874e-01,  9.5489e-01,  9.4759e-01,
            9.7430e-01,  9.7297e-01,  9.6851e-01,  9.6277e-01,  9.5319e-01,
            9.6103e-01,  9.7430e-01,  9.5770e-01,  9.7538e-01,  9.8122e-01,
            9.6510e-01,  9.4646e-01,  9.4849e-01,  9.4342e-01,  9.4197e-01,
            9.5311e-01,  9.5546e-01,  9.4262e-01,  9.6346e-01,  9.6631e-01,
            9.7315e-01,  9.8339e-01],
          [ 9.3041e-01,  9.4574e-01,  9.4164e-01,  9.4234e-01,  9.3366e-01,
            9.4624e-01,  9.5467e-01,  9.6174e-01,  9.6971e-01,  9.5744e-01,
            9.7452e-01,  9.6544e-01,  9.7041e-01,  9.7465e-01,  9.7096e-01,
            9.7133e-01,  9.8130e-01,  9.7924e-01,  9.7703e-01,  9.5072e-01,
            9.5134e-01,  9.3971e-01,  9.6199e-01,  9.5329e-01,  9.6442e-01,
            9.5453e-01,  9.6693e-01,  9.6122e-01,  9.5984e-01,  9.5864e-01,
            9.7317e-01,  9.9845e-01],
          [ 9.4481e-01,  9.3913e-01,  9.4077e-01,  9.7455e-01,  9.4682e-01,
            9.4985e-01,  9.5144e-01,  9.6948e-01,  9.7185e-01,  9.7549e-01,
            9.7315e-01,  9.6024e-01,  9.7615e-01,  9.8486e-01,  9.7776e-01,
            9.8760e-01,  9.8903e-01,  9.8152e-01,  9.4603e-01,  7.5719e-01,
            7.7462e-01,  9.1698e-01,  9.8221e-01,  9.8875e-01,  9.6728e-01,
            9.7000e-01,  9.7643e-01,  9.6585e-01,  9.6674e-01,  9.6822e-01,
            9.7028e-01,  9.8247e-01],
          [ 9.5378e-01,  9.5888e-01,  9.5533e-01,  9.5301e-01,  9.5839e-01,
            9.6721e-01,  9.5622e-01,  9.7224e-01,  9.7052e-01,  9.9026e-01,
            9.7016e-01,  9.8983e-01,  9.8843e-01,  9.7369e-01,  9.9260e-01,
            9.8205e-01,  9.8125e-01,  9.8478e-01,  9.3708e-01,  6.0935e-01,
            5.4524e-01,  8.5376e-01,  9.6758e-01,  1.0028e+00,  9.7607e-01,
            9.6648e-01,  9.6672e-01,  9.5334e-01,  9.4497e-01,  9.7073e-01,
            9.8060e-01,  9.8036e-01],
          [ 9.4618e-01,  9.4462e-01,  9.4966e-01,  9.5633e-01,  9.5210e-01,
            9.5632e-01,  9.5412e-01,  9.6346e-01,  9.8130e-01,  9.7958e-01,
            9.7202e-01,  9.7721e-01,  9.6920e-01,  9.7682e-01,  9.8584e-01,
            9.9332e-01,  9.8599e-01,  9.9375e-01,  9.2845e-01,  5.4227e-01,
            5.3499e-01,  7.6560e-01,  9.2838e-01,  9.9393e-01,  9.9339e-01,
            9.8264e-01,  9.7995e-01,  9.6200e-01,  9.6053e-01,  9.7922e-01,
            9.7971e-01,  9.7900e-01],
          [ 9.3802e-01,  9.4279e-01,  9.4503e-01,  9.5768e-01,  9.4566e-01,
            9.4374e-01,  9.3645e-01,  9.6432e-01,  9.6881e-01,  9.7233e-01,
            9.6869e-01,  9.6683e-01,  9.8269e-01,  9.8049e-01,  9.7897e-01,
            9.8607e-01,  9.8057e-01,  9.9090e-01,  9.4406e-01,  5.6936e-01,
            6.1876e-01,  7.5852e-01,  9.0351e-01,  9.8684e-01,  9.9369e-01,
            9.8154e-01,  9.8321e-01,  9.7015e-01,  9.7521e-01,  9.8321e-01,
            9.8197e-01,  9.8106e-01],
          [ 9.4744e-01,  9.4937e-01,  9.4538e-01,  9.4954e-01,  9.5446e-01,
            9.5169e-01,  9.4869e-01,  9.5685e-01,  9.5582e-01,  9.6634e-01,
            9.6398e-01,  9.8720e-01,  9.9232e-01,  9.9685e-01,  9.8287e-01,
            9.8998e-01,  9.6503e-01,  9.9058e-01,  9.3906e-01,  6.8348e-01,
            6.9257e-01,  7.5439e-01,  7.2769e-01,  8.6795e-01,  9.8839e-01,
            9.7664e-01,  9.8889e-01,  9.8272e-01,  9.8203e-01,  9.8960e-01,
            1.0014e+00,  1.0015e+00],
          [ 9.4553e-01,  9.4438e-01,  9.3195e-01,  9.4013e-01,  9.4359e-01,
            9.3885e-01,  9.3191e-01,  9.5486e-01,  9.6005e-01,  9.4354e-01,
            9.7275e-01,  9.8059e-01,  9.9224e-01,  9.9141e-01,  9.9669e-01,
            9.9388e-01,  9.7002e-01,  9.6975e-01,  1.0007e+00,  6.7811e-01,
            6.3857e-01,  5.6287e-01,  3.8921e-01,  6.4721e-01,  9.6641e-01,
            9.9796e-01,  9.8791e-01,  9.8944e-01,  9.8976e-01,  9.9419e-01,
            9.9216e-01,  9.9270e-01],
          [ 9.3382e-01,  9.3030e-01,  9.2705e-01,  9.3888e-01,  9.3035e-01,
            9.4882e-01,  9.3300e-01,  9.5603e-01,  9.3839e-01,  9.2968e-01,
            9.5038e-01,  9.6073e-01,  9.7907e-01,  9.8796e-01,  9.7620e-01,
            9.7625e-01,  9.6294e-01,  9.5168e-01,  8.6465e-01,  4.2764e-01,
            4.1985e-01,  3.4264e-01,  2.3234e-01,  5.9711e-01,  9.5750e-01,
            9.8403e-01,  9.8936e-01,  9.8648e-01,  9.9043e-01,  9.8997e-01,
            9.7388e-01,  9.9867e-01],
          [ 9.3480e-01,  9.1563e-01,  9.2221e-01,  9.1630e-01,  9.2325e-01,
            8.9910e-01,  9.0433e-01,  9.0781e-01,  8.9878e-01,  9.1533e-01,
            9.1470e-01,  9.2478e-01,  9.5555e-01,  9.4679e-01,  9.3169e-01,
            9.4453e-01,  9.2089e-01,  8.5160e-01,  6.9808e-01,  6.2276e-01,
            7.9131e-01,  5.8226e-01,  3.3692e-01,  5.9735e-01,  9.2334e-01,
            1.0084e+00,  1.0086e+00,  1.0013e+00,  9.9398e-01,  9.9053e-01,
            9.7743e-01,  9.5920e-01],
          [ 9.2179e-01,  9.0181e-01,  8.8706e-01,  8.9197e-01,  8.9386e-01,
            8.7501e-01,  8.7462e-01,  8.6699e-01,  8.5187e-01,  8.4346e-01,
            8.5748e-01,  8.5779e-01,  8.5213e-01,  8.5308e-01,  8.2724e-01,
            7.6877e-01,  7.0275e-01,  8.3225e-01,  8.5025e-01,  8.0368e-01,
            8.4399e-01,  5.4551e-01,  3.9576e-01,  6.4252e-01,  9.3581e-01,
            9.6071e-01,  9.5700e-01,  9.5335e-01,  9.7439e-01,  9.6914e-01,
            9.6490e-01,  9.4438e-01],
          [ 8.6244e-01,  8.3035e-01,  8.3850e-01,  8.1478e-01,  8.1938e-01,
            8.2599e-01,  8.0379e-01,  7.9964e-01,  7.9638e-01,  6.9622e-01,
            6.1534e-01,  4.7974e-01,  4.3412e-01,  4.2761e-01,  3.8364e-01,
            3.6886e-01,  3.0143e-01,  4.4174e-01,  7.9599e-01,  8.0645e-01,
            4.8210e-01,  2.6193e-01,  3.9559e-01,  6.1541e-01,  7.0498e-01,
            6.5500e-01,  6.4737e-01,  6.5191e-01,  7.7798e-01,  8.3208e-01,
            8.2867e-01,  8.4970e-01],
          [ 6.4904e-01,  6.2409e-01,  6.5729e-01,  6.6849e-01,  6.7774e-01,
            6.6308e-01,  6.7388e-01,  5.5529e-01,  3.5352e-01,  1.7296e-01,
            8.8330e-02,  6.7962e-02,  6.2131e-02,  6.9322e-02,  6.1056e-02,
            6.5562e-02,  5.9584e-02,  3.2246e-02,  4.2094e-01,  7.0708e-01,
            2.7736e-01,  1.4579e-01,  2.2740e-01,  2.2959e-01,  2.4503e-01,
            2.1354e-01,  2.9715e-01,  5.2165e-01,  7.1768e-01,  7.8877e-01,
            7.6380e-01,  8.0248e-01],
          [ 4.3160e-01,  4.5107e-01,  4.8274e-01,  5.2228e-01,  5.7482e-01,
            6.0026e-01,  6.5877e-01,  4.1940e-01,  8.2809e-02,  4.8879e-02,
            5.4959e-02,  4.6589e-02,  8.6174e-02,  1.0136e-01,  1.1938e-01,
            6.6597e-02,  6.9877e-02,  2.7036e-02,  1.3605e-01,  4.4246e-01,
            2.3617e-01,  1.3263e-01,  8.7335e-02,  6.0672e-02,  6.0451e-02,
            5.9041e-02,  2.4517e-01,  5.0868e-01,  7.3136e-01,  7.3299e-01,
            7.3218e-01,  7.4874e-01],
          [ 4.5619e-01,  4.8406e-01,  5.0039e-01,  5.1890e-01,  6.1408e-01,
            6.9219e-01,  7.5636e-01,  3.6674e-01,  8.4296e-02,  6.3424e-02,
            4.6363e-02,  7.6058e-02,  1.1317e-01,  1.4502e-01,  1.3892e-01,
            8.2610e-02,  7.6416e-02,  4.3101e-02,  4.8561e-02,  2.4447e-01,
            1.2123e-01,  6.9695e-02,  6.0191e-02,  4.8027e-02,  4.6744e-02,
            3.9231e-02,  3.2640e-01,  6.0679e-01,  7.0474e-01,  7.0156e-01,
            7.0831e-01,  7.0319e-01],
          [ 5.0198e-01,  4.6312e-01,  4.0346e-01,  4.2135e-01,  4.5593e-01,
            5.7485e-01,  6.2599e-01,  2.5753e-01,  6.3032e-02,  3.7999e-02,
            4.3411e-02,  6.3972e-02,  8.9031e-02,  1.0104e-01,  8.4436e-02,
            9.0267e-02,  1.0034e-01,  8.2289e-02,  5.9279e-02,  1.6578e-01,
            9.1248e-02,  3.6589e-02,  8.6431e-02,  6.9684e-02,  1.5926e-02,
            3.2735e-02,  3.2627e-01,  6.4375e-01,  6.8563e-01,  6.8912e-01,
            6.9161e-01,  6.9649e-01],
          [ 6.3916e-01,  5.3448e-01,  4.4882e-01,  4.1310e-01,  4.1527e-01,
            3.6539e-01,  2.8344e-01,  2.2042e-01,  8.5970e-02,  3.8107e-02,
            4.7144e-02,  5.6072e-02,  8.2164e-02,  9.5424e-02,  7.6292e-02,
            5.6358e-02,  6.7766e-02,  8.1913e-02,  7.0218e-02,  1.1671e-01,
            9.0663e-02,  6.9945e-02,  1.2317e-01,  7.3072e-02,  1.7069e-02,
            3.7124e-02,  3.2789e-01,  6.5754e-01,  6.8361e-01,  6.8203e-01,
            6.9654e-01,  6.9454e-01],
          [ 7.0396e-01,  6.3882e-01,  5.5651e-01,  4.9683e-01,  4.7012e-01,
            4.3541e-01,  4.1849e-01,  4.2406e-01,  3.7473e-01,  2.9966e-01,
            2.1902e-01,  1.1448e-01,  1.0930e-01,  1.1137e-01,  7.0315e-02,
            3.3185e-02,  7.2448e-02,  9.9925e-02,  8.3758e-02,  8.2218e-02,
            1.9817e-01,  5.1428e-01,  3.5755e-01,  9.9430e-02,  3.4541e-02,
            7.6675e-02,  3.8327e-01,  6.5679e-01,  6.6806e-01,  6.5291e-01,
            6.7147e-01,  6.7987e-01],
          [ 8.1734e-01,  7.6166e-01,  6.8898e-01,  6.8786e-01,  7.1257e-01,
            6.7226e-01,  6.9174e-01,  7.3425e-01,  7.0339e-01,  6.4535e-01,
            5.5739e-01,  3.5989e-01,  2.5311e-01,  2.5328e-01,  3.4397e-01,
            4.0975e-01,  4.3349e-01,  3.0191e-01,  1.2352e-01,  8.8078e-02,
            2.7863e-01,  7.4513e-01,  7.0950e-01,  4.0508e-01,  2.2330e-01,
            2.5484e-01,  5.2098e-01,  6.7040e-01,  6.6745e-01,  6.6328e-01,
            6.6369e-01,  6.7599e-01],
          [ 7.2394e-01,  5.8375e-01,  4.1569e-01,  3.4565e-01,  3.7434e-01,
            3.8677e-01,  3.7808e-01,  4.0512e-01,  4.7750e-01,  5.2187e-01,
            5.9130e-01,  6.5490e-01,  6.8082e-01,  6.6406e-01,  7.2358e-01,
            7.7855e-01,  8.2563e-01,  7.4166e-01,  3.0848e-01,  6.1846e-02,
            3.4069e-01,  6.1018e-01,  6.6467e-01,  6.0428e-01,  4.5980e-01,
            4.7108e-01,  6.1910e-01,  6.6666e-01,  6.6499e-01,  6.6412e-01,
            6.7742e-01,  6.6084e-01],
          [ 3.7963e-01,  2.2991e-01,  1.3135e-01,  9.4723e-02,  7.9148e-02,
            1.1993e-01,  2.5430e-02,  6.0550e-02,  2.2367e-01,  4.2176e-01,
            4.6677e-01,  4.9076e-01,  5.4799e-01,  6.3668e-01,  7.0953e-01,
            8.0787e-01,  8.1610e-01,  6.9795e-01,  3.9601e-01,  1.8427e-01,
            3.0051e-01,  5.3229e-01,  6.2157e-01,  6.1965e-01,  6.3109e-01,
            6.5182e-01,  6.8249e-01,  6.9887e-01,  7.0033e-01,  6.7876e-01,
            6.8363e-01,  6.7195e-01],
          [ 1.8997e-01,  1.1605e-01,  9.9841e-02,  1.1271e-01,  1.2469e-01,
            2.0172e-01,  1.3848e-01,  2.9805e-01,  5.2236e-01,  5.0049e-01,
            4.5560e-01,  4.6879e-01,  5.1048e-01,  6.1064e-01,  7.4043e-01,
            8.2767e-01,  8.1369e-01,  7.3635e-01,  6.6636e-01,  5.3417e-01,
            4.3705e-01,  5.7736e-01,  7.1458e-01,  6.8208e-01,  7.0968e-01,
            7.0813e-01,  7.0119e-01,  7.1192e-01,  7.0466e-01,  6.9261e-01,
            7.1148e-01,  7.1010e-01],
          [ 1.0829e-01,  9.5548e-02,  8.6155e-02,  5.8306e-02,  1.5284e-01,
            3.8955e-01,  4.8476e-01,  6.0638e-01,  6.3218e-01,  5.3749e-01,
            5.1479e-01,  5.2316e-01,  5.7193e-01,  6.3223e-01,  7.3051e-01,
            7.9659e-01,  8.0992e-01,  7.9474e-01,  7.8201e-01,  7.5924e-01,
            7.2681e-01,  7.4541e-01,  7.6870e-01,  7.4533e-01,  7.5128e-01,
            7.3019e-01,  7.1649e-01,  7.0088e-01,  6.8365e-01,  7.1195e-01,
            7.1035e-01,  7.0395e-01],
          [ 8.8139e-02,  8.7171e-02,  6.2389e-02,  6.8563e-02,  2.3598e-01,
            3.2386e-01,  4.0111e-01,  5.0325e-01,  5.3082e-01,  5.0502e-01,
            4.9852e-01,  4.7612e-01,  5.2685e-01,  5.9021e-01,  6.3318e-01,
            7.0510e-01,  7.5529e-01,  7.3018e-01,  7.4713e-01,  7.1738e-01,
            6.7587e-01,  5.8875e-01,  5.3477e-01,  5.9236e-01,  6.5814e-01,
            6.6219e-01,  6.4872e-01,  6.7115e-01,  7.0160e-01,  7.1758e-01,
            6.9861e-01,  7.1180e-01],
          [ 7.1739e-02,  7.4328e-02,  8.4353e-02,  8.9131e-02,  1.7026e-01,
            1.7785e-01,  2.3059e-01,  2.7366e-01,  3.2175e-01,  4.1089e-01,
            4.4518e-01,  4.1227e-01,  4.6128e-01,  4.9555e-01,  5.3634e-01,
            5.6199e-01,  3.9892e-01,  3.4834e-01,  3.6291e-01,  3.1640e-01,
            2.9041e-01,  2.5296e-01,  2.5756e-01,  3.2695e-01,  4.1156e-01,
            4.0798e-01,  3.9554e-01,  4.3939e-01,  5.2228e-01,  6.0631e-01,
            6.7141e-01,  6.7124e-01],
          [ 5.0218e-02,  5.2304e-02,  4.1207e-02,  6.6701e-02,  1.0273e-01,
            1.4269e-01,  1.5667e-01,  1.6387e-01,  2.0153e-01,  2.7747e-01,
            2.9386e-01,  3.2365e-01,  3.6305e-01,  3.0051e-01,  2.7092e-01,
            2.6679e-01,  1.8666e-01,  1.6756e-01,  1.7621e-01,  1.8625e-01,
            1.9908e-01,  1.8639e-01,  2.1041e-01,  2.4746e-01,  2.2504e-01,
            1.7604e-01,  1.5529e-01,  1.7780e-01,  2.0301e-01,  2.8718e-01,
            4.1279e-01,  4.5932e-01],
          [ 8.3629e-02,  6.1506e-02,  4.2768e-02,  7.0926e-02,  6.8632e-02,
            6.5859e-02,  6.6762e-02,  4.3939e-02,  4.6367e-02,  8.0612e-02,
            7.2632e-02,  6.2669e-02,  7.9948e-02,  8.4085e-02,  8.4993e-02,
            7.8097e-02,  6.0205e-02,  7.1717e-02,  9.5237e-02,  9.3780e-02,
            9.9064e-02,  1.0574e-01,  1.1325e-01,  1.2937e-01,  1.1620e-01,
            1.2440e-01,  1.2306e-01,  9.5838e-02,  9.4315e-02,  8.9553e-02,
            1.3393e-01,  1.7510e-01],
          [ 7.9145e-02,  5.5695e-02,  3.9137e-02,  6.9128e-02,  9.7166e-02,
            8.2913e-02,  7.2592e-02,  6.2471e-02,  5.0094e-02,  4.9972e-02,
            4.9261e-02,  5.7975e-02,  4.8790e-02,  6.5036e-02,  6.4397e-02,
            6.4017e-02,  5.5241e-02,  4.7144e-02,  8.4031e-02,  1.1117e-01,
            5.5196e-02,  3.9238e-02,  6.8486e-02,  8.3999e-02,  9.3107e-02,
            8.9948e-02,  7.1610e-02,  7.9844e-02,  7.7518e-02,  9.4919e-02,
            9.4425e-02,  1.0514e-01],
          [ 6.9846e-02,  5.1572e-02,  5.6390e-02,  5.7538e-02,  6.1045e-02,
            8.1463e-02,  8.1530e-02,  6.7278e-02,  6.6066e-02,  5.1429e-02,
            4.9396e-02,  5.0710e-02,  6.6318e-02,  7.9052e-02,  6.6414e-02,
            6.0883e-02,  6.6261e-02,  7.2071e-02,  8.2700e-02,  6.4848e-02,
            5.6246e-02,  4.2350e-02,  5.6358e-02,  5.4444e-02,  5.3898e-02,
            6.2572e-02,  6.4543e-02,  4.6202e-02,  5.9792e-02,  6.1191e-02,
            6.7430e-02,  7.1455e-02],
          [ 5.3272e-02,  2.9987e-02,  2.0121e-02,  1.5775e-02,  2.6749e-02,
            4.8764e-02,  4.2543e-02,  6.0721e-02,  5.4916e-02,  4.4784e-02,
            4.2755e-02,  5.7394e-02,  5.5922e-02,  4.3772e-02,  5.8589e-02,
            6.8445e-02,  8.7649e-02,  7.5070e-02,  8.8344e-02,  5.1731e-02,
            6.5793e-02,  7.3577e-02,  8.1248e-02,  8.4797e-02,  9.7214e-02,
            8.1348e-02,  6.6532e-02,  3.9240e-02,  4.0135e-02,  6.6840e-02,
            7.7981e-02,  6.8505e-02]],

         [[ 9.1582e-01,  9.0072e-01,  9.1437e-01,  9.0162e-01,  9.0206e-01,
            9.1045e-01,  9.1163e-01,  9.2699e-01,  9.1270e-01,  9.0776e-01,
            9.2023e-01,  9.2363e-01,  9.1378e-01,  9.1944e-01,  9.2796e-01,
            9.2392e-01,  9.2589e-01,  9.4271e-01,  9.4766e-01,  9.3458e-01,
            9.2483e-01,  9.4174e-01,  9.3557e-01,  9.3100e-01,  9.3220e-01,
            9.3902e-01,  9.4589e-01,  9.4211e-01,  9.3096e-01,  9.4656e-01,
            9.3671e-01,  9.4548e-01],
          [ 9.2309e-01,  9.2777e-01,  9.1693e-01,  9.1686e-01,  9.0610e-01,
            9.1096e-01,  9.1174e-01,  9.1856e-01,  9.1797e-01,  9.2667e-01,
            9.2831e-01,  9.2696e-01,  9.3473e-01,  9.3046e-01,  9.2119e-01,
            9.2382e-01,  9.3811e-01,  9.2210e-01,  9.3607e-01,  9.5331e-01,
            9.3450e-01,  9.3694e-01,  9.5288e-01,  9.4500e-01,  9.3993e-01,
            9.4862e-01,  9.4797e-01,  9.3580e-01,  9.3978e-01,  9.3861e-01,
            9.5137e-01,  9.5274e-01],
          [ 9.0620e-01,  8.9887e-01,  9.0430e-01,  9.1317e-01,  8.9384e-01,
            8.8905e-01,  9.0032e-01,  9.1928e-01,  9.1683e-01,  9.1591e-01,
            9.2519e-01,  9.2430e-01,  9.3624e-01,  9.2354e-01,  9.2707e-01,
            9.3467e-01,  9.2897e-01,  9.2662e-01,  9.4470e-01,  9.3956e-01,
            9.4659e-01,  9.4107e-01,  9.3082e-01,  9.3012e-01,  9.3236e-01,
            9.3509e-01,  9.3165e-01,  9.1955e-01,  9.4428e-01,  9.2945e-01,
            9.3537e-01,  9.4283e-01],
          [ 8.9716e-01,  9.0579e-01,  9.0980e-01,  9.0178e-01,  8.9005e-01,
            8.9488e-01,  9.2135e-01,  9.1403e-01,  9.1861e-01,  9.2472e-01,
            9.2662e-01,  9.1310e-01,  9.2969e-01,  9.3459e-01,  9.3579e-01,
            9.5640e-01,  9.4360e-01,  9.5043e-01,  9.4690e-01,  9.1750e-01,
            9.1165e-01,  9.1686e-01,  9.2022e-01,  9.3754e-01,  9.3091e-01,
            9.3886e-01,  9.4123e-01,  9.3889e-01,  9.4091e-01,  9.2840e-01,
            9.4898e-01,  9.5068e-01],
          [ 9.1368e-01,  9.1036e-01,  9.2335e-01,  9.0417e-01,  9.0078e-01,
            8.9759e-01,  9.2229e-01,  9.2761e-01,  9.3853e-01,  9.3050e-01,
            9.4551e-01,  9.4245e-01,  9.4584e-01,  9.3744e-01,  9.3131e-01,
            9.4455e-01,  9.5383e-01,  9.4235e-01,  9.2306e-01,  7.3469e-01,
            7.5518e-01,  8.7449e-01,  9.3232e-01,  9.4575e-01,  9.3270e-01,
            9.3063e-01,  9.1661e-01,  9.4813e-01,  9.3638e-01,  9.4261e-01,
            9.4917e-01,  9.4870e-01],
          [ 9.2210e-01,  9.2870e-01,  9.1674e-01,  9.2884e-01,  8.9930e-01,
            9.1649e-01,  9.2911e-01,  9.3291e-01,  9.3508e-01,  9.4237e-01,
            9.5244e-01,  9.2982e-01,  9.4287e-01,  9.5002e-01,  9.4881e-01,
            9.5635e-01,  9.5172e-01,  9.4111e-01,  9.0749e-01,  5.9183e-01,
            5.2831e-01,  8.0487e-01,  9.3494e-01,  9.3726e-01,  9.3662e-01,
            9.2520e-01,  9.2679e-01,  9.3037e-01,  9.4025e-01,  9.3647e-01,
            9.3792e-01,  9.4800e-01],
          [ 9.2584e-01,  9.2537e-01,  9.2201e-01,  9.3039e-01,  9.2383e-01,
            9.1819e-01,  9.1827e-01,  9.4326e-01,  9.3454e-01,  9.3788e-01,
            9.4065e-01,  9.3260e-01,  9.4095e-01,  9.3773e-01,  9.5199e-01,
            9.5268e-01,  9.4784e-01,  9.6992e-01,  8.9432e-01,  5.0526e-01,
            5.1747e-01,  7.1497e-01,  8.8092e-01,  9.5732e-01,  9.3842e-01,
            9.3558e-01,  9.3900e-01,  9.4145e-01,  9.4210e-01,  9.5222e-01,
            9.5262e-01,  9.5337e-01],
          [ 9.1513e-01,  9.2945e-01,  9.1544e-01,  9.2105e-01,  9.3298e-01,
            9.1959e-01,  9.2549e-01,  9.1636e-01,  9.3184e-01,  9.2072e-01,
            9.2288e-01,  9.3538e-01,  9.3841e-01,  9.2875e-01,  9.3197e-01,
            9.3834e-01,  9.5389e-01,  9.4744e-01,  8.6876e-01,  4.8558e-01,
            5.6024e-01,  7.1222e-01,  8.3637e-01,  9.3070e-01,  9.3708e-01,
            9.5105e-01,  9.4738e-01,  9.5240e-01,  9.4728e-01,  9.6677e-01,
            9.4554e-01,  9.6289e-01],
          [ 9.2161e-01,  9.1288e-01,  9.1724e-01,  9.1433e-01,  9.1555e-01,
            9.1875e-01,  9.2649e-01,  9.3275e-01,  9.0752e-01,  9.0796e-01,
            9.2574e-01,  9.1283e-01,  9.3633e-01,  9.2904e-01,  9.2631e-01,
            9.3460e-01,  9.2999e-01,  9.3168e-01,  8.8607e-01,  5.9803e-01,
            6.3904e-01,  7.1079e-01,  7.0207e-01,  8.2644e-01,  9.4899e-01,
            9.5566e-01,  9.4803e-01,  9.5772e-01,  9.7196e-01,  9.7221e-01,
            9.6897e-01,  9.7014e-01],
          [ 9.1969e-01,  9.1918e-01,  9.0693e-01,  9.0723e-01,  9.1419e-01,
            9.1944e-01,  9.1993e-01,  9.1435e-01,  9.2957e-01,  9.0553e-01,
            9.1146e-01,  9.1976e-01,  9.3588e-01,  9.2808e-01,  9.2854e-01,
            9.4312e-01,  9.3731e-01,  9.2812e-01,  9.4659e-01,  6.5968e-01,
            6.5465e-01,  5.9586e-01,  4.2775e-01,  6.8429e-01,  9.4537e-01,
            9.7355e-01,  9.8030e-01,  9.7737e-01,  9.6896e-01,  9.6960e-01,
            9.7515e-01,  9.7377e-01],
          [ 9.0594e-01,  9.0312e-01,  9.0397e-01,  8.9884e-01,  9.0336e-01,
            9.0816e-01,  9.1718e-01,  9.0836e-01,  9.0532e-01,  8.9950e-01,
            8.8690e-01,  9.0727e-01,  9.3550e-01,  9.0863e-01,  9.2335e-01,
            9.2637e-01,  9.3880e-01,  9.0881e-01,  8.5067e-01,  4.1232e-01,
            4.5980e-01,  3.8263e-01,  2.7993e-01,  6.4051e-01,  9.6062e-01,
            9.9359e-01,  9.7525e-01,  9.7597e-01,  9.6488e-01,  9.7532e-01,
            9.5582e-01,  9.8111e-01],
          [ 8.9722e-01,  9.0388e-01,  9.0246e-01,  9.0292e-01,  8.8949e-01,
            8.9465e-01,  8.8281e-01,  8.9407e-01,  8.8428e-01,  8.8729e-01,
            8.9071e-01,  9.1095e-01,  9.1408e-01,  9.1350e-01,  9.0860e-01,
            9.1632e-01,  8.9177e-01,  8.6794e-01,  7.1959e-01,  6.6724e-01,
            7.9479e-01,  6.1436e-01,  3.8089e-01,  6.3116e-01,  9.4811e-01,
            9.9192e-01,  9.7782e-01,  9.8432e-01,  9.6654e-01,  9.6052e-01,
            9.4641e-01,  9.5387e-01],
          [ 9.0034e-01,  8.8854e-01,  8.8523e-01,  8.9712e-01,  8.7847e-01,
            8.6310e-01,  8.7269e-01,  8.6669e-01,  8.6439e-01,  8.6536e-01,
            8.6697e-01,  8.8580e-01,  8.6037e-01,  8.5067e-01,  8.3841e-01,
            7.8061e-01,  7.2269e-01,  8.3177e-01,  8.8179e-01,  8.4584e-01,
            8.6057e-01,  5.7183e-01,  4.0360e-01,  6.7423e-01,  9.3803e-01,
            9.5566e-01,  9.3860e-01,  9.4647e-01,  9.5607e-01,  9.5811e-01,
            9.3751e-01,  9.2591e-01],
          [ 8.7599e-01,  8.3299e-01,  8.3516e-01,  8.5371e-01,  8.5381e-01,
            8.4327e-01,  8.1687e-01,  8.3670e-01,  8.1422e-01,  7.3152e-01,
            6.5798e-01,  5.2847e-01,  4.5966e-01,  4.3610e-01,  4.0801e-01,
            3.8751e-01,  3.1909e-01,  4.6157e-01,  8.3140e-01,  8.1549e-01,
            5.0861e-01,  2.7989e-01,  4.2125e-01,  6.4141e-01,  7.4230e-01,
            6.7567e-01,  6.5217e-01,  6.5927e-01,  7.8290e-01,  8.3288e-01,
            8.2141e-01,  8.4914e-01],
          [ 6.6270e-01,  6.3962e-01,  6.9893e-01,  7.0613e-01,  7.2266e-01,
            7.2327e-01,  7.1480e-01,  6.2190e-01,  3.9226e-01,  2.0363e-01,
            1.3813e-01,  1.0125e-01,  8.6913e-02,  1.0196e-01,  1.0071e-01,
            9.9257e-02,  8.5254e-02,  5.3099e-02,  4.1914e-01,  6.9638e-01,
            2.9628e-01,  1.7382e-01,  2.5928e-01,  2.5270e-01,  2.7184e-01,
            2.4044e-01,  3.1352e-01,  5.3125e-01,  7.0925e-01,  7.7680e-01,
            7.7452e-01,  8.0145e-01],
          [ 4.4692e-01,  4.6622e-01,  5.1090e-01,  5.5990e-01,  6.3263e-01,
            6.3932e-01,  7.0942e-01,  4.7395e-01,  1.3479e-01,  9.4268e-02,
            9.2540e-02,  1.0383e-01,  1.1665e-01,  1.5078e-01,  1.5603e-01,
            1.2760e-01,  1.0856e-01,  9.8254e-02,  1.4582e-01,  4.1952e-01,
            2.2701e-01,  1.3110e-01,  8.7044e-02,  6.7605e-02,  9.3433e-02,
            7.6374e-02,  2.5978e-01,  5.0551e-01,  7.1527e-01,  7.3642e-01,
            7.3318e-01,  7.5257e-01],
          [ 4.4017e-01,  4.6581e-01,  4.9040e-01,  5.4240e-01,  6.3168e-01,
            7.3604e-01,  7.8098e-01,  4.1326e-01,  1.2792e-01,  1.0037e-01,
            1.0387e-01,  1.2251e-01,  1.6491e-01,  2.0083e-01,  1.9389e-01,
            1.3655e-01,  1.2625e-01,  1.1622e-01,  8.2873e-02,  2.2001e-01,
            1.2586e-01,  7.4233e-02,  3.0609e-02,  2.4784e-02,  3.2641e-02,
            6.0581e-02,  3.2505e-01,  6.1490e-01,  7.1794e-01,  7.0641e-01,
            6.8515e-01,  7.2618e-01],
          [ 4.5404e-01,  4.2563e-01,  3.8205e-01,  3.8632e-01,  4.4341e-01,
            6.2020e-01,  6.7015e-01,  2.9448e-01,  1.0401e-01,  8.7340e-02,
            7.9172e-02,  9.8662e-02,  1.2852e-01,  1.3294e-01,  1.2694e-01,
            1.4840e-01,  1.8164e-01,  1.4280e-01,  1.0884e-01,  1.7878e-01,
            1.0706e-01,  3.8203e-02,  5.5824e-02,  3.7639e-02,  2.2687e-02,
            5.1983e-02,  3.3130e-01,  6.6097e-01,  6.9361e-01,  7.0305e-01,
            6.8973e-01,  6.9318e-01],
          [ 6.5521e-01,  5.1295e-01,  4.2279e-01,  3.9213e-01,  4.1745e-01,
            4.0344e-01,  3.4327e-01,  2.4187e-01,  1.3163e-01,  1.0129e-01,
            9.7895e-02,  1.1074e-01,  1.1591e-01,  1.2641e-01,  9.1104e-02,
            1.0962e-01,  1.2644e-01,  1.3434e-01,  1.2569e-01,  1.6522e-01,
            1.2444e-01,  7.4645e-02,  6.8869e-02,  5.6110e-02,  8.3107e-03,
            5.1350e-02,  3.4946e-01,  6.7636e-01,  6.7228e-01,  6.8227e-01,
            6.9365e-01,  6.9465e-01],
          [ 7.2774e-01,  6.6169e-01,  5.4534e-01,  5.0665e-01,  5.0716e-01,
            4.8131e-01,  4.7995e-01,  4.8413e-01,  4.4870e-01,  3.9848e-01,
            2.8119e-01,  1.3974e-01,  1.1320e-01,  9.8212e-02,  8.7007e-02,
            7.3863e-02,  8.1579e-02,  1.0587e-01,  1.3352e-01,  1.4472e-01,
            2.3162e-01,  5.4084e-01,  3.7707e-01,  9.9213e-02,  4.7240e-02,
            1.0347e-01,  4.0432e-01,  6.7752e-01,  6.7087e-01,  6.7147e-01,
            6.8191e-01,  6.8885e-01],
          [ 8.3907e-01,  8.0022e-01,  7.2846e-01,  7.1837e-01,  7.4745e-01,
            6.9417e-01,  7.3520e-01,  7.9170e-01,  7.5700e-01,  7.0397e-01,
            6.2960e-01,  3.9157e-01,  2.5919e-01,  2.5427e-01,  3.3399e-01,
            4.0470e-01,  4.3102e-01,  2.9413e-01,  1.5374e-01,  1.5495e-01,
            3.2350e-01,  7.5618e-01,  7.3572e-01,  3.9551e-01,  1.9036e-01,
            2.6134e-01,  5.0595e-01,  6.7655e-01,  7.0101e-01,  6.7999e-01,
            6.7612e-01,  6.7498e-01],
          [ 7.3752e-01,  6.1084e-01,  4.4594e-01,  3.9610e-01,  4.0966e-01,
            4.0737e-01,  3.9950e-01,  4.0690e-01,  4.8953e-01,  5.6218e-01,
            6.1382e-01,  6.5841e-01,  6.6353e-01,  6.5252e-01,  6.8137e-01,
            7.3547e-01,  7.8895e-01,  7.2013e-01,  3.1209e-01,  6.8455e-02,
            3.5226e-01,  6.1383e-01,  6.6079e-01,  5.8390e-01,  4.4978e-01,
            4.6506e-01,  6.1846e-01,  6.8544e-01,  6.7705e-01,  6.7775e-01,
            6.8967e-01,  6.8330e-01],
          [ 3.8301e-01,  2.1847e-01,  1.2615e-01,  1.1689e-01,  1.1829e-01,
            1.3929e-01,  3.9124e-02,  7.0354e-02,  2.3040e-01,  4.4256e-01,
            4.7907e-01,  4.8832e-01,  5.2952e-01,  6.0745e-01,  6.5850e-01,
            7.7843e-01,  7.8281e-01,  6.8517e-01,  3.9184e-01,  1.8997e-01,
            3.2065e-01,  5.3654e-01,  6.1629e-01,  6.2829e-01,  6.0834e-01,
            6.4694e-01,  6.8212e-01,  7.1251e-01,  6.9964e-01,  6.7437e-01,
            6.8825e-01,  6.8703e-01],
          [ 1.6515e-01,  9.5141e-02,  7.8209e-02,  9.4635e-02,  1.3026e-01,
            2.0559e-01,  1.5889e-01,  2.9630e-01,  5.1746e-01,  5.3007e-01,
            4.7607e-01,  4.8316e-01,  4.9999e-01,  5.9652e-01,  7.1782e-01,
            8.1681e-01,  7.7735e-01,  7.3053e-01,  6.4272e-01,  5.4997e-01,
            4.2226e-01,  5.6746e-01,  7.0260e-01,  6.8420e-01,  7.0457e-01,
            7.0130e-01,  6.9912e-01,  7.1174e-01,  7.0876e-01,  7.1097e-01,
            7.0502e-01,  7.2401e-01],
          [ 1.1359e-01,  6.3520e-02,  8.0148e-02,  5.4986e-02,  1.2727e-01,
            3.9091e-01,  4.9076e-01,  5.7297e-01,  6.1040e-01,  5.5223e-01,
            5.1901e-01,  5.2153e-01,  5.8546e-01,  6.3303e-01,  7.2410e-01,
            7.8252e-01,  7.8943e-01,  7.8027e-01,  7.6839e-01,  7.3208e-01,
            7.1253e-01,  7.4588e-01,  7.6362e-01,  7.5175e-01,  7.5042e-01,
            7.3352e-01,  7.1736e-01,  7.1413e-01,  7.0233e-01,  7.0469e-01,
            7.0084e-01,  7.3621e-01],
          [ 6.5673e-02,  7.1875e-02,  5.0289e-02,  5.6783e-02,  2.2141e-01,
            2.7768e-01,  3.5365e-01,  4.4256e-01,  4.8695e-01,  4.8126e-01,
            5.0646e-01,  4.9022e-01,  5.2717e-01,  5.7527e-01,  6.1381e-01,
            6.8695e-01,  7.2946e-01,  7.1437e-01,  7.1734e-01,  6.8147e-01,
            6.7417e-01,  5.8096e-01,  5.3264e-01,  5.8627e-01,  6.5836e-01,
            6.6973e-01,  6.7353e-01,  6.9624e-01,  7.2511e-01,  7.2021e-01,
            7.1776e-01,  7.2826e-01],
          [ 6.2601e-02,  6.9634e-02,  8.0326e-02,  1.2096e-01,  1.5832e-01,
            1.6170e-01,  1.8387e-01,  2.1672e-01,  2.7046e-01,  4.1335e-01,
            4.5834e-01,  4.1817e-01,  4.7431e-01,  5.1664e-01,  5.2102e-01,
            5.4389e-01,  3.9577e-01,  3.1569e-01,  3.3528e-01,  2.9969e-01,
            2.8765e-01,  2.5479e-01,  2.6570e-01,  3.2866e-01,  4.2582e-01,
            3.9532e-01,  4.0709e-01,  4.6541e-01,  5.4715e-01,  6.2859e-01,
            6.7521e-01,  6.9107e-01],
          [ 5.6478e-02,  4.4500e-02,  7.5164e-02,  7.3636e-02,  9.0332e-02,
            1.3405e-01,  1.5220e-01,  1.4094e-01,  1.6313e-01,  2.2858e-01,
            3.0952e-01,  3.1897e-01,  3.4853e-01,  2.8669e-01,  2.5249e-01,
            2.8306e-01,  1.8703e-01,  1.7726e-01,  2.0233e-01,  1.8466e-01,
            1.9436e-01,  1.7574e-01,  2.1430e-01,  2.5606e-01,  2.3698e-01,
            1.8603e-01,  1.8400e-01,  1.9129e-01,  2.0726e-01,  2.8568e-01,
            4.2391e-01,  4.5684e-01],
          [ 8.9617e-02,  7.9091e-02,  6.1903e-02,  8.5424e-02,  7.5179e-02,
            6.3049e-02,  8.4275e-02,  7.6946e-02,  6.5753e-02,  6.8631e-02,
            7.2698e-02,  7.2727e-02,  9.9129e-02,  1.0385e-01,  9.4529e-02,
            7.8955e-02,  6.4756e-02,  7.3189e-02,  9.0527e-02,  1.0483e-01,
            8.8918e-02,  1.1018e-01,  1.0638e-01,  1.1007e-01,  1.3719e-01,
            1.4966e-01,  1.4741e-01,  1.1611e-01,  9.7576e-02,  9.2770e-02,
            1.4341e-01,  1.8004e-01],
          [ 8.9928e-02,  7.4460e-02,  6.5127e-02,  8.8845e-02,  9.4191e-02,
            8.8615e-02,  7.6418e-02,  6.2046e-02,  6.3284e-02,  7.2530e-02,
            6.1617e-02,  8.0122e-02,  1.0124e-01,  1.0803e-01,  8.9243e-02,
            6.6354e-02,  7.0597e-02,  6.9492e-02,  7.4218e-02,  7.6207e-02,
            4.8344e-02,  4.8595e-02,  8.6405e-02,  1.0024e-01,  1.0016e-01,
            9.5505e-02,  9.4548e-02,  7.9280e-02,  8.0018e-02,  8.8179e-02,
            9.4685e-02,  1.0667e-01],
          [ 8.4310e-02,  5.0361e-02,  6.6638e-02,  7.1499e-02,  6.3371e-02,
            9.1624e-02,  8.3593e-02,  7.6990e-02,  6.3410e-02,  5.1964e-02,
            5.8344e-02,  4.5628e-02,  6.7796e-02,  9.9556e-02,  7.6412e-02,
            8.0919e-02,  7.0569e-02,  6.0737e-02,  6.6872e-02,  5.3533e-02,
            4.8498e-02,  5.8940e-02,  6.7919e-02,  6.9496e-02,  7.5480e-02,
            7.8897e-02,  7.5007e-02,  7.5649e-02,  6.8381e-02,  5.8552e-02,
            7.5927e-02,  7.6780e-02],
          [ 5.0413e-02,  5.3366e-02,  3.5352e-02,  4.1716e-02,  2.6747e-02,
            3.6157e-02,  4.1856e-02,  3.9003e-02,  5.5012e-02,  4.6495e-02,
            4.0149e-02,  3.0161e-02,  5.0980e-02,  3.1671e-02,  5.4839e-02,
            6.9244e-02,  7.8673e-02,  6.5165e-02,  7.6539e-02,  5.6936e-02,
            7.3048e-02,  8.9079e-02,  7.6879e-02,  9.0425e-02,  1.0441e-01,
            9.0280e-02,  8.2480e-02,  6.1074e-02,  5.0091e-02,  8.4757e-02,
            7.1836e-02,  5.8370e-02]],

         [[ 9.3921e-01,  9.0922e-01,  9.2175e-01,  9.2402e-01,  9.3667e-01,
            9.2720e-01,  9.3619e-01,  9.3031e-01,  9.3591e-01,  9.3453e-01,
            9.2184e-01,  9.3308e-01,  9.2882e-01,  9.4371e-01,  9.4628e-01,
            9.5542e-01,  9.6535e-01,  9.7431e-01,  9.6251e-01,  9.5594e-01,
            9.7077e-01,  9.6799e-01,  9.6180e-01,  9.6915e-01,  9.6382e-01,
            9.8016e-01,  9.5688e-01,  9.7010e-01,  9.5890e-01,  9.7310e-01,
            9.8300e-01,  9.5166e-01],
          [ 9.3377e-01,  9.3471e-01,  9.3632e-01,  9.3311e-01,  9.2078e-01,
            9.3470e-01,  9.3085e-01,  9.2889e-01,  9.3335e-01,  9.3795e-01,
            9.4123e-01,  9.3508e-01,  9.6162e-01,  9.5673e-01,  9.5596e-01,
            9.3925e-01,  9.5265e-01,  9.4478e-01,  9.6294e-01,  9.5682e-01,
            9.5719e-01,  9.5178e-01,  9.7273e-01,  9.6233e-01,  9.7520e-01,
            9.6506e-01,  9.6214e-01,  9.6601e-01,  9.5550e-01,  9.8976e-01,
            9.8775e-01,  9.7722e-01],
          [ 9.2054e-01,  9.0668e-01,  9.2134e-01,  9.2213e-01,  9.3972e-01,
            9.1799e-01,  9.2145e-01,  9.3113e-01,  9.1583e-01,  9.3139e-01,
            9.4117e-01,  9.3979e-01,  9.3986e-01,  9.3990e-01,  9.3630e-01,
            9.6122e-01,  9.5723e-01,  9.6558e-01,  9.6315e-01,  9.6339e-01,
            9.6428e-01,  9.5316e-01,  9.5457e-01,  9.4232e-01,  9.4021e-01,
            9.4572e-01,  9.4015e-01,  9.4269e-01,  9.6542e-01,  9.4747e-01,
            9.5140e-01,  9.6085e-01],
          [ 9.1057e-01,  9.1602e-01,  9.1888e-01,  9.1354e-01,  9.1004e-01,
            9.1426e-01,  9.3428e-01,  9.4054e-01,  9.3281e-01,  9.3109e-01,
            9.3800e-01,  9.4655e-01,  9.4562e-01,  9.3835e-01,  9.4947e-01,
            9.5950e-01,  9.3917e-01,  9.6752e-01,  9.7124e-01,  9.2843e-01,
            9.3070e-01,  9.2485e-01,  9.4095e-01,  9.4083e-01,  9.3605e-01,
            9.5178e-01,  9.2986e-01,  9.4058e-01,  9.4710e-01,  9.4348e-01,
            9.5373e-01,  9.6374e-01],
          [ 9.3412e-01,  9.1425e-01,  9.2766e-01,  9.2420e-01,  9.1408e-01,
            9.2521e-01,  9.4435e-01,  9.5234e-01,  9.3312e-01,  9.2235e-01,
            9.2993e-01,  9.4169e-01,  9.4868e-01,  9.3276e-01,  9.2986e-01,
            9.3477e-01,  9.4509e-01,  9.3966e-01,  9.1179e-01,  7.4477e-01,
            7.6477e-01,  8.7606e-01,  9.4206e-01,  9.5176e-01,  9.4911e-01,
            9.5318e-01,  9.3646e-01,  9.4798e-01,  9.5294e-01,  9.5338e-01,
            9.6918e-01,  9.6599e-01],
          [ 9.3373e-01,  9.3403e-01,  9.4150e-01,  9.2985e-01,  9.1602e-01,
            9.2536e-01,  9.2975e-01,  9.4779e-01,  9.4017e-01,  9.2958e-01,
            9.3086e-01,  9.4944e-01,  9.4250e-01,  9.3044e-01,  9.2935e-01,
            9.3891e-01,  9.3724e-01,  9.3626e-01,  8.9968e-01,  6.0709e-01,
            5.5521e-01,  8.3350e-01,  9.4628e-01,  9.2940e-01,  9.4654e-01,
            9.3815e-01,  9.4452e-01,  9.5255e-01,  9.4983e-01,  9.5872e-01,
            9.4418e-01,  9.5223e-01],
          [ 9.2949e-01,  9.2920e-01,  9.4464e-01,  9.3598e-01,  9.3772e-01,
            9.2159e-01,  9.3025e-01,  9.3828e-01,  9.3688e-01,  9.3407e-01,
            9.2661e-01,  9.2666e-01,  9.3398e-01,  9.4552e-01,  9.4065e-01,
            9.3598e-01,  9.3973e-01,  9.4272e-01,  8.9921e-01,  5.2529e-01,
            5.4450e-01,  7.7016e-01,  8.9612e-01,  9.5322e-01,  9.6069e-01,
            9.4319e-01,  9.4049e-01,  9.5889e-01,  9.5801e-01,  9.4881e-01,
            9.5335e-01,  9.4915e-01],
          [ 9.2181e-01,  9.3578e-01,  9.3885e-01,  9.3192e-01,  9.4183e-01,
            9.1521e-01,  9.2020e-01,  9.3137e-01,  9.3696e-01,  9.3505e-01,
            9.3040e-01,  9.2958e-01,  9.4644e-01,  9.3938e-01,  9.5657e-01,
            9.3044e-01,  9.2806e-01,  9.5176e-01,  8.9444e-01,  5.2717e-01,
            6.0755e-01,  7.4095e-01,  8.5320e-01,  9.5285e-01,  9.4919e-01,
            9.3958e-01,  9.4541e-01,  9.3828e-01,  9.5241e-01,  9.6177e-01,
            9.8350e-01,  9.5600e-01],
          [ 9.2860e-01,  9.2087e-01,  9.3184e-01,  9.2945e-01,  9.1672e-01,
            9.2652e-01,  9.3257e-01,  9.2207e-01,  9.1491e-01,  9.1830e-01,
            9.2670e-01,  9.4073e-01,  9.5472e-01,  9.3421e-01,  9.5140e-01,
            9.4029e-01,  9.2602e-01,  9.3431e-01,  8.9309e-01,  6.0312e-01,
            6.6832e-01,  7.5728e-01,  7.1982e-01,  8.5543e-01,  9.4443e-01,
            9.4385e-01,  9.6112e-01,  9.4865e-01,  9.5772e-01,  9.7324e-01,
            9.7385e-01,  9.7637e-01],
          [ 9.1646e-01,  9.0353e-01,  9.0719e-01,  9.1815e-01,  9.1395e-01,
            9.1703e-01,  9.0869e-01,  9.1613e-01,  9.1099e-01,  9.1222e-01,
            9.2369e-01,  9.2265e-01,  9.3487e-01,  9.3969e-01,  9.4846e-01,
            9.4913e-01,  9.4001e-01,  9.4390e-01,  9.8301e-01,  6.8852e-01,
            6.9593e-01,  6.4877e-01,  4.8263e-01,  7.1521e-01,  9.5932e-01,
            9.5828e-01,  9.6400e-01,  9.5984e-01,  9.5361e-01,  9.6313e-01,
            9.6156e-01,  9.8136e-01],
          [ 9.0575e-01,  9.1848e-01,  8.9667e-01,  8.9528e-01,  8.9068e-01,
            8.9762e-01,  9.0203e-01,  9.1545e-01,  9.0405e-01,  9.1746e-01,
            9.1491e-01,  9.2417e-01,  9.3832e-01,  9.3743e-01,  9.5678e-01,
            9.4799e-01,  9.4304e-01,  9.5395e-01,  8.6841e-01,  4.4166e-01,
            5.0153e-01,  4.4259e-01,  3.4963e-01,  6.8848e-01,  9.6889e-01,
            9.6381e-01,  9.6636e-01,  9.6015e-01,  9.5504e-01,  9.5749e-01,
            9.5462e-01,  9.6020e-01],
          [ 9.2171e-01,  8.9779e-01,  9.0678e-01,  9.0761e-01,  8.8569e-01,
            8.9005e-01,  9.0115e-01,  8.8331e-01,  9.0537e-01,  9.0273e-01,
            9.1537e-01,  9.2287e-01,  9.3784e-01,  9.3610e-01,  9.4296e-01,
            9.2770e-01,  9.2076e-01,  8.7528e-01,  7.4792e-01,  6.9674e-01,
            8.5466e-01,  6.7542e-01,  4.2210e-01,  6.5884e-01,  9.5131e-01,
            9.5603e-01,  9.4626e-01,  9.6515e-01,  9.5841e-01,  9.5795e-01,
            9.5158e-01,  9.3362e-01],
          [ 9.0680e-01,  8.9967e-01,  8.9432e-01,  8.9581e-01,  8.9743e-01,
            8.8984e-01,  9.0573e-01,  8.9680e-01,  9.0972e-01,  9.2655e-01,
            9.4471e-01,  9.3483e-01,  9.0737e-01,  9.0075e-01,  8.6871e-01,
            8.3203e-01,  7.6805e-01,  8.9629e-01,  9.2492e-01,  8.7220e-01,
            8.7493e-01,  6.1736e-01,  4.4575e-01,  7.1196e-01,  9.6891e-01,
            9.6856e-01,  9.2947e-01,  9.2767e-01,  9.3142e-01,  9.4288e-01,
            9.3456e-01,  9.2989e-01],
          [ 8.9816e-01,  8.5872e-01,  8.7939e-01,  8.9565e-01,  9.0008e-01,
            8.9922e-01,  8.8415e-01,  8.9386e-01,  8.5528e-01,  7.9143e-01,
            7.1987e-01,  5.9345e-01,  5.2537e-01,  4.9072e-01,  4.6579e-01,
            4.6932e-01,  4.1040e-01,  5.3956e-01,  8.7294e-01,  8.3697e-01,
            5.0833e-01,  3.3591e-01,  4.6160e-01,  6.8259e-01,  7.7590e-01,
            6.7613e-01,  6.4728e-01,  6.5685e-01,  7.9336e-01,  8.5058e-01,
            8.5768e-01,  8.7615e-01],
          [ 7.1475e-01,  7.1241e-01,  7.5310e-01,  7.8780e-01,  8.1315e-01,
            7.9461e-01,  7.9860e-01,  6.9112e-01,  4.3406e-01,  2.3773e-01,
            1.4902e-01,  1.1272e-01,  1.1220e-01,  1.3588e-01,  1.2672e-01,
            1.6215e-01,  1.5920e-01,  1.2882e-01,  4.7000e-01,  7.1033e-01,
            2.9528e-01,  2.1530e-01,  3.2512e-01,  3.0928e-01,  3.1699e-01,
            2.6840e-01,  3.4833e-01,  5.6261e-01,  7.6636e-01,  8.3165e-01,
            8.1696e-01,  8.4100e-01],
          [ 5.2400e-01,  5.5238e-01,  5.9838e-01,  6.5034e-01,  7.2712e-01,
            7.4560e-01,  7.9970e-01,  5.2305e-01,  1.6705e-01,  1.0365e-01,
            1.0038e-01,  1.2814e-01,  1.3842e-01,  1.6732e-01,  1.7950e-01,
            1.5473e-01,  2.1226e-01,  1.6937e-01,  2.0524e-01,  4.1386e-01,
            2.3712e-01,  1.5890e-01,  1.1979e-01,  1.0876e-01,  1.2076e-01,
            1.1443e-01,  2.8918e-01,  5.6879e-01,  7.8782e-01,  7.8865e-01,
            7.8504e-01,  7.9911e-01],
          [ 4.5443e-01,  4.6718e-01,  4.9699e-01,  5.5393e-01,  6.7341e-01,
            7.8022e-01,  8.3535e-01,  4.5247e-01,  1.3768e-01,  1.2256e-01,
            1.3281e-01,  1.4892e-01,  1.9418e-01,  2.3950e-01,  2.3855e-01,
            2.0102e-01,  2.1835e-01,  2.2069e-01,  1.1827e-01,  2.1559e-01,
            1.2749e-01,  8.8101e-02,  4.2285e-02,  6.1930e-02,  6.0231e-02,
            6.8899e-02,  3.7444e-01,  6.7018e-01,  7.6524e-01,  7.6601e-01,
            7.5959e-01,  7.6002e-01],
          [ 4.1926e-01,  3.7527e-01,  3.4954e-01,  3.5585e-01,  4.2826e-01,
            6.3268e-01,  7.4558e-01,  3.6195e-01,  1.4167e-01,  1.2829e-01,
            1.2116e-01,  1.4888e-01,  1.8150e-01,  1.8704e-01,  1.8428e-01,
            2.1737e-01,  2.5044e-01,  2.2252e-01,  1.5051e-01,  1.8399e-01,
            1.2410e-01,  6.1949e-02,  5.2526e-02,  5.3566e-02,  4.5613e-02,
            8.3978e-02,  3.8839e-01,  7.0478e-01,  7.4413e-01,  7.4441e-01,
            7.3572e-01,  7.4227e-01],
          [ 6.0882e-01,  4.6411e-01,  3.7974e-01,  3.7186e-01,  4.2095e-01,
            4.5560e-01,  4.0977e-01,  3.2056e-01,  1.8931e-01,  1.4227e-01,
            1.5541e-01,  1.4326e-01,  1.6248e-01,  1.4847e-01,  1.5466e-01,
            1.7572e-01,  1.7860e-01,  2.0223e-01,  1.4672e-01,  1.6845e-01,
            1.4132e-01,  9.6461e-02,  9.5746e-02,  6.2473e-02,  2.2679e-02,
            8.1601e-02,  4.0112e-01,  7.2478e-01,  7.2300e-01,  7.3635e-01,
            7.3320e-01,  7.2992e-01],
          [ 7.3309e-01,  6.5474e-01,  5.3485e-01,  5.0104e-01,  5.1850e-01,
            5.2564e-01,  5.3016e-01,  5.5538e-01,  5.1767e-01,  4.6153e-01,
            3.3633e-01,  2.0220e-01,  1.4111e-01,  1.3695e-01,  1.0333e-01,
            1.0657e-01,  1.2444e-01,  1.5061e-01,  1.4308e-01,  1.4468e-01,
            2.5686e-01,  5.7628e-01,  4.0525e-01,  1.1984e-01,  5.0312e-02,
            1.1004e-01,  4.5072e-01,  7.1366e-01,  7.2210e-01,  7.2718e-01,
            7.1286e-01,  7.3209e-01],
          [ 8.9165e-01,  8.5263e-01,  7.7519e-01,  7.5569e-01,  7.8811e-01,
            7.7625e-01,  8.3527e-01,  8.8223e-01,  8.4766e-01,  7.8050e-01,
            6.6275e-01,  4.3302e-01,  2.9462e-01,  2.8176e-01,  3.4509e-01,
            4.7033e-01,  4.8644e-01,  3.3967e-01,  1.6296e-01,  1.4758e-01,
            3.4347e-01,  8.0769e-01,  7.9825e-01,  4.6863e-01,  2.2705e-01,
            3.0318e-01,  5.6330e-01,  7.4720e-01,  7.3801e-01,  7.4150e-01,
            7.2866e-01,  7.2800e-01],
          [ 7.9760e-01,  6.7514e-01,  5.0457e-01,  4.3824e-01,  4.5821e-01,
            4.5044e-01,  4.6492e-01,  4.8232e-01,  5.6210e-01,  5.9190e-01,
            6.4889e-01,  6.8938e-01,  6.9455e-01,  6.8686e-01,  7.1539e-01,
            7.7517e-01,  8.2563e-01,  7.5566e-01,  3.5253e-01,  1.0719e-01,
            3.6226e-01,  6.6187e-01,  7.1744e-01,  6.4594e-01,  4.7713e-01,
            5.1678e-01,  6.7296e-01,  7.3680e-01,  7.4050e-01,  7.3793e-01,
            7.3259e-01,  7.2681e-01],
          [ 4.3274e-01,  2.6729e-01,  1.6694e-01,  1.6601e-01,  1.6527e-01,
            1.9922e-01,  1.0058e-01,  1.0446e-01,  2.5629e-01,  4.5143e-01,
            4.8109e-01,  4.9956e-01,  5.4586e-01,  6.3472e-01,  6.7949e-01,
            7.8430e-01,  8.1343e-01,  7.0619e-01,  4.2489e-01,  2.1327e-01,
            3.5360e-01,  5.8467e-01,  6.6426e-01,  6.4765e-01,  6.4073e-01,
            6.8338e-01,  7.3556e-01,  7.5756e-01,  7.5082e-01,  7.3505e-01,
            7.4518e-01,  7.2214e-01],
          [ 1.9753e-01,  1.2734e-01,  1.0446e-01,  1.1860e-01,  1.6145e-01,
            2.4688e-01,  1.7227e-01,  3.1263e-01,  5.2657e-01,  5.0726e-01,
            4.7490e-01,  4.7833e-01,  5.3102e-01,  6.2331e-01,  7.3297e-01,
            8.0537e-01,  7.9644e-01,  7.2583e-01,  6.6660e-01,  5.6984e-01,
            4.5392e-01,  6.0273e-01,  7.4607e-01,  7.2020e-01,  7.2951e-01,
            7.4960e-01,  7.6959e-01,  7.6375e-01,  7.7036e-01,  7.4399e-01,
            7.5510e-01,  7.8013e-01],
          [ 1.0034e-01,  9.2545e-02,  7.4033e-02,  6.1892e-02,  1.4319e-01,
            3.6608e-01,  4.5383e-01,  5.4645e-01,  6.1103e-01,  5.5871e-01,
            5.4441e-01,  5.3892e-01,  6.1209e-01,  6.5499e-01,  7.3329e-01,
            8.1098e-01,  7.9665e-01,  7.8350e-01,  7.9150e-01,  7.5572e-01,
            7.5447e-01,  7.7451e-01,  7.7662e-01,  7.6208e-01,  7.7872e-01,
            7.7131e-01,  7.7853e-01,  7.6366e-01,  7.5958e-01,  7.6475e-01,
            7.4927e-01,  7.7253e-01],
          [ 8.5740e-02,  8.6985e-02,  5.2164e-02,  6.8593e-02,  2.1560e-01,
            2.0762e-01,  3.0116e-01,  3.8483e-01,  4.5233e-01,  5.1866e-01,
            5.4939e-01,  5.0493e-01,  5.7594e-01,  5.9207e-01,  6.4262e-01,
            7.1063e-01,  7.4654e-01,  7.2182e-01,  7.3068e-01,  7.0829e-01,
            6.8616e-01,  6.1234e-01,  5.4566e-01,  6.0884e-01,  6.8449e-01,
            7.0277e-01,  7.2079e-01,  7.3298e-01,  7.6079e-01,  7.7309e-01,
            7.6306e-01,  7.6773e-01],
          [ 6.0569e-02,  7.5526e-02,  6.0416e-02,  9.0201e-02,  1.1227e-01,
            1.2578e-01,  1.5788e-01,  1.7794e-01,  2.5339e-01,  4.0818e-01,
            4.9702e-01,  4.6843e-01,  5.0153e-01,  5.1863e-01,  5.3475e-01,
            5.7331e-01,  4.4520e-01,  3.4477e-01,  3.5495e-01,  2.8905e-01,
            2.7580e-01,  2.6199e-01,  2.6849e-01,  3.5921e-01,  4.3933e-01,
            4.0261e-01,  4.2028e-01,  4.8383e-01,  5.8849e-01,  6.7674e-01,
            7.4110e-01,  7.3862e-01],
          [ 4.1868e-02,  3.3107e-02,  6.5806e-02,  6.7132e-02,  8.6571e-02,
            1.2382e-01,  1.3897e-01,  1.3715e-01,  1.5115e-01,  2.1596e-01,
            3.0773e-01,  3.2491e-01,  3.6244e-01,  2.7665e-01,  2.5208e-01,
            2.9934e-01,  2.1905e-01,  1.6917e-01,  1.8347e-01,  1.7559e-01,
            1.6979e-01,  1.7516e-01,  2.1365e-01,  2.5230e-01,  2.3224e-01,
            1.8562e-01,  1.7983e-01,  2.0142e-01,  2.1945e-01,  3.1834e-01,
            4.6350e-01,  5.1728e-01],
          [ 5.2668e-02,  4.8383e-02,  6.2168e-02,  7.3512e-02,  6.9757e-02,
            7.4483e-02,  7.9821e-02,  4.8336e-02,  7.0152e-02,  6.5060e-02,
            5.8784e-02,  5.8660e-02,  6.5940e-02,  6.8716e-02,  5.7876e-02,
            8.1824e-02,  5.9397e-02,  6.6973e-02,  7.1802e-02,  6.6361e-02,
            5.4729e-02,  7.0774e-02,  8.0448e-02,  1.0247e-01,  9.5315e-02,
            1.0610e-01,  1.0078e-01,  9.3409e-02,  5.6316e-02,  7.6552e-02,
            1.4336e-01,  1.9733e-01],
          [ 5.5171e-02,  5.6675e-02,  6.7876e-02,  6.5629e-02,  5.7341e-02,
            6.6736e-02,  6.5028e-02,  5.8008e-02,  5.8602e-02,  5.5928e-02,
            5.0822e-02,  5.3724e-02,  6.8646e-02,  7.0646e-02,  7.1317e-02,
            3.2359e-02,  4.1942e-03,  8.9915e-03,  3.2044e-02,  4.2837e-02,
            2.3597e-02,  2.7258e-02,  3.7697e-02,  6.0301e-02,  6.0893e-02,
            6.3031e-02,  6.2677e-02,  4.3621e-02,  3.8001e-02,  5.4264e-02,
            8.5757e-02,  1.2577e-01],
          [ 5.9910e-02,  4.3393e-02,  4.2827e-02,  4.1609e-02,  6.1013e-02,
            6.1138e-02,  6.0755e-02,  6.7355e-02,  4.0526e-02,  3.2430e-02,
            4.7095e-02,  3.6461e-02,  4.6795e-02,  6.0146e-02,  4.8510e-02,
            2.8769e-02,  2.8016e-02,  3.9492e-02,  3.4891e-02,  1.2903e-02,
            1.6484e-02,  1.5067e-02,  2.9523e-02,  2.1635e-02,  5.2113e-02,
            4.9412e-02,  3.6646e-02,  4.0079e-02,  4.7702e-02,  5.4889e-02,
            6.3596e-02,  9.1210e-02],
          [ 2.7673e-02,  2.1819e-02,  8.2537e-03,  1.7400e-02,  7.4826e-03,
            8.5038e-03,  1.9652e-02,  3.3976e-02,  2.7040e-02,  1.3378e-02,
            2.2338e-02,  2.5056e-02,  1.6816e-02,  1.6176e-02,  1.6762e-02,
            4.0404e-02,  4.6374e-02,  3.0283e-02,  3.9427e-02,  3.1606e-02,
            3.2179e-02,  4.6640e-02,  4.9066e-02,  3.0550e-02,  5.7736e-02,
            6.9306e-02,  5.8096e-02,  3.2753e-02,  2.5440e-02,  4.5554e-02,
            5.9578e-02,  5.6744e-02]]],


        [[[ 5.7387e-01,  5.8595e-01,  5.7007e-01,  5.8378e-01,  5.8917e-01,
            5.9566e-01,  6.1867e-01,  5.9101e-01,  6.0744e-01,  4.8276e-01,
            3.7297e-01,  4.5495e-01,  4.1483e-01,  3.4816e-01,  3.2339e-01,
            3.8938e-01,  3.9697e-01,  4.2057e-01,  3.6375e-01,  3.4706e-01,
            2.7864e-01,  2.9674e-01,  3.1972e-01,  3.0476e-01,  3.3357e-01,
            2.3274e-01,  2.2659e-01,  3.1893e-01,  3.3796e-01,  4.3361e-01,
            5.4446e-01,  5.8570e-01],
          [ 5.4767e-01,  5.9526e-01,  5.4281e-01,  5.3896e-01,  5.9384e-01,
            5.9549e-01,  6.0818e-01,  5.8652e-01,  5.7071e-01,  5.2996e-01,
            4.9238e-01,  4.5456e-01,  4.1354e-01,  3.1094e-01,  3.0658e-01,
            3.5125e-01,  3.7065e-01,  3.3000e-01,  3.0516e-01,  3.1761e-01,
            2.3693e-01,  2.4223e-01,  2.4303e-01,  2.3553e-01,  2.4835e-01,
            2.4268e-01,  2.2606e-01,  3.1769e-01,  3.4069e-01,  4.6892e-01,
            5.7682e-01,  5.9000e-01],
          [ 5.1955e-01,  5.6742e-01,  5.4397e-01,  5.7218e-01,  5.8774e-01,
            5.8071e-01,  5.8738e-01,  5.5125e-01,  4.8801e-01,  5.0047e-01,
            5.3217e-01,  4.7897e-01,  4.5274e-01,  3.9495e-01,  3.6277e-01,
            4.0064e-01,  4.0707e-01,  3.4024e-01,  2.9773e-01,  3.2160e-01,
            3.0416e-01,  2.7472e-01,  2.8778e-01,  2.8325e-01,  2.6997e-01,
            2.7256e-01,  2.7107e-01,  3.0525e-01,  3.9152e-01,  5.0919e-01,
            5.6180e-01,  5.9565e-01],
          [ 5.3046e-01,  5.2658e-01,  5.0696e-01,  5.3761e-01,  5.4663e-01,
            5.1477e-01,  5.4226e-01,  5.4715e-01,  4.7112e-01,  4.5745e-01,
            5.0442e-01,  5.0473e-01,  5.1557e-01,  5.0985e-01,  5.1798e-01,
            5.5018e-01,  5.5576e-01,  5.7571e-01,  5.6284e-01,  5.6211e-01,
            5.1632e-01,  5.1590e-01,  5.1921e-01,  5.2783e-01,  5.4480e-01,
            5.3113e-01,  5.6711e-01,  5.1992e-01,  4.4232e-01,  5.7547e-01,
            6.0322e-01,  6.1760e-01],
          [ 5.9813e-01,  5.6231e-01,  5.2858e-01,  4.7804e-01,  4.8888e-01,
            5.4717e-01,  6.0092e-01,  5.7626e-01,  5.9418e-01,  5.7691e-01,
            5.9868e-01,  5.6644e-01,  5.4803e-01,  5.9756e-01,  6.2459e-01,
            6.2468e-01,  6.0955e-01,  5.9700e-01,  6.0010e-01,  6.1718e-01,
            6.2427e-01,  7.1289e-01,  6.7404e-01,  6.9088e-01,  7.3475e-01,
            7.8352e-01,  7.3825e-01,  5.9844e-01,  5.0920e-01,  6.1937e-01,
            6.1929e-01,  6.1332e-01],
          [ 6.6912e-01,  6.1028e-01,  6.1009e-01,  5.2282e-01,  5.8237e-01,
            6.5241e-01,  6.4387e-01,  6.2988e-01,  6.6789e-01,  6.6118e-01,
            6.3543e-01,  6.2026e-01,  5.5111e-01,  5.8885e-01,  6.0504e-01,
            5.9221e-01,  5.9528e-01,  5.4617e-01,  5.1500e-01,  5.3854e-01,
            5.3936e-01,  6.8777e-01,  7.3374e-01,  7.1595e-01,  6.5782e-01,
            5.9889e-01,  5.2560e-01,  5.6458e-01,  5.5844e-01,  5.8161e-01,
            6.3493e-01,  6.4306e-01],
          [ 6.5213e-01,  6.0122e-01,  5.8311e-01,  5.5449e-01,  6.0607e-01,
            6.4654e-01,  6.4444e-01,  6.5618e-01,  6.8776e-01,  6.8804e-01,
            6.8548e-01,  6.6725e-01,  5.9970e-01,  5.4323e-01,  5.2927e-01,
            5.2223e-01,  5.0636e-01,  4.7613e-01,  4.4757e-01,  4.9741e-01,
            4.9400e-01,  5.3488e-01,  6.0876e-01,  6.7332e-01,  6.0569e-01,
            5.6343e-01,  5.3979e-01,  5.5984e-01,  5.7315e-01,  5.4540e-01,
            6.1603e-01,  6.2271e-01],
          [ 4.8899e-01,  4.7799e-01,  5.0661e-01,  5.7517e-01,  6.4415e-01,
            6.4964e-01,  6.4981e-01,  6.5667e-01,  6.8838e-01,  7.0859e-01,
            6.8628e-01,  6.6673e-01,  5.7472e-01,  4.7681e-01,  4.8201e-01,
            4.4699e-01,  4.1221e-01,  3.7511e-01,  3.5674e-01,  4.4540e-01,
            4.9761e-01,  4.7375e-01,  4.2113e-01,  5.3280e-01,  6.2439e-01,
            5.9576e-01,  5.7530e-01,  5.7053e-01,  5.4700e-01,  5.1385e-01,
            5.6585e-01,  5.7953e-01],
          [ 4.2786e-01,  4.2714e-01,  3.9490e-01,  4.9294e-01,  6.2715e-01,
            6.5431e-01,  6.0388e-01,  6.2313e-01,  6.4982e-01,  6.7901e-01,
            6.5823e-01,  6.3200e-01,  5.1108e-01,  4.5369e-01,  4.1070e-01,
            3.7068e-01,  3.8612e-01,  3.8825e-01,  3.7892e-01,  4.2824e-01,
            4.9894e-01,  4.4908e-01,  3.3873e-01,  3.6889e-01,  4.8014e-01,
            5.4885e-01,  5.2044e-01,  5.2137e-01,  4.8581e-01,  4.8457e-01,
            5.1401e-01,  4.9596e-01],
          [ 3.7811e-01,  3.3838e-01,  2.7455e-01,  3.0593e-01,  4.6286e-01,
            5.8141e-01,  5.5809e-01,  6.0198e-01,  6.1476e-01,  6.5246e-01,
            6.4975e-01,  5.9385e-01,  4.9896e-01,  3.6268e-01,  3.0951e-01,
            3.1430e-01,  3.3951e-01,  3.5415e-01,  3.4996e-01,  3.6243e-01,
            3.8268e-01,  3.8737e-01,  3.3224e-01,  3.4986e-01,  4.2680e-01,
            4.5858e-01,  4.4256e-01,  4.4470e-01,  4.4012e-01,  5.1076e-01,
            4.3604e-01,  4.1493e-01],
          [ 3.3579e-01,  2.7302e-01,  1.8643e-01,  2.2129e-01,  2.8590e-01,
            3.5785e-01,  3.8736e-01,  4.6735e-01,  5.0273e-01,  5.7930e-01,
            6.0939e-01,  5.5733e-01,  4.8866e-01,  3.3018e-01,  2.8353e-01,
            2.6398e-01,  2.9842e-01,  2.6555e-01,  2.9474e-01,  3.1168e-01,
            3.1553e-01,  3.2754e-01,  2.9900e-01,  3.6881e-01,  4.2281e-01,
            4.1133e-01,  4.1386e-01,  4.2519e-01,  4.8295e-01,  4.6565e-01,
            3.8078e-01,  3.7210e-01],
          [ 3.2694e-01,  3.6971e-01,  3.4282e-01,  3.6643e-01,  3.8314e-01,
            3.5006e-01,  3.7541e-01,  3.9866e-01,  4.8650e-01,  5.9076e-01,
            5.8383e-01,  4.7299e-01,  4.7815e-01,  4.5172e-01,  3.0557e-01,
            2.6949e-01,  3.1122e-01,  2.4003e-01,  2.6076e-01,  3.0972e-01,
            3.1734e-01,  3.7239e-01,  3.9214e-01,  4.6478e-01,  5.2193e-01,
            4.9452e-01,  4.9905e-01,  5.1509e-01,  5.2811e-01,  4.9017e-01,
            4.7453e-01,  4.7933e-01],
          [ 4.9589e-01,  5.4757e-01,  5.7063e-01,  5.8633e-01,  5.2466e-01,
            4.9235e-01,  4.8730e-01,  4.6866e-01,  6.4195e-01,  6.1239e-01,
            3.9364e-01,  3.8426e-01,  4.0584e-01,  3.7738e-01,  2.7993e-01,
            1.4153e-01,  1.8725e-01,  2.2130e-01,  1.2649e-01,  1.1963e-01,
            2.0579e-01,  3.1144e-01,  3.9141e-01,  5.1165e-01,  5.7221e-01,
            5.8112e-01,  5.8026e-01,  5.6310e-01,  5.5239e-01,  5.5039e-01,
            5.7199e-01,  6.0188e-01],
          [ 5.7133e-01,  5.3432e-01,  5.1997e-01,  5.4498e-01,  5.3265e-01,
            4.8402e-01,  4.6878e-01,  5.2696e-01,  6.1621e-01,  4.6118e-01,
            2.8102e-01,  3.1087e-01,  2.5589e-01,  1.8812e-01,  1.3803e-01,
            1.3221e-01,  1.7187e-01,  1.6904e-01,  1.2286e-01,  1.0047e-01,
            1.2605e-01,  2.1588e-01,  2.2111e-01,  3.3098e-01,  5.2728e-01,
            5.8360e-01,  5.9249e-01,  5.5465e-01,  5.7503e-01,  5.8286e-01,
            5.9852e-01,  6.4478e-01],
          [ 5.3636e-01,  5.1204e-01,  5.0017e-01,  4.9139e-01,  5.1073e-01,
            4.8665e-01,  4.8900e-01,  4.8133e-01,  4.2803e-01,  2.4061e-01,
            2.2078e-01,  2.3873e-01,  1.5602e-01,  1.3807e-01,  2.3315e-01,
            2.2136e-01,  1.7671e-01,  1.5227e-01,  1.5394e-01,  1.4627e-01,
            1.2964e-01,  1.2970e-01,  1.1883e-01,  1.9601e-01,  3.4034e-01,
            3.6990e-01,  5.1493e-01,  6.1525e-01,  6.1316e-01,  6.1573e-01,
            5.9828e-01,  6.5189e-01],
          [ 4.7202e-01,  4.7660e-01,  4.8142e-01,  4.9387e-01,  4.8020e-01,
            4.9348e-01,  5.3835e-01,  5.4576e-01,  4.3605e-01,  2.9542e-01,
            2.5253e-01,  2.2256e-01,  1.3739e-01,  1.5863e-01,  1.8132e-01,
            1.5964e-01,  1.2891e-01,  1.0301e-01,  1.2821e-01,  1.6034e-01,
            1.3603e-01,  1.1325e-01,  1.1986e-01,  1.3376e-01,  1.8788e-01,
            2.1260e-01,  4.5061e-01,  6.0113e-01,  6.2837e-01,  5.8471e-01,
            5.2032e-01,  5.1696e-01],
          [ 4.3061e-01,  3.7758e-01,  3.8643e-01,  4.1313e-01,  4.3394e-01,
            5.0928e-01,  5.5241e-01,  5.4541e-01,  4.5096e-01,  3.9765e-01,
            3.4472e-01,  2.9474e-01,  2.4383e-01,  1.8472e-01,  2.0429e-01,
            1.8330e-01,  1.8640e-01,  1.1899e-01,  1.1530e-01,  1.6104e-01,
            1.6068e-01,  1.1176e-01,  1.5460e-01,  1.2589e-01,  1.0683e-01,
            2.1102e-01,  4.7335e-01,  5.3159e-01,  5.3709e-01,  5.3028e-01,
            4.8179e-01,  3.4310e-01],
          [ 4.0845e-01,  4.1005e-01,  4.0332e-01,  3.7688e-01,  3.7899e-01,
            3.8940e-01,  4.4194e-01,  4.5291e-01,  4.4657e-01,  4.4293e-01,
            3.9397e-01,  3.9162e-01,  3.8012e-01,  3.4284e-01,  3.2555e-01,
            3.2564e-01,  3.0067e-01,  1.9189e-01,  1.2837e-01,  1.8057e-01,
            1.4074e-01,  1.2156e-01,  1.4606e-01,  1.1972e-01,  8.1408e-02,
            2.4136e-01,  4.6131e-01,  4.6878e-01,  4.9577e-01,  5.3885e-01,
            4.7715e-01,  3.2992e-01],
          [ 3.6289e-01,  3.4273e-01,  2.9346e-01,  2.5663e-01,  2.6926e-01,
            2.9535e-01,  3.2007e-01,  3.3456e-01,  4.5016e-01,  5.4490e-01,
            5.4241e-01,  4.9018e-01,  4.5861e-01,  4.4020e-01,  4.2139e-01,
            3.7998e-01,  3.5864e-01,  2.3394e-01,  9.2512e-02,  1.3789e-01,
            1.2589e-01,  1.6721e-01,  2.1140e-01,  1.8793e-01,  1.1382e-01,
            2.3064e-01,  4.1189e-01,  4.2729e-01,  4.6213e-01,  5.1864e-01,
            4.9292e-01,  4.7789e-01],
          [ 3.3135e-01,  2.9779e-01,  1.9135e-01,  1.6696e-01,  2.3077e-01,
            2.8985e-01,  2.2551e-01,  2.1089e-01,  2.6176e-01,  3.8860e-01,
            5.7324e-01,  6.4177e-01,  6.3402e-01,  5.1536e-01,  5.2618e-01,
            5.7315e-01,  4.8333e-01,  2.1055e-01,  5.1674e-02,  6.6521e-02,
            1.2637e-01,  2.0909e-01,  2.4231e-01,  2.1688e-01,  1.7383e-01,
            2.0398e-01,  3.6786e-01,  3.8247e-01,  4.4760e-01,  4.7739e-01,
            4.6429e-01,  4.5111e-01],
          [ 3.0795e-01,  3.2706e-01,  2.2961e-01,  1.3768e-01,  2.3636e-01,
            2.4147e-01,  2.1225e-01,  1.6806e-01,  1.6987e-01,  2.0359e-01,
            3.5213e-01,  4.8303e-01,  5.5602e-01,  5.9860e-01,  7.1188e-01,
            8.3327e-01,  7.5198e-01,  2.3318e-01,  4.7322e-02,  8.2457e-02,
            1.2197e-01,  1.7672e-01,  1.9783e-01,  2.0261e-01,  2.2310e-01,
            1.4792e-01,  3.3138e-01,  3.9115e-01,  4.2213e-01,  4.4237e-01,
            4.4261e-01,  4.3242e-01],
          [ 3.1324e-01,  2.9569e-01,  2.9633e-01,  1.9305e-01,  1.3074e-01,
            1.0742e-01,  1.2212e-01,  7.0772e-02,  4.3909e-02,  6.3902e-02,
            1.0694e-01,  7.9648e-02,  1.6675e-01,  4.4695e-01,  5.5348e-01,
            7.0285e-01,  6.7925e-01,  3.6746e-01,  1.1493e-01,  5.5887e-02,
            1.2021e-01,  1.6608e-01,  1.7128e-01,  1.8966e-01,  1.5404e-01,
            1.2283e-01,  3.0840e-01,  3.9285e-01,  4.1712e-01,  4.1420e-01,
            4.2568e-01,  3.8673e-01],
          [ 3.0016e-01,  2.7820e-01,  2.8026e-01,  2.4189e-01,  1.6163e-01,
            1.4154e-01,  1.2036e-01,  9.7864e-02,  8.7467e-02,  8.1444e-02,
            1.4802e-01,  1.5744e-01,  1.0853e-01,  1.0228e-01,  2.0436e-01,
            3.9689e-01,  4.1458e-01,  5.1293e-01,  3.1692e-01,  1.3431e-01,
            1.4841e-01,  2.0346e-01,  1.9063e-01,  1.6900e-01,  1.4973e-01,
            2.2008e-01,  3.7183e-01,  4.1303e-01,  4.1171e-01,  3.8494e-01,
            3.7606e-01,  3.6519e-01],
          [ 2.7614e-01,  2.6696e-01,  2.6757e-01,  2.4153e-01,  1.9140e-01,
            1.6344e-01,  1.8008e-01,  1.7028e-01,  1.1704e-01,  1.1995e-01,
            1.3248e-01,  1.5243e-01,  9.3462e-02,  8.3554e-02,  1.2115e-01,
            1.4758e-01,  2.1091e-01,  4.5504e-01,  4.3275e-01,  1.5477e-01,
            1.6131e-01,  1.9275e-01,  1.7976e-01,  1.4721e-01,  1.8760e-01,
            3.3702e-01,  3.9198e-01,  4.0102e-01,  3.9236e-01,  3.6129e-01,
            3.6061e-01,  3.4961e-01],
          [ 2.8838e-01,  2.6324e-01,  2.4159e-01,  2.3615e-01,  2.2531e-01,
            1.7284e-01,  1.7978e-01,  1.9057e-01,  1.6616e-01,  1.0069e-01,
            9.8292e-02,  1.2111e-01,  1.0799e-01,  9.0882e-02,  1.1809e-01,
            1.1552e-01,  1.2007e-01,  2.6325e-01,  2.5191e-01,  1.3668e-01,
            1.3859e-01,  1.8149e-01,  1.7388e-01,  1.7954e-01,  2.5778e-01,
            3.6198e-01,  3.7479e-01,  3.7479e-01,  3.8414e-01,  3.7725e-01,
            3.6271e-01,  3.6884e-01],
          [ 2.4032e-01,  2.2924e-01,  2.2804e-01,  2.0911e-01,  2.1965e-01,
            1.8952e-01,  1.8371e-01,  2.0223e-01,  1.9063e-01,  1.8377e-01,
            1.3486e-01,  1.4463e-01,  1.1329e-01,  9.0164e-02,  9.6022e-02,
            9.1143e-02,  9.8238e-02,  1.6426e-01,  1.4960e-01,  1.3868e-01,
            1.4606e-01,  1.7500e-01,  1.7464e-01,  1.8316e-01,  2.7542e-01,
            3.2928e-01,  3.5706e-01,  3.4824e-01,  3.6711e-01,  3.6366e-01,
            3.6875e-01,  3.6649e-01],
          [ 2.0798e-01,  2.0723e-01,  2.0158e-01,  2.2204e-01,  2.0203e-01,
            2.0778e-01,  2.1765e-01,  2.2250e-01,  2.2113e-01,  2.0838e-01,
            1.8197e-01,  1.5866e-01,  1.4343e-01,  1.1357e-01,  1.0281e-01,
            1.0873e-01,  1.2638e-01,  1.2665e-01,  1.4278e-01,  1.5659e-01,
            1.6182e-01,  1.6398e-01,  1.3987e-01,  2.1678e-01,  3.0287e-01,
            3.3748e-01,  3.6699e-01,  3.4780e-01,  3.4742e-01,  3.5437e-01,
            3.5327e-01,  3.4835e-01],
          [ 2.1174e-01,  2.3400e-01,  2.0641e-01,  2.0311e-01,  2.2031e-01,
            1.7695e-01,  2.2439e-01,  2.0972e-01,  2.3444e-01,  2.1250e-01,
            2.0353e-01,  2.1105e-01,  1.8852e-01,  2.0280e-01,  1.7142e-01,
            1.5937e-01,  1.4526e-01,  1.6308e-01,  1.9469e-01,  2.0030e-01,
            2.1109e-01,  2.0381e-01,  2.6226e-01,  3.0392e-01,  3.2673e-01,
            3.5851e-01,  3.5665e-01,  3.4037e-01,  3.1623e-01,  3.2656e-01,
            3.5531e-01,  3.6235e-01],
          [ 2.2771e-01,  2.3754e-01,  2.2319e-01,  2.0031e-01,  1.7281e-01,
            1.6189e-01,  2.0776e-01,  2.1412e-01,  2.0921e-01,  2.0118e-01,
            1.8869e-01,  2.0981e-01,  2.2442e-01,  2.4197e-01,  2.5194e-01,
            2.1848e-01,  1.9947e-01,  2.1495e-01,  2.0911e-01,  2.0906e-01,
            2.3549e-01,  2.7961e-01,  4.1150e-01,  3.9365e-01,  3.4467e-01,
            3.5627e-01,  3.3871e-01,  3.2275e-01,  3.2411e-01,  3.0763e-01,
            3.0892e-01,  3.2631e-01],
          [ 2.3596e-01,  2.2601e-01,  2.3325e-01,  2.0871e-01,  1.9952e-01,
            1.7879e-01,  2.1577e-01,  2.2837e-01,  2.1554e-01,  2.0038e-01,
            2.0379e-01,  1.9302e-01,  2.1414e-01,  2.2689e-01,  2.4047e-01,
            2.3762e-01,  2.3961e-01,  2.4558e-01,  2.5498e-01,  2.3546e-01,
            2.4648e-01,  3.1268e-01,  3.7475e-01,  3.7541e-01,  3.7942e-01,
            3.3634e-01,  3.4004e-01,  3.4330e-01,  3.3115e-01,  3.4191e-01,
            3.1254e-01,  2.8240e-01],
          [ 2.1803e-01,  2.1467e-01,  2.2549e-01,  2.2499e-01,  2.3400e-01,
            2.0089e-01,  2.1347e-01,  2.1244e-01,  1.9442e-01,  1.9759e-01,
            2.1374e-01,  2.0323e-01,  1.9318e-01,  2.1246e-01,  2.3572e-01,
            2.7346e-01,  2.9302e-01,  3.0134e-01,  2.7340e-01,  2.6444e-01,
            2.8321e-01,  3.5031e-01,  4.1500e-01,  4.0395e-01,  3.6730e-01,
            3.3800e-01,  3.4586e-01,  3.4747e-01,  3.2865e-01,  3.0268e-01,
            3.0265e-01,  2.8250e-01],
          [ 2.2393e-01,  2.3731e-01,  2.2396e-01,  2.2102e-01,  2.2948e-01,
            2.4633e-01,  2.5915e-01,  2.3321e-01,  2.1719e-01,  2.2225e-01,
            2.1796e-01,  2.2357e-01,  2.0545e-01,  2.1326e-01,  2.3934e-01,
            2.7884e-01,  3.1580e-01,  3.2733e-01,  3.2644e-01,  3.3403e-01,
            3.5311e-01,  4.0888e-01,  4.1097e-01,  4.1007e-01,  3.5931e-01,
            3.4125e-01,  3.2538e-01,  3.0268e-01,  3.0373e-01,  2.9397e-01,
            2.8623e-01,  2.7893e-01]],

         [[ 2.9642e-01,  2.8396e-01,  2.6461e-01,  3.0326e-01,  3.4050e-01,
            3.4765e-01,  3.5242e-01,  3.4260e-01,  3.2870e-01,  2.7770e-01,
            2.2202e-01,  3.0100e-01,  3.3562e-01,  3.1536e-01,  2.8303e-01,
            3.0769e-01,  3.0553e-01,  3.1796e-01,  2.8289e-01,  2.6395e-01,
            2.1381e-01,  2.0868e-01,  2.2539e-01,  2.1705e-01,  2.4855e-01,
            1.7218e-01,  1.4774e-01,  2.4654e-01,  2.3344e-01,  3.1058e-01,
            3.6434e-01,  3.8228e-01],
          [ 2.5700e-01,  2.7966e-01,  2.3649e-01,  2.4145e-01,  3.2346e-01,
            3.1478e-01,  3.1957e-01,  2.9698e-01,  3.1001e-01,  2.9504e-01,
            2.6342e-01,  2.9535e-01,  3.2880e-01,  2.6337e-01,  2.8788e-01,
            3.2948e-01,  3.0826e-01,  2.5357e-01,  2.3948e-01,  2.3324e-01,
            1.7498e-01,  1.6312e-01,  1.6493e-01,  1.5220e-01,  1.7663e-01,
            1.5077e-01,  1.5116e-01,  2.1054e-01,  2.1886e-01,  3.0736e-01,
            3.8717e-01,  4.0313e-01],
          [ 2.6586e-01,  2.9686e-01,  2.4322e-01,  2.6962e-01,  3.2416e-01,
            2.9235e-01,  2.7295e-01,  2.5796e-01,  2.3129e-01,  2.3503e-01,
            2.5764e-01,  2.4291e-01,  2.8432e-01,  2.5550e-01,  2.8310e-01,
            2.8631e-01,  2.7358e-01,  2.0562e-01,  1.8841e-01,  1.8924e-01,
            1.8247e-01,  1.5015e-01,  1.9041e-01,  1.7845e-01,  1.5020e-01,
            1.6480e-01,  1.5598e-01,  1.5530e-01,  2.0918e-01,  3.4305e-01,
            3.8387e-01,  3.8222e-01],
          [ 2.7986e-01,  2.8863e-01,  2.5193e-01,  2.7187e-01,  2.6883e-01,
            2.2019e-01,  2.2787e-01,  2.4492e-01,  2.0544e-01,  1.8422e-01,
            1.8284e-01,  1.9287e-01,  2.2635e-01,  2.4914e-01,  2.7257e-01,
            3.0106e-01,  2.8446e-01,  2.8937e-01,  3.0087e-01,  2.6918e-01,
            2.8357e-01,  2.7682e-01,  2.7251e-01,  3.0804e-01,  3.1798e-01,
            3.2177e-01,  3.6904e-01,  3.4152e-01,  2.3943e-01,  3.9126e-01,
            4.0195e-01,  3.9285e-01],
          [ 3.4405e-01,  3.1210e-01,  3.0374e-01,  2.3939e-01,  2.2590e-01,
            2.6336e-01,  3.0489e-01,  3.0023e-01,  3.0754e-01,  2.8191e-01,
            2.6416e-01,  2.3054e-01,  2.2135e-01,  2.8205e-01,  2.9737e-01,
            2.9651e-01,  3.0446e-01,  2.8965e-01,  2.9836e-01,  2.9974e-01,
            3.4251e-01,  4.1662e-01,  3.7602e-01,  3.8957e-01,  4.2672e-01,
            4.7088e-01,  4.7209e-01,  3.6116e-01,  2.1787e-01,  3.8438e-01,
            4.0295e-01,  3.9394e-01],
          [ 4.3608e-01,  3.8450e-01,  3.5959e-01,  2.6837e-01,  2.9970e-01,
            3.4452e-01,  3.3453e-01,  3.2702e-01,  3.4851e-01,  3.0314e-01,
            2.8012e-01,  2.4267e-01,  2.2182e-01,  2.4433e-01,  2.7585e-01,
            2.7541e-01,  2.8565e-01,  2.7840e-01,  2.8505e-01,  3.0563e-01,
            3.0678e-01,  4.2519e-01,  4.2844e-01,  4.1898e-01,  3.6401e-01,
            3.0921e-01,  2.6959e-01,  2.7629e-01,  2.4104e-01,  3.1484e-01,
            3.8220e-01,  3.8348e-01],
          [ 4.4367e-01,  3.9006e-01,  3.4247e-01,  3.0137e-01,  3.0198e-01,
            3.0784e-01,  3.1745e-01,  3.0827e-01,  3.1909e-01,  2.9521e-01,
            2.8144e-01,  2.8392e-01,  2.3934e-01,  2.0977e-01,  2.3393e-01,
            2.3018e-01,  2.2151e-01,  2.2790e-01,  2.5703e-01,  3.0718e-01,
            2.8286e-01,  2.9630e-01,  3.6103e-01,  3.9274e-01,  3.2842e-01,
            2.8952e-01,  2.4939e-01,  2.5026e-01,  2.7529e-01,  2.8371e-01,
            3.6374e-01,  3.6612e-01],
          [ 3.0058e-01,  2.8038e-01,  2.7588e-01,  2.8601e-01,  2.9696e-01,
            3.0802e-01,  2.7911e-01,  2.9041e-01,  3.1010e-01,  3.0555e-01,
            2.9549e-01,  2.7893e-01,  2.7328e-01,  2.0075e-01,  2.2068e-01,
            1.8423e-01,  1.5858e-01,  1.4723e-01,  1.5554e-01,  1.8910e-01,
            2.2974e-01,  2.0970e-01,  2.0924e-01,  2.4310e-01,  2.8220e-01,
            2.5052e-01,  2.3914e-01,  2.6978e-01,  2.5508e-01,  2.6215e-01,
            3.5965e-01,  3.5353e-01],
          [ 2.4502e-01,  2.4321e-01,  1.8557e-01,  2.1974e-01,  2.7518e-01,
            3.0938e-01,  2.8621e-01,  2.6719e-01,  2.7075e-01,  2.6294e-01,
            2.5320e-01,  2.8907e-01,  2.4505e-01,  2.2344e-01,  2.0916e-01,
            1.8261e-01,  1.7593e-01,  1.6073e-01,  1.5837e-01,  1.9145e-01,
            2.2257e-01,  2.0799e-01,  1.7203e-01,  1.4663e-01,  1.7991e-01,
            1.8651e-01,  2.0887e-01,  2.7955e-01,  2.4273e-01,  2.5715e-01,
            2.8123e-01,  2.7753e-01],
          [ 2.6441e-01,  2.4077e-01,  1.7147e-01,  1.9275e-01,  2.4024e-01,
            2.1572e-01,  2.0231e-01,  2.4321e-01,  2.2200e-01,  2.5308e-01,
            2.3053e-01,  2.1055e-01,  2.4187e-01,  1.7492e-01,  1.8486e-01,
            1.4746e-01,  1.6611e-01,  1.6000e-01,  1.5453e-01,  1.7216e-01,
            1.7920e-01,  1.7639e-01,  1.3102e-01,  1.5592e-01,  1.9181e-01,
            2.0569e-01,  2.4804e-01,  2.2663e-01,  2.2537e-01,  2.6854e-01,
            2.1011e-01,  1.7636e-01],
          [ 2.5972e-01,  1.9500e-01,  1.9461e-01,  2.2529e-01,  2.3032e-01,
            1.6351e-01,  1.5826e-01,  1.9335e-01,  2.1239e-01,  2.6054e-01,
            2.4381e-01,  2.0606e-01,  2.6400e-01,  1.8547e-01,  1.5571e-01,
            1.5542e-01,  1.3412e-01,  1.2757e-01,  1.3463e-01,  1.7458e-01,
            1.7034e-01,  1.8146e-01,  1.7754e-01,  1.9811e-01,  2.1445e-01,
            2.2699e-01,  2.4103e-01,  2.4236e-01,  2.6917e-01,  2.8354e-01,
            2.0664e-01,  1.3648e-01],
          [ 2.0114e-01,  2.2310e-01,  2.5904e-01,  2.6178e-01,  2.3885e-01,
            2.2661e-01,  2.6254e-01,  2.5633e-01,  3.0656e-01,  2.4736e-01,
            2.1630e-01,  2.2186e-01,  3.3227e-01,  3.1263e-01,  2.1430e-01,
            1.5661e-01,  1.1305e-01,  1.1930e-01,  1.4157e-01,  2.0222e-01,
            2.1669e-01,  2.5438e-01,  2.8548e-01,  2.8729e-01,  2.8495e-01,
            2.9013e-01,  2.7852e-01,  3.0288e-01,  3.0496e-01,  3.0019e-01,
            2.7684e-01,  1.8032e-01],
          [ 2.7245e-01,  2.8686e-01,  2.8267e-01,  2.5662e-01,  2.6806e-01,
            3.2417e-01,  4.0636e-01,  3.5831e-01,  3.9880e-01,  2.2129e-01,
            1.0970e-01,  2.0944e-01,  3.1525e-01,  2.7537e-01,  2.3515e-01,
            8.1287e-02,  6.5034e-02,  1.4865e-01,  9.0621e-02,  1.0528e-01,
            1.2449e-01,  1.9761e-01,  2.5446e-01,  3.2514e-01,  3.6148e-01,
            3.8370e-01,  3.3997e-01,  3.1908e-01,  3.0565e-01,  2.8705e-01,
            2.4922e-01,  2.3847e-01],
          [ 3.2313e-01,  2.9675e-01,  2.8514e-01,  2.7722e-01,  2.9042e-01,
            3.6708e-01,  4.0589e-01,  3.9061e-01,  3.9047e-01,  2.0071e-01,
            1.2061e-01,  2.3205e-01,  2.3702e-01,  1.6051e-01,  1.5311e-01,
            1.4727e-01,  1.9015e-01,  1.9137e-01,  1.2113e-01,  8.2244e-02,
            8.3528e-02,  1.4270e-01,  1.7033e-01,  2.0865e-01,  3.3149e-01,
            3.9834e-01,  3.6468e-01,  3.2746e-01,  3.0092e-01,  2.4475e-01,
            2.5452e-01,  2.6671e-01],
          [ 2.7268e-01,  2.6902e-01,  2.8942e-01,  2.9494e-01,  3.1618e-01,
            3.9146e-01,  4.6608e-01,  4.2002e-01,  3.7880e-01,  2.1378e-01,
            2.7768e-01,  3.3204e-01,  2.3475e-01,  1.9076e-01,  2.3234e-01,
            2.3962e-01,  2.0529e-01,  1.9601e-01,  1.4036e-01,  9.9385e-02,
            1.0044e-01,  1.1371e-01,  8.5535e-02,  1.4157e-01,  2.3286e-01,
            2.6749e-01,  3.5817e-01,  4.1112e-01,  3.3874e-01,  3.2315e-01,
            3.1514e-01,  3.1421e-01],
          [ 2.3327e-01,  2.6148e-01,  2.7620e-01,  2.8856e-01,  3.1503e-01,
            4.1930e-01,  5.5092e-01,  5.6086e-01,  4.5087e-01,  3.6756e-01,
            4.0155e-01,  3.7492e-01,  2.4667e-01,  2.2579e-01,  2.2616e-01,
            2.1740e-01,  1.9394e-01,  1.6536e-01,  1.0984e-01,  1.0880e-01,
            1.0596e-01,  8.8146e-02,  8.9639e-02,  1.1350e-01,  1.3147e-01,
            1.9346e-01,  3.6868e-01,  4.3868e-01,  4.0995e-01,  3.8425e-01,
            3.0903e-01,  2.9944e-01],
          [ 2.4051e-01,  2.4026e-01,  2.5510e-01,  2.8778e-01,  2.9276e-01,
            4.1446e-01,  5.5806e-01,  5.5806e-01,  4.7351e-01,  4.4904e-01,
            4.5628e-01,  4.4457e-01,  3.5559e-01,  2.8860e-01,  2.8425e-01,
            2.7405e-01,  2.5863e-01,  1.9972e-01,  1.0549e-01,  7.9494e-02,
            1.3855e-01,  1.3622e-01,  1.3693e-01,  1.0996e-01,  6.7249e-02,
            1.9193e-01,  4.0447e-01,  3.9708e-01,  4.3115e-01,  4.1747e-01,
            3.3692e-01,  1.7388e-01],
          [ 2.8747e-01,  2.8775e-01,  2.8654e-01,  2.7074e-01,  2.6617e-01,
            2.7896e-01,  3.8075e-01,  4.4602e-01,  4.8595e-01,  4.9052e-01,
            4.7891e-01,  4.8504e-01,  4.8978e-01,  4.4872e-01,  4.4312e-01,
            4.3439e-01,  4.1239e-01,  2.8409e-01,  1.3831e-01,  1.1914e-01,
            1.6288e-01,  1.6264e-01,  1.6559e-01,  1.1342e-01,  5.9443e-02,
            2.0957e-01,  4.0500e-01,  3.9705e-01,  4.2653e-01,  4.6625e-01,
            3.8329e-01,  2.0237e-01],
          [ 2.8182e-01,  2.6911e-01,  2.2593e-01,  1.8908e-01,  1.8111e-01,
            1.9491e-01,  2.1522e-01,  2.7825e-01,  4.4780e-01,  5.8562e-01,
            5.6238e-01,  5.1788e-01,  5.1638e-01,  4.8725e-01,  4.8349e-01,
            4.7605e-01,  4.5848e-01,  3.6200e-01,  1.8829e-01,  1.7235e-01,
            1.7112e-01,  2.1540e-01,  2.4760e-01,  1.8468e-01,  1.0035e-01,
            1.9651e-01,  3.6852e-01,  3.5945e-01,  4.2382e-01,  4.5481e-01,
            4.0434e-01,  3.7211e-01],
          [ 2.9902e-01,  2.6855e-01,  1.8027e-01,  1.4471e-01,  1.5109e-01,
            2.1962e-01,  1.5553e-01,  1.2920e-01,  2.3192e-01,  3.9309e-01,
            6.1554e-01,  6.6153e-01,  6.5058e-01,  5.4376e-01,  5.6511e-01,
            6.2800e-01,  5.2052e-01,  3.0009e-01,  1.7483e-01,  1.4199e-01,
            1.6354e-01,  2.2466e-01,  2.5290e-01,  2.1429e-01,  1.4660e-01,
            1.9475e-01,  3.3352e-01,  3.3346e-01,  3.7394e-01,  4.0937e-01,
            3.9595e-01,  3.4728e-01],
          [ 2.8790e-01,  3.1936e-01,  2.2530e-01,  1.1932e-01,  1.8715e-01,
            1.8342e-01,  1.5428e-01,  1.4077e-01,  1.3206e-01,  1.8564e-01,
            3.2916e-01,  4.8085e-01,  5.8476e-01,  6.4679e-01,  7.4810e-01,
            8.5671e-01,  7.9136e-01,  2.7001e-01,  1.0588e-01,  1.1806e-01,
            1.3321e-01,  1.6667e-01,  2.0105e-01,  2.0950e-01,  2.1994e-01,
            1.4304e-01,  2.9316e-01,  3.2847e-01,  3.5350e-01,  3.6051e-01,
            3.7941e-01,  3.3204e-01],
          [ 2.9216e-01,  2.8274e-01,  2.9592e-01,  1.9866e-01,  1.0532e-01,
            6.7101e-02,  7.5911e-02,  6.2345e-02,  2.3798e-02,  2.8347e-02,
            7.5212e-02,  5.9381e-02,  1.2547e-01,  4.3397e-01,  5.6807e-01,
            7.0527e-01,  6.7552e-01,  3.7096e-01,  1.7244e-01,  1.0159e-01,
            1.3125e-01,  1.6509e-01,  1.8132e-01,  2.1380e-01,  1.7192e-01,
            1.2305e-01,  2.7793e-01,  3.4052e-01,  3.4754e-01,  3.3489e-01,
            3.4668e-01,  3.1506e-01],
          [ 2.8419e-01,  2.5607e-01,  2.7322e-01,  2.2822e-01,  1.4585e-01,
            1.2373e-01,  9.3753e-02,  7.4160e-02,  5.5064e-02,  5.0567e-02,
            1.0418e-01,  7.7391e-02,  4.0909e-02,  6.8564e-02,  1.7485e-01,
            3.5698e-01,  3.7953e-01,  5.1719e-01,  3.6705e-01,  1.6065e-01,
            1.6929e-01,  1.9950e-01,  2.2056e-01,  1.7227e-01,  1.3129e-01,
            1.9713e-01,  3.0895e-01,  3.4905e-01,  3.3152e-01,  3.0343e-01,
            3.0114e-01,  2.9293e-01],
          [ 2.5775e-01,  2.5706e-01,  2.6336e-01,  2.4541e-01,  1.8013e-01,
            1.2818e-01,  1.6309e-01,  1.4413e-01,  7.6996e-02,  5.6581e-02,
            7.1025e-02,  6.1746e-02,  3.4060e-02,  4.2811e-02,  6.2293e-02,
            7.8743e-02,  1.7322e-01,  4.5422e-01,  4.4327e-01,  1.7627e-01,
            1.6803e-01,  2.0222e-01,  1.8521e-01,  1.5696e-01,  1.6689e-01,
            2.9058e-01,  3.4546e-01,  3.3061e-01,  3.3650e-01,  2.9554e-01,
            2.8639e-01,  2.6849e-01],
          [ 2.6990e-01,  2.4439e-01,  2.4435e-01,  2.3627e-01,  2.1881e-01,
            1.5295e-01,  1.5541e-01,  1.4994e-01,  1.4258e-01,  5.7803e-02,
            5.2749e-02,  5.8682e-02,  4.3370e-02,  5.1371e-02,  7.4495e-02,
            7.9636e-02,  8.9521e-02,  2.5379e-01,  2.5911e-01,  1.3722e-01,
            1.5548e-01,  1.7930e-01,  1.8857e-01,  1.6097e-01,  2.1545e-01,
            2.9799e-01,  3.0314e-01,  3.1504e-01,  3.3493e-01,  3.0657e-01,
            2.7879e-01,  2.6458e-01],
          [ 2.3921e-01,  2.2515e-01,  2.2256e-01,  2.2156e-01,  2.0991e-01,
            1.6543e-01,  1.6072e-01,  1.6834e-01,  1.6907e-01,  1.4869e-01,
            1.0517e-01,  1.1074e-01,  7.8706e-02,  5.9651e-02,  6.1100e-02,
            6.1070e-02,  8.2309e-02,  1.4741e-01,  1.3665e-01,  1.6261e-01,
            1.5336e-01,  1.6483e-01,  1.6894e-01,  1.6079e-01,  2.4842e-01,
            2.7152e-01,  2.8791e-01,  2.7117e-01,  3.0496e-01,  2.7632e-01,
            2.7925e-01,  2.6707e-01],
          [ 2.0541e-01,  2.0447e-01,  1.9714e-01,  2.2365e-01,  2.0517e-01,
            1.6699e-01,  2.0342e-01,  2.0163e-01,  1.9516e-01,  1.7681e-01,
            1.5349e-01,  1.3870e-01,  1.1920e-01,  8.3144e-02,  6.6152e-02,
            7.5772e-02,  8.5373e-02,  1.1531e-01,  1.5020e-01,  1.7263e-01,
            1.8281e-01,  1.6903e-01,  1.3351e-01,  1.9548e-01,  2.8020e-01,
            2.8418e-01,  3.0024e-01,  2.6569e-01,  2.8484e-01,  2.7695e-01,
            2.5587e-01,  2.5549e-01],
          [ 2.1281e-01,  2.3055e-01,  2.0275e-01,  2.2066e-01,  2.2148e-01,
            1.5739e-01,  1.9544e-01,  1.8410e-01,  1.9032e-01,  1.8170e-01,
            1.7412e-01,  1.8516e-01,  1.6419e-01,  1.6195e-01,  1.5331e-01,
            1.4754e-01,  1.2407e-01,  1.4656e-01,  1.7747e-01,  2.0249e-01,
            2.1474e-01,  1.9941e-01,  2.3288e-01,  2.6132e-01,  2.9135e-01,
            3.0572e-01,  2.9980e-01,  2.8974e-01,  2.6521e-01,  2.4832e-01,
            2.6493e-01,  2.6829e-01],
          [ 2.3288e-01,  2.4370e-01,  2.3053e-01,  2.0775e-01,  1.9615e-01,
            1.5509e-01,  1.7097e-01,  1.9725e-01,  1.7509e-01,  1.7019e-01,
            1.6962e-01,  1.7504e-01,  1.9201e-01,  1.9349e-01,  2.0466e-01,
            1.9237e-01,  1.8662e-01,  1.8123e-01,  1.9553e-01,  2.1095e-01,
            2.1494e-01,  2.5022e-01,  3.4862e-01,  3.3999e-01,  3.1520e-01,
            3.2001e-01,  2.9361e-01,  2.7714e-01,  2.5602e-01,  2.4145e-01,
            2.3826e-01,  2.4708e-01],
          [ 2.5063e-01,  2.3401e-01,  2.2231e-01,  2.1545e-01,  2.0659e-01,
            1.6705e-01,  1.8465e-01,  1.8779e-01,  1.8861e-01,  1.7899e-01,
            1.7364e-01,  1.6765e-01,  1.8115e-01,  1.9201e-01,  2.1418e-01,
            2.1172e-01,  2.1880e-01,  2.3251e-01,  2.3868e-01,  2.0315e-01,
            1.9592e-01,  2.5208e-01,  3.2632e-01,  3.3308e-01,  3.2410e-01,
            3.2542e-01,  2.9920e-01,  2.8458e-01,  2.7768e-01,  2.5481e-01,
            2.4043e-01,  2.1100e-01],
          [ 2.3232e-01,  2.1989e-01,  2.2415e-01,  2.3171e-01,  2.2625e-01,
            1.9047e-01,  1.8714e-01,  1.9838e-01,  1.7870e-01,  1.7483e-01,
            1.6193e-01,  1.7103e-01,  1.7085e-01,  1.8047e-01,  2.0879e-01,
            2.4819e-01,  2.4897e-01,  2.7009e-01,  2.3138e-01,  2.1348e-01,
            2.2346e-01,  3.0083e-01,  3.3403e-01,  3.2395e-01,  3.2087e-01,
            3.0233e-01,  2.8828e-01,  2.8391e-01,  2.6364e-01,  2.4197e-01,
            2.4423e-01,  2.2257e-01],
          [ 2.2733e-01,  2.2431e-01,  1.9580e-01,  1.9902e-01,  2.1943e-01,
            2.2636e-01,  2.1969e-01,  2.0320e-01,  1.9290e-01,  1.7984e-01,
            1.7602e-01,  1.7696e-01,  1.6810e-01,  1.6904e-01,  1.9927e-01,
            2.3618e-01,  2.6177e-01,  2.7567e-01,  2.5868e-01,  2.5810e-01,
            2.9168e-01,  3.2884e-01,  3.5460e-01,  3.3628e-01,  3.0687e-01,
            3.0426e-01,  2.6547e-01,  2.6046e-01,  2.4723e-01,  2.3196e-01,
            2.2155e-01,  2.2948e-01]],

         [[ 1.7414e-01,  1.4029e-01,  1.4765e-01,  1.9130e-01,  2.1095e-01,
            1.9897e-01,  1.8480e-01,  1.9911e-01,  1.9441e-01,  1.7267e-01,
            1.1150e-01,  1.5205e-01,  1.6813e-01,  1.1865e-01,  1.3804e-01,
            1.4837e-01,  1.4472e-01,  1.6751e-01,  1.2504e-01,  1.2274e-01,
            1.1012e-01,  1.2118e-01,  1.4603e-01,  1.1958e-01,  1.6399e-01,
            1.0750e-01,  1.1505e-01,  1.5379e-01,  1.1463e-01,  1.6002e-01,
            2.3820e-01,  2.7037e-01],
          [ 1.6149e-01,  1.4872e-01,  1.4383e-01,  1.3794e-01,  1.9620e-01,
            1.6485e-01,  1.7870e-01,  1.7068e-01,  1.9891e-01,  1.7357e-01,
            1.5540e-01,  1.5528e-01,  1.6245e-01,  9.3387e-02,  1.1741e-01,
            1.7043e-01,  1.3702e-01,  1.1397e-01,  1.1067e-01,  1.0493e-01,
            1.0730e-01,  1.0146e-01,  1.1250e-01,  9.9826e-02,  8.7912e-02,
            1.0674e-01,  1.1314e-01,  1.4114e-01,  1.1831e-01,  1.4749e-01,
            2.1065e-01,  2.2452e-01],
          [ 1.6958e-01,  1.8208e-01,  1.6389e-01,  1.7795e-01,  1.8807e-01,
            1.4939e-01,  1.3454e-01,  1.2020e-01,  1.1690e-01,  1.2613e-01,
            1.6161e-01,  1.3859e-01,  1.5448e-01,  1.1975e-01,  1.2931e-01,
            1.3586e-01,  1.1205e-01,  8.1068e-02,  8.7218e-02,  9.2169e-02,
            1.0953e-01,  1.1141e-01,  1.2907e-01,  9.2863e-02,  6.0276e-02,
            1.1198e-01,  8.5904e-02,  7.3053e-02,  1.1226e-01,  1.7506e-01,
            1.4290e-01,  1.7429e-01],
          [ 1.9145e-01,  1.9609e-01,  1.3947e-01,  1.6168e-01,  1.3446e-01,
            8.7034e-02,  8.7352e-02,  9.6812e-02,  9.7026e-02,  8.1163e-02,
            8.1665e-02,  9.2483e-02,  1.0354e-01,  7.4455e-02,  9.7495e-02,
            9.7915e-02,  7.0630e-02,  8.0079e-02,  1.3845e-01,  1.4766e-01,
            1.3912e-01,  1.3812e-01,  1.4378e-01,  1.3216e-01,  1.4149e-01,
            1.5487e-01,  2.0975e-01,  1.6667e-01,  7.7288e-02,  1.9596e-01,
            1.7369e-01,  1.6909e-01],
          [ 2.2059e-01,  1.8387e-01,  1.4284e-01,  1.0924e-01,  9.4770e-02,
            1.1097e-01,  1.4386e-01,  1.2960e-01,  1.3895e-01,  1.5336e-01,
            1.2293e-01,  9.8536e-02,  6.6452e-02,  8.8509e-02,  1.0865e-01,
            9.3652e-02,  8.1665e-02,  6.8152e-02,  9.3608e-02,  1.1127e-01,
            1.5005e-01,  1.7336e-01,  9.7431e-02,  1.0043e-01,  1.2728e-01,
            2.0142e-01,  2.3775e-01,  1.1658e-01,  4.1176e-02,  1.8435e-01,
            1.7604e-01,  1.7350e-01],
          [ 2.7877e-01,  2.1939e-01,  1.9986e-01,  1.4074e-01,  1.6923e-01,
            2.0779e-01,  1.7969e-01,  1.6142e-01,  1.8256e-01,  1.6910e-01,
            1.4789e-01,  1.1516e-01,  8.4767e-02,  1.1293e-01,  1.2128e-01,
            1.0270e-01,  8.1822e-02,  8.0767e-02,  9.7927e-02,  1.1717e-01,
            1.0152e-01,  1.8143e-01,  1.4139e-01,  1.5495e-01,  1.1503e-01,
            6.6598e-02,  6.8967e-02,  9.0268e-02,  1.1864e-01,  1.5594e-01,
            1.5035e-01,  1.7969e-01],
          [ 2.9262e-01,  2.2586e-01,  2.0420e-01,  1.7160e-01,  1.6052e-01,
            1.4875e-01,  1.3486e-01,  1.3908e-01,  1.5181e-01,  1.3853e-01,
            1.5809e-01,  1.5798e-01,  1.2250e-01,  9.7810e-02,  1.0140e-01,
            9.6969e-02,  7.3854e-02,  7.6226e-02,  9.1973e-02,  1.4453e-01,
            1.1639e-01,  1.0709e-01,  1.4340e-01,  1.6560e-01,  1.4344e-01,
            9.7850e-02,  1.0424e-01,  1.3779e-01,  1.4362e-01,  1.4628e-01,
            1.8071e-01,  1.4443e-01],
          [ 1.7003e-01,  1.4155e-01,  1.5368e-01,  1.5230e-01,  1.3448e-01,
            1.1481e-01,  1.1612e-01,  1.2846e-01,  1.4410e-01,  1.6642e-01,
            1.5346e-01,  1.6054e-01,  1.2843e-01,  1.0218e-01,  9.7526e-02,
            9.4653e-02,  6.4060e-02,  5.1630e-02,  7.3390e-02,  1.0312e-01,
            7.8412e-02,  7.7114e-02,  5.8585e-02,  8.5481e-02,  1.4463e-01,
            1.3122e-01,  1.2528e-01,  1.3648e-01,  1.2621e-01,  1.1753e-01,
            1.9423e-01,  1.5491e-01],
          [ 1.5503e-01,  1.6614e-01,  1.1503e-01,  1.0411e-01,  1.0345e-01,
            1.0065e-01,  8.7463e-02,  9.2120e-02,  1.0034e-01,  1.1895e-01,
            1.3696e-01,  1.4018e-01,  1.1287e-01,  1.1157e-01,  1.2292e-01,
            1.0345e-01,  8.9676e-02,  7.5279e-02,  7.0757e-02,  9.2964e-02,
            7.7436e-02,  4.7400e-02,  3.5952e-02,  3.1816e-02,  5.1445e-02,
            7.9578e-02,  9.3176e-02,  1.3427e-01,  1.0871e-01,  1.0707e-01,
            1.6133e-01,  1.1784e-01],
          [ 1.7231e-01,  1.7608e-01,  1.1724e-01,  1.2447e-01,  1.4828e-01,
            1.2476e-01,  9.5334e-02,  7.6370e-02,  7.9658e-02,  1.1582e-01,
            1.4767e-01,  1.1164e-01,  1.3285e-01,  9.0927e-02,  9.5462e-02,
            9.7170e-02,  9.9585e-02,  9.6372e-02,  6.1558e-02,  5.2610e-02,
            5.2792e-02,  4.1086e-02,  4.9739e-02,  6.7375e-02,  7.3211e-02,
            1.0236e-01,  1.2426e-01,  1.2355e-01,  1.3486e-01,  1.9251e-01,
            1.3162e-01,  8.4537e-02],
          [ 1.9622e-01,  1.5027e-01,  1.1921e-01,  1.4582e-01,  1.7840e-01,
            1.1857e-01,  8.8420e-02,  9.3489e-02,  1.1607e-01,  1.5072e-01,
            1.4111e-01,  1.1604e-01,  1.8743e-01,  9.9799e-02,  1.0836e-01,
            8.6154e-02,  7.3982e-02,  8.8890e-02,  5.0910e-02,  6.8829e-02,
            9.1644e-02,  1.0919e-01,  7.4535e-02,  1.0151e-01,  8.7191e-02,
            1.0416e-01,  1.2587e-01,  1.2797e-01,  1.8974e-01,  2.1430e-01,
            1.5175e-01,  8.3351e-02],
          [ 1.8072e-01,  1.8872e-01,  2.0005e-01,  1.9954e-01,  1.9649e-01,
            2.0238e-01,  1.9915e-01,  1.8472e-01,  2.1933e-01,  1.8363e-01,
            1.3631e-01,  1.7709e-01,  3.1054e-01,  2.9456e-01,  1.8436e-01,
            1.0943e-01,  8.7320e-02,  8.5012e-02,  8.8494e-02,  1.6204e-01,
            1.5244e-01,  1.8938e-01,  1.5350e-01,  1.5018e-01,  1.3721e-01,
            1.2430e-01,  1.1708e-01,  1.8405e-01,  1.9915e-01,  1.8490e-01,
            1.7843e-01,  1.0174e-01],
          [ 2.0675e-01,  2.4234e-01,  2.4490e-01,  2.2558e-01,  2.1596e-01,
            2.6049e-01,  3.4405e-01,  2.9843e-01,  3.1998e-01,  1.6542e-01,
            6.8733e-02,  2.2172e-01,  3.7102e-01,  3.1645e-01,  2.5070e-01,
            1.2200e-01,  9.7550e-02,  1.3034e-01,  6.9837e-02,  8.4352e-02,
            9.7254e-02,  1.4514e-01,  1.7027e-01,  1.9150e-01,  1.5691e-01,
            1.7994e-01,  1.6117e-01,  1.8395e-01,  1.7790e-01,  1.3882e-01,
            1.1208e-01,  1.2605e-01],
          [ 2.4229e-01,  2.1772e-01,  2.0995e-01,  2.2766e-01,  2.4716e-01,
            3.0603e-01,  3.8489e-01,  3.5323e-01,  3.2917e-01,  1.7420e-01,
            1.3124e-01,  2.8547e-01,  3.0738e-01,  2.2411e-01,  1.8800e-01,
            1.7328e-01,  1.9294e-01,  2.0193e-01,  1.1288e-01,  6.3575e-02,
            6.2513e-02,  1.3274e-01,  1.0109e-01,  9.5006e-02,  1.6383e-01,
            2.1903e-01,  1.9781e-01,  2.0252e-01,  1.5544e-01,  1.1468e-01,
            1.0730e-01,  1.3381e-01],
          [ 1.7484e-01,  1.9731e-01,  1.8639e-01,  2.1171e-01,  2.5419e-01,
            3.7956e-01,  4.3841e-01,  3.9366e-01,  3.5158e-01,  2.5802e-01,
            3.9048e-01,  4.3825e-01,  3.0785e-01,  2.2757e-01,  2.4617e-01,
            2.3289e-01,  2.0482e-01,  2.0336e-01,  1.5092e-01,  9.1858e-02,
            8.8737e-02,  1.1693e-01,  7.4639e-02,  8.4857e-02,  1.2526e-01,
            1.6448e-01,  2.4629e-01,  2.8499e-01,  1.6782e-01,  1.5695e-01,
            1.6309e-01,  1.6561e-01],
          [ 1.5828e-01,  1.7394e-01,  1.7045e-01,  2.2319e-01,  2.9527e-01,
            4.0548e-01,  5.3678e-01,  5.4208e-01,  4.8374e-01,  4.6041e-01,
            5.0269e-01,  4.8741e-01,  3.1712e-01,  2.7965e-01,  2.5563e-01,
            2.4412e-01,  2.1859e-01,  1.9361e-01,  1.3069e-01,  1.0407e-01,
            9.5823e-02,  9.7879e-02,  1.0338e-01,  7.7073e-02,  6.2806e-02,
            1.4840e-01,  2.9712e-01,  3.1020e-01,  2.3899e-01,  2.3014e-01,
            1.8044e-01,  1.4319e-01],
          [ 1.5362e-01,  1.7326e-01,  2.1109e-01,  2.4907e-01,  2.7715e-01,
            4.2605e-01,  5.3746e-01,  5.4015e-01,  5.1046e-01,  5.3138e-01,
            5.5708e-01,  5.4072e-01,  4.5054e-01,  3.9210e-01,  3.5775e-01,
            3.1707e-01,  3.1921e-01,  2.5596e-01,  1.5126e-01,  1.1946e-01,
            1.4095e-01,  1.0773e-01,  1.2670e-01,  7.7651e-02,  3.9036e-02,
            1.6909e-01,  3.5475e-01,  2.8138e-01,  3.1195e-01,  2.9818e-01,
            2.5469e-01,  1.2474e-01],
          [ 2.3127e-01,  2.5684e-01,  2.9526e-01,  2.7340e-01,  2.5882e-01,
            2.7731e-01,  3.6518e-01,  4.6422e-01,  5.1788e-01,  5.3906e-01,
            5.3165e-01,  5.3708e-01,  5.7112e-01,  5.3122e-01,  5.1612e-01,
            5.0985e-01,  4.6502e-01,  3.6289e-01,  2.1083e-01,  1.4841e-01,
            1.6171e-01,  1.2657e-01,  1.3595e-01,  8.5758e-02,  3.4396e-02,
            1.9031e-01,  3.7535e-01,  3.2719e-01,  3.5397e-01,  3.6864e-01,
            3.2178e-01,  1.4959e-01],
          [ 2.4894e-01,  2.4222e-01,  2.1547e-01,  1.8778e-01,  1.6118e-01,
            1.6035e-01,  1.9519e-01,  2.9304e-01,  4.7519e-01,  6.0968e-01,
            6.1094e-01,  5.5514e-01,  5.5905e-01,  5.3839e-01,  5.3545e-01,
            5.3075e-01,  5.2810e-01,  4.1793e-01,  2.4195e-01,  1.8100e-01,
            1.8641e-01,  2.1726e-01,  2.2442e-01,  1.5467e-01,  7.2531e-02,
            1.6581e-01,  3.3944e-01,  3.0447e-01,  3.2526e-01,  3.6367e-01,
            3.4610e-01,  2.9323e-01],
          [ 2.5451e-01,  2.5001e-01,  1.5123e-01,  1.1153e-01,  1.4129e-01,
            2.4474e-01,  1.8201e-01,  1.4479e-01,  2.3412e-01,  4.3420e-01,
            6.3708e-01,  6.8916e-01,  6.9032e-01,  5.7375e-01,  5.9953e-01,
            6.6914e-01,  5.8284e-01,  3.3512e-01,  1.8035e-01,  1.4709e-01,
            1.9509e-01,  2.4842e-01,  2.4665e-01,  2.0785e-01,  1.2448e-01,
            1.6465e-01,  3.1762e-01,  2.8037e-01,  3.2363e-01,  3.0168e-01,
            2.9352e-01,  3.0106e-01],
          [ 2.5817e-01,  2.7748e-01,  1.9377e-01,  1.0354e-01,  1.9869e-01,
            1.9961e-01,  1.8465e-01,  1.2882e-01,  1.1451e-01,  1.6972e-01,
            3.1009e-01,  5.3192e-01,  6.6113e-01,  6.7850e-01,  7.9071e-01,
            9.3180e-01,  8.3982e-01,  3.0053e-01,  1.3839e-01,  1.4088e-01,
            1.5227e-01,  1.7794e-01,  1.9824e-01,  1.7953e-01,  1.9857e-01,
            1.3327e-01,  2.6672e-01,  2.9584e-01,  2.9381e-01,  2.7168e-01,
            2.8841e-01,  2.6373e-01],
          [ 2.6795e-01,  2.8844e-01,  2.8925e-01,  1.8906e-01,  9.6437e-02,
            6.8026e-02,  6.0868e-02,  4.5804e-02, -1.2287e-03,  1.3399e-02,
            1.4993e-02,  4.3812e-03,  9.9905e-02,  3.8107e-01,  5.6179e-01,
            7.5288e-01,  7.3417e-01,  4.1720e-01,  1.9640e-01,  1.3062e-01,
            1.3444e-01,  1.5114e-01,  1.7203e-01,  1.6932e-01,  1.4154e-01,
            9.8784e-02,  2.2904e-01,  2.9215e-01,  2.8399e-01,  2.5433e-01,
            2.6622e-01,  2.5718e-01],
          [ 2.6334e-01,  2.7205e-01,  2.5894e-01,  2.1762e-01,  1.3500e-01,
            9.3098e-02,  6.5208e-02,  3.4593e-02,  8.2720e-03,  7.6269e-03,
            1.6058e-02,  7.9967e-03, -4.4254e-02,  4.7641e-03,  1.4514e-01,
            3.3395e-01,  3.7604e-01,  5.4086e-01,  4.0046e-01,  1.6701e-01,
            1.7478e-01,  1.9070e-01,  1.8374e-01,  1.3230e-01,  1.1248e-01,
            1.3376e-01,  2.6708e-01,  2.8077e-01,  2.7319e-01,  2.4502e-01,
            2.3416e-01,  2.3668e-01],
          [ 2.6092e-01,  2.6589e-01,  2.6565e-01,  2.2741e-01,  1.6850e-01,
            1.2382e-01,  1.3632e-01,  9.6257e-02,  1.9680e-02,  2.7739e-02,
            1.7060e-02,  1.3690e-02, -2.4040e-03,  4.2009e-03,  3.9453e-02,
            5.5976e-02,  1.5973e-01,  4.8053e-01,  4.8297e-01,  1.8886e-01,
            1.6966e-01,  2.0987e-01,  1.7993e-01,  1.2744e-01,  1.4116e-01,
            2.4504e-01,  2.9152e-01,  2.7767e-01,  2.6318e-01,  2.2784e-01,
            2.2772e-01,  2.1989e-01],
          [ 2.7153e-01,  2.4942e-01,  2.4863e-01,  2.4389e-01,  2.1385e-01,
            1.2979e-01,  1.2406e-01,  1.4834e-01,  7.5175e-02,  3.3884e-02,
            3.9209e-02,  3.2629e-02,  1.7689e-02,  1.5328e-02,  3.9951e-02,
            6.0844e-02,  7.2173e-02,  2.3948e-01,  2.6440e-01,  1.6118e-01,
            1.7315e-01,  1.9188e-01,  1.8071e-01,  1.3804e-01,  1.8127e-01,
            2.4672e-01,  2.5653e-01,  2.4649e-01,  2.6416e-01,  2.4083e-01,
            2.4553e-01,  2.3933e-01],
          [ 2.3802e-01,  2.3336e-01,  2.2182e-01,  2.2708e-01,  2.1764e-01,
            1.4886e-01,  1.4146e-01,  1.1913e-01,  1.2422e-01,  1.0839e-01,
            7.9744e-02,  6.8598e-02,  4.9754e-02,  1.0071e-02,  6.9738e-03,
            3.3175e-02,  5.4183e-02,  1.2124e-01,  1.3547e-01,  1.5157e-01,
            1.3547e-01,  1.5856e-01,  1.6185e-01,  1.4519e-01,  2.0678e-01,
            2.1714e-01,  2.2401e-01,  2.2047e-01,  2.2948e-01,  2.2874e-01,
            2.3150e-01,  2.2253e-01],
          [ 2.2746e-01,  2.0173e-01,  2.0785e-01,  2.2881e-01,  2.1461e-01,
            1.4254e-01,  1.4848e-01,  1.4343e-01,  1.4119e-01,  1.2810e-01,
            9.1210e-02,  8.2196e-02,  7.1828e-02,  3.5650e-02,  1.5245e-02,
            3.0610e-02,  4.3932e-02,  7.0102e-02,  9.5950e-02,  1.3833e-01,
            1.4314e-01,  1.4048e-01,  1.2147e-01,  1.5968e-01,  2.2191e-01,
            2.3227e-01,  2.3749e-01,  2.2601e-01,  2.1819e-01,  2.2251e-01,
            2.2202e-01,  2.2143e-01],
          [ 2.2684e-01,  2.3726e-01,  1.9698e-01,  2.1798e-01,  2.1553e-01,
            1.5447e-01,  1.6460e-01,  1.4080e-01,  1.3157e-01,  1.2348e-01,
            1.2866e-01,  1.2657e-01,  1.2764e-01,  1.1036e-01,  1.0082e-01,
            9.1158e-02,  8.0348e-02,  9.3147e-02,  1.0974e-01,  1.3631e-01,
            1.5077e-01,  1.3898e-01,  1.7603e-01,  1.9709e-01,  2.3287e-01,
            2.5496e-01,  2.3599e-01,  2.5241e-01,  2.2591e-01,  1.8996e-01,
            2.1293e-01,  2.3421e-01],
          [ 2.6257e-01,  2.5645e-01,  2.2260e-01,  2.1831e-01,  1.8494e-01,
            1.6368e-01,  1.5596e-01,  1.4551e-01,  1.1840e-01,  1.2110e-01,
            1.2463e-01,  1.4321e-01,  1.5592e-01,  1.5920e-01,  1.6806e-01,
            1.4188e-01,  1.2964e-01,  1.4813e-01,  1.3611e-01,  1.4560e-01,
            1.5868e-01,  1.8665e-01,  2.7317e-01,  2.8624e-01,  2.4924e-01,
            2.8276e-01,  2.4937e-01,  2.3002e-01,  2.2018e-01,  1.9301e-01,
            1.9634e-01,  2.0531e-01],
          [ 2.5887e-01,  2.5148e-01,  2.4851e-01,  2.3390e-01,  2.0698e-01,
            1.7749e-01,  1.7199e-01,  1.6975e-01,  1.5084e-01,  1.1878e-01,
            1.3190e-01,  1.3562e-01,  1.4392e-01,  1.4789e-01,  1.7445e-01,
            1.6676e-01,  1.6804e-01,  2.0512e-01,  1.8272e-01,  1.4533e-01,
            1.3244e-01,  2.0823e-01,  2.5289e-01,  2.5745e-01,  2.6792e-01,
            2.6471e-01,  2.5504e-01,  2.5277e-01,  2.5128e-01,  2.3292e-01,
            2.0880e-01,  1.9499e-01],
          [ 2.2712e-01,  2.3777e-01,  2.5247e-01,  2.3953e-01,  2.3922e-01,
            2.1727e-01,  1.8337e-01,  1.6354e-01,  1.6589e-01,  1.4106e-01,
            1.4232e-01,  1.3163e-01,  1.3583e-01,  1.4622e-01,  1.7897e-01,
            2.0166e-01,  2.0864e-01,  2.3225e-01,  1.9145e-01,  1.4999e-01,
            1.7123e-01,  2.4011e-01,  2.7574e-01,  2.5854e-01,  2.6073e-01,
            2.6260e-01,  2.5333e-01,  2.6222e-01,  2.3286e-01,  2.1713e-01,
            2.0582e-01,  2.0321e-01],
          [ 2.4191e-01,  2.3880e-01,  2.1939e-01,  2.1700e-01,  2.2735e-01,
            2.3096e-01,  2.2331e-01,  1.9265e-01,  1.5992e-01,  1.6816e-01,
            1.5914e-01,  1.5288e-01,  1.5558e-01,  1.6316e-01,  1.5846e-01,
            1.8536e-01,  2.0809e-01,  2.3927e-01,  2.0529e-01,  1.7738e-01,
            2.1139e-01,  2.4397e-01,  2.7939e-01,  2.4430e-01,  2.4753e-01,
            2.5400e-01,  2.4936e-01,  2.2809e-01,  2.2489e-01,  1.9765e-01,
            2.0440e-01,  2.0375e-01]]],


        [[[ 5.7055e-01,  5.6804e-01,  5.4867e-01,  5.3331e-01,  5.3843e-01,
            5.3078e-01,  5.0495e-01,  4.9978e-01,  4.9129e-01,  4.8844e-01,
            4.4063e-01,  3.9426e-01,  3.9173e-01,  3.9310e-01,  4.1723e-01,
            4.1316e-01,  4.4669e-01,  5.0601e-01,  5.2030e-01,  4.7754e-01,
            4.5391e-01,  4.5755e-01,  4.5129e-01,  4.2889e-01,  4.7182e-01,
            5.3245e-01,  5.3430e-01,  5.2777e-01,  5.2183e-01,  5.2148e-01,
            5.1458e-01,  5.0199e-01],
          [ 5.5191e-01,  5.0840e-01,  5.3629e-01,  5.1388e-01,  5.3151e-01,
            5.3907e-01,  5.0706e-01,  5.0947e-01,  4.7443e-01,  4.6956e-01,
            4.6690e-01,  4.3294e-01,  4.2368e-01,  4.4327e-01,  4.5617e-01,
            4.9187e-01,  4.7306e-01,  4.2339e-01,  4.5353e-01,  4.9721e-01,
            4.7049e-01,  4.5575e-01,  4.4581e-01,  4.8200e-01,  5.4709e-01,
            5.6632e-01,  5.3097e-01,  5.3295e-01,  5.2777e-01,  5.4176e-01,
            5.1667e-01,  5.1129e-01],
          [ 5.3869e-01,  5.2158e-01,  5.2690e-01,  4.9447e-01,  4.8305e-01,
            4.8872e-01,  4.5313e-01,  4.4276e-01,  4.4219e-01,  4.4605e-01,
            4.4948e-01,  4.5332e-01,  4.3821e-01,  4.4717e-01,  4.6431e-01,
            4.6433e-01,  4.0970e-01,  4.6534e-01,  5.0204e-01,  6.3015e-01,
            6.1769e-01,  5.0984e-01,  4.1220e-01,  4.5116e-01,  5.1361e-01,
            5.2130e-01,  5.2309e-01,  5.3148e-01,  5.5114e-01,  5.5375e-01,
            5.4052e-01,  5.3300e-01],
          [ 5.2249e-01,  5.4007e-01,  5.1975e-01,  5.2821e-01,  5.1961e-01,
            4.5483e-01,  3.7909e-01,  3.4978e-01,  3.9438e-01,  4.4325e-01,
            4.5330e-01,  4.6850e-01,  4.5847e-01,  4.6260e-01,  4.5530e-01,
            4.8412e-01,  4.2574e-01,  5.4959e-01,  7.3851e-01,  7.7241e-01,
            7.6628e-01,  6.5177e-01,  4.4576e-01,  3.6177e-01,  4.1835e-01,
            4.4956e-01,  5.1169e-01,  5.1996e-01,  5.5565e-01,  5.7340e-01,
            5.7283e-01,  5.5198e-01],
          [ 5.1891e-01,  5.3247e-01,  5.3558e-01,  5.1977e-01,  5.1748e-01,
            4.6638e-01,  3.7085e-01,  2.9648e-01,  3.0896e-01,  3.6218e-01,
            3.8162e-01,  4.0714e-01,  4.1684e-01,  4.1187e-01,  4.1446e-01,
            4.6450e-01,  5.9406e-01,  7.2473e-01,  7.7208e-01,  7.3974e-01,
            8.1047e-01,  8.0416e-01,  6.1786e-01,  4.3439e-01,  3.6575e-01,
            4.1385e-01,  4.8285e-01,  5.1115e-01,  5.3694e-01,  5.8102e-01,
            5.6834e-01,  5.5589e-01],
          [ 5.3421e-01,  5.1534e-01,  5.1255e-01,  5.0875e-01,  4.2765e-01,
            3.3674e-01,  2.6187e-01,  2.3880e-01,  2.3380e-01,  1.9947e-01,
            2.1927e-01,  2.3173e-01,  2.4189e-01,  2.9106e-01,  3.2272e-01,
            3.8081e-01,  6.0428e-01,  7.9633e-01,  6.5762e-01,  5.8018e-01,
            7.3603e-01,  7.7715e-01,  7.3555e-01,  6.0575e-01,  3.9414e-01,
            3.6390e-01,  4.2778e-01,  4.8146e-01,  4.9854e-01,  5.3893e-01,
            5.8969e-01,  5.8353e-01],
          [ 5.3976e-01,  5.1154e-01,  5.1221e-01,  4.4785e-01,  3.2556e-01,
            2.5595e-01,  2.2340e-01,  2.3110e-01,  2.2583e-01,  1.8468e-01,
            1.7301e-01,  1.7928e-01,  1.9958e-01,  2.0234e-01,  3.3682e-01,
            3.5298e-01,  6.6587e-01,  7.8417e-01,  5.5409e-01,  5.4809e-01,
            7.2605e-01,  7.8679e-01,  8.0981e-01,  8.0033e-01,  5.3783e-01,
            3.3455e-01,  3.7187e-01,  4.3093e-01,  4.5750e-01,  4.7034e-01,
            5.6082e-01,  5.5348e-01],
          [ 5.0636e-01,  4.3282e-01,  3.9520e-01,  3.3152e-01,  2.8211e-01,
            2.5177e-01,  2.3036e-01,  2.2876e-01,  2.1615e-01,  1.8987e-01,
            1.7028e-01,  1.8577e-01,  1.8039e-01,  1.8236e-01,  2.6955e-01,
            4.8990e-01,  7.8182e-01,  6.9719e-01,  5.5483e-01,  5.8463e-01,
            7.2794e-01,  8.0880e-01,  8.2211e-01,  8.4118e-01,  6.3626e-01,
            3.7743e-01,  3.5966e-01,  3.8092e-01,  3.6780e-01,  3.8383e-01,
            4.3326e-01,  4.8281e-01],
          [ 4.6834e-01,  3.7847e-01,  3.2596e-01,  3.2427e-01,  2.7595e-01,
            2.4166e-01,  2.3792e-01,  2.1470e-01,  2.2202e-01,  2.0853e-01,
            1.8432e-01,  1.5853e-01,  1.5992e-01,  1.5937e-01,  2.6027e-01,
            5.5874e-01,  7.6073e-01,  6.8458e-01,  6.0902e-01,  6.3102e-01,
            8.1162e-01,  9.0445e-01,  9.0434e-01,  8.8276e-01,  6.4166e-01,
            4.0855e-01,  3.6469e-01,  3.3291e-01,  3.0165e-01,  3.1693e-01,
            3.1792e-01,  3.9552e-01],
          [ 4.1992e-01,  3.6878e-01,  3.4746e-01,  3.1999e-01,  2.7892e-01,
            2.4890e-01,  2.5343e-01,  2.5812e-01,  2.3970e-01,  1.9856e-01,
            1.7759e-01,  1.5227e-01,  1.3258e-01,  1.5495e-01,  2.6791e-01,
            5.5060e-01,  7.0730e-01,  6.5850e-01,  6.6846e-01,  7.0438e-01,
            8.4151e-01,  8.9671e-01,  8.7813e-01,  8.2120e-01,  6.1615e-01,
            4.0285e-01,  3.6062e-01,  3.7461e-01,  3.6780e-01,  2.9523e-01,
            2.4428e-01,  2.8043e-01],
          [ 3.9880e-01,  3.6649e-01,  3.3587e-01,  3.1898e-01,  2.9129e-01,
            2.7972e-01,  2.7259e-01,  2.4084e-01,  2.3862e-01,  2.1989e-01,
            2.2149e-01,  2.0840e-01,  1.8620e-01,  1.8286e-01,  2.6668e-01,
            4.6024e-01,  6.2810e-01,  6.4667e-01,  6.8796e-01,  7.7122e-01,
            8.1506e-01,  8.3839e-01,  7.9994e-01,  7.8234e-01,  5.8089e-01,
            3.9721e-01,  3.5329e-01,  4.0577e-01,  4.4569e-01,  4.5295e-01,
            3.4583e-01,  3.1094e-01],
          [ 3.9978e-01,  3.7196e-01,  3.3280e-01,  3.4276e-01,  3.3699e-01,
            3.2185e-01,  2.9922e-01,  2.7121e-01,  2.5900e-01,  2.7550e-01,
            2.7078e-01,  2.5238e-01,  2.4579e-01,  2.5281e-01,  2.4747e-01,
            3.6616e-01,  4.8206e-01,  5.3258e-01,  6.1235e-01,  6.8036e-01,
            7.5288e-01,  8.0198e-01,  7.7611e-01,  7.4088e-01,  5.3191e-01,
            3.7162e-01,  3.4891e-01,  3.8749e-01,  5.0133e-01,  5.8005e-01,
            5.7115e-01,  4.6599e-01],
          [ 4.0464e-01,  3.8364e-01,  3.4481e-01,  3.4246e-01,  3.5436e-01,
            3.7861e-01,  3.6435e-01,  3.3161e-01,  3.2936e-01,  3.3120e-01,
            3.0713e-01,  3.0522e-01,  3.2555e-01,  3.2273e-01,  3.1522e-01,
            3.2825e-01,  4.0706e-01,  4.4997e-01,  4.7491e-01,  5.3328e-01,
            6.8150e-01,  7.2205e-01,  6.8965e-01,  6.1951e-01,  5.1105e-01,
            4.1049e-01,  3.6407e-01,  3.8267e-01,  5.5927e-01,  7.3825e-01,
            7.2277e-01,  6.3321e-01],
          [ 3.6892e-01,  3.7366e-01,  3.4469e-01,  3.7426e-01,  3.7100e-01,
            3.7388e-01,  3.9561e-01,  3.8369e-01,  3.8901e-01,  3.7676e-01,
            3.8139e-01,  3.8273e-01,  3.7746e-01,  3.6838e-01,  3.3419e-01,
            3.5211e-01,  3.4931e-01,  3.6884e-01,  3.9793e-01,  5.0910e-01,
            5.8740e-01,  5.5185e-01,  5.5628e-01,  5.0224e-01,  4.4839e-01,
            4.3754e-01,  3.6510e-01,  4.9149e-01,  7.6362e-01,  9.0975e-01,
            9.2332e-01,  8.1317e-01],
          [ 3.5771e-01,  3.4776e-01,  3.6509e-01,  3.9678e-01,  4.0778e-01,
            4.2054e-01,  4.0946e-01,  4.1087e-01,  4.1996e-01,  4.0748e-01,
            4.1259e-01,  3.9860e-01,  3.8096e-01,  3.5067e-01,  3.4077e-01,
            3.5496e-01,  3.5348e-01,  3.5424e-01,  3.8625e-01,  4.5940e-01,
            5.0500e-01,  4.9844e-01,  4.4084e-01,  3.7044e-01,  3.5765e-01,
            4.1180e-01,  4.2312e-01,  6.0724e-01,  8.4993e-01,  8.9535e-01,
            9.1613e-01,  8.5868e-01],
          [ 3.6787e-01,  3.9789e-01,  4.0922e-01,  4.1851e-01,  4.3024e-01,
            4.3817e-01,  4.4748e-01,  4.2657e-01,  4.2816e-01,  4.4875e-01,
            4.1293e-01,  4.0621e-01,  3.8050e-01,  3.3697e-01,  3.5449e-01,
            3.5053e-01,  4.0585e-01,  3.9517e-01,  3.8461e-01,  4.0882e-01,
            5.0180e-01,  4.7940e-01,  3.8437e-01,  3.6126e-01,  3.2873e-01,
            4.1177e-01,  4.5902e-01,  6.2954e-01,  8.5624e-01,  8.7683e-01,
            8.7105e-01,  8.3599e-01],
          [ 3.7902e-01,  4.3415e-01,  4.2508e-01,  4.4002e-01,  4.3789e-01,
            4.6650e-01,  4.5184e-01,  4.4857e-01,  4.4358e-01,  4.4427e-01,
            4.3792e-01,  4.2152e-01,  3.8637e-01,  3.5273e-01,  3.5353e-01,
            3.6606e-01,  4.2298e-01,  4.3169e-01,  3.8079e-01,  4.3813e-01,
            5.5134e-01,  4.6499e-01,  3.8793e-01,  3.6855e-01,  3.0737e-01,
            3.7971e-01,  5.0847e-01,  7.0256e-01,  8.1890e-01,  8.3514e-01,
            8.3405e-01,  8.2686e-01],
          [ 4.3029e-01,  4.6994e-01,  4.5541e-01,  4.6241e-01,  4.6059e-01,
            4.7933e-01,  4.6194e-01,  4.5528e-01,  4.5858e-01,  4.6013e-01,
            4.7221e-01,  4.6008e-01,  4.1772e-01,  3.9054e-01,  3.4232e-01,
            3.4265e-01,  3.6743e-01,  4.0278e-01,  4.0481e-01,  4.3647e-01,
            5.4121e-01,  4.2588e-01,  3.1348e-01,  2.3300e-01,  2.4851e-01,
            3.8655e-01,  5.8690e-01,  7.4606e-01,  8.3233e-01,  8.1919e-01,
            8.2525e-01,  8.4323e-01],
          [ 4.3605e-01,  4.8544e-01,  4.8496e-01,  4.9582e-01,  4.7334e-01,
            4.7891e-01,  4.6899e-01,  4.5630e-01,  4.7681e-01,  4.8331e-01,
            4.8886e-01,  4.8434e-01,  4.2409e-01,  3.4977e-01,  3.0952e-01,
            2.9795e-01,  3.4490e-01,  3.5949e-01,  4.1504e-01,  4.2518e-01,
            3.1679e-01,  1.9818e-01,  1.1999e-01,  8.0201e-02,  8.3540e-02,
            1.8774e-01,  5.2168e-01,  7.5676e-01,  8.3289e-01,  8.1963e-01,
            8.2281e-01,  8.1963e-01],
          [ 4.3525e-01,  4.8840e-01,  4.9495e-01,  5.0550e-01,  4.8221e-01,
            4.8804e-01,  4.9259e-01,  4.8724e-01,  4.6781e-01,  5.0282e-01,
            4.9949e-01,  5.0226e-01,  4.2568e-01,  3.4962e-01,  2.9998e-01,
            2.8943e-01,  3.2175e-01,  4.2525e-01,  4.1503e-01,  2.6541e-01,
            1.0719e-01,  4.6419e-02,  5.0799e-02,  6.4905e-02,  8.9470e-02,
            1.4758e-01,  4.2853e-01,  7.3819e-01,  8.2958e-01,  8.2550e-01,
            8.1782e-01,  8.0835e-01],
          [ 4.3724e-01,  4.9547e-01,  5.1998e-01,  5.0474e-01,  5.1119e-01,
            4.8027e-01,  4.9172e-01,  4.9840e-01,  5.0216e-01,  5.2000e-01,
            5.3754e-01,  5.2117e-01,  4.5039e-01,  3.8145e-01,  3.1520e-01,
            2.7780e-01,  2.5275e-01,  3.0015e-01,  1.9912e-01,  9.1150e-02,
            3.5542e-02,  4.9905e-02,  7.1508e-02,  1.0551e-01,  1.4466e-01,
            1.9215e-01,  3.7079e-01,  7.3851e-01,  8.4920e-01,  8.2669e-01,
            8.4886e-01,  7.7148e-01],
          [ 4.6428e-01,  5.1512e-01,  5.3219e-01,  5.1983e-01,  5.2497e-01,
            4.9510e-01,  5.0002e-01,  5.1591e-01,  5.2153e-01,  5.3946e-01,
            5.5071e-01,  5.5319e-01,  4.7196e-01,  3.4931e-01,  2.3208e-01,
            2.5331e-01,  2.4067e-01,  1.3651e-01,  8.8024e-02,  3.9313e-02,
            3.1775e-02,  4.9715e-02,  7.2759e-02,  1.1587e-01,  1.7235e-01,
            2.2644e-01,  3.7870e-01,  7.4085e-01,  8.4850e-01,  8.4693e-01,
            8.9520e-01,  8.4081e-01],
          [ 4.3861e-01,  5.1607e-01,  5.6942e-01,  5.8115e-01,  5.4710e-01,
            5.0181e-01,  5.0157e-01,  5.3237e-01,  5.3531e-01,  5.5798e-01,
            5.6757e-01,  5.7338e-01,  4.7133e-01,  2.6594e-01,  1.9346e-01,
            2.0338e-01,  1.8921e-01,  1.1249e-01,  6.8031e-02,  3.6932e-02,
            2.6076e-02,  4.1027e-02,  4.9780e-02,  9.9740e-02,  2.2014e-01,
            3.6513e-01,  4.9452e-01,  6.9120e-01,  8.4567e-01,  8.9927e-01,
            9.5903e-01,  9.5385e-01],
          [ 4.0057e-01,  4.4470e-01,  5.1528e-01,  6.0021e-01,  5.7607e-01,
            4.9810e-01,  5.0755e-01,  5.1700e-01,  5.1401e-01,  5.5690e-01,
            6.0348e-01,  5.7310e-01,  4.7870e-01,  3.7819e-01,  1.8828e-01,
            1.9307e-01,  1.8804e-01,  1.2639e-01,  5.3343e-02,  1.3955e-02,
            2.6613e-02,  5.0802e-02,  5.7978e-02,  1.1159e-01,  3.8615e-01,
            5.2440e-01,  5.3435e-01,  6.7282e-01,  8.6399e-01,  9.2507e-01,
            9.3820e-01,  9.3516e-01],
          [ 3.6352e-01,  4.0116e-01,  4.7196e-01,  6.2264e-01,  5.9702e-01,
            5.2564e-01,  5.2686e-01,  5.3826e-01,  5.5461e-01,  6.1366e-01,
            6.1417e-01,  4.8637e-01,  3.4723e-01,  2.7110e-01,  1.8123e-01,
            1.8680e-01,  1.5860e-01,  1.2634e-01,  5.7316e-02,  4.1152e-02,
            2.0239e-02,  3.3796e-02,  4.7129e-02,  2.1004e-01,  6.4171e-01,
            6.9928e-01,  4.5151e-01,  4.3995e-01,  6.3593e-01,  8.4890e-01,
            9.3050e-01,  8.7307e-01],
          [ 4.7121e-01,  5.1999e-01,  6.2716e-01,  7.5681e-01,  7.1028e-01,
            6.1360e-01,  6.8566e-01,  7.3565e-01,  7.7033e-01,  7.6829e-01,
            6.8749e-01,  5.4398e-01,  4.1257e-01,  3.3335e-01,  3.0081e-01,
            2.4397e-01,  2.4608e-01,  2.5431e-01,  2.5707e-01,  1.7886e-01,
            1.3258e-01,  1.8290e-01,  2.5709e-01,  3.9235e-01,  6.9454e-01,
            7.4582e-01,  4.5803e-01,  2.6951e-01,  3.1246e-01,  5.5505e-01,
            7.5737e-01,  6.1268e-01],
          [ 6.9531e-01,  7.9219e-01,  8.7166e-01,  8.8736e-01,  8.4845e-01,
            8.2042e-01,  8.2839e-01,  8.5080e-01,  8.5912e-01,  8.6091e-01,
            8.2937e-01,  7.4063e-01,  6.5087e-01,  6.3321e-01,  5.9479e-01,
            5.8496e-01,  6.6243e-01,  7.6071e-01,  7.8854e-01,  7.3134e-01,
            6.5984e-01,  6.3717e-01,  6.8864e-01,  7.0127e-01,  7.2083e-01,
            6.9803e-01,  6.3011e-01,  3.9887e-01,  2.7236e-01,  2.4501e-01,
            5.7757e-01,  7.1421e-01],
          [ 8.4372e-01,  9.4754e-01,  9.3995e-01,  9.1265e-01,  8.8629e-01,
            8.9435e-01,  8.6670e-01,  8.7920e-01,  8.5993e-01,  8.6763e-01,
            8.4594e-01,  8.3289e-01,  7.9630e-01,  7.9355e-01,  7.5902e-01,
            7.8145e-01,  8.3935e-01,  9.1857e-01,  9.5458e-01,  9.2721e-01,
            9.0416e-01,  8.7055e-01,  8.4913e-01,  8.6608e-01,  8.6203e-01,
            8.3506e-01,  8.5365e-01,  8.1040e-01,  7.2448e-01,  4.7675e-01,
            5.5631e-01,  7.9840e-01],
          [ 8.5768e-01,  9.2107e-01,  9.3175e-01,  8.7546e-01,  8.9551e-01,
            8.9551e-01,  8.9107e-01,  9.0908e-01,  8.7531e-01,  8.8526e-01,
            8.7858e-01,  8.6072e-01,  8.3327e-01,  8.3174e-01,  8.4346e-01,
            8.0560e-01,  8.0289e-01,  8.5880e-01,  8.9586e-01,  8.7885e-01,
            8.9449e-01,  8.7987e-01,  8.9731e-01,  8.8609e-01,  8.5737e-01,
            8.5368e-01,  8.4454e-01,  8.3817e-01,  8.1462e-01,  7.7949e-01,
            7.5029e-01,  8.3619e-01],
          [ 8.4923e-01,  8.6937e-01,  8.8795e-01,  8.7232e-01,  8.9656e-01,
            9.1484e-01,  9.0001e-01,  8.7487e-01,  8.3308e-01,  8.6590e-01,
            8.3681e-01,  8.1633e-01,  8.5607e-01,  8.5843e-01,  8.8064e-01,
            8.7437e-01,  9.1271e-01,  9.1222e-01,  9.0606e-01,  9.0259e-01,
            9.1258e-01,  8.8143e-01,  8.7642e-01,  8.7231e-01,  8.6785e-01,
            8.6718e-01,  8.7050e-01,  8.5881e-01,  8.3817e-01,  8.3741e-01,
            7.9947e-01,  8.3277e-01],
          [ 8.7335e-01,  8.5005e-01,  8.2705e-01,  8.5022e-01,  8.5149e-01,
            8.8180e-01,  8.5268e-01,  8.0746e-01,  7.7425e-01,  7.9444e-01,
            8.5463e-01,  7.7145e-01,  8.4404e-01,  8.4259e-01,  7.9533e-01,
            8.5890e-01,  8.9756e-01,  9.0963e-01,  9.1305e-01,  9.0805e-01,
            9.1696e-01,  8.9853e-01,  8.5674e-01,  8.4622e-01,  8.6374e-01,
            8.6852e-01,  8.6425e-01,  8.5592e-01,  8.5342e-01,  8.1780e-01,
            8.1646e-01,  8.1123e-01],
          [ 8.0879e-01,  7.8851e-01,  7.7601e-01,  8.0210e-01,  8.1801e-01,
            8.6395e-01,  8.4555e-01,  7.8984e-01,  7.9319e-01,  7.6780e-01,
            9.0990e-01,  7.8926e-01,  7.2539e-01,  8.6026e-01,  8.4080e-01,
            8.6038e-01,  8.8151e-01,  8.8046e-01,  8.7322e-01,  8.5731e-01,
            8.6184e-01,  8.6101e-01,  8.5731e-01,  8.3894e-01,  8.5810e-01,
            8.7053e-01,  8.8693e-01,  8.5568e-01,  8.5800e-01,  8.8086e-01,
            8.5063e-01,  8.3610e-01]],

         [[ 5.5708e-01,  5.7771e-01,  5.6848e-01,  5.7041e-01,  5.8880e-01,
            5.8432e-01,  5.5671e-01,  5.3534e-01,  5.1104e-01,  5.0310e-01,
            4.9540e-01,  4.4205e-01,  4.3443e-01,  4.2989e-01,  4.4872e-01,
            4.5159e-01,  4.7864e-01,  5.5563e-01,  5.6696e-01,  5.3949e-01,
            5.4501e-01,  5.3157e-01,  5.3773e-01,  5.2327e-01,  5.4863e-01,
            5.9919e-01,  5.9187e-01,  5.7760e-01,  5.6563e-01,  5.7174e-01,
            5.7871e-01,  5.5356e-01],
          [ 5.7313e-01,  5.4026e-01,  5.5292e-01,  5.6693e-01,  5.6720e-01,
            5.9138e-01,  5.5535e-01,  5.4034e-01,  5.2119e-01,  5.0246e-01,
            5.1823e-01,  4.9920e-01,  4.9668e-01,  4.8916e-01,  5.1303e-01,
            5.3444e-01,  5.4735e-01,  5.0800e-01,  5.0010e-01,  5.4319e-01,
            5.2864e-01,  5.2418e-01,  5.1739e-01,  5.6690e-01,  6.1123e-01,
            6.4151e-01,  5.9007e-01,  5.8661e-01,  5.9038e-01,  5.8765e-01,
            5.8045e-01,  5.5075e-01],
          [ 5.7934e-01,  5.5446e-01,  5.6210e-01,  5.3691e-01,  5.3103e-01,
            5.1045e-01,  4.7199e-01,  4.6990e-01,  4.9245e-01,  5.1428e-01,
            5.3639e-01,  5.2783e-01,  5.2177e-01,  5.3105e-01,  5.5859e-01,
            5.4415e-01,  4.6792e-01,  5.0612e-01,  5.4039e-01,  6.4041e-01,
            6.2996e-01,  5.5366e-01,  4.7244e-01,  5.4713e-01,  5.9216e-01,
            6.0162e-01,  6.0309e-01,  5.9008e-01,  6.0065e-01,  5.9619e-01,
            5.8527e-01,  5.9358e-01],
          [ 5.5546e-01,  5.6885e-01,  5.5362e-01,  5.6378e-01,  5.5218e-01,
            4.7408e-01,  4.1854e-01,  3.7899e-01,  4.6014e-01,  5.2529e-01,
            5.3557e-01,  5.2695e-01,  5.5491e-01,  5.5211e-01,  5.8108e-01,
            5.5646e-01,  4.6819e-01,  5.6294e-01,  7.5580e-01,  7.6730e-01,
            7.5583e-01,  6.6847e-01,  4.9364e-01,  4.6303e-01,  5.1851e-01,
            5.3743e-01,  5.9178e-01,  6.0161e-01,  6.1203e-01,  6.2269e-01,
            6.1342e-01,  6.1428e-01],
          [ 5.6979e-01,  5.6122e-01,  5.6749e-01,  5.6551e-01,  5.5871e-01,
            4.7277e-01,  3.7981e-01,  3.1087e-01,  3.2117e-01,  3.8484e-01,
            4.3205e-01,  4.6554e-01,  4.6790e-01,  4.8420e-01,  4.7858e-01,
            4.8493e-01,  6.1074e-01,  7.2084e-01,  7.3447e-01,  7.0849e-01,
            7.6230e-01,  7.3830e-01,  5.9584e-01,  4.8991e-01,  4.3886e-01,
            4.9982e-01,  5.7277e-01,  5.9121e-01,  6.0100e-01,  6.3095e-01,
            6.4140e-01,  6.3797e-01],
          [ 5.7131e-01,  5.7126e-01,  5.7562e-01,  5.5692e-01,  4.6640e-01,
            3.5425e-01,  2.8880e-01,  2.4932e-01,  2.3496e-01,  2.1044e-01,
            2.5250e-01,  2.7163e-01,  2.6364e-01,  3.1673e-01,  3.4165e-01,
            3.9482e-01,  5.9583e-01,  7.7140e-01,  6.1180e-01,  5.2339e-01,
            6.5797e-01,  6.9049e-01,  6.8629e-01,  6.2895e-01,  4.7395e-01,
            4.7025e-01,  5.1629e-01,  5.6679e-01,  5.8406e-01,  6.1692e-01,
            6.5865e-01,  6.4995e-01],
          [ 5.5489e-01,  5.3859e-01,  5.3947e-01,  4.8869e-01,  3.4933e-01,
            2.6735e-01,  2.2110e-01,  2.3444e-01,  2.2610e-01,  2.0172e-01,
            1.8526e-01,  1.9530e-01,  2.0313e-01,  2.1721e-01,  3.4928e-01,
            3.5249e-01,  6.7995e-01,  7.7656e-01,  4.9564e-01,  4.9414e-01,
            6.4342e-01,  7.2773e-01,  7.7662e-01,  8.0244e-01,  6.0128e-01,
            4.3711e-01,  4.6716e-01,  5.1849e-01,  5.3580e-01,  5.5796e-01,
            6.2132e-01,  6.2858e-01],
          [ 5.2921e-01,  4.4026e-01,  4.0546e-01,  3.5045e-01,  2.8378e-01,
            2.5636e-01,  2.2820e-01,  2.2556e-01,  1.9947e-01,  1.7477e-01,
            1.7121e-01,  1.8423e-01,  1.8726e-01,  2.0551e-01,  2.9464e-01,
            5.0454e-01,  7.9163e-01,  6.7775e-01,  4.9367e-01,  5.0648e-01,
            6.5805e-01,  7.5839e-01,  7.9216e-01,  8.2316e-01,  6.6223e-01,
            4.7382e-01,  4.5450e-01,  4.6877e-01,  4.7305e-01,  4.8854e-01,
            5.2473e-01,  5.4754e-01],
          [ 4.5995e-01,  3.8948e-01,  3.2738e-01,  3.0692e-01,  2.5978e-01,
            2.3472e-01,  2.3837e-01,  2.1056e-01,  1.9915e-01,  1.9269e-01,
            1.9272e-01,  1.8980e-01,  1.9020e-01,  2.0058e-01,  2.7199e-01,
            5.5935e-01,  7.4298e-01,  6.3701e-01,  5.4152e-01,  5.9043e-01,
            8.0476e-01,  9.0045e-01,  8.8339e-01,  8.5486e-01,  6.4172e-01,
            5.0108e-01,  4.6401e-01,  4.3549e-01,  3.9936e-01,  4.4495e-01,
            4.4099e-01,  4.8827e-01],
          [ 4.1664e-01,  3.8962e-01,  3.2996e-01,  2.9652e-01,  2.6512e-01,
            2.2057e-01,  2.4008e-01,  2.4855e-01,  2.5104e-01,  2.1450e-01,
            2.2351e-01,  2.0419e-01,  1.9449e-01,  2.0787e-01,  2.8874e-01,
            5.4273e-01,  6.8121e-01,  6.0737e-01,  6.2825e-01,  7.0505e-01,
            8.3317e-01,  8.8605e-01,  8.6636e-01,  7.9237e-01,  6.0216e-01,
            4.5895e-01,  4.4385e-01,  4.3880e-01,  4.3646e-01,  3.9879e-01,
            3.6759e-01,  3.7457e-01],
          [ 4.0074e-01,  3.8673e-01,  3.4695e-01,  3.2047e-01,  2.7791e-01,
            2.6386e-01,  2.6917e-01,  2.6349e-01,  2.6157e-01,  2.4889e-01,
            2.5132e-01,  2.3799e-01,  2.1284e-01,  2.1215e-01,  3.0999e-01,
            4.7790e-01,  6.0919e-01,  6.1275e-01,  6.7902e-01,  7.5699e-01,
            8.2503e-01,  8.2760e-01,  7.9606e-01,  7.3032e-01,  5.4128e-01,
            4.0444e-01,  4.0147e-01,  4.4416e-01,  4.8579e-01,  4.8971e-01,
            4.1864e-01,  3.6342e-01],
          [ 4.2282e-01,  3.8587e-01,  3.8320e-01,  3.6785e-01,  3.4732e-01,
            3.4910e-01,  3.2890e-01,  3.3938e-01,  3.0332e-01,  3.2058e-01,
            2.9458e-01,  2.8362e-01,  2.8423e-01,  2.8729e-01,  3.0671e-01,
            4.0989e-01,  5.1551e-01,  5.4225e-01,  5.8076e-01,  6.8272e-01,
            7.4848e-01,  7.7788e-01,  7.5889e-01,  7.0274e-01,  5.1731e-01,
            3.9070e-01,  3.9556e-01,  4.2681e-01,  5.1904e-01,  5.8117e-01,
            5.6251e-01,  4.7479e-01],
          [ 4.4123e-01,  4.1104e-01,  3.9344e-01,  3.7815e-01,  3.9492e-01,
            4.2600e-01,  3.9105e-01,  3.9709e-01,  3.6319e-01,  3.6110e-01,
            3.5498e-01,  3.3249e-01,  3.5465e-01,  3.4905e-01,  3.5460e-01,
            3.8024e-01,  4.4027e-01,  4.3666e-01,  4.2612e-01,  4.8480e-01,
            6.0191e-01,  6.7176e-01,  6.3879e-01,  5.8390e-01,  5.0221e-01,
            4.4509e-01,  4.1681e-01,  4.4013e-01,  5.8718e-01,  7.2464e-01,
            7.1443e-01,  6.1845e-01],
          [ 4.2316e-01,  4.0641e-01,  3.9515e-01,  4.0376e-01,  4.0452e-01,
            4.4297e-01,  4.3931e-01,  4.3568e-01,  4.3716e-01,  4.2805e-01,
            4.0142e-01,  4.0660e-01,  3.9608e-01,  3.9538e-01,  3.8693e-01,
            4.0647e-01,  4.2086e-01,  3.7386e-01,  3.5872e-01,  4.5764e-01,
            5.2233e-01,  4.9120e-01,  5.1959e-01,  4.6201e-01,  4.2390e-01,
            4.4263e-01,  3.7616e-01,  5.3617e-01,  7.8145e-01,  9.0291e-01,
            9.2388e-01,  8.1597e-01],
          [ 4.2437e-01,  4.1713e-01,  4.1339e-01,  4.3859e-01,  4.5459e-01,
            4.6836e-01,  4.5714e-01,  4.5231e-01,  4.6038e-01,  4.6094e-01,
            4.5595e-01,  4.4205e-01,  4.3060e-01,  4.0887e-01,  4.0742e-01,
            4.2188e-01,  4.0981e-01,  3.8684e-01,  3.8243e-01,  4.4930e-01,
            4.6958e-01,  4.2708e-01,  3.8832e-01,  3.4632e-01,  3.4086e-01,
            4.2594e-01,  4.3884e-01,  6.1481e-01,  8.6169e-01,  9.0703e-01,
            9.0232e-01,  8.6979e-01],
          [ 4.3578e-01,  4.6459e-01,  4.5527e-01,  4.6652e-01,  4.6599e-01,
            4.9768e-01,  4.8673e-01,  4.6159e-01,  4.8159e-01,  4.7582e-01,
            4.8149e-01,  4.5870e-01,  4.3414e-01,  4.3401e-01,  4.3094e-01,
            4.2686e-01,  4.5986e-01,  4.2147e-01,  3.9134e-01,  4.1573e-01,
            4.7194e-01,  4.0562e-01,  3.4554e-01,  3.5205e-01,  3.4885e-01,
            4.1107e-01,  4.5577e-01,  6.3424e-01,  8.6222e-01,  8.9483e-01,
            8.7348e-01,  8.3035e-01],
          [ 4.2660e-01,  4.7156e-01,  4.8758e-01,  4.7303e-01,  4.8667e-01,
            5.1069e-01,  4.9407e-01,  4.9213e-01,  4.8378e-01,  4.7442e-01,
            4.6736e-01,  4.8124e-01,  4.5890e-01,  4.3845e-01,  4.3326e-01,
            4.2519e-01,  4.7502e-01,  4.4395e-01,  3.9254e-01,  4.5432e-01,
            5.0987e-01,  4.0955e-01,  3.4916e-01,  3.6163e-01,  3.2021e-01,
            3.6835e-01,  5.0278e-01,  7.0346e-01,  8.2973e-01,  8.4543e-01,
            8.2318e-01,  8.1285e-01],
          [ 4.3196e-01,  5.1538e-01,  5.0452e-01,  5.1717e-01,  5.0005e-01,
            5.0892e-01,  4.9863e-01,  4.9449e-01,  4.9256e-01,  5.0135e-01,
            5.0616e-01,  4.9943e-01,  4.7299e-01,  4.6374e-01,  4.2453e-01,
            4.2185e-01,  4.2792e-01,  4.3112e-01,  4.2493e-01,  4.8930e-01,
            5.0972e-01,  3.6790e-01,  2.7539e-01,  2.0960e-01,  2.1715e-01,
            3.4868e-01,  5.5185e-01,  7.3501e-01,  8.2069e-01,  8.2557e-01,
            8.0885e-01,  8.2101e-01],
          [ 4.2232e-01,  4.9500e-01,  5.2020e-01,  5.3390e-01,  5.0701e-01,
            5.2362e-01,  5.1853e-01,  5.1510e-01,  5.2225e-01,  5.2434e-01,
            5.3658e-01,  5.3149e-01,  4.7424e-01,  4.5194e-01,  4.0936e-01,
            3.8888e-01,  4.0893e-01,  3.9987e-01,  4.5166e-01,  4.3233e-01,
            2.6352e-01,  1.3457e-01,  7.4195e-02,  4.9029e-02,  6.4292e-02,
            1.6877e-01,  4.8507e-01,  7.4213e-01,  8.1692e-01,  8.2126e-01,
            7.9042e-01,  8.0507e-01],
          [ 4.0413e-01,  4.9224e-01,  5.3255e-01,  5.4817e-01,  5.2964e-01,
            5.1642e-01,  5.2895e-01,  5.5040e-01,  5.4566e-01,  5.5343e-01,
            5.6244e-01,  5.4011e-01,  4.7777e-01,  4.0880e-01,  3.9123e-01,
            3.5208e-01,  3.3613e-01,  4.1234e-01,  3.8048e-01,  2.3097e-01,
            6.7319e-02,  3.2083e-02,  4.0157e-02,  6.8421e-02,  8.3084e-02,
            1.2567e-01,  3.8776e-01,  7.1049e-01,  8.2374e-01,  8.1771e-01,
            8.1271e-01,  8.0240e-01],
          [ 4.4703e-01,  4.9304e-01,  5.3985e-01,  5.4544e-01,  5.2749e-01,
            5.1907e-01,  5.3864e-01,  5.6116e-01,  5.5885e-01,  5.9038e-01,
            5.7494e-01,  5.5060e-01,  4.8664e-01,  4.2859e-01,  3.6055e-01,
            2.9651e-01,  2.3629e-01,  2.8580e-01,  1.5962e-01,  6.6179e-02,
            2.9447e-02,  5.3573e-02,  6.5216e-02,  1.0365e-01,  1.3369e-01,
            1.7586e-01,  3.2667e-01,  7.0312e-01,  8.4110e-01,  8.4002e-01,
            8.3129e-01,  7.6378e-01],
          [ 4.7435e-01,  5.0502e-01,  5.3502e-01,  5.3566e-01,  5.4589e-01,
            5.3428e-01,  5.3829e-01,  5.6540e-01,  5.8599e-01,  5.9573e-01,
            6.0053e-01,  5.8121e-01,  5.0512e-01,  3.7711e-01,  2.7676e-01,
            2.6539e-01,  2.2742e-01,  1.2136e-01,  8.2790e-02,  3.9529e-02,
            3.7500e-02,  4.6774e-02,  8.6461e-02,  1.2209e-01,  1.5721e-01,
            2.0473e-01,  3.3956e-01,  7.2313e-01,  8.6062e-01,  8.7516e-01,
            8.9253e-01,  8.1903e-01],
          [ 4.8444e-01,  5.0625e-01,  5.5587e-01,  5.7884e-01,  5.4126e-01,
            5.4531e-01,  5.5078e-01,  5.7890e-01,  6.0200e-01,  6.0516e-01,
            6.0109e-01,  5.9406e-01,  4.8879e-01,  2.9810e-01,  2.1049e-01,
            2.0555e-01,  1.7492e-01,  1.2604e-01,  8.3259e-02,  6.5697e-02,
            3.7329e-02,  3.9632e-02,  8.2368e-02,  1.2199e-01,  1.9716e-01,
            3.3353e-01,  4.5646e-01,  6.9101e-01,  8.6235e-01,  9.1985e-01,
            9.5408e-01,  9.4279e-01],
          [ 4.5645e-01,  4.6416e-01,  4.9288e-01,  5.7193e-01,  5.4678e-01,
            5.1506e-01,  5.4092e-01,  5.6429e-01,  5.8178e-01,  5.8688e-01,
            6.0360e-01,  5.6068e-01,  4.4301e-01,  3.3411e-01,  1.5994e-01,
            1.4900e-01,  1.6614e-01,  1.2164e-01,  8.5536e-02,  3.9368e-02,
            1.7979e-02,  2.4544e-02,  3.6339e-02,  1.0069e-01,  3.7310e-01,
            5.0452e-01,  5.1082e-01,  6.7126e-01,  8.7985e-01,  9.4274e-01,
            9.4854e-01,  9.3788e-01],
          [ 4.0915e-01,  4.2591e-01,  4.6968e-01,  5.8365e-01,  5.8317e-01,
            5.1684e-01,  5.2990e-01,  5.6785e-01,  6.0468e-01,  6.2061e-01,
            5.8885e-01,  4.4990e-01,  2.7958e-01,  1.9605e-01,  1.3114e-01,
            1.1356e-01,  1.1454e-01,  1.2325e-01,  3.2128e-02,  1.4788e-02,
           -6.4699e-03,  2.3458e-02,  2.6385e-02,  1.9515e-01,  6.4168e-01,
            6.6085e-01,  4.0670e-01,  4.1386e-01,  6.3518e-01,  8.5562e-01,
            9.1965e-01,  8.6464e-01],
          [ 5.0266e-01,  5.3984e-01,  6.1726e-01,  7.2713e-01,  6.8574e-01,
            5.9359e-01,  6.8003e-01,  7.5867e-01,  7.8348e-01,  7.4019e-01,
            6.6051e-01,  4.9511e-01,  3.4827e-01,  2.8315e-01,  2.6687e-01,
            2.2277e-01,  2.0382e-01,  2.6994e-01,  2.1667e-01,  1.3487e-01,
            9.3865e-02,  1.5161e-01,  2.2962e-01,  3.8706e-01,  6.6894e-01,
            7.0985e-01,  4.1218e-01,  2.0819e-01,  2.8404e-01,  5.4854e-01,
            7.6918e-01,  6.3046e-01],
          [ 7.2330e-01,  7.9969e-01,  8.4588e-01,  8.7344e-01,  8.3216e-01,
            7.9507e-01,  8.1695e-01,  8.3662e-01,  8.3407e-01,  8.2686e-01,
            7.9408e-01,  7.2269e-01,  6.2263e-01,  6.3222e-01,  6.1114e-01,
            5.9682e-01,  6.7942e-01,  7.6893e-01,  7.6819e-01,  6.8034e-01,
            6.1388e-01,  6.0591e-01,  6.7128e-01,  6.8961e-01,  6.8168e-01,
            6.9702e-01,  6.0776e-01,  3.7027e-01,  2.4650e-01,  2.3656e-01,
            5.8816e-01,  7.3885e-01],
          [ 8.5909e-01,  9.4313e-01,  9.3845e-01,  8.9889e-01,  8.8158e-01,
            8.7561e-01,  8.6748e-01,  8.7473e-01,  8.4166e-01,  8.3530e-01,
            8.3529e-01,  8.2165e-01,  8.0374e-01,  7.8259e-01,  7.7097e-01,
            7.8469e-01,  8.6753e-01,  9.0001e-01,  9.0586e-01,  8.7349e-01,
            8.6449e-01,  8.5628e-01,  8.4106e-01,  8.4974e-01,  8.3374e-01,
            8.3854e-01,  8.4600e-01,  8.1820e-01,  7.0911e-01,  4.7894e-01,
            5.6355e-01,  8.0906e-01],
          [ 8.7283e-01,  9.2028e-01,  9.1725e-01,  8.7844e-01,  9.0018e-01,
            9.0047e-01,  8.9123e-01,  8.9170e-01,  8.7394e-01,  8.4791e-01,
            8.6201e-01,  8.5319e-01,  8.2238e-01,  8.0941e-01,  8.1870e-01,
            8.2514e-01,  8.2199e-01,  8.4228e-01,  8.6162e-01,  8.4564e-01,
            8.6499e-01,  8.7554e-01,  8.7955e-01,  8.7325e-01,  8.5859e-01,
            8.4615e-01,  8.5537e-01,  8.2947e-01,  8.1718e-01,  7.9447e-01,
            7.6143e-01,  8.4411e-01],
          [ 8.4829e-01,  8.7227e-01,  8.8623e-01,  8.6918e-01,  8.8779e-01,
            8.9909e-01,  8.7227e-01,  8.6010e-01,  7.8259e-01,  8.3137e-01,
            8.1901e-01,  7.9669e-01,  8.2204e-01,  8.4643e-01,  8.6122e-01,
            8.7580e-01,  8.8419e-01,  8.8190e-01,  8.8523e-01,  8.7277e-01,
            8.9276e-01,  8.7140e-01,  8.6526e-01,  8.7749e-01,  8.7013e-01,
            8.7341e-01,  8.6349e-01,  8.6710e-01,  8.4287e-01,  8.4628e-01,
            8.1445e-01,  8.1275e-01],
          [ 8.5160e-01,  8.3046e-01,  8.2026e-01,  8.4808e-01,  8.4257e-01,
            8.5839e-01,  8.5477e-01,  7.9082e-01,  7.4188e-01,  7.6508e-01,
            8.4644e-01,  7.2317e-01,  8.1039e-01,  8.1577e-01,  7.8930e-01,
            8.3785e-01,  8.9691e-01,  8.9537e-01,  8.9253e-01,  8.8361e-01,
            8.9167e-01,  8.8673e-01,  8.4779e-01,  8.4437e-01,  8.5858e-01,
            8.7088e-01,  8.7669e-01,  8.5972e-01,  8.6527e-01,  8.4230e-01,
            8.1452e-01,  8.0508e-01],
          [ 8.1765e-01,  7.7548e-01,  7.8538e-01,  8.0174e-01,  8.1397e-01,
            8.3221e-01,  8.2096e-01,  7.6104e-01,  7.5534e-01,  7.3000e-01,
            9.0330e-01,  7.3818e-01,  7.0280e-01,  8.2708e-01,  8.2845e-01,
            8.4025e-01,  8.6406e-01,  8.6786e-01,  8.2811e-01,  8.2432e-01,
            8.5091e-01,  8.3600e-01,  8.3673e-01,  8.3940e-01,  8.4461e-01,
            8.6287e-01,  8.5782e-01,  8.6146e-01,  8.6407e-01,  8.6840e-01,
            8.4929e-01,  8.3137e-01]],

         [[ 3.1881e-01,  3.1883e-01,  3.1606e-01,  2.9555e-01,  2.9688e-01,
            3.0701e-01,  2.9388e-01,  2.7472e-01,  2.8509e-01,  2.7614e-01,
            2.2124e-01,  1.7706e-01,  1.6073e-01,  1.7777e-01,  2.0760e-01,
            1.9284e-01,  2.2019e-01,  3.1987e-01,  3.3369e-01,  2.6557e-01,
            2.4660e-01,  2.4735e-01,  2.3904e-01,  2.2964e-01,  2.7099e-01,
            3.2631e-01,  3.6601e-01,  3.5664e-01,  3.6219e-01,  3.5649e-01,
            3.4360e-01,  3.4656e-01],
          [ 2.8055e-01,  2.5645e-01,  2.8798e-01,  2.6551e-01,  2.7855e-01,
            3.1014e-01,  2.8369e-01,  2.7288e-01,  2.5485e-01,  2.4117e-01,
            2.3088e-01,  1.7996e-01,  1.8702e-01,  1.9391e-01,  2.2426e-01,
            2.4228e-01,  2.4097e-01,  2.4590e-01,  2.7752e-01,  3.3367e-01,
            2.8037e-01,  2.5608e-01,  2.2371e-01,  2.6314e-01,  2.9958e-01,
            3.4774e-01,  3.1865e-01,  3.1425e-01,  3.5329e-01,  3.3071e-01,
            3.2054e-01,  3.2182e-01],
          [ 2.5591e-01,  2.5477e-01,  2.5230e-01,  2.4352e-01,  2.2362e-01,
            2.2114e-01,  2.2142e-01,  2.2431e-01,  2.3982e-01,  2.4728e-01,
            2.3347e-01,  2.1646e-01,  1.8554e-01,  1.7980e-01,  1.8339e-01,
            1.9861e-01,  2.1646e-01,  3.0134e-01,  3.6508e-01,  5.1659e-01,
            4.9708e-01,  3.4134e-01,  2.0145e-01,  2.1589e-01,  2.4266e-01,
            2.7514e-01,  2.9038e-01,  3.0782e-01,  3.3816e-01,  3.1192e-01,
            3.0150e-01,  2.7373e-01],
          [ 2.5856e-01,  2.6222e-01,  2.6387e-01,  2.7671e-01,  2.7475e-01,
            2.2590e-01,  1.8097e-01,  1.8019e-01,  2.4141e-01,  2.8129e-01,
            2.6208e-01,  2.5116e-01,  2.2527e-01,  2.0586e-01,  2.1020e-01,
            2.5783e-01,  2.8478e-01,  4.4348e-01,  6.5693e-01,  7.2182e-01,
            7.2140e-01,  5.7874e-01,  2.5657e-01,  1.3747e-01,  1.8377e-01,
            2.1646e-01,  2.7918e-01,  3.1309e-01,  3.3324e-01,  3.3753e-01,
            3.0064e-01,  2.9792e-01],
          [ 2.5330e-01,  2.5793e-01,  2.5736e-01,  3.0158e-01,  3.0579e-01,
            2.6886e-01,  1.9850e-01,  1.7254e-01,  2.0087e-01,  2.1667e-01,
            2.3360e-01,  2.3762e-01,  2.3502e-01,  2.2812e-01,  1.9813e-01,
            2.9009e-01,  5.1760e-01,  6.7186e-01,  7.0128e-01,  6.8494e-01,
            7.5265e-01,  7.3437e-01,  5.1191e-01,  2.7099e-01,  1.3645e-01,
            1.6408e-01,  2.4474e-01,  2.8956e-01,  3.0605e-01,  3.7147e-01,
            3.6422e-01,  3.5427e-01],
          [ 2.6756e-01,  2.6187e-01,  2.7300e-01,  2.8712e-01,  2.7002e-01,
            2.0778e-01,  1.5835e-01,  1.6937e-01,  1.6092e-01,  1.5921e-01,
            1.5549e-01,  1.2430e-01,  1.2996e-01,  1.4849e-01,  1.6818e-01,
            2.4364e-01,  5.1155e-01,  7.1895e-01,  5.5537e-01,  4.7911e-01,
            6.8696e-01,  7.4053e-01,  7.1496e-01,  5.2220e-01,  2.2487e-01,
            1.4089e-01,  1.9623e-01,  2.5850e-01,  2.8696e-01,  3.2563e-01,
            3.7271e-01,  3.9024e-01],
          [ 2.8276e-01,  2.6895e-01,  2.7210e-01,  2.4608e-01,  1.7887e-01,
            1.5994e-01,  1.3782e-01,  1.5912e-01,  1.5908e-01,  1.4733e-01,
            1.3199e-01,  1.1823e-01,  1.2706e-01,  1.1531e-01,  2.2645e-01,
            2.6316e-01,  6.4764e-01,  7.6623e-01,  4.4990e-01,  4.5312e-01,
            6.4549e-01,  7.5735e-01,  8.0384e-01,  7.7207e-01,  4.2436e-01,
            1.4617e-01,  1.8578e-01,  2.1950e-01,  2.5810e-01,  2.6566e-01,
            3.3256e-01,  3.7230e-01],
          [ 2.5552e-01,  2.1829e-01,  2.0324e-01,  1.3591e-01,  9.4600e-02,
            1.1515e-01,  1.0743e-01,  1.1301e-01,  1.1891e-01,  9.7398e-02,
            9.6407e-02,  8.9332e-02,  7.8622e-02,  1.1782e-01,  2.2499e-01,
            4.5596e-01,  7.3752e-01,  6.5070e-01,  4.4901e-01,  4.7464e-01,
            6.6155e-01,  7.8610e-01,  7.9423e-01,  7.9961e-01,  5.2893e-01,
            2.0712e-01,  1.8541e-01,  2.1602e-01,  1.9649e-01,  2.1492e-01,
            2.5811e-01,  3.0574e-01],
          [ 2.2855e-01,  1.7168e-01,  1.5139e-01,  1.3519e-01,  8.1113e-02,
            5.1861e-02,  9.9290e-02,  8.2705e-02,  6.8049e-02,  6.3696e-02,
            7.4861e-02,  8.1434e-02,  9.0159e-02,  1.2166e-01,  2.0703e-01,
            5.4467e-01,  6.9087e-01,  5.8870e-01,  5.0108e-01,  5.3850e-01,
            7.9878e-01,  8.9592e-01,  8.6187e-01,  8.3927e-01,  5.2510e-01,
            2.3874e-01,  1.7155e-01,  1.6474e-01,  1.5872e-01,  1.7692e-01,
            1.8765e-01,  2.5531e-01],
          [ 1.8925e-01,  1.6616e-01,  1.6565e-01,  1.2830e-01,  8.6071e-02,
            8.3238e-02,  1.0509e-01,  1.1546e-01,  1.2218e-01,  8.9528e-02,
            8.1740e-02,  7.9252e-02,  8.0513e-02,  9.9057e-02,  2.1334e-01,
            5.1151e-01,  6.3331e-01,  5.7438e-01,  5.7659e-01,  6.5926e-01,
            8.4254e-01,  8.8953e-01,  8.3296e-01,  7.5882e-01,  4.7720e-01,
            2.5542e-01,  1.5853e-01,  1.9072e-01,  1.9672e-01,  1.9362e-01,
            1.2978e-01,  1.5503e-01],
          [ 1.7316e-01,  1.6351e-01,  1.5727e-01,  1.4307e-01,  1.0952e-01,
            1.0837e-01,  1.0628e-01,  1.1304e-01,  1.2323e-01,  1.2032e-01,
            1.0420e-01,  1.0188e-01,  8.6892e-02,  8.5179e-02,  1.6824e-01,
            3.8554e-01,  5.5791e-01,  5.8490e-01,  6.2621e-01,  7.3614e-01,
            8.3107e-01,  8.2926e-01,  7.7496e-01,  6.9866e-01,  4.6361e-01,
            2.6989e-01,  1.7491e-01,  2.3367e-01,  3.0893e-01,  3.5846e-01,
            2.5579e-01,  1.8937e-01],
          [ 1.9516e-01,  1.5239e-01,  1.5537e-01,  1.3916e-01,  1.3048e-01,
            1.1058e-01,  1.0818e-01,  9.6573e-02,  9.7691e-02,  1.3877e-01,
            1.2180e-01,  1.0297e-01,  1.0712e-01,  1.0925e-01,  1.2413e-01,
            2.2842e-01,  4.0813e-01,  4.9927e-01,  5.5725e-01,  6.4235e-01,
            7.3457e-01,  7.5501e-01,  7.1052e-01,  6.1744e-01,  4.2484e-01,
            2.9347e-01,  2.1478e-01,  2.2208e-01,  3.9362e-01,  5.3341e-01,
            5.2819e-01,  3.9485e-01],
          [ 1.9260e-01,  1.7369e-01,  1.4857e-01,  1.4436e-01,  1.4406e-01,
            1.3414e-01,  1.2509e-01,  1.2218e-01,  1.2686e-01,  1.4709e-01,
            1.4770e-01,  1.2900e-01,  1.4069e-01,  1.5649e-01,  1.4475e-01,
            1.7313e-01,  2.6624e-01,  3.5761e-01,  3.8879e-01,  4.2889e-01,
            5.6654e-01,  6.4083e-01,  6.0920e-01,  4.6356e-01,  3.5633e-01,
            3.2910e-01,  2.2263e-01,  2.4711e-01,  4.8956e-01,  7.0170e-01,
            7.3651e-01,  6.1506e-01],
          [ 1.9830e-01,  1.6052e-01,  1.4445e-01,  1.5528e-01,  1.6503e-01,
            1.5225e-01,  1.3514e-01,  1.4555e-01,  1.3552e-01,  1.4999e-01,
            1.5695e-01,  1.4339e-01,  1.5309e-01,  1.6603e-01,  1.6358e-01,
            1.6516e-01,  2.1533e-01,  2.5905e-01,  2.6982e-01,  3.9909e-01,
            4.3980e-01,  4.1794e-01,  4.2426e-01,  3.0264e-01,  2.5551e-01,
            2.3666e-01,  1.8467e-01,  3.8066e-01,  7.4029e-01,  9.3810e-01,
            9.6317e-01,  8.5935e-01],
          [ 1.5899e-01,  1.5230e-01,  1.4707e-01,  1.6641e-01,  1.7252e-01,
            1.6385e-01,  1.3903e-01,  1.3720e-01,  1.5435e-01,  1.3808e-01,
            1.6109e-01,  1.3919e-01,  1.3738e-01,  1.2907e-01,  1.3561e-01,
            1.1564e-01,  1.5530e-01,  1.9507e-01,  2.2962e-01,  3.7970e-01,
            3.6651e-01,  3.4222e-01,  2.6181e-01,  1.6167e-01,  1.4138e-01,
            1.9933e-01,  2.5775e-01,  5.1579e-01,  8.5797e-01,  9.6344e-01,
            9.5666e-01,  9.0788e-01],
          [ 1.8649e-01,  1.7555e-01,  1.7401e-01,  1.8726e-01,  1.8870e-01,
            1.8738e-01,  1.7886e-01,  1.7042e-01,  1.7155e-01,  1.7285e-01,
            1.5603e-01,  1.4539e-01,  1.5337e-01,  1.1873e-01,  1.3571e-01,
            1.1706e-01,  1.3228e-01,  1.5670e-01,  2.4294e-01,  3.4237e-01,
            3.3638e-01,  2.9774e-01,  2.0803e-01,  1.6151e-01,  1.2332e-01,
            2.0550e-01,  3.1154e-01,  5.8839e-01,  8.7701e-01,  9.3814e-01,
            9.0407e-01,  8.5886e-01],
          [ 1.6708e-01,  1.9422e-01,  1.9780e-01,  1.9654e-01,  1.9520e-01,
            2.0857e-01,  2.0064e-01,  1.9319e-01,  1.8452e-01,  1.7593e-01,
            1.4676e-01,  1.6044e-01,  1.4421e-01,  1.2445e-01,  1.3241e-01,
            1.2292e-01,  1.5978e-01,  2.2658e-01,  2.3969e-01,  3.5848e-01,
            3.3002e-01,  2.3668e-01,  2.1572e-01,  1.8679e-01,  1.3898e-01,
            2.1373e-01,  4.1424e-01,  7.1191e-01,  8.8580e-01,  8.9575e-01,
            8.3068e-01,  8.4403e-01],
          [ 2.0132e-01,  2.3696e-01,  2.1636e-01,  2.3049e-01,  2.0870e-01,
            2.1585e-01,  2.0985e-01,  2.0277e-01,  2.0616e-01,  1.9799e-01,
            2.0894e-01,  2.0287e-01,  1.5286e-01,  1.3794e-01,  1.1705e-01,
            8.8338e-02,  1.1923e-01,  2.1270e-01,  2.4567e-01,  3.2614e-01,
            3.4848e-01,  2.6240e-01,  1.8337e-01,  8.9695e-02,  1.3437e-01,
            2.6909e-01,  5.4382e-01,  7.9287e-01,  8.5996e-01,  8.7516e-01,
            8.1369e-01,  8.4560e-01],
          [ 2.0556e-01,  2.2701e-01,  2.2003e-01,  2.6319e-01,  2.1047e-01,
            2.0966e-01,  2.3644e-01,  2.2615e-01,  2.2136e-01,  2.4163e-01,
            2.4637e-01,  2.3065e-01,  1.6854e-01,  1.2187e-01,  8.2677e-02,
            8.2968e-02,  1.2627e-01,  1.5676e-01,  1.8152e-01,  1.9838e-01,
            1.9140e-01,  1.2038e-01,  7.2859e-02,  4.2264e-02,  4.3032e-02,
            1.4082e-01,  4.9821e-01,  7.7403e-01,  8.6714e-01,  8.4525e-01,
            8.2015e-01,  8.4073e-01],
          [ 1.8908e-01,  2.3184e-01,  2.3777e-01,  2.5475e-01,  2.2634e-01,
            2.3290e-01,  2.3826e-01,  2.5268e-01,  2.3712e-01,  2.5715e-01,
            2.5656e-01,  2.4307e-01,  1.7658e-01,  1.1132e-01,  8.9550e-02,
            8.3515e-02,  9.0706e-02,  1.5775e-01,  1.4085e-01,  1.2567e-01,
            5.5585e-02,  2.9067e-02,  4.0553e-02,  6.3800e-02,  6.9899e-02,
            1.1724e-01,  4.0458e-01,  7.6510e-01,  8.7100e-01,  8.5482e-01,
            8.4838e-01,  8.4667e-01],
          [ 2.1781e-01,  2.2770e-01,  2.4041e-01,  2.5648e-01,  2.4231e-01,
            2.3483e-01,  2.7171e-01,  2.7049e-01,  2.6718e-01,  2.4049e-01,
            2.7730e-01,  2.5236e-01,  1.9999e-01,  1.5719e-01,  1.3144e-01,
            1.0988e-01,  1.0350e-01,  1.5634e-01,  7.9004e-02,  4.3069e-02,
            3.0743e-02,  3.6955e-02,  3.4218e-02,  4.8715e-02,  1.1041e-01,
            1.6945e-01,  3.3017e-01,  7.4749e-01,  8.6473e-01,  8.4101e-01,
            8.5898e-01,  7.8080e-01],
          [ 2.1607e-01,  2.5621e-01,  2.8022e-01,  2.5967e-01,  2.5161e-01,
            2.5174e-01,  2.8039e-01,  2.8813e-01,  2.7310e-01,  2.5942e-01,
            2.7193e-01,  2.6686e-01,  2.1842e-01,  1.5549e-01,  9.7254e-02,
            1.1240e-01,  1.5749e-01,  8.4687e-02,  5.9418e-02,  3.5098e-02,
            1.2656e-02, -6.8758e-03,  9.9538e-03,  4.6072e-02,  1.1798e-01,
            1.9814e-01,  3.3308e-01,  7.4615e-01,  8.7774e-01,  8.5692e-01,
            9.0801e-01,  8.5238e-01],
          [ 2.0520e-01,  2.5818e-01,  2.9542e-01,  2.9504e-01,  2.7303e-01,
            2.6480e-01,  2.9037e-01,  3.0789e-01,  2.8257e-01,  2.6683e-01,
            2.7304e-01,  3.1590e-01,  2.4926e-01,  1.4476e-01,  9.2715e-02,
            1.2075e-01,  1.3408e-01,  8.4043e-02,  6.8903e-02,  2.2220e-02,
           -2.6231e-02, -1.3118e-02,  3.8592e-03,  4.7045e-02,  1.7043e-01,
            3.2743e-01,  4.6905e-01,  7.1820e-01,  8.8761e-01,  9.1193e-01,
            9.8106e-01,  9.6307e-01],
          [ 1.6371e-01,  2.0190e-01,  2.5127e-01,  3.3117e-01,  3.0948e-01,
            2.7844e-01,  2.8326e-01,  2.9016e-01,  2.6445e-01,  2.7596e-01,
            3.3659e-01,  3.4740e-01,  2.9975e-01,  2.7658e-01,  1.3731e-01,
            1.1182e-01,  1.4103e-01,  1.0270e-01,  5.6213e-02, -1.5003e-02,
           -4.1534e-03,  2.8053e-02,  2.8230e-02,  7.0403e-02,  3.5993e-01,
            4.9614e-01,  5.1818e-01,  6.9458e-01,  8.8344e-01,  9.5991e-01,
            9.7360e-01,  9.6826e-01],
          [ 1.2828e-01,  1.7695e-01,  2.5191e-01,  3.9972e-01,  3.8915e-01,
            3.1647e-01,  3.3444e-01,  3.5921e-01,  3.6277e-01,  3.9229e-01,
            4.0860e-01,  3.3516e-01,  2.2844e-01,  1.8289e-01,  1.0706e-01,
            1.1722e-01,  1.0992e-01,  1.0211e-01,  6.1751e-03,  3.0995e-03,
            1.6860e-03,  6.5763e-02,  6.0556e-02,  2.0696e-01,  6.4594e-01,
            6.8598e-01,  3.9768e-01,  3.6378e-01,  5.9557e-01,  8.3218e-01,
            9.4051e-01,  8.5240e-01],
          [ 2.8512e-01,  3.3838e-01,  4.6746e-01,  6.1489e-01,  5.7131e-01,
            5.1055e-01,  6.2062e-01,  6.8923e-01,  7.0299e-01,  6.6270e-01,
            5.9855e-01,  4.8200e-01,  3.5494e-01,  2.6229e-01,  2.3827e-01,
            2.1165e-01,  2.1181e-01,  2.4351e-01,  2.1404e-01,  1.0679e-01,
            7.8463e-02,  1.8707e-01,  2.7332e-01,  3.8029e-01,  6.8349e-01,
            7.2380e-01,  4.0024e-01,  1.4708e-01,  1.8304e-01,  4.7343e-01,
            6.9877e-01,  5.4486e-01],
          [ 5.9621e-01,  6.9469e-01,  7.7768e-01,  8.6567e-01,  8.4233e-01,
            8.2830e-01,  8.6009e-01,  8.6130e-01,  8.5809e-01,  8.6409e-01,
            8.3856e-01,  7.7809e-01,  6.8506e-01,  6.6879e-01,  6.0420e-01,
            5.8809e-01,  6.9560e-01,  7.9711e-01,  7.9397e-01,  6.6283e-01,
            6.1789e-01,  5.9569e-01,  6.6816e-01,  6.8714e-01,  7.1861e-01,
            7.2524e-01,  6.1981e-01,  3.4575e-01,  1.9208e-01,  2.0736e-01,
            5.1219e-01,  6.6392e-01],
          [ 8.4025e-01,  9.6844e-01,  9.7019e-01,  9.5529e-01,  9.6308e-01,
            9.3218e-01,  9.2581e-01,  9.0562e-01,  8.8706e-01,  8.9312e-01,
            8.9334e-01,  8.7551e-01,  8.5710e-01,  8.4867e-01,  8.3477e-01,
            8.5218e-01,  9.2609e-01,  9.6398e-01,  9.7131e-01,  9.1275e-01,
            8.9865e-01,  8.6495e-01,  8.4894e-01,  8.7549e-01,  8.7437e-01,
            8.8012e-01,  8.9476e-01,  8.5557e-01,  7.4515e-01,  5.1711e-01,
            5.6178e-01,  7.7953e-01],
          [ 8.9931e-01,  9.8595e-01,  9.9501e-01,  9.3812e-01,  9.6841e-01,
            9.6843e-01,  9.2779e-01,  9.4508e-01,  9.1213e-01,  9.1329e-01,
            9.1428e-01,  8.9591e-01,  8.8202e-01,  8.7886e-01,  8.9459e-01,
            8.6634e-01,  8.5335e-01,  8.9993e-01,  9.2585e-01,  9.0188e-01,
            9.1938e-01,  9.1923e-01,  9.1189e-01,  9.0545e-01,  8.8190e-01,
            8.9090e-01,  9.0803e-01,  8.8007e-01,  8.8495e-01,  8.3955e-01,
            8.1082e-01,  8.6334e-01],
          [ 9.0646e-01,  9.1398e-01,  9.1833e-01,  9.2759e-01,  9.5417e-01,
            9.7105e-01,  9.4343e-01,  8.9964e-01,  8.4173e-01,  8.7222e-01,
            8.6422e-01,  8.3705e-01,  8.9064e-01,  9.0682e-01,  8.9204e-01,
            8.8352e-01,  9.2546e-01,  9.5426e-01,  9.4750e-01,  9.3161e-01,
            9.3984e-01,  9.3279e-01,  9.0373e-01,  9.2330e-01,  9.0314e-01,
            9.1448e-01,  9.1651e-01,  8.9673e-01,  8.9719e-01,  8.8130e-01,
            8.7305e-01,  8.7782e-01],
          [ 9.1212e-01,  8.8272e-01,  8.6853e-01,  8.8832e-01,  8.9035e-01,
            9.0802e-01,  8.8534e-01,  8.1590e-01,  7.8085e-01,  8.0162e-01,
            8.5345e-01,  7.3474e-01,  8.3369e-01,  9.1325e-01,  8.0941e-01,
            8.4786e-01,  9.2764e-01,  9.4156e-01,  9.4198e-01,  9.4322e-01,
            9.4920e-01,  9.2785e-01,  9.0390e-01,  9.0141e-01,  8.9968e-01,
            9.2811e-01,  9.1973e-01,  9.0968e-01,  9.1297e-01,  8.8596e-01,
            8.7901e-01,  8.6487e-01],
          [ 8.3376e-01,  8.1821e-01,  8.1385e-01,  8.4135e-01,  8.5695e-01,
            8.6817e-01,  8.5330e-01,  8.0345e-01,  7.9702e-01,  7.7494e-01,
            8.9401e-01,  7.0756e-01,  7.1605e-01,  9.0892e-01,  8.5997e-01,
            8.7864e-01,  8.7917e-01,  8.8808e-01,  8.7932e-01,  8.8311e-01,
            8.7896e-01,  8.8331e-01,  8.8429e-01,  9.0957e-01,  8.9836e-01,
            9.3316e-01,  9.1528e-01,  8.9897e-01,  9.1753e-01,  9.0549e-01,
            9.0278e-01,  8.8100e-01]]],


        [[[ 3.1516e-01,  3.2504e-01,  3.7350e-01,  3.8108e-01,  3.6318e-01,
            3.6009e-01,  3.3013e-01,  3.5211e-01,  3.7954e-01,  3.9184e-01,
            4.0524e-01,  4.2916e-01,  4.6101e-01,  4.9355e-01,  4.9972e-01,
            5.4097e-01,  5.6797e-01,  5.9030e-01,  5.9173e-01,  5.1812e-01,
            4.8599e-01,  3.9370e-01,  1.3273e-01,  4.6756e-02,  2.4895e-02,
            1.3090e-02,  3.3697e-02,  7.3647e-02,  1.8098e-01,  2.1288e-01,
            2.3654e-01,  2.5545e-01],
          [ 3.0834e-01,  3.2905e-01,  3.7368e-01,  3.7982e-01,  3.5874e-01,
            3.4757e-01,  3.3597e-01,  3.5565e-01,  3.4521e-01,  3.8983e-01,
            4.1203e-01,  4.3988e-01,  4.7131e-01,  4.9727e-01,  5.0905e-01,
            5.5117e-01,  5.7261e-01,  6.0739e-01,  6.0942e-01,  5.9317e-01,
            5.7482e-01,  5.2699e-01,  2.4317e-01,  4.3978e-02,  3.0962e-02,
            2.0707e-02,  3.8626e-02,  5.6134e-02,  1.2629e-01,  1.9807e-01,
            2.4421e-01,  2.4720e-01],
          [ 3.1917e-01,  3.3386e-01,  3.4495e-01,  3.5059e-01,  3.4972e-01,
            3.3401e-01,  3.0954e-01,  2.9197e-01,  2.8214e-01,  2.7918e-01,
            3.2475e-01,  3.5794e-01,  4.0977e-01,  4.7084e-01,  5.0319e-01,
            5.7043e-01,  5.7349e-01,  6.1326e-01,  6.2162e-01,  6.1983e-01,
            5.8967e-01,  5.8288e-01,  4.0817e-01,  1.0184e-01,  5.3509e-02,
            6.0027e-02,  4.0336e-02,  4.4566e-02,  8.8986e-02,  1.5477e-01,
            2.1353e-01,  2.5201e-01],
          [ 3.0651e-01,  3.3187e-01,  3.1912e-01,  3.1541e-01,  3.1084e-01,
            2.5089e-01,  2.4595e-01,  2.1726e-01,  1.9102e-01,  2.0926e-01,
            2.0644e-01,  2.4970e-01,  2.8849e-01,  3.7044e-01,  4.5445e-01,
            5.3356e-01,  6.4211e-01,  6.4539e-01,  6.5443e-01,  6.2538e-01,
            5.6347e-01,  5.5912e-01,  4.7719e-01,  1.2230e-01,  7.6486e-02,
            6.2778e-02,  5.2353e-02,  1.0392e-01,  1.4667e-01,  1.4146e-01,
            1.8281e-01,  2.3041e-01],
          [ 2.3655e-01,  2.1508e-01,  2.1521e-01,  2.5702e-01,  2.6393e-01,
            2.1496e-01,  2.0850e-01,  2.0809e-01,  2.2487e-01,  2.1949e-01,
            2.1526e-01,  2.2627e-01,  2.4571e-01,  2.4304e-01,  2.8672e-01,
            4.1270e-01,  5.9950e-01,  6.9763e-01,  6.7764e-01,  6.2892e-01,
            5.4836e-01,  4.9432e-01,  4.4904e-01,  9.4256e-02,  8.7637e-02,
            1.1420e-01,  1.3361e-01,  1.9589e-01,  1.7306e-01,  1.5766e-01,
            2.1266e-01,  2.8128e-01],
          [ 1.3641e-01,  1.5538e-01,  1.9082e-01,  2.2166e-01,  2.2811e-01,
            2.1607e-01,  2.2334e-01,  2.2499e-01,  2.2368e-01,  2.4099e-01,
            2.3930e-01,  2.4819e-01,  2.3940e-01,  2.2354e-01,  2.2521e-01,
            2.8575e-01,  3.4228e-01,  5.5728e-01,  7.3013e-01,  7.2063e-01,
            6.8958e-01,  5.3041e-01,  3.5735e-01,  1.6404e-01,  1.3557e-01,
            1.6178e-01,  2.1649e-01,  1.7301e-01,  8.9078e-02,  1.1478e-01,
            2.0189e-01,  3.3829e-01],
          [ 1.2520e-01,  1.7446e-01,  2.4510e-01,  2.5071e-01,  2.1375e-01,
            1.9008e-01,  2.2573e-01,  2.3811e-01,  1.9389e-01,  2.2436e-01,
            2.5322e-01,  2.4774e-01,  2.3892e-01,  2.4445e-01,  2.4441e-01,
            2.4703e-01,  2.3876e-01,  2.7860e-01,  5.4294e-01,  8.0207e-01,
            8.3508e-01,  6.5388e-01,  4.0340e-01,  2.3498e-01,  1.6229e-01,
            2.2906e-01,  3.2944e-01,  2.0618e-01,  7.0547e-02,  8.6147e-02,
            1.3288e-01,  3.1177e-01],
          [ 1.3368e-01,  1.9200e-01,  2.6633e-01,  3.0010e-01,  2.1851e-01,
            2.0143e-01,  2.6243e-01,  2.4534e-01,  2.3830e-01,  2.6166e-01,
            2.8764e-01,  2.7096e-01,  2.7863e-01,  2.4545e-01,  2.5037e-01,
            2.5133e-01,  3.0271e-01,  2.9244e-01,  3.0519e-01,  5.1351e-01,
            7.9717e-01,  7.4967e-01,  4.5698e-01,  2.5238e-01,  2.3540e-01,
            2.9475e-01,  3.7882e-01,  2.5169e-01,  1.0168e-01,  7.7648e-02,
            1.3151e-01,  2.4428e-01],
          [ 1.4036e-01,  1.8934e-01,  2.7043e-01,  2.9529e-01,  2.3102e-01,
            2.2568e-01,  2.8386e-01,  2.3537e-01,  2.5031e-01,  2.9527e-01,
            3.0015e-01,  2.9019e-01,  2.9071e-01,  2.7937e-01,  2.4531e-01,
            2.3278e-01,  2.7773e-01,  3.6337e-01,  3.5733e-01,  3.7333e-01,
            5.9498e-01,  7.7953e-01,  5.4350e-01,  2.7184e-01,  3.4649e-01,
            3.7639e-01,  4.9219e-01,  4.1576e-01,  1.1124e-01,  1.0203e-01,
            1.7508e-01,  2.6227e-01],
          [ 1.3350e-01,  1.9721e-01,  2.6171e-01,  3.0468e-01,  2.6269e-01,
            2.3161e-01,  2.2003e-01,  1.9189e-01,  2.2633e-01,  2.3342e-01,
            2.7133e-01,  2.9209e-01,  2.9007e-01,  2.8793e-01,  2.8498e-01,
            2.4326e-01,  2.6036e-01,  3.5127e-01,  4.4453e-01,  4.0170e-01,
            4.5820e-01,  7.7126e-01,  6.9529e-01,  3.1625e-01,  3.7434e-01,
            4.3752e-01,  6.5744e-01,  4.8406e-01,  1.5005e-01,  1.4136e-01,
            2.2147e-01,  3.5715e-01],
          [ 1.2730e-01,  1.8910e-01,  3.1239e-01,  3.1511e-01,  2.8586e-01,
            2.6418e-01,  2.0132e-01,  1.6380e-01,  1.6350e-01,  1.9718e-01,
            2.8450e-01,  2.7882e-01,  2.5607e-01,  2.8685e-01,  2.6324e-01,
            2.6122e-01,  2.3256e-01,  2.8122e-01,  3.8663e-01,  3.9253e-01,
            4.4254e-01,  7.3051e-01,  7.4804e-01,  3.7478e-01,  4.2741e-01,
            5.1312e-01,  6.2175e-01,  5.1799e-01,  1.9142e-01,  1.1811e-01,
            3.6782e-01,  5.2060e-01],
          [ 1.5089e-01,  2.0216e-01,  2.9745e-01,  3.5124e-01,  2.9870e-01,
            2.8202e-01,  1.8447e-01,  1.1926e-01,  1.4114e-01,  1.7936e-01,
            3.3268e-01,  2.3253e-01,  2.2555e-01,  2.7713e-01,  2.6138e-01,
            2.5609e-01,  2.4401e-01,  1.9634e-01,  2.3289e-01,  3.2014e-01,
            4.1724e-01,  6.1387e-01,  7.2112e-01,  5.3646e-01,  5.0683e-01,
            5.7684e-01,  6.0291e-01,  4.9828e-01,  1.2154e-01,  2.0603e-01,
            4.9990e-01,  5.2927e-01],
          [ 1.5718e-01,  1.9176e-01,  3.0763e-01,  3.9150e-01,  3.6235e-01,
            3.0969e-01,  2.3575e-01,  1.6856e-01,  1.6014e-01,  1.7743e-01,
            3.3372e-01,  2.4818e-01,  2.4974e-01,  3.1236e-01,  2.6198e-01,
            2.7160e-01,  2.5094e-01,  1.5417e-01,  1.6192e-01,  2.3862e-01,
            3.9444e-01,  5.8095e-01,  6.6657e-01,  6.2363e-01,  5.5422e-01,
            6.0860e-01,  6.2072e-01,  3.6025e-01,  8.5499e-02,  3.5605e-01,
            5.3916e-01,  4.9862e-01],
          [ 1.8215e-01,  2.0217e-01,  3.4065e-01,  4.2442e-01,  4.0907e-01,
            3.7264e-01,  2.1029e-01,  1.6015e-01,  1.4447e-01,  2.2856e-01,
            3.5350e-01,  2.9430e-01,  2.5997e-01,  2.9642e-01,  2.7912e-01,
            2.8835e-01,  2.4883e-01,  1.6483e-01,  1.6215e-01,  2.2597e-01,
            3.8641e-01,  5.6019e-01,  6.7267e-01,  6.4230e-01,  6.5066e-01,
            6.6492e-01,  5.2859e-01,  2.0141e-01,  8.1664e-02,  2.4238e-01,
            4.8904e-01,  4.9306e-01],
          [ 3.2574e-01,  2.4113e-01,  3.7688e-01,  4.2332e-01,  4.3560e-01,
            3.6552e-01,  1.5390e-01,  1.0418e-01,  1.1151e-01,  2.6945e-01,
            3.5208e-01,  3.0496e-01,  2.6534e-01,  2.8556e-01,  2.7973e-01,
            2.5096e-01,  2.3394e-01,  1.7622e-01,  1.5481e-01,  2.3839e-01,
            3.2802e-01,  5.2730e-01,  6.6918e-01,  6.8137e-01,  7.0183e-01,
            7.3290e-01,  4.9234e-01,  2.0970e-01,  1.4026e-01,  1.2106e-01,
            3.6407e-01,  4.3177e-01],
          [ 4.1056e-01,  2.6002e-01,  3.9158e-01,  4.3299e-01,  4.3196e-01,
            3.3190e-01,  1.2458e-01,  1.0180e-01,  1.0369e-01,  2.5261e-01,
            3.6529e-01,  3.3850e-01,  2.9382e-01,  2.8859e-01,  2.8659e-01,
            2.7148e-01,  2.0676e-01,  1.7284e-01,  1.6684e-01,  2.2617e-01,
            2.9855e-01,  4.9207e-01,  6.6184e-01,  7.0481e-01,  7.0597e-01,
            7.7688e-01,  6.0003e-01,  2.4326e-01,  1.7148e-01,  1.2963e-01,
            2.7700e-01,  4.2733e-01],
          [ 4.0823e-01,  2.9171e-01,  4.0274e-01,  4.2346e-01,  4.1984e-01,
            3.0704e-01,  1.4191e-01,  1.3042e-01,  1.5130e-01,  2.6401e-01,
            3.4462e-01,  3.3587e-01,  3.2997e-01,  2.9538e-01,  2.8822e-01,
            2.6841e-01,  1.8396e-01,  1.7371e-01,  1.8434e-01,  2.3009e-01,
            2.7351e-01,  4.3543e-01,  6.2467e-01,  7.1518e-01,  7.0411e-01,
            7.3442e-01,  7.0707e-01,  4.3752e-01,  3.0203e-01,  1.5701e-01,
            2.8076e-01,  4.4928e-01],
          [ 4.7773e-01,  4.2235e-01,  4.3178e-01,  4.4100e-01,  4.2027e-01,
            2.8858e-01,  1.8997e-01,  1.3289e-01,  1.4866e-01,  2.7359e-01,
            3.1718e-01,  3.3084e-01,  3.1441e-01,  2.9869e-01,  2.9538e-01,
            2.5469e-01,  1.8186e-01,  1.7145e-01,  2.1174e-01,  2.2694e-01,
            2.6766e-01,  3.6729e-01,  5.9245e-01,  6.9230e-01,  7.1002e-01,
            7.1199e-01,  6.6717e-01,  5.3088e-01,  4.0494e-01,  3.1476e-01,
            4.8581e-01,  5.8275e-01],
          [ 4.7866e-01,  4.9580e-01,  4.5604e-01,  4.4431e-01,  3.8872e-01,
            2.0277e-01,  9.5067e-02,  9.2300e-02,  1.9709e-01,  3.3195e-01,
            3.8078e-01,  3.5676e-01,  3.2395e-01,  3.1346e-01,  2.9576e-01,
            2.6457e-01,  1.9250e-01,  1.8357e-01,  2.5036e-01,  2.7242e-01,
            3.0174e-01,  3.2551e-01,  5.3128e-01,  6.9626e-01,  7.1774e-01,
            6.7627e-01,  5.4960e-01,  4.9889e-01,  5.4794e-01,  5.8320e-01,
            6.3270e-01,  6.5581e-01],
          [ 4.5866e-01,  4.5412e-01,  4.8170e-01,  4.2437e-01,  2.5517e-01,
            9.3244e-02,  7.5495e-02,  1.4843e-01,  2.9956e-01,  4.0043e-01,
            4.2685e-01,  3.8346e-01,  3.1326e-01,  3.0226e-01,  2.8867e-01,
            2.5878e-01,  1.9239e-01,  2.1231e-01,  2.7240e-01,  3.3466e-01,
            3.8134e-01,  3.1391e-01,  4.5787e-01,  6.1995e-01,  6.6621e-01,
            6.0666e-01,  4.5773e-01,  5.0655e-01,  6.2695e-01,  6.3965e-01,
            6.5048e-01,  6.4848e-01],
          [ 4.7646e-01,  4.4330e-01,  4.3115e-01,  3.5593e-01,  1.6965e-01,
            9.6480e-02,  1.0686e-01,  2.0665e-01,  3.0322e-01,  3.9633e-01,
            3.7369e-01,  3.3859e-01,  3.1339e-01,  3.0498e-01,  2.8380e-01,
            2.4085e-01,  2.0138e-01,  2.0542e-01,  2.5398e-01,  3.4297e-01,
            4.1282e-01,  3.3248e-01,  4.6582e-01,  6.0753e-01,  6.4037e-01,
            4.7171e-01,  3.2058e-01,  5.2984e-01,  6.4409e-01,  6.8057e-01,
            6.8541e-01,  6.7317e-01],
          [ 4.6442e-01,  4.7722e-01,  3.9252e-01,  3.0196e-01,  1.1326e-01,
            6.6751e-02,  1.1453e-01,  1.5996e-01,  3.1075e-01,  3.6636e-01,
            2.9038e-01,  2.9655e-01,  2.9717e-01,  2.8165e-01,  2.8000e-01,
            2.4874e-01,  2.1609e-01,  2.3321e-01,  2.6927e-01,  3.2252e-01,
            3.8999e-01,  3.6377e-01,  5.0475e-01,  5.5602e-01,  4.9387e-01,
            3.3542e-01,  3.6735e-01,  5.7619e-01,  6.8005e-01,  6.7532e-01,
            6.7986e-01,  6.7263e-01],
          [ 4.8419e-01,  4.8957e-01,  4.1237e-01,  2.5129e-01,  9.9750e-02,
            8.8767e-02,  9.8012e-02,  1.7765e-01,  3.2036e-01,  3.1768e-01,
            2.5899e-01,  2.8382e-01,  2.9079e-01,  2.8555e-01,  2.6703e-01,
            2.3331e-01,  2.3882e-01,  2.6466e-01,  2.8868e-01,  3.1940e-01,
            3.6233e-01,  3.9245e-01,  4.0111e-01,  3.7680e-01,  3.1360e-01,
            3.0462e-01,  4.9015e-01,  6.4143e-01,  6.9282e-01,  6.9112e-01,
            6.8774e-01,  6.7577e-01],
          [ 5.4441e-01,  5.4199e-01,  4.1690e-01,  2.4700e-01,  1.2042e-01,
            1.3847e-01,  1.9143e-01,  2.7599e-01,  3.2748e-01,  2.7444e-01,
            2.5115e-01,  2.5437e-01,  2.5395e-01,  2.6219e-01,  2.6323e-01,
            2.2080e-01,  2.3635e-01,  2.5246e-01,  2.7110e-01,  2.9443e-01,
            3.0358e-01,  3.2631e-01,  2.7800e-01,  2.4235e-01,  1.8907e-01,
            3.0577e-01,  5.5271e-01,  6.6166e-01,  7.0214e-01,  6.9723e-01,
            7.0459e-01,  6.9305e-01],
          [ 5.5866e-01,  5.4245e-01,  4.2994e-01,  3.1022e-01,  2.0421e-01,
            1.9998e-01,  2.2825e-01,  2.8604e-01,  3.0643e-01,  2.2974e-01,
            2.2311e-01,  2.2840e-01,  2.1792e-01,  2.1969e-01,  2.2297e-01,
            2.2457e-01,  2.3514e-01,  2.5178e-01,  2.6138e-01,  2.4661e-01,
            2.6441e-01,  2.9245e-01,  1.9150e-01,  1.8173e-01,  2.3578e-01,
            4.2344e-01,  5.8814e-01,  6.5719e-01,  6.9551e-01,  6.9112e-01,
            7.2450e-01,  7.0848e-01],
          [ 4.4799e-01,  4.7243e-01,  4.2262e-01,  3.4094e-01,  2.7543e-01,
            2.6663e-01,  1.5579e-01,  2.7150e-01,  3.0335e-01,  2.0076e-01,
            1.7251e-01,  1.6196e-01,  1.6823e-01,  1.8977e-01,  2.2555e-01,
            2.1622e-01,  2.2878e-01,  2.3349e-01,  2.3772e-01,  2.5243e-01,
            2.5693e-01,  2.0646e-01,  1.3623e-01,  2.7219e-01,  4.3213e-01,
            5.4546e-01,  6.2014e-01,  6.5478e-01,  6.7710e-01,  6.9428e-01,
            7.1152e-01,  7.1158e-01],
          [ 4.0976e-01,  4.4439e-01,  3.7094e-01,  2.7369e-01,  3.2201e-01,
            2.7293e-01,  2.7781e-01,  3.5817e-01,  3.4014e-01,  2.6158e-01,
            1.9275e-01,  1.3989e-01,  1.4218e-01,  1.6154e-01,  1.9234e-01,
            2.2034e-01,  2.2595e-01,  2.3518e-01,  2.4856e-01,  2.4782e-01,
            2.0087e-01,  1.4726e-01,  2.3613e-01,  4.0522e-01,  5.2815e-01,
            5.7743e-01,  5.9169e-01,  6.2935e-01,  6.6963e-01,  7.1672e-01,
            7.1569e-01,  7.2574e-01],
          [ 3.9662e-01,  3.7332e-01,  3.2314e-01,  4.0007e-01,  3.8243e-01,
            3.4336e-01,  5.8563e-01,  5.9079e-01,  5.3754e-01,  5.1091e-01,
            4.8979e-01,  4.3712e-01,  3.7266e-01,  3.5066e-01,  3.5925e-01,
            3.3618e-01,  3.1993e-01,  3.1527e-01,  3.4453e-01,  3.5722e-01,
            3.6549e-01,  3.7096e-01,  4.4764e-01,  5.4101e-01,  6.1434e-01,
            6.1018e-01,  5.9711e-01,  6.3923e-01,  6.8005e-01,  7.0241e-01,
            6.9461e-01,  7.2172e-01],
          [ 2.5934e-01,  3.4504e-01,  5.0593e-01,  5.5596e-01,  4.9428e-01,
            4.5413e-01,  6.3603e-01,  6.7532e-01,  6.2658e-01,  6.0710e-01,
            6.2925e-01,  6.2356e-01,  6.0512e-01,  5.5916e-01,  5.7212e-01,
            5.3001e-01,  5.4383e-01,  5.3561e-01,  5.4762e-01,  5.6938e-01,
            5.7934e-01,  5.6866e-01,  6.1092e-01,  6.3775e-01,  6.6846e-01,
            6.2543e-01,  6.1698e-01,  6.3060e-01,  6.6458e-01,  6.8905e-01,
            6.9684e-01,  7.1253e-01],
          [ 3.0964e-01,  3.7536e-01,  3.8433e-01,  3.7494e-01,  4.0103e-01,
            4.4750e-01,  6.0821e-01,  6.1853e-01,  6.1912e-01,  6.0657e-01,
            5.9688e-01,  5.8318e-01,  5.9349e-01,  5.9910e-01,  5.7586e-01,
            5.8245e-01,  5.9944e-01,  5.7978e-01,  5.7589e-01,  6.0279e-01,
            6.2781e-01,  6.3559e-01,  6.4676e-01,  6.5590e-01,  6.5875e-01,
            6.6309e-01,  6.3975e-01,  6.3515e-01,  6.6134e-01,  6.8581e-01,
            7.0066e-01,  7.0530e-01],
          [ 3.4274e-01,  3.2292e-01,  3.1838e-01,  3.5988e-01,  4.0366e-01,
            4.0936e-01,  5.1188e-01,  5.7165e-01,  5.6706e-01,  5.7482e-01,
            5.8134e-01,  5.5842e-01,  5.7234e-01,  5.7910e-01,  5.7381e-01,
            5.8832e-01,  6.2380e-01,  6.1776e-01,  6.3073e-01,  6.3807e-01,
            6.5017e-01,  6.6848e-01,  6.7040e-01,  6.8003e-01,  6.7792e-01,
            6.6377e-01,  6.7352e-01,  6.6434e-01,  6.8282e-01,  6.8641e-01,
            6.9373e-01,  7.0298e-01],
          [ 3.4189e-01,  3.4141e-01,  3.7213e-01,  3.8974e-01,  3.6317e-01,
            4.2628e-01,  4.9270e-01,  5.2963e-01,  5.3252e-01,  5.3169e-01,
            5.4447e-01,  5.4842e-01,  5.2522e-01,  5.6148e-01,  5.7412e-01,
            5.8246e-01,  6.2875e-01,  6.2054e-01,  6.3026e-01,  6.4537e-01,
            6.5139e-01,  6.6023e-01,  6.6674e-01,  6.7942e-01,  6.7730e-01,
            6.6615e-01,  6.6156e-01,  6.6863e-01,  6.8138e-01,  6.8051e-01,
            6.9205e-01,  6.7735e-01]],

         [[ 3.3718e-01,  3.4385e-01,  3.7870e-01,  4.0066e-01,  3.8814e-01,
            3.6658e-01,  3.6145e-01,  3.7931e-01,  3.9253e-01,  4.0740e-01,
            4.3435e-01,  4.4632e-01,  4.8304e-01,  5.1406e-01,  5.2658e-01,
            5.4911e-01,  5.6566e-01,  5.7827e-01,  5.8256e-01,  5.2102e-01,
            5.0315e-01,  4.0486e-01,  1.1151e-01,  3.0880e-02,  1.4001e-02,
            3.7518e-03,  2.4592e-02,  6.1130e-02,  1.7711e-01,  2.2473e-01,
            2.6391e-01,  2.6449e-01],
          [ 3.5153e-01,  3.5303e-01,  3.8312e-01,  3.7941e-01,  3.7991e-01,
            3.5631e-01,  3.4650e-01,  3.5383e-01,  3.5616e-01,  3.7392e-01,
            4.0357e-01,  4.3931e-01,  4.6525e-01,  5.1673e-01,  5.3482e-01,
            5.6889e-01,  6.0409e-01,  6.1264e-01,  6.2614e-01,  6.1109e-01,
            5.9567e-01,  5.0688e-01,  2.1091e-01,  1.9582e-02,  1.8707e-02,
            1.9262e-02,  2.4420e-02,  5.4633e-02,  1.3633e-01,  2.4420e-01,
            2.8308e-01,  2.7615e-01],
          [ 3.1631e-01,  3.3970e-01,  3.3652e-01,  3.5722e-01,  3.5845e-01,
            3.3388e-01,  3.2588e-01,  3.0721e-01,  2.8893e-01,  2.9615e-01,
            3.2479e-01,  3.7044e-01,  4.1399e-01,  4.8803e-01,  5.1784e-01,
            5.8044e-01,  5.8740e-01,  6.1412e-01,  6.2732e-01,  6.1036e-01,
            6.1153e-01,  5.7614e-01,  4.1338e-01,  8.0905e-02,  4.6123e-02,
            4.1671e-02,  3.2755e-02,  4.4717e-02,  8.5602e-02,  1.7080e-01,
            2.5993e-01,  2.6965e-01],
          [ 3.2175e-01,  3.3822e-01,  3.2653e-01,  3.0842e-01,  3.1760e-01,
            2.4859e-01,  2.1219e-01,  1.8391e-01,  1.5986e-01,  1.5960e-01,
            1.8303e-01,  2.3597e-01,  2.7427e-01,  3.6151e-01,  4.3480e-01,
            5.3569e-01,  6.3812e-01,  6.8573e-01,  6.4551e-01,  6.1787e-01,
            5.8309e-01,  5.4286e-01,  4.6957e-01,  1.2524e-01,  5.8141e-02,
            5.1482e-02,  4.6689e-02,  8.7716e-02,  1.1741e-01,  1.4475e-01,
            2.1481e-01,  2.3030e-01],
          [ 2.4134e-01,  2.2831e-01,  2.2499e-01,  2.5439e-01,  2.5204e-01,
            1.6278e-01,  1.4090e-01,  1.3402e-01,  1.3483e-01,  1.3899e-01,
            1.5319e-01,  1.6031e-01,  1.7390e-01,  1.8430e-01,  2.4576e-01,
            3.7632e-01,  5.9294e-01,  7.2001e-01,  6.4641e-01,  6.1141e-01,
            5.4389e-01,  4.9357e-01,  4.1916e-01,  7.0451e-02,  4.9004e-02,
            6.6046e-02,  8.4312e-02,  1.5256e-01,  1.4549e-01,  1.4566e-01,
            2.0724e-01,  2.7608e-01],
          [ 1.1154e-01,  1.4491e-01,  1.7346e-01,  1.8907e-01,  1.9010e-01,
            1.6048e-01,  1.4369e-01,  1.4402e-01,  1.4384e-01,  1.4420e-01,
            1.4684e-01,  1.3601e-01,  1.5053e-01,  1.5038e-01,  1.5146e-01,
            1.9781e-01,  2.8831e-01,  5.6541e-01,  7.3337e-01,  7.1123e-01,
            6.4150e-01,  5.1270e-01,  3.2928e-01,  1.3666e-01,  6.9509e-02,
            9.0609e-02,  1.7262e-01,  1.4823e-01,  7.4922e-02,  9.1282e-02,
            1.8772e-01,  3.2355e-01],
          [ 8.9103e-02,  1.1802e-01,  1.9325e-01,  1.9859e-01,  1.3172e-01,
            1.4166e-01,  1.5744e-01,  1.3216e-01,  1.2538e-01,  1.2609e-01,
            1.4413e-01,  1.6447e-01,  1.4472e-01,  1.5545e-01,  1.4998e-01,
            1.6039e-01,  1.7156e-01,  2.5908e-01,  5.2866e-01,  7.6983e-01,
            7.9201e-01,  6.2412e-01,  3.5645e-01,  2.0892e-01,  1.0535e-01,
            1.5111e-01,  2.7304e-01,  2.0373e-01,  8.6802e-02,  6.0690e-02,
            1.3713e-01,  3.0153e-01],
          [ 8.2488e-02,  1.2975e-01,  2.2134e-01,  2.2825e-01,  1.2224e-01,
            1.2776e-01,  1.8683e-01,  1.6928e-01,  1.5850e-01,  1.6591e-01,
            1.7771e-01,  1.7238e-01,  1.6069e-01,  1.4924e-01,  1.4378e-01,
            1.2580e-01,  1.8470e-01,  2.3704e-01,  2.5119e-01,  5.0134e-01,
            7.4309e-01,  7.0888e-01,  3.9876e-01,  1.8227e-01,  1.5898e-01,
            2.2950e-01,  3.2755e-01,  2.3087e-01,  7.2695e-02,  6.6527e-02,
            1.2692e-01,  2.2610e-01],
          [ 6.3526e-02,  1.2325e-01,  2.0033e-01,  2.2025e-01,  1.4129e-01,
            1.2964e-01,  1.8813e-01,  1.6328e-01,  1.7653e-01,  1.8016e-01,
            1.8596e-01,  1.8387e-01,  1.8875e-01,  1.7558e-01,  1.5718e-01,
            1.3639e-01,  1.3856e-01,  2.6263e-01,  2.8252e-01,  3.2420e-01,
            5.2870e-01,  7.3788e-01,  4.8852e-01,  1.7219e-01,  2.5703e-01,
            3.0780e-01,  4.6070e-01,  3.4619e-01,  8.1399e-02,  8.4836e-02,
            1.4157e-01,  2.4614e-01],
          [ 5.7447e-02,  1.4477e-01,  1.8392e-01,  2.0239e-01,  1.6588e-01,
            1.4952e-01,  1.5150e-01,  1.2161e-01,  1.4145e-01,  1.3267e-01,
            1.5343e-01,  1.9154e-01,  1.7918e-01,  1.9871e-01,  1.8748e-01,
            1.5896e-01,  1.3082e-01,  2.1598e-01,  3.5812e-01,  3.3907e-01,
            3.9255e-01,  7.5258e-01,  6.5954e-01,  2.3748e-01,  2.8817e-01,
            3.6132e-01,  6.0804e-01,  4.4521e-01,  1.2582e-01,  1.1246e-01,
            1.7723e-01,  3.4220e-01],
          [ 5.1034e-02,  1.2261e-01,  2.0083e-01,  2.0931e-01,  1.7814e-01,
            1.6406e-01,  1.1597e-01,  1.0108e-01,  1.2064e-01,  1.1243e-01,
            1.5400e-01,  1.5757e-01,  1.4569e-01,  1.9003e-01,  1.7391e-01,
            1.6780e-01,  1.3888e-01,  1.7731e-01,  2.8730e-01,  3.0492e-01,
            3.8587e-01,  6.8740e-01,  7.3361e-01,  3.1545e-01,  3.5982e-01,
            4.5003e-01,  5.7841e-01,  4.9392e-01,  1.7911e-01,  7.9845e-02,
            3.3868e-01,  4.8696e-01],
          [ 6.8787e-02,  1.3021e-01,  2.3057e-01,  2.4696e-01,  1.9151e-01,
            1.7868e-01,  1.0859e-01,  6.7947e-02,  9.1913e-02,  1.1285e-01,
            1.8527e-01,  1.1816e-01,  1.2856e-01,  1.8805e-01,  1.7941e-01,
            1.8029e-01,  1.5409e-01,  1.0480e-01,  1.4453e-01,  2.4186e-01,
            3.4147e-01,  5.5980e-01,  6.9630e-01,  4.7692e-01,  4.2061e-01,
            4.9720e-01,  5.6585e-01,  4.6957e-01,  1.0127e-01,  1.9600e-01,
            4.9113e-01,  5.1608e-01],
          [ 8.3221e-02,  1.1411e-01,  2.1877e-01,  2.9113e-01,  2.2538e-01,
            2.0194e-01,  1.5134e-01,  1.2297e-01,  1.0815e-01,  9.3477e-02,
            2.0043e-01,  1.4077e-01,  1.6611e-01,  2.0366e-01,  1.7644e-01,
            1.8128e-01,  1.4792e-01,  8.3140e-02,  6.2383e-02,  1.6834e-01,
            3.2644e-01,  5.1296e-01,  6.2214e-01,  5.4719e-01,  4.6879e-01,
            5.3684e-01,  6.0600e-01,  3.3054e-01,  5.5090e-02,  3.3599e-01,
            5.4314e-01,  4.9133e-01],
          [ 1.2326e-01,  1.2064e-01,  2.4582e-01,  3.2843e-01,  2.8734e-01,
            2.5538e-01,  1.5103e-01,  9.9794e-02,  8.9346e-02,  1.4219e-01,
            2.1614e-01,  1.7065e-01,  1.6850e-01,  1.9960e-01,  1.7701e-01,
            1.9695e-01,  1.4677e-01,  9.4422e-02,  8.5486e-02,  1.3814e-01,
            2.9112e-01,  4.9996e-01,  6.1429e-01,  5.9225e-01,  5.7429e-01,
            5.9289e-01,  4.7288e-01,  1.8672e-01,  3.9929e-02,  2.0560e-01,
            4.5628e-01,  4.7830e-01],
          [ 2.7985e-01,  1.6068e-01,  2.7838e-01,  3.3107e-01,  3.1635e-01,
            2.5152e-01,  9.8077e-02,  7.0686e-02,  8.0387e-02,  1.7998e-01,
            2.2853e-01,  1.6336e-01,  1.4544e-01,  1.5699e-01,  1.8503e-01,
            1.6337e-01,  1.4397e-01,  9.1830e-02,  8.8294e-02,  1.2822e-01,
            2.4390e-01,  4.5721e-01,  6.3063e-01,  6.1990e-01,  6.2158e-01,
            6.6012e-01,  4.4170e-01,  1.6890e-01,  8.3666e-02,  7.7452e-02,
            3.2496e-01,  4.1042e-01],
          [ 3.5407e-01,  1.7364e-01,  3.0602e-01,  3.2884e-01,  3.3352e-01,
            2.1903e-01,  7.4034e-02,  6.4849e-02,  7.7922e-02,  1.8735e-01,
            2.1992e-01,  2.1315e-01,  1.7359e-01,  1.6344e-01,  1.6588e-01,
            1.4841e-01,  1.1107e-01,  9.0681e-02,  8.6037e-02,  1.4246e-01,
            2.1732e-01,  4.2450e-01,  5.9535e-01,  6.2942e-01,  6.4453e-01,
            6.9788e-01,  5.2507e-01,  2.0623e-01,  1.4539e-01,  9.9031e-02,
            2.4741e-01,  3.9224e-01],
          [ 3.2693e-01,  2.0176e-01,  3.0350e-01,  3.3425e-01,  3.1764e-01,
            1.9511e-01,  9.1823e-02,  7.7113e-02,  1.1635e-01,  1.8254e-01,
            2.2668e-01,  2.0956e-01,  1.9938e-01,  1.7038e-01,  1.6042e-01,
            1.4401e-01,  8.9467e-02,  8.9015e-02,  1.0317e-01,  1.4052e-01,
            2.0125e-01,  3.7646e-01,  5.6520e-01,  6.5018e-01,  6.6276e-01,
            6.5869e-01,  6.4337e-01,  3.9079e-01,  2.5582e-01,  1.2794e-01,
            2.5238e-01,  4.2015e-01],
          [ 3.7003e-01,  3.2040e-01,  3.4194e-01,  3.2734e-01,  3.1333e-01,
            2.1004e-01,  1.3309e-01,  7.8949e-02,  9.2563e-02,  1.7133e-01,
            2.0419e-01,  1.9510e-01,  1.7318e-01,  1.7522e-01,  1.6508e-01,
            1.4196e-01,  9.0725e-02,  1.0166e-01,  1.2205e-01,  1.5875e-01,
            1.8775e-01,  2.9939e-01,  5.2580e-01,  6.2454e-01,  6.5852e-01,
            6.4091e-01,  6.1248e-01,  4.6825e-01,  3.6590e-01,  2.7052e-01,
            4.4328e-01,  5.4833e-01],
          [ 3.8284e-01,  3.7649e-01,  3.5781e-01,  3.4034e-01,  2.8572e-01,
            1.3516e-01,  3.2751e-02,  3.9138e-02,  1.3136e-01,  2.3156e-01,
            2.5727e-01,  2.3117e-01,  1.8976e-01,  1.8379e-01,  1.7295e-01,
            1.4382e-01,  1.0497e-01,  1.0018e-01,  1.5704e-01,  1.9337e-01,
            2.3700e-01,  2.5114e-01,  4.4908e-01,  6.4779e-01,  6.6383e-01,
            6.2402e-01,  5.0244e-01,  4.5656e-01,  5.1133e-01,  5.4136e-01,
            6.0414e-01,  6.1986e-01],
          [ 3.5683e-01,  3.5404e-01,  3.6668e-01,  3.1386e-01,  1.8944e-01,
            4.3408e-02,  2.8622e-02,  9.9934e-02,  2.1749e-01,  2.9753e-01,
            3.1354e-01,  2.4635e-01,  2.0037e-01,  1.7867e-01,  1.6055e-01,
            1.3921e-01,  1.1492e-01,  1.2730e-01,  1.6802e-01,  2.4998e-01,
            2.9586e-01,  2.3280e-01,  3.7633e-01,  5.5836e-01,  6.1783e-01,
            5.3299e-01,  3.9190e-01,  4.7576e-01,  6.0750e-01,  6.0475e-01,
            6.3071e-01,  6.2000e-01],
          [ 4.0489e-01,  3.5252e-01,  3.2316e-01,  2.6742e-01,  1.1405e-01,
            4.7981e-02,  5.6622e-02,  1.3925e-01,  2.1607e-01,  2.9941e-01,
            2.6553e-01,  2.1697e-01,  1.8740e-01,  1.6763e-01,  1.5603e-01,
            1.3395e-01,  9.4572e-02,  1.1468e-01,  1.6703e-01,  2.7196e-01,
            3.2813e-01,  2.5877e-01,  3.9389e-01,  5.4187e-01,  5.9467e-01,
            4.0362e-01,  2.9225e-01,  5.1871e-01,  6.2999e-01,  6.5429e-01,
            6.6576e-01,  6.5651e-01],
          [ 3.6910e-01,  3.8051e-01,  3.0476e-01,  2.3250e-01,  9.9173e-02,
            4.8253e-02,  6.6212e-02,  9.4963e-02,  2.2902e-01,  2.7050e-01,
            1.9014e-01,  1.7902e-01,  1.7374e-01,  1.6263e-01,  1.4939e-01,
            1.4065e-01,  1.2293e-01,  1.2377e-01,  1.7033e-01,  2.4030e-01,
            3.1887e-01,  2.7591e-01,  4.1838e-01,  5.0355e-01,  4.3558e-01,
            2.7806e-01,  3.3926e-01,  5.7556e-01,  6.6320e-01,  6.7330e-01,
            6.6628e-01,  6.5797e-01],
          [ 3.9212e-01,  3.8607e-01,  3.2539e-01,  2.0850e-01,  8.4758e-02,
            6.8386e-02,  7.5668e-02,  1.2457e-01,  2.2983e-01,  2.1465e-01,
            1.5544e-01,  1.6249e-01,  1.7199e-01,  1.5934e-01,  1.5415e-01,
            1.2648e-01,  1.2949e-01,  1.4931e-01,  1.8476e-01,  2.2312e-01,
            2.7502e-01,  2.8428e-01,  3.3655e-01,  3.1403e-01,  2.6458e-01,
            2.6921e-01,  4.7363e-01,  6.2026e-01,  6.8025e-01,  6.7540e-01,
            6.6274e-01,  6.7107e-01],
          [ 4.5579e-01,  4.6184e-01,  3.5485e-01,  2.2151e-01,  1.0559e-01,
            1.1557e-01,  1.4072e-01,  2.1653e-01,  2.6107e-01,  1.8195e-01,
            1.4519e-01,  1.4374e-01,  1.4779e-01,  1.4137e-01,  1.2438e-01,
            1.1919e-01,  1.1757e-01,  1.3580e-01,  1.6166e-01,  1.8893e-01,
            2.2196e-01,  2.6263e-01,  2.0351e-01,  1.8145e-01,  1.4810e-01,
            2.8770e-01,  5.3271e-01,  6.4173e-01,  6.8824e-01,  6.8939e-01,
            6.8926e-01,  6.8048e-01],
          [ 4.8641e-01,  4.6714e-01,  3.6772e-01,  2.4837e-01,  1.3237e-01,
            1.4095e-01,  1.7710e-01,  2.4182e-01,  2.4249e-01,  1.5797e-01,
            1.1264e-01,  9.1559e-02,  1.0062e-01,  1.1834e-01,  1.0544e-01,
            1.0912e-01,  1.2573e-01,  1.3975e-01,  1.5053e-01,  1.3653e-01,
            1.8110e-01,  2.2322e-01,  1.4750e-01,  1.4842e-01,  2.2571e-01,
            4.1962e-01,  5.7633e-01,  6.3325e-01,  6.7679e-01,  7.0450e-01,
            6.9848e-01,  6.9157e-01],
          [ 3.9439e-01,  4.0974e-01,  3.6226e-01,  2.7865e-01,  2.2892e-01,
            2.1450e-01,  1.1261e-01,  2.2140e-01,  2.4452e-01,  1.0823e-01,
            7.8812e-02,  5.4522e-02,  6.7543e-02,  9.5915e-02,  1.0917e-01,
            1.1542e-01,  1.1705e-01,  1.3385e-01,  1.4317e-01,  1.4155e-01,
            1.6576e-01,  1.5228e-01,  1.0143e-01,  2.5240e-01,  4.4275e-01,
            5.3703e-01,  6.1252e-01,  6.5239e-01,  6.6948e-01,  6.8591e-01,
            7.0301e-01,  6.9558e-01],
          [ 3.1961e-01,  4.0690e-01,  3.2325e-01,  2.2074e-01,  2.6762e-01,
            2.1430e-01,  2.4985e-01,  3.4546e-01,  3.1880e-01,  2.0577e-01,
            1.0726e-01,  7.2066e-02,  6.8320e-02,  8.8374e-02,  1.2501e-01,
            1.3078e-01,  1.4326e-01,  1.3564e-01,  1.4496e-01,  1.6729e-01,
            1.3563e-01,  9.6047e-02,  1.9905e-01,  3.9399e-01,  5.1314e-01,
            5.7041e-01,  6.0458e-01,  6.0625e-01,  6.7723e-01,  6.8582e-01,
            6.9178e-01,  7.1203e-01],
          [ 2.5102e-01,  3.0091e-01,  3.0585e-01,  3.7140e-01,  3.1154e-01,
            2.7065e-01,  5.5589e-01,  5.9993e-01,  5.3625e-01,  4.7565e-01,
            4.2483e-01,  3.8972e-01,  3.2363e-01,  3.0122e-01,  3.0756e-01,
            2.9526e-01,  2.8714e-01,  2.6102e-01,  2.8073e-01,  3.1296e-01,
            3.3326e-01,  3.4926e-01,  3.9783e-01,  5.4261e-01,  6.0088e-01,
            6.0028e-01,  5.9090e-01,  6.1990e-01,  6.5362e-01,  6.8283e-01,
            6.7371e-01,  6.8766e-01],
          [ 1.6130e-01,  2.8605e-01,  4.7399e-01,  4.7739e-01,  3.9841e-01,
            4.0282e-01,  6.2647e-01,  6.5671e-01,  6.1838e-01,  5.9748e-01,
            5.9626e-01,  6.0051e-01,  5.6489e-01,  5.5325e-01,  5.3358e-01,
            5.0938e-01,  5.1993e-01,  5.0261e-01,  5.1468e-01,  5.4145e-01,
            5.4061e-01,  5.5319e-01,  5.8694e-01,  6.3352e-01,  6.4038e-01,
            6.1378e-01,  5.9072e-01,  6.0880e-01,  6.2329e-01,  6.7223e-01,
            6.6657e-01,  6.6611e-01],
          [ 2.8073e-01,  3.4151e-01,  3.3088e-01,  3.2986e-01,  3.0637e-01,
            3.8124e-01,  5.7404e-01,  5.9228e-01,  5.9565e-01,  5.9031e-01,
            5.6478e-01,  5.6431e-01,  5.5299e-01,  5.6494e-01,  5.4347e-01,
            5.5340e-01,  5.5420e-01,  5.5321e-01,  5.4547e-01,  5.6623e-01,
            5.9664e-01,  6.1827e-01,  6.2837e-01,  6.5365e-01,  6.4286e-01,
            6.3345e-01,  6.2339e-01,  6.0540e-01,  6.3446e-01,  6.5376e-01,
            6.6504e-01,  6.7970e-01],
          [ 2.9162e-01,  2.7692e-01,  2.5815e-01,  2.9709e-01,  3.0390e-01,
            3.5352e-01,  4.8272e-01,  5.3504e-01,  5.3538e-01,  5.3509e-01,
            5.3319e-01,  5.1336e-01,  5.2423e-01,  5.5370e-01,  5.5093e-01,
            5.5551e-01,  5.8090e-01,  5.7261e-01,  5.9149e-01,  6.0881e-01,
            6.3245e-01,  6.3988e-01,  6.5188e-01,  6.4756e-01,  6.4650e-01,
            6.5871e-01,  6.5027e-01,  6.4121e-01,  6.3240e-01,  6.5952e-01,
            6.6730e-01,  6.6419e-01],
          [ 2.6695e-01,  2.8207e-01,  2.9563e-01,  3.1781e-01,  2.9264e-01,
            3.7412e-01,  4.4536e-01,  4.8892e-01,  4.9193e-01,  4.8403e-01,
            5.2567e-01,  5.0454e-01,  5.0972e-01,  5.3780e-01,  5.3262e-01,
            5.5463e-01,  5.9693e-01,  6.1062e-01,  5.9879e-01,  6.2377e-01,
            6.0963e-01,  6.1228e-01,  6.2608e-01,  6.3943e-01,  6.3716e-01,
            6.3644e-01,  6.2824e-01,  6.2162e-01,  6.3583e-01,  6.3974e-01,
            6.5519e-01,  6.4140e-01]],

         [[ 3.4314e-01,  3.5346e-01,  3.8971e-01,  4.1945e-01,  3.8877e-01,
            3.7260e-01,  3.4918e-01,  3.5887e-01,  3.8524e-01,  3.8578e-01,
            4.0908e-01,  4.5010e-01,  4.8093e-01,  5.1729e-01,  5.4008e-01,
            5.5522e-01,  5.8368e-01,  5.7837e-01,  6.0400e-01,  5.5326e-01,
            4.9963e-01,  3.8384e-01,  5.6945e-02,  1.5506e-02, -1.5668e-02,
           -2.8547e-02,  1.1632e-02,  6.3978e-02,  1.9270e-01,  2.2523e-01,
            2.5671e-01,  2.3535e-01],
          [ 3.3686e-01,  3.5042e-01,  4.0101e-01,  4.0222e-01,  4.0202e-01,
            3.7311e-01,  3.4717e-01,  3.4253e-01,  3.5300e-01,  3.6312e-01,
            3.8234e-01,  4.2807e-01,  4.5995e-01,  5.1606e-01,  5.4714e-01,
            5.8183e-01,  6.1518e-01,  6.2223e-01,  6.2945e-01,  6.2246e-01,
            5.9783e-01,  5.0429e-01,  1.9773e-01, -4.4579e-03, -3.0440e-02,
           -5.0057e-03,  1.3569e-02,  5.1830e-02,  1.2113e-01,  2.2949e-01,
            2.7065e-01,  2.4409e-01],
          [ 3.3751e-01,  3.4484e-01,  3.6838e-01,  3.8264e-01,  3.6813e-01,
            3.3796e-01,  3.2110e-01,  2.8638e-01,  2.8116e-01,  2.9589e-01,
            3.2176e-01,  3.7317e-01,  4.0903e-01,  4.9959e-01,  5.4417e-01,
            6.1201e-01,  6.1728e-01,  6.3500e-01,  6.4574e-01,  6.4696e-01,
            6.2474e-01,  5.9406e-01,  3.9329e-01,  6.4278e-02,  1.7637e-02,
            2.7075e-03,  1.0398e-02,  4.7851e-02,  8.7494e-02,  1.6809e-01,
            2.6301e-01,  2.3872e-01],
          [ 3.2159e-01,  3.4473e-01,  3.4361e-01,  3.4436e-01,  3.2653e-01,
            2.2151e-01,  1.8534e-01,  1.6891e-01,  1.2618e-01,  1.2277e-01,
            1.7997e-01,  2.2650e-01,  2.8886e-01,  3.7024e-01,  4.5773e-01,
            5.8033e-01,  6.8572e-01,  6.9552e-01,  6.6847e-01,  6.4700e-01,
            6.0894e-01,  5.6360e-01,  4.4713e-01,  7.2401e-02,  2.0743e-02,
            1.7386e-02,  3.2335e-02,  7.3650e-02,  1.0820e-01,  1.5554e-01,
            2.2263e-01,  2.1291e-01],
          [ 2.3726e-01,  2.2332e-01,  2.2510e-01,  2.3696e-01,  2.2587e-01,
            1.2821e-01,  1.0090e-01,  1.0694e-01,  1.0385e-01,  1.0036e-01,
            1.1287e-01,  1.3158e-01,  1.2640e-01,  1.6099e-01,  2.3542e-01,
            3.6984e-01,  6.2889e-01,  7.5659e-01,  6.9723e-01,  6.4669e-01,
            5.7769e-01,  4.9982e-01,  4.0381e-01,  2.3669e-02,  1.2248e-02,
            4.3980e-02,  9.4091e-02,  1.3417e-01,  1.4449e-01,  1.3961e-01,
            2.2090e-01,  2.7471e-01],
          [ 1.1620e-01,  1.2843e-01,  1.4569e-01,  1.6072e-01,  1.6639e-01,
            1.1882e-01,  9.9664e-02,  8.9055e-02,  9.0573e-02,  9.3096e-02,
            9.2325e-02,  1.0009e-01,  1.0402e-01,  1.0489e-01,  1.1899e-01,
            1.7858e-01,  2.7507e-01,  5.6714e-01,  7.4231e-01,  7.3945e-01,
            6.7208e-01,  5.2217e-01,  2.8248e-01,  7.2076e-02,  3.4123e-02,
            7.5986e-02,  1.6034e-01,  1.2776e-01,  6.7562e-02,  9.1336e-02,
            1.6457e-01,  3.1713e-01],
          [ 6.3048e-02,  9.2872e-02,  1.5147e-01,  1.5094e-01,  8.3347e-02,
            1.0527e-01,  9.1394e-02,  9.4453e-02,  6.2816e-02,  6.4045e-02,
            7.6996e-02,  1.0278e-01,  9.9405e-02,  1.0488e-01,  9.8156e-02,
            1.2942e-01,  1.4907e-01,  2.3145e-01,  5.1996e-01,  7.7971e-01,
            8.1732e-01,  6.2487e-01,  2.9801e-01,  1.3993e-01,  6.1314e-02,
            1.2437e-01,  2.4650e-01,  1.5313e-01,  6.5381e-02,  5.6023e-02,
            1.0343e-01,  2.6484e-01],
          [ 3.9305e-02,  6.6927e-02,  1.4970e-01,  1.7150e-01,  9.0575e-02,
            8.3250e-02,  1.1257e-01,  9.6406e-02,  9.5755e-02,  1.0334e-01,
            9.9240e-02,  1.0083e-01,  1.1125e-01,  1.2652e-01,  1.2857e-01,
            1.0225e-01,  1.6215e-01,  1.9640e-01,  2.3804e-01,  4.4746e-01,
            7.2996e-01,  7.1399e-01,  3.6615e-01,  1.4716e-01,  1.2702e-01,
            1.9179e-01,  2.9361e-01,  1.9014e-01,  7.6306e-02,  7.4077e-02,
            9.7375e-02,  1.9965e-01],
          [ 3.1496e-02,  6.1369e-02,  1.3550e-01,  1.5894e-01,  9.7246e-02,
            8.3992e-02,  1.2942e-01,  1.0425e-01,  1.4078e-01,  1.3660e-01,
            1.1759e-01,  1.2221e-01,  1.1354e-01,  1.4391e-01,  1.5120e-01,
            1.2439e-01,  1.0724e-01,  2.2496e-01,  2.3908e-01,  2.5785e-01,
            5.0577e-01,  7.3509e-01,  4.9089e-01,  1.4461e-01,  2.0689e-01,
            2.5353e-01,  3.9585e-01,  3.1755e-01,  8.4783e-02,  7.7366e-02,
            1.4566e-01,  2.1802e-01],
          [ 4.3886e-02,  8.8189e-02,  1.2390e-01,  1.3489e-01,  1.2657e-01,
            1.0134e-01,  8.2642e-02,  7.7335e-02,  1.0648e-01,  8.2470e-02,
            9.5705e-02,  1.1237e-01,  1.0492e-01,  1.6224e-01,  1.5758e-01,
            1.5146e-01,  8.9366e-02,  1.6101e-01,  2.7897e-01,  2.8092e-01,
            3.4331e-01,  7.5773e-01,  6.6475e-01,  2.0995e-01,  2.3603e-01,
            3.0432e-01,  5.8092e-01,  4.3613e-01,  1.3698e-01,  1.1138e-01,
            1.8382e-01,  3.1323e-01],
          [ 4.4302e-02,  8.1802e-02,  1.3371e-01,  1.5141e-01,  1.3784e-01,
            1.0779e-01,  5.9376e-02,  5.8235e-02,  7.8662e-02,  6.1624e-02,
            9.6707e-02,  9.3071e-02,  6.7205e-02,  1.3603e-01,  1.4832e-01,
            1.5653e-01,  1.0749e-01,  1.1554e-01,  2.1719e-01,  2.5493e-01,
            3.4755e-01,  6.9989e-01,  7.3002e-01,  2.7529e-01,  2.8968e-01,
            3.5879e-01,  5.2343e-01,  4.8929e-01,  1.6792e-01,  9.6563e-02,
            3.3419e-01,  4.3793e-01],
          [ 7.3728e-02,  8.7717e-02,  1.6782e-01,  1.7295e-01,  1.5546e-01,
            1.1569e-01,  4.5590e-02,  1.5765e-02,  5.3435e-02,  6.5616e-02,
            1.3178e-01,  5.7195e-02,  6.9581e-02,  1.5180e-01,  1.4870e-01,
            1.4449e-01,  1.1521e-01,  6.6230e-02,  1.1923e-01,  1.8730e-01,
            2.8762e-01,  5.3755e-01,  6.8410e-01,  4.4958e-01,  3.5246e-01,
            4.1966e-01,  5.1694e-01,  4.6674e-01,  9.9609e-02,  2.0492e-01,
            5.2855e-01,  4.8430e-01],
          [ 6.3861e-02,  9.4770e-02,  1.6736e-01,  2.3429e-01,  1.9053e-01,
            1.5456e-01,  8.7328e-02,  5.3287e-02,  6.6063e-02,  7.9999e-02,
            1.4673e-01,  8.2784e-02,  1.0113e-01,  1.8100e-01,  1.5330e-01,
            1.3928e-01,  1.0295e-01,  3.1570e-02,  3.6176e-02,  1.2264e-01,
            2.8274e-01,  4.8732e-01,  6.0885e-01,  5.2375e-01,  4.1007e-01,
            4.7312e-01,  5.4632e-01,  3.2055e-01,  5.8601e-02,  3.5125e-01,
            5.8959e-01,  5.1199e-01],
          [ 1.0833e-01,  9.4751e-02,  2.0301e-01,  2.5192e-01,  2.5549e-01,
            2.1901e-01,  1.0692e-01,  4.1855e-02,  5.0379e-02,  1.0106e-01,
            1.5896e-01,  1.2953e-01,  1.2790e-01,  1.6240e-01,  1.5232e-01,
            1.3977e-01,  8.2929e-02,  4.7387e-02,  4.2158e-02,  8.5918e-02,
            2.6591e-01,  4.7180e-01,  5.9434e-01,  5.7019e-01,  5.5183e-01,
            5.4700e-01,  4.3342e-01,  1.5061e-01,  2.3443e-02,  1.8940e-01,
            4.8761e-01,  4.8497e-01],
          [ 2.6676e-01,  1.4253e-01,  2.0829e-01,  2.5778e-01,  2.5988e-01,
            1.9813e-01,  5.9448e-02,  3.1151e-02,  4.7202e-02,  1.4310e-01,
            1.7629e-01,  1.3295e-01,  1.1384e-01,  1.1085e-01,  1.4731e-01,
            1.4217e-01,  9.0820e-02,  6.2466e-02,  6.2068e-02,  1.0290e-01,
            2.1881e-01,  4.1904e-01,  5.9299e-01,  6.1004e-01,  5.8589e-01,
            6.3973e-01,  4.1597e-01,  1.3801e-01,  5.2256e-02,  4.9539e-02,
            3.1733e-01,  4.0831e-01],
          [ 3.5082e-01,  1.2953e-01,  2.2121e-01,  2.5324e-01,  2.5434e-01,
            1.7051e-01,  4.2521e-02,  3.4187e-02,  3.9714e-02,  1.4061e-01,
            1.8186e-01,  1.6021e-01,  1.4477e-01,  1.3177e-01,  1.3918e-01,
            1.3848e-01,  8.1197e-02,  6.5482e-02,  5.9605e-02,  1.2010e-01,
            1.6490e-01,  3.8352e-01,  5.6288e-01,  6.2185e-01,  6.2403e-01,
            6.8100e-01,  5.1696e-01,  1.6354e-01,  1.0339e-01,  9.0421e-02,
            2.2934e-01,  3.9973e-01],
          [ 2.8998e-01,  1.3458e-01,  2.3222e-01,  2.4659e-01,  2.4826e-01,
            1.5671e-01,  5.6949e-02,  5.0530e-02,  7.7649e-02,  1.4973e-01,
            1.7409e-01,  1.4897e-01,  1.5975e-01,  1.2273e-01,  1.2799e-01,
            1.0343e-01,  5.1140e-02,  6.0769e-02,  7.2444e-02,  1.1332e-01,
            1.6475e-01,  3.3816e-01,  5.4158e-01,  6.1265e-01,  6.2586e-01,
            6.3792e-01,  6.2515e-01,  3.4869e-01,  2.2832e-01,  1.1667e-01,
            2.3674e-01,  4.1567e-01],
          [ 3.2026e-01,  2.6222e-01,  2.3632e-01,  2.4331e-01,  2.4146e-01,
            1.4852e-01,  9.2391e-02,  4.3944e-02,  6.4171e-02,  1.3732e-01,
            1.5543e-01,  1.5192e-01,  1.3175e-01,  1.3332e-01,  1.1558e-01,
            1.0084e-01,  5.0637e-02,  6.4166e-02,  8.5895e-02,  1.2873e-01,
            1.6338e-01,  2.5934e-01,  4.9120e-01,  5.9328e-01,  5.8913e-01,
            6.0624e-01,  5.8827e-01,  4.6202e-01,  3.7127e-01,  2.9651e-01,
            4.3742e-01,  5.4708e-01],
          [ 3.5148e-01,  3.3227e-01,  2.9368e-01,  2.5591e-01,  2.3404e-01,
            9.2394e-02,  2.8552e-02,  2.0706e-02,  1.0381e-01,  1.8786e-01,
            2.1870e-01,  1.8988e-01,  1.3497e-01,  1.0978e-01,  1.2035e-01,
            1.1135e-01,  5.2067e-02,  7.2384e-02,  1.1408e-01,  1.5424e-01,
            2.0403e-01,  1.9923e-01,  4.0630e-01,  6.2297e-01,  6.1689e-01,
            5.6451e-01,  4.7316e-01,  4.2638e-01,  5.5304e-01,  5.6116e-01,
            6.1578e-01,  6.2628e-01],
          [ 3.2944e-01,  2.8341e-01,  2.9920e-01,  2.5847e-01,  1.5105e-01,
            2.0340e-02,  9.0672e-03,  7.2897e-02,  1.7786e-01,  2.5048e-01,
            2.6754e-01,  1.9559e-01,  1.3950e-01,  1.2135e-01,  1.1158e-01,
            9.4549e-02,  4.6887e-02,  8.5341e-02,  1.4200e-01,  2.2089e-01,
            2.5850e-01,  1.7348e-01,  3.2812e-01,  5.1605e-01,  5.7715e-01,
            5.0976e-01,  3.7821e-01,  4.8486e-01,  6.4630e-01,  6.5008e-01,
            6.3875e-01,  6.2957e-01],
          [ 3.4526e-01,  2.9602e-01,  2.5886e-01,  2.0746e-01,  5.4045e-02,
            2.3775e-02,  2.0193e-02,  1.1278e-01,  1.7968e-01,  2.4608e-01,
            2.1045e-01,  1.5677e-01,  1.3789e-01,  1.1546e-01,  8.4870e-02,
            7.9538e-02,  6.3269e-02,  8.6856e-02,  1.1810e-01,  2.1420e-01,
            2.9471e-01,  2.0828e-01,  3.3639e-01,  5.0452e-01,  5.3907e-01,
            3.6276e-01,  2.8716e-01,  5.3442e-01,  6.5668e-01,  6.7836e-01,
            6.7777e-01,  6.8245e-01],
          [ 3.2187e-01,  3.4110e-01,  2.5506e-01,  1.9532e-01,  5.9561e-02,
            1.6989e-02,  2.0760e-02,  4.7023e-02,  1.7575e-01,  2.2727e-01,
            1.3378e-01,  1.3021e-01,  1.1841e-01,  1.0338e-01,  7.6238e-02,
            8.0468e-02,  7.6526e-02,  9.1066e-02,  1.2719e-01,  1.9689e-01,
            2.8809e-01,  2.3771e-01,  3.8657e-01,  4.6053e-01,  4.0362e-01,
            2.6153e-01,  3.5646e-01,  5.8517e-01,  6.9727e-01,  6.8417e-01,
            6.8798e-01,  6.7867e-01],
          [ 3.6088e-01,  3.6003e-01,  2.8892e-01,  1.5851e-01,  4.6253e-02,
            3.4415e-02,  3.7644e-02,  7.7881e-02,  1.7891e-01,  1.7206e-01,
            1.0261e-01,  1.0211e-01,  1.0315e-01,  1.1091e-01,  7.8016e-02,
            6.5199e-02,  7.9641e-02,  9.0647e-02,  1.2499e-01,  1.8444e-01,
            2.2681e-01,  2.5478e-01,  2.9813e-01,  2.9177e-01,  2.4531e-01,
            2.4973e-01,  4.8378e-01,  6.3961e-01,  7.1292e-01,  7.0341e-01,
            6.7299e-01,  6.6779e-01],
          [ 4.1837e-01,  4.1764e-01,  3.1038e-01,  1.7678e-01,  6.1355e-02,
            7.0270e-02,  1.0669e-01,  1.7164e-01,  1.9564e-01,  1.1896e-01,
            9.0278e-02,  8.1764e-02,  8.1805e-02,  8.4725e-02,  7.3009e-02,
            5.1825e-02,  6.2764e-02,  5.7076e-02,  8.2351e-02,  1.1773e-01,
            1.6108e-01,  2.0924e-01,  1.6691e-01,  1.5916e-01,  1.4021e-01,
            2.8679e-01,  5.3974e-01,  6.5952e-01,  6.9568e-01,  6.9929e-01,
            6.9995e-01,  7.0977e-01],
          [ 4.7489e-01,  4.5312e-01,  3.3774e-01,  2.3402e-01,  1.0633e-01,
            1.0197e-01,  1.4752e-01,  2.0201e-01,  1.8748e-01,  9.1853e-02,
            7.3078e-02,  5.6256e-02,  4.6931e-02,  4.8961e-02,  5.0518e-02,
            6.4123e-02,  3.4640e-02,  5.7159e-02,  7.4284e-02,  7.1838e-02,
            1.1489e-01,  1.8278e-01,  1.0399e-01,  1.2937e-01,  2.2547e-01,
            4.1352e-01,  5.8048e-01,  6.6703e-01,  7.0394e-01,  7.0638e-01,
            7.1866e-01,  7.1790e-01],
          [ 3.9542e-01,  3.9946e-01,  3.2090e-01,  2.4117e-01,  1.9133e-01,
            1.8907e-01,  8.6482e-02,  2.0611e-01,  2.2149e-01,  9.8749e-02,
            3.1547e-02,  1.4030e-02,  3.5185e-02,  3.8736e-02,  4.7536e-02,
            5.1539e-02,  5.5718e-02,  6.3848e-02,  7.6457e-02,  9.5615e-02,
            1.3214e-01,  1.2371e-01,  5.9308e-02,  2.3811e-01,  4.3760e-01,
            5.5551e-01,  6.2422e-01,  6.6132e-01,  7.0474e-01,  6.9881e-01,
            6.8909e-01,  7.0896e-01],
          [ 3.3206e-01,  4.0004e-01,  2.8248e-01,  1.7593e-01,  2.2845e-01,
            1.8074e-01,  2.3083e-01,  3.4748e-01,  3.2725e-01,  2.1494e-01,
            1.1612e-01,  5.8686e-02,  5.4490e-02,  4.1570e-02,  6.7732e-02,
            7.8577e-02,  8.1264e-02,  8.5492e-02,  8.1732e-02,  1.2070e-01,
            9.5261e-02,  6.5579e-02,  2.1365e-01,  4.0150e-01,  5.2321e-01,
            5.8365e-01,  6.1119e-01,  6.2300e-01,  6.9928e-01,  6.9404e-01,
            7.0699e-01,  7.0732e-01],
          [ 2.3474e-01,  2.7052e-01,  2.7130e-01,  3.3430e-01,  2.9554e-01,
            2.5609e-01,  5.5448e-01,  6.1794e-01,  5.5500e-01,  5.0208e-01,
            4.5066e-01,  4.0230e-01,  3.2180e-01,  2.8193e-01,  2.6732e-01,
            2.5767e-01,  2.7341e-01,  2.6470e-01,  2.7434e-01,  3.0710e-01,
            3.1634e-01,  3.5305e-01,  4.2448e-01,  5.3370e-01,  5.9923e-01,
            6.0304e-01,  6.0424e-01,  6.1773e-01,  6.6648e-01,  6.8054e-01,
            6.8657e-01,  6.9249e-01],
          [ 1.7029e-01,  2.8626e-01,  4.4538e-01,  4.6600e-01,  3.7298e-01,
            3.7308e-01,  6.4584e-01,  6.6667e-01,  6.4843e-01,  5.8768e-01,
            5.9540e-01,  6.0592e-01,  5.6608e-01,  5.3456e-01,  5.2723e-01,
            5.0438e-01,  5.1168e-01,  5.0953e-01,  5.2905e-01,  5.5775e-01,
            5.7138e-01,  5.7999e-01,  5.9943e-01,  6.5011e-01,  6.7105e-01,
            6.0750e-01,  6.0170e-01,  6.1953e-01,  6.4236e-01,  6.5570e-01,
            6.7725e-01,  6.9859e-01],
          [ 2.5795e-01,  3.2189e-01,  3.2527e-01,  3.0028e-01,  2.9487e-01,
            3.8358e-01,  5.7816e-01,  6.0688e-01,  5.8102e-01,  5.7474e-01,
            5.4729e-01,  5.4349e-01,  5.2696e-01,  5.6145e-01,  5.6031e-01,
            5.5586e-01,  5.6375e-01,  5.5365e-01,  5.5796e-01,  5.9240e-01,
            6.2137e-01,  6.2708e-01,  6.5604e-01,  6.5955e-01,  6.6245e-01,
            6.5503e-01,  6.4463e-01,  6.4153e-01,  6.4871e-01,  6.6233e-01,
            6.8258e-01,  6.7818e-01],
          [ 2.9741e-01,  2.6418e-01,  2.5705e-01,  2.8865e-01,  3.0827e-01,
            3.4582e-01,  4.7855e-01,  5.2001e-01,  5.1110e-01,  5.2277e-01,
            5.1394e-01,  5.2056e-01,  5.2577e-01,  5.5303e-01,  5.5901e-01,
            5.6389e-01,  5.9726e-01,  5.6192e-01,  6.1204e-01,  6.1677e-01,
            6.4409e-01,  6.5809e-01,  6.5621e-01,  6.7203e-01,  6.6966e-01,
            6.6887e-01,  6.6898e-01,  6.5694e-01,  6.5043e-01,  6.7211e-01,
            6.8576e-01,  6.8122e-01],
          [ 2.7384e-01,  2.9540e-01,  2.9829e-01,  3.0874e-01,  2.9306e-01,
            3.4990e-01,  4.4605e-01,  4.8165e-01,  4.8266e-01,  4.8299e-01,
            5.2042e-01,  4.9499e-01,  5.0647e-01,  5.3749e-01,  5.6043e-01,
            5.6050e-01,  5.9944e-01,  5.9871e-01,  6.0141e-01,  6.2872e-01,
            6.2617e-01,  6.4044e-01,  6.5089e-01,  6.6295e-01,  6.5712e-01,
            6.5554e-01,  6.4381e-01,  6.6526e-01,  6.5219e-01,  6.5801e-01,
            6.5579e-01,  6.4057e-01]]],


        [[[ 2.1448e-01,  1.8825e-01,  1.8655e-01,  1.9491e-01,  2.2566e-01,
            2.6311e-01,  2.7376e-01,  3.1107e-01,  2.9444e-01,  2.7691e-01,
            2.8943e-01,  2.5928e-01,  2.4064e-01,  2.3500e-01,  2.4111e-01,
            2.4849e-01,  2.7529e-01,  2.9556e-01,  3.1276e-01,  3.0495e-01,
            2.8028e-01,  2.7353e-01,  2.8884e-01,  3.5157e-01,  3.5251e-01,
            2.6989e-01,  2.2144e-01,  2.6271e-01,  2.9483e-01,  2.6582e-01,
            2.4779e-01,  2.6829e-01],
          [ 1.9682e-01,  1.8145e-01,  1.8979e-01,  2.0144e-01,  2.6004e-01,
            2.7784e-01,  2.7212e-01,  2.9234e-01,  3.0734e-01,  2.8732e-01,
            2.7664e-01,  2.2894e-01,  2.3002e-01,  2.2397e-01,  2.2720e-01,
            2.2376e-01,  2.3652e-01,  2.4816e-01,  2.4824e-01,  2.4619e-01,
            2.4778e-01,  2.4544e-01,  2.6824e-01,  2.9706e-01,  2.8074e-01,
            2.2539e-01,  2.1337e-01,  2.5424e-01,  2.7732e-01,  2.4417e-01,
            2.1505e-01,  2.0629e-01],
          [ 2.1176e-01,  1.8220e-01,  2.0419e-01,  2.4454e-01,  2.6559e-01,
            2.7258e-01,  2.7793e-01,  2.7564e-01,  2.8281e-01,  2.5319e-01,
            2.4906e-01,  2.4238e-01,  2.4222e-01,  2.3988e-01,  2.2200e-01,
            2.2824e-01,  2.2945e-01,  2.3341e-01,  2.3738e-01,  2.4761e-01,
            2.4887e-01,  2.4996e-01,  2.4527e-01,  2.5842e-01,  2.1818e-01,
            2.0846e-01,  2.1069e-01,  2.4775e-01,  2.4987e-01,  2.3035e-01,
            2.1005e-01,  1.9485e-01],
          [ 2.2076e-01,  1.8928e-01,  2.3224e-01,  2.4635e-01,  2.6547e-01,
            2.6937e-01,  2.8025e-01,  2.9481e-01,  2.8377e-01,  2.6800e-01,
            2.6211e-01,  2.4257e-01,  2.4514e-01,  2.6051e-01,  2.3235e-01,
            2.3749e-01,  2.5258e-01,  2.4118e-01,  2.5101e-01,  2.4957e-01,
            2.2198e-01,  2.3857e-01,  2.4105e-01,  2.4696e-01,  2.3259e-01,
            2.2151e-01,  2.2324e-01,  2.4010e-01,  2.3845e-01,  2.3305e-01,
            2.3109e-01,  2.0468e-01],
          [ 2.0615e-01,  1.8773e-01,  2.3648e-01,  2.4793e-01,  2.5948e-01,
            2.8990e-01,  2.8808e-01,  3.0191e-01,  3.0136e-01,  2.7264e-01,
            2.7093e-01,  2.6970e-01,  2.5843e-01,  2.7277e-01,  2.5421e-01,
            2.4426e-01,  2.4012e-01,  2.4540e-01,  2.4377e-01,  2.4512e-01,
            2.1872e-01,  2.4001e-01,  2.4715e-01,  2.3795e-01,  2.4262e-01,
            2.2830e-01,  2.1943e-01,  2.3214e-01,  2.3597e-01,  2.5562e-01,
            2.4085e-01,  2.4020e-01],
          [ 2.2625e-01,  2.0021e-01,  2.3008e-01,  2.4108e-01,  2.7008e-01,
            3.1329e-01,  3.1887e-01,  3.2667e-01,  2.8175e-01,  2.5626e-01,
            2.7369e-01,  2.9510e-01,  2.8308e-01,  2.9306e-01,  2.9292e-01,
            2.5847e-01,  2.3963e-01,  2.4593e-01,  2.5317e-01,  2.5150e-01,
            2.3703e-01,  2.4229e-01,  2.4517e-01,  2.5370e-01,  2.4784e-01,
            2.5040e-01,  2.2336e-01,  2.1079e-01,  2.3535e-01,  2.4050e-01,
            2.4334e-01,  2.3838e-01],
          [ 2.1739e-01,  1.8216e-01,  2.0675e-01,  2.3559e-01,  2.5454e-01,
            2.9833e-01,  3.2430e-01,  3.2856e-01,  2.7133e-01,  2.6725e-01,
            2.9352e-01,  2.7664e-01,  2.8353e-01,  2.9155e-01,  2.9888e-01,
            2.8093e-01,  2.5003e-01,  2.4664e-01,  2.4212e-01,  2.4267e-01,
            2.5197e-01,  2.5413e-01,  2.2784e-01,  2.5041e-01,  2.2783e-01,
            2.4487e-01,  2.3057e-01,  2.3257e-01,  2.3053e-01,  2.4122e-01,
            2.5974e-01,  2.4537e-01],
          [ 2.2123e-01,  1.7949e-01,  2.2018e-01,  2.1872e-01,  2.6242e-01,
            2.9028e-01,  3.0352e-01,  3.1169e-01,  2.7579e-01,  2.7631e-01,
            3.1886e-01,  2.7146e-01,  2.7663e-01,  2.9056e-01,  2.8178e-01,
            2.8237e-01,  2.5396e-01,  2.5722e-01,  2.5408e-01,  2.4887e-01,
            2.5094e-01,  2.4568e-01,  2.2644e-01,  2.3530e-01,  2.3694e-01,
            2.4413e-01,  2.4204e-01,  2.3407e-01,  2.4083e-01,  2.4512e-01,
            2.4329e-01,  2.3474e-01],
          [ 2.1378e-01,  1.8475e-01,  2.2594e-01,  2.2282e-01,  2.3679e-01,
            2.6677e-01,  2.6853e-01,  2.8129e-01,  2.7941e-01,  2.9548e-01,
            2.9426e-01,  2.8244e-01,  2.8810e-01,  2.8693e-01,  2.9028e-01,
            2.6690e-01,  2.5808e-01,  2.4233e-01,  2.4991e-01,  2.6136e-01,
            2.5581e-01,  2.4094e-01,  2.4130e-01,  2.4064e-01,  2.5310e-01,
            2.3855e-01,  2.4633e-01,  2.4002e-01,  2.2995e-01,  2.5210e-01,
            2.4159e-01,  2.3768e-01],
          [ 1.9392e-01,  1.7539e-01,  1.8862e-01,  2.2083e-01,  2.2436e-01,
            2.4159e-01,  2.4521e-01,  2.7350e-01,  2.8616e-01,  3.0755e-01,
            3.0289e-01,  2.9466e-01,  2.8600e-01,  2.8990e-01,  2.7412e-01,
            2.7528e-01,  2.7447e-01,  2.5617e-01,  2.5813e-01,  2.5504e-01,
            2.5732e-01,  2.5916e-01,  2.4763e-01,  2.4879e-01,  2.5120e-01,
            2.5149e-01,  2.4559e-01,  2.3935e-01,  2.2435e-01,  2.1473e-01,
            2.1348e-01,  2.0847e-01],
          [ 2.0923e-01,  1.9910e-01,  2.0175e-01,  2.1793e-01,  2.2687e-01,
            2.3770e-01,  2.6897e-01,  2.5684e-01,  2.9389e-01,  2.8207e-01,
            3.0223e-01,  2.7915e-01,  2.9529e-01,  2.9768e-01,  3.0142e-01,
            2.8676e-01,  2.7906e-01,  2.8152e-01,  2.7158e-01,  2.7299e-01,
            2.6606e-01,  2.3776e-01,  2.5698e-01,  2.4954e-01,  2.5459e-01,
            2.6153e-01,  2.4702e-01,  2.1463e-01,  2.0165e-01,  2.1367e-01,
            2.1426e-01,  2.2261e-01],
          [ 2.1914e-01,  2.3454e-01,  2.2043e-01,  2.2389e-01,  2.2947e-01,
            2.5240e-01,  2.9570e-01,  2.9663e-01,  3.0131e-01,  3.0099e-01,
            3.0596e-01,  2.9769e-01,  2.7266e-01,  2.9745e-01,  3.2638e-01,
            2.8782e-01,  2.6360e-01,  2.7283e-01,  2.7006e-01,  2.8830e-01,
            2.8785e-01,  2.6500e-01,  2.5727e-01,  2.7449e-01,  2.5929e-01,
            2.7736e-01,  2.5938e-01,  1.7896e-01,  2.0163e-01,  2.5454e-01,
            2.2256e-01,  2.4150e-01],
          [ 2.0546e-01,  2.0944e-01,  2.0568e-01,  2.1337e-01,  2.2637e-01,
            2.6239e-01,  2.9210e-01,  3.0230e-01,  3.0082e-01,  2.8658e-01,
            2.9375e-01,  2.7660e-01,  2.7057e-01,  2.8081e-01,  3.0401e-01,
            3.0266e-01,  2.9152e-01,  2.6059e-01,  2.6448e-01,  2.7830e-01,
            2.5875e-01,  2.6504e-01,  2.7297e-01,  2.7974e-01,  2.8323e-01,
            3.5116e-01,  3.0280e-01,  2.2134e-01,  2.0811e-01,  2.6059e-01,
            2.4839e-01,  2.4507e-01],
          [ 2.3828e-01,  2.0518e-01,  2.1099e-01,  2.2232e-01,  2.3895e-01,
            2.5261e-01,  2.5754e-01,  2.7019e-01,  2.8624e-01,  2.5389e-01,
            2.8437e-01,  2.7463e-01,  2.8007e-01,  3.0392e-01,  2.9245e-01,
            3.0523e-01,  3.1585e-01,  2.7980e-01,  2.8239e-01,  2.6533e-01,
            2.9669e-01,  3.0049e-01,  2.8478e-01,  2.8678e-01,  3.8627e-01,
            4.1443e-01,  3.4143e-01,  2.5199e-01,  2.4526e-01,  2.4352e-01,
            2.5406e-01,  2.6830e-01],
          [ 2.1636e-01,  2.1938e-01,  2.1034e-01,  2.0503e-01,  2.3988e-01,
            2.2803e-01,  2.1946e-01,  2.5146e-01,  2.4859e-01,  2.3058e-01,
            2.7372e-01,  2.5375e-01,  2.7437e-01,  2.5564e-01,  2.3792e-01,
            2.8484e-01,  3.1964e-01,  3.1394e-01,  3.1749e-01,  3.3364e-01,
            3.6283e-01,  3.9305e-01,  3.8105e-01,  3.4205e-01,  4.6048e-01,
            4.4846e-01,  3.5477e-01,  2.5254e-01,  2.4457e-01,  2.8355e-01,
            3.0979e-01,  3.0358e-01],
          [ 2.0355e-01,  2.2725e-01,  2.2143e-01,  2.2177e-01,  2.3294e-01,
            2.3530e-01,  2.2529e-01,  2.3916e-01,  2.3686e-01,  2.4904e-01,
            2.6520e-01,  2.3227e-01,  2.7995e-01,  2.7169e-01,  2.3651e-01,
            2.8220e-01,  2.7305e-01,  2.7443e-01,  3.3278e-01,  3.8689e-01,
            3.8689e-01,  4.0984e-01,  3.7427e-01,  3.5385e-01,  4.4165e-01,
            4.2616e-01,  3.2379e-01,  2.4064e-01,  2.6727e-01,  2.3061e-01,
            3.1494e-01,  3.3189e-01],
          [ 1.6890e-01,  2.1753e-01,  2.2110e-01,  2.2204e-01,  2.2372e-01,
            2.2497e-01,  2.2875e-01,  2.3949e-01,  2.6250e-01,  2.3624e-01,
            2.2197e-01,  2.3215e-01,  3.3943e-01,  3.2059e-01,  2.5358e-01,
            2.6765e-01,  2.5810e-01,  2.6060e-01,  3.2599e-01,  3.7555e-01,
            3.9491e-01,  4.0453e-01,  3.8178e-01,  3.5983e-01,  4.0515e-01,
            3.6975e-01,  2.4516e-01,  2.2212e-01,  2.6130e-01,  2.4162e-01,
            2.6156e-01,  2.9046e-01],
          [ 3.3516e-01,  3.7305e-01,  3.6709e-01,  3.5680e-01,  3.7177e-01,
            4.1045e-01,  4.0669e-01,  3.9084e-01,  3.6274e-01,  3.1640e-01,
            3.3571e-01,  3.8614e-01,  4.8634e-01,  4.5959e-01,  4.4536e-01,
            4.2250e-01,  3.6514e-01,  3.3668e-01,  3.4415e-01,  3.4857e-01,
            3.8366e-01,  4.3799e-01,  3.8263e-01,  2.9242e-01,  3.3358e-01,
            2.9278e-01,  2.1680e-01,  2.0518e-01,  2.2899e-01,  2.3132e-01,
            2.4506e-01,  2.3545e-01],
          [ 5.5421e-01,  5.6240e-01,  5.6877e-01,  5.8915e-01,  6.1557e-01,
            6.3345e-01,  6.5715e-01,  6.6225e-01,  6.2312e-01,  5.4940e-01,
            5.5565e-01,  6.5869e-01,  7.2135e-01,  6.9870e-01,  7.1198e-01,
            6.8894e-01,  5.2978e-01,  3.9462e-01,  3.7124e-01,  3.6890e-01,
            3.8491e-01,  4.5877e-01,  2.9110e-01,  1.8099e-01,  2.8079e-01,
            2.7479e-01,  2.1740e-01,  2.0752e-01,  2.1428e-01,  2.0669e-01,
            2.4056e-01,  2.4570e-01],
          [ 5.9472e-01,  5.9132e-01,  5.9965e-01,  6.4416e-01,  6.4665e-01,
            6.5397e-01,  6.6084e-01,  7.0102e-01,  6.7914e-01,  6.5467e-01,
            6.2668e-01,  6.7148e-01,  7.2923e-01,  7.0693e-01,  7.1714e-01,
            6.9347e-01,  6.1827e-01,  5.0635e-01,  4.1937e-01,  4.1209e-01,
            3.8431e-01,  3.3178e-01,  2.0371e-01,  1.9095e-01,  2.9103e-01,
            2.8755e-01,  2.5992e-01,  2.5996e-01,  2.2383e-01,  2.2443e-01,
            2.6137e-01,  2.7667e-01],
          [ 5.9156e-01,  6.0926e-01,  6.1641e-01,  6.3325e-01,  6.4420e-01,
            6.2518e-01,  6.3943e-01,  6.5148e-01,  6.6217e-01,  6.7150e-01,
            6.4164e-01,  6.3943e-01,  6.7874e-01,  6.4574e-01,  6.5730e-01,
            6.5224e-01,  6.3996e-01,  5.8071e-01,  4.5275e-01,  4.1617e-01,
            3.9612e-01,  2.8036e-01,  2.5847e-01,  2.7045e-01,  2.5829e-01,
            2.8158e-01,  2.8985e-01,  2.4679e-01,  2.1886e-01,  2.5745e-01,
            2.6260e-01,  2.6819e-01],
          [ 5.9178e-01,  6.1246e-01,  6.1496e-01,  6.1734e-01,  6.2467e-01,
            6.2827e-01,  6.5195e-01,  6.3948e-01,  6.3389e-01,  6.4877e-01,
            6.3708e-01,  6.2683e-01,  6.5084e-01,  6.5962e-01,  6.2984e-01,
            6.2617e-01,  6.2742e-01,  5.8270e-01,  5.0257e-01,  4.9985e-01,
            4.1703e-01,  3.2020e-01,  2.3548e-01,  2.7479e-01,  2.8654e-01,
            2.9168e-01,  2.6798e-01,  2.2130e-01,  2.2297e-01,  2.4370e-01,
            2.5546e-01,  2.4073e-01],
          [ 5.9602e-01,  6.0849e-01,  6.1985e-01,  6.0275e-01,  6.1205e-01,
            6.3351e-01,  6.3616e-01,  6.1554e-01,  5.9969e-01,  6.2273e-01,
            6.1651e-01,  6.2229e-01,  6.4341e-01,  6.6917e-01,  6.5294e-01,
            6.3509e-01,  6.4322e-01,  6.2617e-01,  6.4151e-01,  7.0388e-01,
            5.4569e-01,  2.9787e-01,  2.7967e-01,  2.7849e-01,  2.9081e-01,
            2.8744e-01,  2.6449e-01,  2.5333e-01,  2.2009e-01,  2.3986e-01,
            2.5123e-01,  2.5080e-01],
          [ 5.8733e-01,  6.0093e-01,  6.0378e-01,  6.0384e-01,  6.1755e-01,
            6.1624e-01,  6.1818e-01,  5.9609e-01,  5.8686e-01,  5.8732e-01,
            5.6714e-01,  5.6866e-01,  5.7945e-01,  7.0479e-01,  7.6342e-01,
            7.8170e-01,  8.2225e-01,  8.2350e-01,  8.7778e-01,  8.3614e-01,
            6.4974e-01,  3.0211e-01,  2.5675e-01,  2.8854e-01,  2.8156e-01,
            2.7373e-01,  2.5875e-01,  2.4562e-01,  2.5125e-01,  2.5182e-01,
            2.6149e-01,  2.7778e-01],
          [ 5.7116e-01,  5.8822e-01,  6.0335e-01,  5.7786e-01,  5.5036e-01,
            5.6268e-01,  6.1628e-01,  5.9836e-01,  5.5904e-01,  5.0361e-01,
            4.8634e-01,  5.2453e-01,  5.7958e-01,  7.4447e-01,  8.7290e-01,
            9.0195e-01,  8.6045e-01,  8.2842e-01,  8.4031e-01,  8.2088e-01,
            7.0590e-01,  3.2994e-01,  2.6776e-01,  2.9299e-01,  2.5947e-01,
            2.6927e-01,  2.6106e-01,  2.4759e-01,  2.3767e-01,  2.5896e-01,
            2.7734e-01,  3.0031e-01],
          [ 5.4276e-01,  5.9772e-01,  5.9097e-01,  5.3136e-01,  5.1583e-01,
            5.0651e-01,  5.6647e-01,  5.4753e-01,  5.0806e-01,  4.0740e-01,
            4.0540e-01,  5.0816e-01,  6.8505e-01,  7.4254e-01,  8.2028e-01,
            8.0660e-01,  7.2687e-01,  6.7546e-01,  6.7703e-01,  6.8265e-01,
            6.2562e-01,  3.5964e-01,  2.9409e-01,  2.8006e-01,  2.6625e-01,
            2.7486e-01,  2.7885e-01,  2.4125e-01,  2.4064e-01,  2.7340e-01,
            2.7307e-01,  2.9004e-01],
          [ 4.9508e-01,  6.0395e-01,  6.2031e-01,  5.9506e-01,  5.4156e-01,
            5.0445e-01,  5.0495e-01,  4.5452e-01,  4.0611e-01,  3.7564e-01,
            3.8788e-01,  5.3800e-01,  6.6424e-01,  6.1166e-01,  5.5582e-01,
            5.2752e-01,  5.2161e-01,  5.2452e-01,  5.0262e-01,  4.7523e-01,
            4.1367e-01,  3.5688e-01,  3.3023e-01,  3.3207e-01,  3.0358e-01,
            3.2097e-01,  3.2801e-01,  3.0926e-01,  2.9534e-01,  3.0615e-01,
            2.8515e-01,  2.9106e-01],
          [ 4.4096e-01,  5.9511e-01,  6.8359e-01,  6.1541e-01,  4.5844e-01,
            5.2232e-01,  5.5563e-01,  4.6637e-01,  3.8782e-01,  3.6869e-01,
            3.8491e-01,  4.5598e-01,  4.6544e-01,  4.5681e-01,  4.2783e-01,
            4.1984e-01,  4.2348e-01,  4.3375e-01,  4.0420e-01,  3.9885e-01,
            3.3314e-01,  3.3655e-01,  3.5792e-01,  3.3074e-01,  3.2865e-01,
            3.3513e-01,  3.4259e-01,  3.3244e-01,  3.2044e-01,  3.1838e-01,
            3.0446e-01,  2.8542e-01],
          [ 4.1927e-01,  5.0415e-01,  5.7149e-01,  5.1821e-01,  4.4784e-01,
            4.3707e-01,  5.4538e-01,  4.8006e-01,  3.7199e-01,  3.3348e-01,
            3.7567e-01,  3.7573e-01,  3.8048e-01,  3.9331e-01,  4.0874e-01,
            3.9921e-01,  3.9493e-01,  3.9867e-01,  3.7363e-01,  3.6384e-01,
            3.3146e-01,  3.3679e-01,  3.4884e-01,  3.4194e-01,  3.4754e-01,
            3.3490e-01,  3.5365e-01,  3.3848e-01,  3.2568e-01,  3.2882e-01,
            3.2595e-01,  3.3315e-01],
          [ 3.9973e-01,  4.2777e-01,  4.0325e-01,  4.0568e-01,  4.6409e-01,
            4.1722e-01,  4.7642e-01,  4.7328e-01,  3.6655e-01,  3.3805e-01,
            4.0213e-01,  4.1613e-01,  4.0930e-01,  3.8101e-01,  3.9648e-01,
            3.9518e-01,  3.8343e-01,  3.7792e-01,  3.6818e-01,  3.6466e-01,
            3.6419e-01,  3.7611e-01,  3.6627e-01,  3.6628e-01,  3.5826e-01,
            3.2539e-01,  3.3869e-01,  3.3885e-01,  3.4746e-01,  3.4686e-01,
            3.6889e-01,  3.6368e-01],
          [ 3.7120e-01,  3.8165e-01,  3.7560e-01,  3.7653e-01,  4.2488e-01,
            3.9869e-01,  3.7311e-01,  4.3815e-01,  3.8417e-01,  3.6247e-01,
            4.2247e-01,  4.7084e-01,  4.1817e-01,  3.8702e-01,  3.9368e-01,
            4.0258e-01,  3.7358e-01,  3.5742e-01,  3.8986e-01,  3.7058e-01,
            3.6445e-01,  3.7755e-01,  3.7317e-01,  3.5976e-01,  3.5452e-01,
            3.5060e-01,  3.5295e-01,  3.5129e-01,  3.7544e-01,  3.5444e-01,
            3.7059e-01,  3.5706e-01],
          [ 3.4796e-01,  3.4521e-01,  3.6722e-01,  3.7136e-01,  4.1890e-01,
            4.4475e-01,  4.1772e-01,  4.4604e-01,  4.1686e-01,  3.8727e-01,
            4.2612e-01,  4.5971e-01,  4.0647e-01,  4.1069e-01,  4.0974e-01,
            3.9301e-01,  3.4968e-01,  3.4811e-01,  3.6450e-01,  3.7726e-01,
            3.8029e-01,  3.6354e-01,  3.5864e-01,  3.7084e-01,  3.5489e-01,
            3.4108e-01,  3.4412e-01,  3.4715e-01,  3.6796e-01,  3.5870e-01,
            3.7756e-01,  3.9173e-01]],

         [[ 1.9723e-01,  1.9167e-01,  1.8202e-01,  1.7842e-01,  2.2411e-01,
            2.4083e-01,  2.5957e-01,  2.8790e-01,  2.7194e-01,  2.3802e-01,
            2.5177e-01,  2.2994e-01,  2.2455e-01,  2.1613e-01,  2.4622e-01,
            2.4200e-01,  2.5532e-01,  2.5816e-01,  2.7802e-01,  2.7468e-01,
            2.6913e-01,  2.6008e-01,  2.4891e-01,  2.7919e-01,  2.7887e-01,
            2.2632e-01,  1.8239e-01,  2.1016e-01,  2.3503e-01,  2.2518e-01,
            2.2977e-01,  2.5460e-01],
          [ 2.0134e-01,  1.8357e-01,  1.7825e-01,  2.0403e-01,  2.5982e-01,
            2.7402e-01,  2.7106e-01,  2.5391e-01,  2.7474e-01,  2.3356e-01,
            2.4767e-01,  2.1306e-01,  2.0902e-01,  2.0587e-01,  2.3561e-01,
            2.3491e-01,  2.3488e-01,  2.5253e-01,  2.5012e-01,  2.4488e-01,
            2.4207e-01,  2.3527e-01,  2.3749e-01,  2.5553e-01,  2.4703e-01,
            2.2443e-01,  2.0923e-01,  2.2998e-01,  2.2827e-01,  2.2118e-01,
            2.2329e-01,  2.1577e-01],
          [ 1.9697e-01,  1.7904e-01,  1.9133e-01,  2.3941e-01,  2.7146e-01,
            2.6722e-01,  2.5556e-01,  2.5933e-01,  2.3953e-01,  2.3422e-01,
            2.3390e-01,  2.1354e-01,  2.0986e-01,  2.1199e-01,  2.0192e-01,
            2.3448e-01,  2.3575e-01,  2.4515e-01,  2.5332e-01,  2.6264e-01,
            2.3175e-01,  2.4566e-01,  2.3935e-01,  2.5468e-01,  2.2059e-01,
            2.0574e-01,  2.2766e-01,  2.4451e-01,  2.2390e-01,  2.2824e-01,
            2.2553e-01,  1.9901e-01],
          [ 1.9462e-01,  1.8712e-01,  2.1780e-01,  2.4264e-01,  2.6322e-01,
            2.5951e-01,  2.6476e-01,  2.6202e-01,  2.5088e-01,  2.6430e-01,
            2.5879e-01,  2.3457e-01,  2.3187e-01,  2.2033e-01,  2.0016e-01,
            1.9226e-01,  2.3165e-01,  2.3098e-01,  2.4821e-01,  2.4532e-01,
            2.2359e-01,  2.2150e-01,  2.3203e-01,  2.2942e-01,  2.3123e-01,
            2.4455e-01,  2.4074e-01,  2.4610e-01,  2.2924e-01,  2.2386e-01,
            2.1501e-01,  2.1335e-01],
          [ 2.0502e-01,  1.9149e-01,  2.2588e-01,  2.4490e-01,  2.6558e-01,
            2.7860e-01,  2.9335e-01,  2.8708e-01,  2.6209e-01,  2.5486e-01,
            2.5948e-01,  2.2954e-01,  2.4842e-01,  2.3392e-01,  2.0164e-01,
            2.1556e-01,  2.3203e-01,  2.3729e-01,  2.3605e-01,  2.2166e-01,
            2.0337e-01,  2.1813e-01,  2.2730e-01,  2.2738e-01,  2.5884e-01,
            2.4235e-01,  2.4167e-01,  2.2000e-01,  2.1997e-01,  2.3518e-01,
            2.3567e-01,  2.2852e-01],
          [ 1.9378e-01,  1.9508e-01,  2.2589e-01,  2.3319e-01,  2.6111e-01,
            3.0063e-01,  3.0793e-01,  3.1552e-01,  2.7360e-01,  2.5954e-01,
            2.5859e-01,  2.6767e-01,  2.7843e-01,  2.4503e-01,  2.3521e-01,
            2.3222e-01,  2.3056e-01,  2.4705e-01,  2.3106e-01,  2.3989e-01,
            2.2000e-01,  2.1531e-01,  2.2093e-01,  2.3152e-01,  2.4873e-01,
            2.5846e-01,  2.2859e-01,  2.2601e-01,  2.1872e-01,  2.3870e-01,
            2.2736e-01,  2.4895e-01],
          [ 2.0015e-01,  1.8142e-01,  2.1842e-01,  2.4168e-01,  2.3257e-01,
            2.7019e-01,  3.1151e-01,  3.1168e-01,  2.7957e-01,  2.2267e-01,
            2.6985e-01,  2.7357e-01,  2.6068e-01,  2.6206e-01,  2.5453e-01,
            2.2769e-01,  2.3316e-01,  2.3899e-01,  2.2643e-01,  2.3811e-01,
            2.3250e-01,  2.3801e-01,  2.1996e-01,  2.3653e-01,  2.3573e-01,
            2.4758e-01,  2.4228e-01,  2.3916e-01,  2.2845e-01,  2.3102e-01,
            2.4576e-01,  2.3083e-01],
          [ 2.1692e-01,  1.8723e-01,  2.2512e-01,  2.2133e-01,  2.3763e-01,
            2.7995e-01,  3.0729e-01,  3.0592e-01,  2.6831e-01,  2.4687e-01,
            2.7786e-01,  2.5653e-01,  2.5665e-01,  2.6826e-01,  2.4518e-01,
            2.5915e-01,  2.3716e-01,  2.4357e-01,  2.5963e-01,  2.4769e-01,
            2.2928e-01,  2.2639e-01,  2.3089e-01,  2.4158e-01,  2.3606e-01,
            2.4408e-01,  2.4314e-01,  2.2476e-01,  2.1762e-01,  2.3576e-01,
            2.4772e-01,  2.4343e-01],
          [ 2.1117e-01,  2.0238e-01,  2.3982e-01,  2.2888e-01,  2.3673e-01,
            2.5588e-01,  2.6876e-01,  2.7414e-01,  2.6266e-01,  2.5385e-01,
            2.6870e-01,  2.6078e-01,  2.5192e-01,  2.4824e-01,  2.3254e-01,
            2.3937e-01,  2.5166e-01,  2.5389e-01,  2.5507e-01,  2.5148e-01,
            2.6418e-01,  2.4022e-01,  2.3976e-01,  2.3542e-01,  2.3483e-01,
            2.6359e-01,  2.5670e-01,  2.4234e-01,  2.2985e-01,  2.4330e-01,
            2.2374e-01,  2.3975e-01],
          [ 1.7841e-01,  1.7457e-01,  1.9468e-01,  2.3503e-01,  2.2751e-01,
            2.3105e-01,  2.6465e-01,  2.6408e-01,  2.5500e-01,  2.5327e-01,
            2.7095e-01,  2.7331e-01,  2.6650e-01,  2.4885e-01,  2.3429e-01,
            2.3563e-01,  2.5974e-01,  2.6458e-01,  2.5944e-01,  2.5023e-01,
            2.5186e-01,  2.3774e-01,  2.2755e-01,  2.4089e-01,  2.2947e-01,
            2.5283e-01,  2.4968e-01,  2.3245e-01,  2.3227e-01,  2.1679e-01,
            2.2318e-01,  2.1625e-01],
          [ 1.9990e-01,  1.9668e-01,  2.1094e-01,  2.2600e-01,  2.2487e-01,
            2.4688e-01,  2.6760e-01,  2.5002e-01,  2.5660e-01,  2.5137e-01,
            2.7716e-01,  2.8318e-01,  2.6667e-01,  2.5983e-01,  2.6346e-01,
            2.4343e-01,  2.5669e-01,  2.6253e-01,  2.6634e-01,  2.6379e-01,
            2.4497e-01,  2.3594e-01,  2.3932e-01,  2.4544e-01,  2.5357e-01,
            2.6609e-01,  2.3565e-01,  2.0767e-01,  1.9014e-01,  2.1489e-01,
            2.2587e-01,  2.2409e-01],
          [ 2.1905e-01,  2.2470e-01,  2.2885e-01,  2.2981e-01,  2.3367e-01,
            2.5496e-01,  3.0493e-01,  2.7990e-01,  2.6350e-01,  2.7403e-01,
            2.8026e-01,  2.9737e-01,  2.7911e-01,  2.7115e-01,  2.7644e-01,
            2.6113e-01,  2.5393e-01,  2.5929e-01,  2.6869e-01,  2.6587e-01,
            2.6106e-01,  2.6259e-01,  2.5318e-01,  2.4591e-01,  2.5659e-01,
            2.9569e-01,  2.5194e-01,  1.8128e-01,  1.9300e-01,  2.2743e-01,
            2.2728e-01,  2.2844e-01],
          [ 2.2137e-01,  2.0144e-01,  2.2996e-01,  2.2332e-01,  2.3996e-01,
            2.8304e-01,  3.1192e-01,  2.9323e-01,  2.7492e-01,  2.4688e-01,
            2.8088e-01,  2.7645e-01,  2.7013e-01,  2.6242e-01,  2.9169e-01,
            2.8202e-01,  2.8325e-01,  2.6040e-01,  2.3838e-01,  2.5908e-01,
            2.5385e-01,  2.4457e-01,  2.6270e-01,  2.4990e-01,  2.8742e-01,
            3.5865e-01,  2.9168e-01,  2.2048e-01,  2.0452e-01,  2.5432e-01,
            2.3283e-01,  2.4372e-01],
          [ 2.1940e-01,  2.1334e-01,  2.1364e-01,  2.1509e-01,  2.4929e-01,
            2.7083e-01,  2.7052e-01,  2.8135e-01,  2.6211e-01,  2.2966e-01,
            2.3828e-01,  2.6430e-01,  2.7313e-01,  2.6787e-01,  2.7295e-01,
            2.6935e-01,  3.1448e-01,  2.7593e-01,  2.5852e-01,  2.6080e-01,
            2.6019e-01,  2.6758e-01,  2.6523e-01,  2.5886e-01,  3.9408e-01,
            4.3002e-01,  3.4034e-01,  2.5847e-01,  2.1252e-01,  2.4122e-01,
            2.4650e-01,  2.5849e-01],
          [ 2.0937e-01,  2.1623e-01,  2.2104e-01,  2.0414e-01,  2.2515e-01,
            2.5399e-01,  2.4324e-01,  2.7785e-01,  2.3676e-01,  2.0306e-01,
            2.5208e-01,  2.4571e-01,  2.6344e-01,  2.5406e-01,  2.2194e-01,
            2.4633e-01,  3.0232e-01,  3.1166e-01,  3.1402e-01,  3.3925e-01,
            3.4230e-01,  3.7385e-01,  3.6715e-01,  3.3631e-01,  4.9424e-01,
            4.5313e-01,  3.4608e-01,  2.3742e-01,  2.3080e-01,  2.4814e-01,
            2.9232e-01,  2.9889e-01],
          [ 2.1554e-01,  2.1489e-01,  2.3220e-01,  2.2502e-01,  2.2435e-01,
            2.4072e-01,  2.4042e-01,  2.6759e-01,  2.2826e-01,  2.2237e-01,
            2.3434e-01,  2.3447e-01,  2.6716e-01,  2.5901e-01,  2.1289e-01,
            2.4973e-01,  2.6711e-01,  2.8582e-01,  3.1751e-01,  3.6504e-01,
            3.7995e-01,  4.0761e-01,  3.7780e-01,  3.8959e-01,  4.8276e-01,
            4.3801e-01,  3.1697e-01,  2.3858e-01,  2.3647e-01,  2.2642e-01,
            3.0512e-01,  3.0243e-01],
          [ 2.0059e-01,  2.1437e-01,  2.3617e-01,  2.2155e-01,  2.4188e-01,
            2.6015e-01,  2.4795e-01,  2.5739e-01,  2.6386e-01,  2.5212e-01,
            2.4595e-01,  2.3686e-01,  3.3088e-01,  3.0903e-01,  2.5840e-01,
            2.6260e-01,  2.3006e-01,  2.4516e-01,  3.0074e-01,  3.4879e-01,
            3.8882e-01,  4.1507e-01,  3.8928e-01,  3.8084e-01,  4.3623e-01,
            3.6203e-01,  2.4397e-01,  2.1473e-01,  2.4896e-01,  2.3178e-01,
            2.5833e-01,  2.9306e-01],
          [ 3.5573e-01,  4.0518e-01,  3.9247e-01,  3.7134e-01,  4.0397e-01,
            4.2353e-01,  4.2516e-01,  3.9935e-01,  3.6957e-01,  3.2692e-01,
            3.3924e-01,  4.0703e-01,  4.8329e-01,  4.7688e-01,  4.2854e-01,
            4.0292e-01,  3.4719e-01,  3.2420e-01,  3.1804e-01,  3.4901e-01,
            3.7358e-01,  4.4249e-01,  4.0188e-01,  3.1920e-01,  3.4468e-01,
            2.9837e-01,  2.1759e-01,  1.8449e-01,  2.2456e-01,  2.2986e-01,
            2.4309e-01,  2.5528e-01],
          [ 6.0511e-01,  6.0331e-01,  6.1471e-01,  6.2730e-01,  6.5445e-01,
            6.8198e-01,  6.8169e-01,  6.9422e-01,  6.4999e-01,  5.7773e-01,
            5.7067e-01,  6.7091e-01,  7.3389e-01,  7.3706e-01,  7.1164e-01,
            6.7406e-01,  5.1782e-01,  3.8963e-01,  3.7034e-01,  3.7940e-01,
            4.0936e-01,  4.6344e-01,  3.0914e-01,  2.1685e-01,  2.9930e-01,
            2.8340e-01,  2.3484e-01,  2.0270e-01,  2.0943e-01,  2.1512e-01,
            2.4551e-01,  2.4083e-01],
          [ 6.3709e-01,  6.3969e-01,  6.5591e-01,  6.7631e-01,  6.8691e-01,
            6.8444e-01,  7.0998e-01,  7.2468e-01,  7.0036e-01,  6.6529e-01,
            6.4540e-01,  7.1268e-01,  7.6943e-01,  7.4043e-01,  7.3359e-01,
            7.1685e-01,  6.3714e-01,  4.9829e-01,  4.2079e-01,  4.0674e-01,
            3.8078e-01,  3.4432e-01,  1.9813e-01,  2.2510e-01,  3.0249e-01,
            3.1219e-01,  2.8321e-01,  2.7793e-01,  2.3840e-01,  2.6005e-01,
            2.8652e-01,  2.7596e-01],
          [ 6.5631e-01,  6.4167e-01,  6.6549e-01,  6.7817e-01,  6.7798e-01,
            6.5099e-01,  6.6271e-01,  6.8189e-01,  6.9475e-01,  6.7015e-01,
            6.6238e-01,  6.6530e-01,  6.8871e-01,  6.7092e-01,  6.8292e-01,
            6.6062e-01,  6.5685e-01,  6.0858e-01,  4.6797e-01,  4.3017e-01,
            4.1902e-01,  2.9931e-01,  2.8212e-01,  2.8008e-01,  2.9932e-01,
            2.9946e-01,  3.0696e-01,  2.6328e-01,  2.3356e-01,  2.5261e-01,
            2.6210e-01,  2.5817e-01],
          [ 6.3851e-01,  6.4966e-01,  6.5277e-01,  6.5167e-01,  6.6163e-01,
            6.3284e-01,  6.5001e-01,  6.5959e-01,  6.5129e-01,  6.5455e-01,
            6.3484e-01,  6.2167e-01,  6.5233e-01,  6.5773e-01,  6.5095e-01,
            6.2898e-01,  6.4382e-01,  5.9922e-01,  5.1922e-01,  5.1174e-01,
            4.3499e-01,  3.3485e-01,  2.6254e-01,  3.0789e-01,  2.9168e-01,
            2.8498e-01,  2.8650e-01,  2.4333e-01,  2.3408e-01,  2.4501e-01,
            2.5247e-01,  2.5288e-01],
          [ 6.3850e-01,  6.4894e-01,  6.4088e-01,  6.4800e-01,  6.3941e-01,
            6.3839e-01,  6.2955e-01,  6.2974e-01,  6.0657e-01,  6.2829e-01,
            6.2372e-01,  6.2231e-01,  6.4290e-01,  6.6928e-01,  6.7697e-01,
            6.6189e-01,  6.5432e-01,  6.3704e-01,  6.5686e-01,  7.1151e-01,
            5.7858e-01,  3.0288e-01,  2.8709e-01,  3.1434e-01,  2.9661e-01,
            2.9884e-01,  2.7396e-01,  2.5786e-01,  2.2994e-01,  2.3997e-01,
            2.5121e-01,  2.4259e-01],
          [ 6.1799e-01,  6.2013e-01,  6.4597e-01,  6.4479e-01,  6.3617e-01,
            6.3969e-01,  6.3145e-01,  5.9921e-01,  5.9836e-01,  5.9275e-01,
            5.7443e-01,  5.8002e-01,  5.8789e-01,  7.0788e-01,  7.8908e-01,
            8.0953e-01,  8.2971e-01,  8.3777e-01,  8.6506e-01,  8.4752e-01,
            6.7582e-01,  3.3310e-01,  2.7314e-01,  2.9326e-01,  2.9779e-01,
            2.9386e-01,  2.8490e-01,  2.7060e-01,  2.4977e-01,  2.4400e-01,
            2.7297e-01,  2.7929e-01],
          [ 6.1147e-01,  6.1232e-01,  6.2344e-01,  5.9248e-01,  5.4317e-01,
            5.6538e-01,  6.2553e-01,  6.1407e-01,  5.5981e-01,  5.0826e-01,
            5.0624e-01,  5.1379e-01,  5.7555e-01,  7.6144e-01,  9.0464e-01,
            9.2075e-01,  8.8013e-01,  8.2561e-01,  8.3611e-01,  8.1338e-01,
            7.2887e-01,  3.4918e-01,  2.6821e-01,  3.0749e-01,  2.8039e-01,
            2.6642e-01,  2.8014e-01,  2.5845e-01,  2.4786e-01,  2.6062e-01,
            2.7616e-01,  2.9283e-01],
          [ 5.7890e-01,  6.0676e-01,  5.8443e-01,  5.2838e-01,  4.8519e-01,
            4.7837e-01,  5.6475e-01,  5.6323e-01,  5.2217e-01,  4.2363e-01,
            4.3490e-01,  4.6516e-01,  6.3284e-01,  7.1637e-01,  8.2314e-01,
            8.2049e-01,  7.2929e-01,  6.7006e-01,  6.4682e-01,  6.6047e-01,
            6.2040e-01,  3.5252e-01,  2.8290e-01,  2.7721e-01,  2.7362e-01,
            2.8528e-01,  2.7133e-01,  2.5677e-01,  2.5054e-01,  2.7707e-01,
            2.8403e-01,  2.8185e-01],
          [ 5.2879e-01,  6.2689e-01,  6.1290e-01,  5.6673e-01,  4.8562e-01,
            4.7968e-01,  4.9196e-01,  4.3537e-01,  3.8760e-01,  3.8980e-01,
            3.8766e-01,  4.8504e-01,  5.7211e-01,  5.6903e-01,  5.5283e-01,
            5.1820e-01,  5.0043e-01,  5.0439e-01,  4.5858e-01,  4.5131e-01,
            4.2525e-01,  3.3482e-01,  3.3805e-01,  3.2726e-01,  2.9201e-01,
            3.3007e-01,  3.2321e-01,  2.9026e-01,  2.9514e-01,  2.9461e-01,
            2.8437e-01,  2.7960e-01],
          [ 4.8516e-01,  6.1080e-01,  6.8419e-01,  6.0432e-01,  4.3635e-01,
            5.0143e-01,  5.5562e-01,  4.6345e-01,  3.7076e-01,  3.6857e-01,
            3.9732e-01,  4.1712e-01,  4.3288e-01,  4.3064e-01,  4.1152e-01,
            4.1078e-01,  4.0048e-01,  3.9595e-01,  3.9161e-01,  3.7669e-01,
            3.2781e-01,  3.2638e-01,  3.4471e-01,  3.4001e-01,  3.2361e-01,
            3.3221e-01,  3.3472e-01,  3.2892e-01,  3.1547e-01,  2.9732e-01,
            3.0934e-01,  3.0206e-01],
          [ 4.6717e-01,  5.5826e-01,  5.6340e-01,  5.2165e-01,  4.5499e-01,
            4.4756e-01,  5.4375e-01,  4.6660e-01,  3.7161e-01,  3.3731e-01,
            3.6434e-01,  3.8518e-01,  3.9185e-01,  4.0253e-01,  4.0324e-01,
            3.8767e-01,  3.7599e-01,  3.8002e-01,  3.5774e-01,  3.3720e-01,
            3.0592e-01,  3.1853e-01,  3.4337e-01,  3.3703e-01,  3.3768e-01,
            3.2147e-01,  3.0981e-01,  3.3402e-01,  2.9829e-01,  3.0580e-01,
            3.2071e-01,  3.0662e-01],
          [ 4.5933e-01,  4.7990e-01,  4.3340e-01,  4.3511e-01,  4.6704e-01,
            4.1230e-01,  4.8092e-01,  4.5923e-01,  3.6518e-01,  3.3322e-01,
            3.9192e-01,  4.0240e-01,  4.0260e-01,  3.8777e-01,  3.7029e-01,
            3.8536e-01,  3.7902e-01,  3.7206e-01,  3.5820e-01,  3.4599e-01,
            3.3540e-01,  3.5407e-01,  3.4012e-01,  3.4266e-01,  3.2492e-01,
            3.1189e-01,  3.1438e-01,  3.0979e-01,  3.1121e-01,  3.1033e-01,
            3.4964e-01,  3.3055e-01],
          [ 4.1355e-01,  4.1841e-01,  4.0937e-01,  4.0296e-01,  4.3847e-01,
            3.8371e-01,  3.8237e-01,  4.3962e-01,  3.7581e-01,  3.4477e-01,
            4.0699e-01,  4.4424e-01,  3.9952e-01,  3.7891e-01,  3.8226e-01,
            3.8723e-01,  3.6455e-01,  3.5932e-01,  3.5960e-01,  3.5313e-01,
            3.4124e-01,  3.4759e-01,  3.5076e-01,  3.3453e-01,  3.2122e-01,
            3.1212e-01,  3.1763e-01,  3.1659e-01,  3.3502e-01,  3.0970e-01,
            3.4293e-01,  3.5593e-01],
          [ 3.8219e-01,  4.0276e-01,  3.8709e-01,  3.8810e-01,  4.2720e-01,
            4.3652e-01,  4.3417e-01,  4.4896e-01,  4.0140e-01,  3.7996e-01,
            4.0613e-01,  4.3447e-01,  3.8101e-01,  3.8797e-01,  4.2006e-01,
            3.9212e-01,  3.4040e-01,  3.5599e-01,  3.4965e-01,  3.5369e-01,
            3.4720e-01,  3.6073e-01,  3.5063e-01,  3.6941e-01,  3.3365e-01,
            3.2430e-01,  3.2029e-01,  3.2423e-01,  3.3962e-01,  3.2238e-01,
            3.4493e-01,  3.5452e-01]],

         [[ 1.6893e-01,  1.2877e-01,  1.5222e-01,  1.3737e-01,  1.8067e-01,
            2.1132e-01,  2.1836e-01,  2.4941e-01,  2.3949e-01,  1.9761e-01,
            2.1332e-01,  2.0216e-01,  1.9179e-01,  1.9992e-01,  2.0542e-01,
            2.0330e-01,  2.3162e-01,  2.2392e-01,  2.2649e-01,  2.0347e-01,
            1.9342e-01,  1.8220e-01,  1.8582e-01,  2.2116e-01,  2.2634e-01,
            1.8941e-01,  1.5430e-01,  1.8649e-01,  1.9384e-01,  1.9819e-01,
            1.8347e-01,  2.0331e-01],
          [ 1.5647e-01,  1.3348e-01,  1.3275e-01,  1.7159e-01,  2.1251e-01,
            2.2758e-01,  2.3830e-01,  2.4382e-01,  2.2305e-01,  2.0406e-01,
            2.1104e-01,  1.9652e-01,  1.9081e-01,  1.9083e-01,  1.9650e-01,
            2.1463e-01,  2.0924e-01,  2.2401e-01,  2.0965e-01,  2.0373e-01,
            2.1222e-01,  1.7481e-01,  1.9935e-01,  2.2479e-01,  2.1119e-01,
            1.9255e-01,  1.7988e-01,  2.0930e-01,  2.1370e-01,  1.9444e-01,
            1.7150e-01,  1.7055e-01],
          [ 1.5641e-01,  1.3510e-01,  1.5794e-01,  1.9936e-01,  2.2852e-01,
            2.4775e-01,  2.3430e-01,  2.4606e-01,  2.1306e-01,  2.0496e-01,
            2.1288e-01,  1.7770e-01,  1.7388e-01,  1.8245e-01,  1.8467e-01,
            2.0626e-01,  2.2508e-01,  2.1116e-01,  2.2241e-01,  2.1442e-01,
            1.9756e-01,  2.0300e-01,  1.9856e-01,  2.0886e-01,  1.8575e-01,
            1.8640e-01,  1.9271e-01,  2.0643e-01,  2.1322e-01,  1.9617e-01,
            1.9611e-01,  1.5501e-01],
          [ 1.6698e-01,  1.6662e-01,  1.8889e-01,  2.1806e-01,  2.2431e-01,
            2.3808e-01,  2.4034e-01,  2.4780e-01,  2.2508e-01,  2.2101e-01,
            2.2009e-01,  1.9593e-01,  1.8961e-01,  1.8126e-01,  1.7828e-01,
            1.8996e-01,  2.1882e-01,  2.1189e-01,  2.0406e-01,  2.0374e-01,
            1.8921e-01,  1.6772e-01,  1.8948e-01,  2.1129e-01,  2.1503e-01,
            2.1347e-01,  2.0064e-01,  1.9952e-01,  2.0543e-01,  1.9461e-01,
            1.8920e-01,  1.8225e-01],
          [ 1.8133e-01,  1.6699e-01,  2.1451e-01,  2.2907e-01,  2.4106e-01,
            2.5506e-01,  2.6597e-01,  2.7741e-01,  2.4852e-01,  2.1643e-01,
            2.0561e-01,  1.9300e-01,  2.0116e-01,  2.1462e-01,  1.9321e-01,
            1.8995e-01,  2.0060e-01,  1.9342e-01,  1.9027e-01,  1.9364e-01,
            1.7155e-01,  1.8525e-01,  1.8777e-01,  2.0341e-01,  2.2783e-01,
            2.0180e-01,  1.8781e-01,  1.7599e-01,  2.0542e-01,  2.2042e-01,
            2.1383e-01,  1.8797e-01],
          [ 1.6217e-01,  1.4896e-01,  2.0349e-01,  2.0902e-01,  2.3006e-01,
            2.3957e-01,  2.8724e-01,  3.0587e-01,  2.6107e-01,  2.1288e-01,
            1.9819e-01,  2.0562e-01,  2.3396e-01,  2.1553e-01,  2.1023e-01,
            2.0447e-01,  1.8582e-01,  2.1056e-01,  2.0390e-01,  2.1093e-01,
            2.0173e-01,  1.9428e-01,  2.1394e-01,  2.2069e-01,  2.1434e-01,
            2.1520e-01,  1.7846e-01,  1.8756e-01,  2.0172e-01,  2.2035e-01,
            2.1021e-01,  2.0208e-01],
          [ 1.8118e-01,  1.6641e-01,  1.8745e-01,  2.0424e-01,  1.9511e-01,
            2.2911e-01,  2.8482e-01,  3.2870e-01,  2.6423e-01,  1.9666e-01,
            2.1680e-01,  2.2804e-01,  2.1643e-01,  2.1688e-01,  2.1282e-01,
            1.9648e-01,  1.9460e-01,  2.1354e-01,  2.0873e-01,  2.0244e-01,
            1.9797e-01,  1.9304e-01,  1.9330e-01,  2.2467e-01,  2.1624e-01,
            2.3281e-01,  2.1269e-01,  1.9150e-01,  2.0188e-01,  2.2280e-01,
            2.2127e-01,  2.1292e-01],
          [ 2.0444e-01,  1.8575e-01,  1.9668e-01,  2.0716e-01,  2.0638e-01,
            2.3624e-01,  2.6613e-01,  2.9807e-01,  2.5459e-01,  2.0889e-01,
            2.1940e-01,  2.1087e-01,  2.1079e-01,  2.3857e-01,  2.2895e-01,
            2.1847e-01,  2.0448e-01,  2.1086e-01,  2.2008e-01,  2.2026e-01,
            2.1107e-01,  2.0636e-01,  1.9288e-01,  2.0274e-01,  2.0762e-01,
            2.3512e-01,  2.2587e-01,  2.2100e-01,  2.1169e-01,  2.1128e-01,
            2.2933e-01,  2.2846e-01],
          [ 2.0120e-01,  1.8045e-01,  1.8521e-01,  1.9332e-01,  1.8262e-01,
            2.1704e-01,  2.4094e-01,  2.6359e-01,  2.3515e-01,  1.8611e-01,
            1.9739e-01,  2.0951e-01,  2.1846e-01,  2.3105e-01,  2.2856e-01,
            2.3112e-01,  2.1848e-01,  2.3290e-01,  2.1819e-01,  2.1083e-01,
            2.1758e-01,  2.0758e-01,  1.9151e-01,  1.9905e-01,  1.8973e-01,
            2.4280e-01,  2.3076e-01,  2.1040e-01,  1.9796e-01,  2.2424e-01,
            2.1078e-01,  2.0895e-01],
          [ 1.9107e-01,  1.5868e-01,  1.7077e-01,  2.0115e-01,  1.9332e-01,
            1.9018e-01,  2.2568e-01,  2.3488e-01,  2.3434e-01,  1.7300e-01,
            2.2680e-01,  2.3851e-01,  2.2144e-01,  2.2247e-01,  2.1772e-01,
            1.9995e-01,  2.3472e-01,  2.2947e-01,  2.2727e-01,  2.2214e-01,
            2.2043e-01,  2.0948e-01,  2.0175e-01,  1.9748e-01,  2.0163e-01,
            2.3959e-01,  2.5206e-01,  2.1491e-01,  1.9921e-01,  1.9503e-01,
            2.0091e-01,  2.0125e-01],
          [ 2.0598e-01,  1.7323e-01,  1.8243e-01,  1.8219e-01,  2.0477e-01,
            2.3094e-01,  2.6948e-01,  2.3280e-01,  2.2007e-01,  1.8107e-01,
            2.2718e-01,  2.5587e-01,  2.2782e-01,  2.2428e-01,  2.1015e-01,
            2.1945e-01,  2.2730e-01,  2.2739e-01,  2.2897e-01,  2.3172e-01,
            2.2525e-01,  2.1606e-01,  1.8824e-01,  2.0173e-01,  2.2952e-01,
            2.4989e-01,  2.2486e-01,  1.6595e-01,  1.6807e-01,  1.9121e-01,
            2.0349e-01,  1.9067e-01],
          [ 2.2658e-01,  2.0248e-01,  2.0410e-01,  1.9835e-01,  2.1447e-01,
            2.4985e-01,  2.9759e-01,  2.6675e-01,  2.3836e-01,  1.8479e-01,
            2.2267e-01,  2.2741e-01,  2.3365e-01,  2.2205e-01,  2.3806e-01,
            2.0263e-01,  2.0818e-01,  2.1505e-01,  2.4032e-01,  2.3239e-01,
            2.3611e-01,  2.4012e-01,  2.2148e-01,  2.2560e-01,  2.2610e-01,
            2.6452e-01,  2.3138e-01,  1.4355e-01,  1.6074e-01,  2.0173e-01,
            1.9818e-01,  1.9672e-01],
          [ 2.1716e-01,  1.9006e-01,  1.8241e-01,  1.8927e-01,  2.1890e-01,
            2.7147e-01,  2.9862e-01,  2.8545e-01,  2.4709e-01,  1.9607e-01,
            2.1723e-01,  2.2567e-01,  2.2781e-01,  2.2517e-01,  2.4425e-01,
            2.3441e-01,  2.3389e-01,  2.0859e-01,  2.0227e-01,  2.2622e-01,
            2.2568e-01,  2.1630e-01,  2.3548e-01,  2.3558e-01,  2.5313e-01,
            3.3146e-01,  2.5476e-01,  1.7474e-01,  1.8157e-01,  2.2701e-01,
            2.2298e-01,  2.3997e-01],
          [ 2.2174e-01,  1.8110e-01,  1.9512e-01,  2.0079e-01,  2.2307e-01,
            2.7380e-01,  2.6379e-01,  2.7062e-01,  2.3959e-01,  1.9131e-01,
            1.9635e-01,  2.0896e-01,  2.4250e-01,  2.2623e-01,  2.3287e-01,
            2.4058e-01,  2.5335e-01,  2.1992e-01,  2.0305e-01,  1.9510e-01,
            2.2042e-01,  2.4742e-01,  2.3705e-01,  2.4170e-01,  3.8545e-01,
            4.0968e-01,  2.9306e-01,  2.0125e-01,  1.8368e-01,  2.1459e-01,
            2.1306e-01,  2.4212e-01],
          [ 2.2192e-01,  2.0870e-01,  2.0107e-01,  1.9898e-01,  2.2253e-01,
            2.1725e-01,  2.2377e-01,  2.5410e-01,  2.1200e-01,  1.6050e-01,
            2.0132e-01,  2.0530e-01,  2.1547e-01,  2.1890e-01,  2.0780e-01,
            2.1594e-01,  2.5906e-01,  2.5243e-01,  2.5340e-01,  2.7697e-01,
            2.9164e-01,  3.3418e-01,  3.3568e-01,  3.0123e-01,  4.5931e-01,
            4.1261e-01,  3.0399e-01,  2.0877e-01,  2.1944e-01,  2.5407e-01,
            2.8770e-01,  2.7507e-01],
          [ 2.1132e-01,  2.1629e-01,  2.1955e-01,  2.1546e-01,  2.1865e-01,
            2.2050e-01,  2.3199e-01,  2.4271e-01,  2.0946e-01,  2.0192e-01,
            1.9550e-01,  2.0511e-01,  1.9782e-01,  1.8125e-01,  1.6172e-01,
            1.9665e-01,  2.1987e-01,  2.4152e-01,  2.7473e-01,  3.1951e-01,
            3.1699e-01,  3.5262e-01,  3.3109e-01,  3.4830e-01,  4.5213e-01,
            4.1683e-01,  2.9048e-01,  2.5988e-01,  2.6684e-01,  2.4793e-01,
            2.7933e-01,  3.0754e-01],
          [ 1.8886e-01,  2.1776e-01,  2.2260e-01,  2.2619e-01,  2.2696e-01,
            2.1798e-01,  2.2463e-01,  2.2592e-01,  2.4614e-01,  2.2741e-01,
            1.9575e-01,  1.8594e-01,  2.6037e-01,  2.3775e-01,  1.9603e-01,
            2.0687e-01,  1.8406e-01,  1.9386e-01,  2.5252e-01,  2.9841e-01,
            3.3990e-01,  3.6418e-01,  3.4574e-01,  3.4411e-01,  3.9665e-01,
            3.3016e-01,  2.4259e-01,  2.3638e-01,  2.4537e-01,  2.2108e-01,
            2.4993e-01,  2.8507e-01],
          [ 3.8076e-01,  4.1941e-01,  4.1992e-01,  3.9214e-01,  4.0445e-01,
            4.1789e-01,  4.2251e-01,  3.9482e-01,  3.4925e-01,  3.0740e-01,
            3.3129e-01,  3.5372e-01,  4.1925e-01,  4.2087e-01,  3.9699e-01,
            3.5248e-01,  2.4651e-01,  2.2091e-01,  2.4537e-01,  2.6018e-01,
            3.1420e-01,  4.0988e-01,  3.6546e-01,  2.8666e-01,  3.3820e-01,
            2.9095e-01,  2.1518e-01,  1.9056e-01,  2.0774e-01,  2.2726e-01,
            2.3215e-01,  2.5316e-01],
          [ 6.9573e-01,  6.9194e-01,  6.8680e-01,  7.0357e-01,  7.0644e-01,
            7.2895e-01,  7.2512e-01,  7.1587e-01,  6.6083e-01,  5.8235e-01,
            5.7843e-01,  6.4744e-01,  7.1223e-01,  7.3859e-01,  7.1586e-01,
            6.3887e-01,  4.6371e-01,  3.2128e-01,  2.9048e-01,  3.0918e-01,
            3.4774e-01,  4.4152e-01,  2.9010e-01,  1.8647e-01,  2.9947e-01,
            2.9295e-01,  2.2342e-01,  2.1029e-01,  1.9262e-01,  2.1326e-01,
            2.4108e-01,  2.5373e-01],
          [ 7.1119e-01,  7.2811e-01,  7.3075e-01,  7.5460e-01,  7.4600e-01,
            7.3852e-01,  7.4755e-01,  7.7229e-01,  7.4387e-01,  6.9361e-01,
            6.7561e-01,  7.0063e-01,  7.7253e-01,  7.5547e-01,  7.3655e-01,
            7.3473e-01,  6.5074e-01,  5.2311e-01,  3.7303e-01,  3.5116e-01,
            3.5209e-01,  3.2110e-01,  1.8856e-01,  2.0302e-01,  2.8740e-01,
            3.0466e-01,  2.7185e-01,  2.5303e-01,  2.1968e-01,  2.2919e-01,
            2.7384e-01,  2.7222e-01],
          [ 7.1388e-01,  7.1454e-01,  7.1931e-01,  7.2646e-01,  7.2246e-01,
            6.8011e-01,  6.8909e-01,  7.1707e-01,  7.1304e-01,  7.0825e-01,
            6.9358e-01,  7.0394e-01,  7.2599e-01,  6.9729e-01,  7.0002e-01,
            7.0604e-01,  7.1638e-01,  6.4778e-01,  4.7486e-01,  4.1677e-01,
            3.9065e-01,  2.8812e-01,  2.5113e-01,  2.6721e-01,  2.7344e-01,
            2.7837e-01,  2.9544e-01,  2.4652e-01,  2.1551e-01,  2.6479e-01,
            2.5831e-01,  2.4977e-01],
          [ 7.0508e-01,  7.2847e-01,  7.1615e-01,  7.1297e-01,  6.9830e-01,
            6.8172e-01,  6.7923e-01,  7.0021e-01,  6.8880e-01,  7.0358e-01,
            6.8349e-01,  6.6977e-01,  7.1165e-01,  6.9928e-01,  6.8600e-01,
            6.6762e-01,  6.7921e-01,  6.5931e-01,  5.5019e-01,  5.1276e-01,
            4.3018e-01,  3.1833e-01,  2.5320e-01,  2.7940e-01,  2.7489e-01,
            2.8605e-01,  2.6429e-01,  2.2576e-01,  2.1993e-01,  2.5232e-01,
            2.3686e-01,  2.4319e-01],
          [ 6.9519e-01,  7.0571e-01,  7.2409e-01,  7.0588e-01,  6.8453e-01,
            6.9590e-01,  6.8091e-01,  6.6554e-01,  6.6728e-01,  6.8619e-01,
            6.7119e-01,  6.6750e-01,  6.8050e-01,  7.1930e-01,  7.0795e-01,
            6.9049e-01,  6.8939e-01,  6.6490e-01,  6.5812e-01,  7.3911e-01,
            5.9723e-01,  3.1626e-01,  2.7297e-01,  2.7412e-01,  2.8490e-01,
            2.7150e-01,  2.6956e-01,  2.4430e-01,  2.2945e-01,  2.3588e-01,
            2.4780e-01,  2.3770e-01],
          [ 6.7308e-01,  6.9306e-01,  7.0551e-01,  6.8654e-01,  6.5784e-01,
            6.6098e-01,  6.6267e-01,  6.4426e-01,  6.4539e-01,  6.3236e-01,
            6.2440e-01,  6.2731e-01,  6.5290e-01,  7.6689e-01,  8.1859e-01,
            8.2356e-01,  8.6676e-01,  8.5912e-01,  8.7455e-01,  8.4593e-01,
            7.0049e-01,  3.0905e-01,  2.3747e-01,  2.8159e-01,  2.8015e-01,
            2.8501e-01,  2.8545e-01,  2.5071e-01,  2.4399e-01,  2.4273e-01,
            2.5467e-01,  2.6339e-01],
          [ 6.5843e-01,  6.5869e-01,  6.6325e-01,  5.9958e-01,  5.3440e-01,
            5.4956e-01,  6.4424e-01,  6.4339e-01,  6.1184e-01,  5.2465e-01,
            5.3590e-01,  5.4933e-01,  5.9652e-01,  7.6796e-01,  9.2918e-01,
            9.2065e-01,  8.6797e-01,  8.1147e-01,  8.0188e-01,  7.9632e-01,
            7.0545e-01,  2.9773e-01,  2.3631e-01,  2.8168e-01,  2.6759e-01,
            2.7825e-01,  2.6986e-01,  2.4152e-01,  2.3040e-01,  2.5600e-01,
            2.5383e-01,  2.8967e-01],
          [ 6.4634e-01,  6.5105e-01,  5.9091e-01,  4.9936e-01,  4.3753e-01,
            4.2475e-01,  5.4634e-01,  5.5702e-01,  5.2563e-01,  4.3734e-01,
            4.3323e-01,  4.7443e-01,  6.2935e-01,  7.0865e-01,  8.1908e-01,
            7.7392e-01,  6.9800e-01,  6.4290e-01,  6.0907e-01,  6.2510e-01,
            5.7934e-01,  3.0250e-01,  2.4893e-01,  2.6107e-01,  2.7354e-01,
            2.6420e-01,  2.4755e-01,  2.5062e-01,  2.4317e-01,  2.6990e-01,
            2.6904e-01,  2.7152e-01],
          [ 5.7912e-01,  6.5890e-01,  6.2619e-01,  5.4894e-01,  4.4656e-01,
            4.3448e-01,  4.5931e-01,  4.1190e-01,  3.7791e-01,  3.6038e-01,
            3.7561e-01,  4.5371e-01,  5.4411e-01,  5.3507e-01,  4.9385e-01,
            4.6774e-01,  4.7892e-01,  4.8687e-01,  4.5888e-01,  4.3062e-01,
            3.9474e-01,  3.0312e-01,  3.0433e-01,  2.9857e-01,  2.7497e-01,
            2.8005e-01,  3.1666e-01,  2.9213e-01,  2.7447e-01,  2.8820e-01,
            3.0841e-01,  2.6999e-01],
          [ 5.4953e-01,  6.4992e-01,  6.8795e-01,  5.9032e-01,  3.7842e-01,
            4.6371e-01,  5.1276e-01,  4.2802e-01,  3.4146e-01,  3.4348e-01,
            3.4593e-01,  3.6319e-01,  3.6022e-01,  3.7225e-01,  3.5529e-01,
            3.3442e-01,  3.7396e-01,  3.6310e-01,  3.5971e-01,  3.4993e-01,
            3.0466e-01,  2.9185e-01,  2.9964e-01,  3.1411e-01,  3.0475e-01,
            3.0596e-01,  3.2733e-01,  3.0882e-01,  2.9700e-01,  3.0556e-01,
            3.0160e-01,  2.8408e-01],
          [ 5.1994e-01,  5.6710e-01,  5.8419e-01,  4.9296e-01,  3.8618e-01,
            3.7553e-01,  4.9284e-01,  4.4783e-01,  3.4140e-01,  3.0368e-01,
            3.2888e-01,  3.1160e-01,  3.2399e-01,  3.4989e-01,  3.4588e-01,
            3.3122e-01,  3.4362e-01,  3.4257e-01,  3.2266e-01,  3.1259e-01,
            2.7912e-01,  2.9563e-01,  2.9700e-01,  3.0258e-01,  3.0059e-01,
            3.0163e-01,  3.0720e-01,  2.9946e-01,  2.7920e-01,  2.9428e-01,
            3.1278e-01,  3.0683e-01],
          [ 4.7766e-01,  4.7524e-01,  4.2401e-01,  4.2561e-01,  4.2983e-01,
            3.7134e-01,  4.3727e-01,  4.2590e-01,  3.4036e-01,  3.0778e-01,
            3.7415e-01,  3.6887e-01,  3.3956e-01,  3.2035e-01,  3.2672e-01,
            3.2499e-01,  3.1526e-01,  3.1682e-01,  3.0551e-01,  2.9469e-01,
            3.1280e-01,  3.1090e-01,  3.0152e-01,  3.0600e-01,  2.9127e-01,
            2.8095e-01,  2.8592e-01,  2.8366e-01,  3.0345e-01,  2.8103e-01,
            3.1689e-01,  2.8917e-01],
          [ 4.2044e-01,  4.2268e-01,  3.9840e-01,  4.0237e-01,  3.8798e-01,
            3.3410e-01,  3.5144e-01,  4.1209e-01,  3.5129e-01,  3.5020e-01,
            3.9724e-01,  4.1166e-01,  3.4364e-01,  3.2795e-01,  3.4025e-01,
            3.3651e-01,  3.1474e-01,  2.9682e-01,  3.2108e-01,  3.0844e-01,
            3.1386e-01,  3.2733e-01,  3.0825e-01,  3.1547e-01,  2.9081e-01,
            2.8465e-01,  2.8206e-01,  2.9348e-01,  3.1241e-01,  3.0270e-01,
            3.0565e-01,  3.0913e-01],
          [ 3.8101e-01,  3.9035e-01,  3.9153e-01,  3.8776e-01,  4.1103e-01,
            3.9603e-01,  3.9506e-01,  4.3877e-01,  3.7955e-01,  3.5406e-01,
            3.8889e-01,  3.9509e-01,  3.4655e-01,  3.5983e-01,  3.6598e-01,
            3.5295e-01,  2.9211e-01,  2.9464e-01,  3.0871e-01,  3.1050e-01,
            3.1299e-01,  3.2000e-01,  3.0950e-01,  3.1342e-01,  3.0726e-01,
            2.8615e-01,  2.8494e-01,  2.8473e-01,  3.1520e-01,  2.9496e-01,
            3.0914e-01,  3.0633e-01]]],


        [[[ 2.9481e-02,  3.3401e-02,  4.6145e-02,  4.5235e-02,  3.3001e-02,
            3.2585e-02,  2.3136e-02,  5.4245e-02,  4.2493e-02,  5.6828e-02,
            4.6986e-02,  3.6752e-02,  2.5073e-02,  3.4729e-02,  3.4909e-02,
            5.6386e-02,  4.9676e-02,  4.2611e-02,  7.0727e-02,  4.6916e-02,
            6.6866e-02,  4.4254e-02,  6.9013e-02,  8.0599e-02,  6.9398e-02,
            4.7173e-02,  3.6694e-02,  4.9348e-02,  6.5878e-02,  6.3244e-02,
            7.5163e-02,  5.9242e-02],
          [ 3.1643e-02,  4.3825e-02,  3.2628e-02,  5.0100e-02,  4.6607e-02,
            4.3556e-02,  3.5970e-02,  3.9220e-02,  4.6679e-02,  4.0971e-02,
            3.3835e-02,  3.8676e-02,  2.8063e-02,  4.0843e-02,  4.3657e-02,
            6.4829e-02,  5.5766e-02,  5.3783e-02,  6.2594e-02,  6.2528e-02,
            5.6809e-02,  7.0276e-02,  7.2304e-02,  6.4544e-02,  7.5677e-02,
            5.0361e-02,  5.4637e-02,  5.7079e-02,  6.7350e-02,  6.8026e-02,
            7.7937e-02,  7.7085e-02],
          [ 2.5244e-02,  3.6362e-02,  3.2534e-02,  3.9789e-02,  4.4368e-02,
            4.1764e-02,  3.9073e-02,  4.2714e-02,  4.1180e-02,  5.5897e-02,
            5.7225e-02,  4.1182e-02,  2.0030e-02,  3.9216e-02,  5.3407e-02,
            4.9786e-02,  6.5298e-02,  7.7510e-02,  4.6516e-02,  5.5087e-02,
            6.5319e-02,  6.2385e-02,  7.7494e-02,  7.7541e-02,  6.2710e-02,
            7.5278e-02,  5.1382e-02,  5.7913e-02,  5.1272e-02,  6.5123e-02,
            5.8261e-02,  5.8652e-02],
          [ 1.9746e-02,  3.8252e-02,  4.7094e-02,  3.4692e-02,  3.9128e-02,
            4.0293e-02,  2.9805e-02,  2.6406e-02,  4.7243e-02,  4.9381e-02,
            5.7461e-02,  4.1944e-02,  4.3779e-02,  6.0384e-02,  3.8216e-02,
            5.2310e-02,  4.0600e-02,  5.5995e-02,  4.7082e-02,  4.5575e-02,
            7.7183e-02,  7.3275e-02,  7.6833e-02,  7.8060e-02,  7.7806e-02,
            7.1391e-02,  5.7387e-02,  6.0035e-02,  5.8268e-02,  6.9213e-02,
            6.2281e-02,  5.0791e-02],
          [ 3.9356e-02,  4.3000e-02,  3.4715e-02,  5.8783e-02,  4.2991e-02,
            3.5577e-02,  3.7650e-02,  4.3420e-02,  5.4688e-02,  6.6827e-02,
            6.2179e-02,  8.0962e-02,  8.4071e-02,  7.8660e-02,  7.4792e-02,
            6.7550e-02,  5.6899e-02,  5.5638e-02,  7.6167e-02,  8.9786e-02,
            1.0344e-01,  8.0966e-02,  7.8431e-02,  6.4898e-02,  6.1043e-02,
            5.7159e-02,  5.2979e-02,  6.2420e-02,  8.9826e-02,  8.4158e-02,
            5.1419e-02,  2.4908e-02],
          [ 4.0217e-02,  3.2791e-02,  2.8332e-02,  5.9955e-02,  3.0738e-02,
            2.7094e-02,  1.5884e-02,  2.6196e-02,  4.4173e-02,  8.2439e-02,
            1.2367e-01,  1.5418e-01,  1.9362e-01,  1.9312e-01,  1.9191e-01,
            2.1035e-01,  1.7852e-01,  1.7132e-01,  1.8753e-01,  2.0078e-01,
            1.3548e-01,  6.1795e-02,  5.7980e-02,  5.8394e-02,  7.0622e-02,
            4.9096e-02,  5.0610e-02,  7.7164e-02,  7.9234e-02,  6.9095e-02,
            5.6544e-02,  5.5681e-02],
          [ 3.7738e-02,  3.7062e-02,  3.6629e-02,  4.6981e-02,  4.8710e-02,
            2.0407e-02,  2.5317e-02,  2.7949e-02,  6.9515e-02,  1.7720e-01,
            3.5218e-01,  4.0264e-01,  3.9230e-01,  3.9434e-01,  4.0480e-01,
            3.8544e-01,  3.4240e-01,  3.0479e-01,  3.1739e-01,  2.9305e-01,
            1.2055e-01,  3.8863e-02,  6.0321e-02,  6.7848e-02,  7.3820e-02,
            6.4452e-02,  7.5876e-02,  8.5110e-02,  7.5709e-02,  7.0894e-02,
            7.4978e-02,  5.7067e-02],
          [ 3.9865e-02,  5.2007e-02,  7.2784e-02,  1.2094e-01,  1.8521e-01,
            1.0666e-01,  5.3035e-02,  5.4053e-02,  1.8102e-01,  4.3229e-01,
            4.9185e-01,  4.8031e-01,  4.0654e-01,  3.9093e-01,  4.6989e-01,
            4.6719e-01,  4.0417e-01,  4.0264e-01,  4.1025e-01,  3.4510e-01,
            1.1783e-01,  4.4575e-02,  4.7772e-02,  4.7933e-02,  5.5356e-02,
            8.2639e-02,  5.7908e-02,  7.1247e-02,  5.0089e-02,  5.5829e-02,
            5.9754e-02,  5.9520e-02],
          [ 6.5654e-02,  9.3831e-02,  1.2106e-01,  2.3948e-01,  2.9280e-01,
            1.2640e-01,  5.1212e-02,  9.2707e-02,  3.5372e-01,  4.2520e-01,
            3.5646e-01,  3.1066e-01,  3.1218e-01,  3.9717e-01,  4.8097e-01,
            4.9748e-01,  4.4793e-01,  3.8216e-01,  4.4902e-01,  3.7578e-01,
            1.5025e-01,  5.7982e-02,  5.3853e-02,  6.4521e-02,  5.7399e-02,
            6.2093e-02,  6.4566e-02,  5.0955e-02,  4.7372e-02,  6.7006e-02,
            1.0106e-01,  1.0463e-01],
          [ 1.3103e-01,  2.0547e-01,  2.3627e-01,  3.3848e-01,  3.7597e-01,
            9.5816e-02,  2.5947e-02,  1.8217e-01,  4.5643e-01,  4.1382e-01,
            3.9260e-01,  3.6953e-01,  4.6769e-01,  5.9778e-01,  5.5826e-01,
            4.9056e-01,  4.5828e-01,  3.8045e-01,  4.2678e-01,  3.7649e-01,
            1.7135e-01,  1.1599e-01,  1.4829e-01,  1.0377e-01,  6.0749e-02,
            6.9027e-02,  6.3528e-02,  6.6851e-02,  6.8260e-02,  9.3842e-02,
            1.3692e-01,  1.3923e-01],
          [ 1.3721e-01,  1.7603e-01,  2.0154e-01,  3.7901e-01,  3.6320e-01,
            5.5026e-02,  6.0256e-02,  2.9996e-01,  5.0069e-01,  4.8432e-01,
            5.0360e-01,  5.2400e-01,  5.3041e-01,  5.4817e-01,  5.2949e-01,
            4.8139e-01,  4.0180e-01,  3.8962e-01,  3.8486e-01,  3.4962e-01,
            1.8860e-01,  2.2629e-01,  3.1637e-01,  1.5601e-01,  9.5452e-02,
            6.8484e-02,  8.4375e-02,  1.0315e-01,  9.2193e-02,  1.3035e-01,
            1.8756e-01,  2.5604e-01],
          [ 7.0835e-02,  8.2256e-02,  1.9188e-01,  5.2705e-01,  3.3142e-01,
            2.3591e-02,  1.2089e-01,  4.2305e-01,  5.1741e-01,  5.2368e-01,
            5.6282e-01,  5.9557e-01,  5.4030e-01,  5.1749e-01,  5.1422e-01,
            4.3747e-01,  3.9012e-01,  4.0322e-01,  4.0570e-01,  3.2087e-01,
            2.0057e-01,  2.3755e-01,  3.3525e-01,  1.7917e-01,  9.8526e-02,
            8.8896e-02,  9.8334e-02,  1.0584e-01,  1.1152e-01,  1.7094e-01,
            2.9124e-01,  3.8946e-01],
          [ 5.1476e-02,  3.8674e-02,  1.3803e-01,  4.3655e-01,  2.7949e-01,
            1.4524e-02,  1.9319e-01,  4.7019e-01,  5.3563e-01,  5.4385e-01,
            5.8254e-01,  6.5979e-01,  6.2841e-01,  4.2423e-01,  2.7555e-01,
            3.8050e-01,  4.6651e-01,  4.4751e-01,  4.3202e-01,  3.7681e-01,
            2.1553e-01,  1.9350e-01,  2.3137e-01,  1.5032e-01,  1.0272e-01,
            8.3054e-02,  8.2939e-02,  1.0119e-01,  1.2156e-01,  1.9711e-01,
            3.6376e-01,  4.8319e-01],
          [ 8.0870e-02,  1.2748e-01,  1.8915e-01,  2.7573e-01,  1.8681e-01,
            2.4213e-02,  1.6679e-01,  3.9322e-01,  4.7384e-01,  5.3247e-01,
            5.9435e-01,  6.7276e-01,  6.4483e-01,  3.8495e-01,  2.8071e-01,
            3.8678e-01,  5.8686e-01,  5.6035e-01,  5.5645e-01,  5.4362e-01,
            3.0236e-01,  1.8678e-01,  1.5835e-01,  1.4865e-01,  1.2408e-01,
            9.2788e-02,  1.0929e-01,  1.0512e-01,  1.3035e-01,  2.4518e-01,
            4.2458e-01,  5.2300e-01],
          [ 2.9333e-01,  3.0778e-01,  3.4952e-01,  3.3328e-01,  1.9630e-01,
            6.3864e-02,  1.7692e-01,  2.7952e-01,  3.5049e-01,  4.8348e-01,
            5.7011e-01,  5.9923e-01,  6.4381e-01,  4.8742e-01,  5.3143e-01,
            6.7108e-01,  7.0182e-01,  6.6097e-01,  6.5110e-01,  5.8280e-01,
            4.5359e-01,  3.1287e-01,  2.3318e-01,  1.8108e-01,  1.3330e-01,
            1.1432e-01,  1.4057e-01,  1.3727e-01,  1.6002e-01,  3.2756e-01,
            4.9116e-01,  5.0197e-01],
          [ 4.0177e-01,  4.2171e-01,  3.8174e-01,  4.3117e-01,  3.0115e-01,
            1.3555e-01,  1.6060e-01,  2.0242e-01,  3.3271e-01,  5.1105e-01,
            6.0129e-01,  5.6761e-01,  6.3450e-01,  6.7240e-01,  7.2634e-01,
            7.4725e-01,  7.1967e-01,  7.1097e-01,  6.3932e-01,  5.9829e-01,
            5.8178e-01,  4.5751e-01,  2.9069e-01,  2.0363e-01,  1.2825e-01,
            1.2011e-01,  1.2699e-01,  1.5520e-01,  2.2485e-01,  4.0836e-01,
            5.3107e-01,  5.0576e-01],
          [ 4.2665e-01,  4.5683e-01,  4.2410e-01,  4.2665e-01,  3.9516e-01,
            2.7560e-01,  1.4334e-01,  1.3196e-01,  2.7094e-01,  4.8349e-01,
            6.1933e-01,  5.8056e-01,  5.8915e-01,  7.3734e-01,  7.5782e-01,
            7.4044e-01,  7.1920e-01,  7.6106e-01,  6.6273e-01,  6.3314e-01,
            6.2538e-01,  4.9670e-01,  2.7827e-01,  1.9039e-01,  1.2970e-01,
            1.0855e-01,  1.1778e-01,  1.7187e-01,  3.2745e-01,  4.8636e-01,
            5.5808e-01,  5.3078e-01],
          [ 4.8023e-01,  5.0364e-01,  4.8076e-01,  4.7742e-01,  4.3983e-01,
            3.1540e-01,  1.6723e-01,  1.2993e-01,  2.1805e-01,  4.1380e-01,
            5.7823e-01,  6.5758e-01,  6.7770e-01,  6.9939e-01,  7.4590e-01,
            7.9219e-01,  7.7659e-01,  7.8330e-01,  7.3731e-01,  6.8637e-01,
            6.2613e-01,  4.5969e-01,  2.0926e-01,  1.2961e-01,  1.1760e-01,
            1.0503e-01,  1.5350e-01,  2.4959e-01,  4.6400e-01,  5.5373e-01,
            5.8074e-01,  5.2660e-01],
          [ 4.9869e-01,  5.6469e-01,  5.6046e-01,  5.6324e-01,  5.0538e-01,
            3.3996e-01,  1.6802e-01,  1.3074e-01,  1.6980e-01,  3.4432e-01,
            5.5620e-01,  7.0066e-01,  7.0046e-01,  6.2593e-01,  7.2325e-01,
            7.8373e-01,  7.5401e-01,  8.0991e-01,  7.6505e-01,  6.9591e-01,
            6.2526e-01,  3.9847e-01,  1.5601e-01,  1.0895e-01,  9.8114e-02,
            1.0697e-01,  2.1013e-01,  3.5783e-01,  5.1289e-01,  5.6886e-01,
            5.7748e-01,  5.6975e-01],
          [ 4.7812e-01,  6.2018e-01,  6.2596e-01,  5.6722e-01,  5.4094e-01,
            3.3967e-01,  2.2078e-01,  1.6591e-01,  1.6881e-01,  3.4229e-01,
            5.7096e-01,  6.9895e-01,  6.8442e-01,  6.0322e-01,  6.6839e-01,
            7.6359e-01,  7.4318e-01,  7.8184e-01,  7.7266e-01,  6.7209e-01,
            6.2183e-01,  3.8882e-01,  1.3708e-01,  1.0857e-01,  8.8804e-02,
            1.6371e-01,  2.9421e-01,  4.3321e-01,  5.4221e-01,  5.3938e-01,
            5.7690e-01,  5.9692e-01],
          [ 4.7056e-01,  7.5764e-01,  7.2380e-01,  6.0129e-01,  4.9659e-01,
            3.4097e-01,  2.2937e-01,  1.7815e-01,  1.9094e-01,  3.4905e-01,
            5.7081e-01,  6.4558e-01,  6.5522e-01,  6.0873e-01,  6.5071e-01,
            7.3775e-01,  7.4806e-01,  7.5405e-01,  7.3314e-01,  6.9008e-01,
            6.0500e-01,  3.2088e-01,  1.2427e-01,  1.1334e-01,  1.3617e-01,
            2.7732e-01,  4.6382e-01,  5.3618e-01,  5.3520e-01,  5.2009e-01,
            6.3165e-01,  6.1089e-01],
          [ 6.1627e-01,  8.4159e-01,  7.7683e-01,  6.9479e-01,  6.4070e-01,
            4.2105e-01,  2.5028e-01,  1.6912e-01,  1.8776e-01,  2.8429e-01,
            3.6502e-01,  5.0211e-01,  5.7950e-01,  5.9815e-01,  6.2956e-01,
            6.9427e-01,  7.3388e-01,  7.3048e-01,  7.2502e-01,  6.8399e-01,
            5.7791e-01,  2.5981e-01,  1.4380e-01,  1.0216e-01,  1.4265e-01,
            3.5170e-01,  6.0110e-01,  6.0797e-01,  5.7953e-01,  6.0098e-01,
            6.5701e-01,  6.5928e-01],
          [ 6.9894e-01,  7.9564e-01,  7.3527e-01,  6.7743e-01,  6.5679e-01,
            5.2165e-01,  3.6262e-01,  1.8741e-01,  1.3110e-01,  1.4176e-01,
            1.7668e-01,  3.1531e-01,  4.4463e-01,  5.1907e-01,  5.5873e-01,
            6.4374e-01,  7.1162e-01,  7.6261e-01,  7.1482e-01,  6.3754e-01,
            5.0187e-01,  2.2459e-01,  1.3805e-01,  9.6506e-02,  1.4775e-01,
            4.0717e-01,  6.6285e-01,  6.4304e-01,  6.5935e-01,  6.6123e-01,
            6.9753e-01,  6.6552e-01],
          [ 7.5833e-01,  7.3359e-01,  6.3857e-01,  5.8446e-01,  5.7390e-01,
            5.6503e-01,  3.9826e-01,  1.8729e-01,  1.0653e-01,  1.2343e-01,
            1.6341e-01,  2.0656e-01,  2.6704e-01,  4.0745e-01,  4.9573e-01,
            5.7572e-01,  6.3743e-01,  6.9825e-01,  7.0893e-01,  5.2859e-01,
            3.3241e-01,  1.2737e-01,  1.0193e-01,  1.1127e-01,  1.6089e-01,
            3.9529e-01,  6.2575e-01,  6.6207e-01,  6.9448e-01,  7.3432e-01,
            7.0813e-01,  6.3487e-01],
          [ 7.6584e-01,  6.7958e-01,  5.9371e-01,  5.4262e-01,  5.9805e-01,
            5.7963e-01,  3.7834e-01,  2.3984e-01,  2.5026e-01,  2.2799e-01,
            1.7594e-01,  1.4488e-01,  1.4608e-01,  1.8355e-01,  2.8299e-01,
            4.0480e-01,  5.2170e-01,  5.7202e-01,  5.8576e-01,  3.7398e-01,
            1.6344e-01,  8.6757e-02,  1.1524e-01,  1.8144e-01,  2.4811e-01,
            4.6581e-01,  6.0300e-01,  6.8301e-01,  7.2623e-01,  7.2463e-01,
            7.2702e-01,  6.8819e-01],
          [ 6.7626e-01,  6.2239e-01,  5.3795e-01,  4.9233e-01,  5.6107e-01,
            5.6455e-01,  4.6466e-01,  3.0525e-01,  2.3062e-01,  1.1622e-01,
            6.9128e-02,  7.9834e-02,  1.0306e-01,  1.2180e-01,  1.7076e-01,
            2.7428e-01,  3.0823e-01,  3.0563e-01,  3.2306e-01,  1.8158e-01,
            9.1204e-02,  1.0500e-01,  1.7669e-01,  2.9670e-01,  4.9173e-01,
            6.0486e-01,  6.6127e-01,  6.7714e-01,  6.8093e-01,  6.5478e-01,
            7.0066e-01,  7.4601e-01],
          [ 6.4799e-01,  6.0523e-01,  5.4671e-01,  5.2110e-01,  5.5981e-01,
            5.1050e-01,  3.9833e-01,  3.0162e-01,  2.3087e-01,  1.7304e-01,
            1.3608e-01,  1.6142e-01,  1.8441e-01,  1.6102e-01,  1.5042e-01,
            1.2795e-01,  1.1491e-01,  9.7438e-02,  1.0429e-01,  1.0312e-01,
            5.6794e-02,  9.5682e-02,  1.4711e-01,  3.1263e-01,  5.7631e-01,
            5.7732e-01,  5.5460e-01,  5.9958e-01,  6.4106e-01,  6.5031e-01,
            6.7197e-01,  7.2453e-01],
          [ 6.5815e-01,  6.5631e-01,  5.9590e-01,  5.6075e-01,  5.9128e-01,
            5.9780e-01,  5.4904e-01,  5.1898e-01,  4.4726e-01,  3.5162e-01,
            2.6673e-01,  1.4196e-01,  5.0260e-02,  5.5312e-02,  6.0244e-02,
            6.3039e-02,  7.0992e-02,  6.4536e-02,  8.1001e-02,  4.9973e-02,
            7.0763e-02,  1.0364e-01,  1.3086e-01,  2.8711e-01,  4.6307e-01,
            5.1353e-01,  5.6198e-01,  6.2304e-01,  7.0290e-01,  7.0632e-01,
            6.5286e-01,  6.4707e-01],
          [ 6.8241e-01,  6.7991e-01,  6.8017e-01,  6.2009e-01,  6.4588e-01,
            6.5793e-01,  6.1331e-01,  6.2291e-01,  4.7587e-01,  2.8750e-01,
            2.7620e-01,  2.7465e-01,  9.6537e-02,  3.4821e-02,  3.4133e-02,
            9.7922e-03,  9.6443e-02,  1.1494e-01,  4.7063e-02,  3.4272e-02,
            5.1372e-02,  9.6070e-02,  1.1484e-01,  1.9876e-01,  3.4060e-01,
            5.0754e-01,  6.4557e-01,  6.5645e-01,  7.3336e-01,  7.8332e-01,
            7.3962e-01,  7.2605e-01],
          [ 4.7828e-01,  5.0520e-01,  6.4076e-01,  7.3644e-01,  7.1035e-01,
            7.3152e-01,  6.8215e-01,  6.6682e-01,  6.6532e-01,  3.8425e-01,
            1.9882e-01,  2.9172e-01,  2.7759e-01,  1.0082e-01,  9.2791e-02,
            9.7931e-02,  1.5242e-01,  1.2359e-01,  5.0243e-02,  4.9142e-02,
            5.6139e-02,  7.0984e-02,  9.4441e-02,  1.2428e-01,  3.4946e-01,
            5.6524e-01,  6.3557e-01,  7.1962e-01,  7.7418e-01,  7.5253e-01,
            7.4519e-01,  7.7652e-01],
          [ 3.9811e-01,  3.6936e-01,  4.6160e-01,  6.5330e-01,  7.4555e-01,
            7.8084e-01,  7.6316e-01,  7.3304e-01,  7.6318e-01,  6.6973e-01,
            4.9711e-01,  4.7564e-01,  3.7507e-01,  3.2679e-01,  3.3689e-01,
            2.8559e-01,  2.8189e-01,  1.8404e-01,  7.4052e-02,  6.2982e-02,
            8.5297e-02,  9.2311e-02,  1.0070e-01,  1.5633e-01,  3.2739e-01,
            5.7683e-01,  6.9778e-01,  7.5965e-01,  8.0529e-01,  7.9862e-01,
            7.8864e-01,  7.9133e-01],
          [ 5.2547e-01,  4.4365e-01,  4.4791e-01,  5.2720e-01,  6.6569e-01,
            7.5637e-01,  7.8553e-01,  7.5893e-01,  7.4871e-01,  7.0074e-01,
            6.5707e-01,  6.0122e-01,  4.9909e-01,  5.2048e-01,  5.2520e-01,
            4.4895e-01,  3.6920e-01,  2.5009e-01,  1.2421e-01,  1.0238e-01,
            6.3997e-02,  8.2135e-02,  1.0091e-01,  1.2903e-01,  1.9223e-01,
            4.3213e-01,  6.5499e-01,  7.5803e-01,  7.8126e-01,  7.9464e-01,
            8.1004e-01,  7.5763e-01]],

         [[ 3.5943e-03,  1.3464e-02,  1.8374e-02,  9.3325e-03,  8.3559e-03,
            5.4275e-03,  5.9796e-04,  3.3524e-02,  2.8989e-02,  2.5003e-02,
            2.2190e-02, -3.0484e-03,  1.4171e-04, -1.5584e-02, -7.5096e-03,
           -8.2897e-03, -4.7076e-03, -1.3281e-03,  8.1895e-03, -9.7847e-03,
            1.3653e-02,  1.6273e-02,  1.9730e-02,  2.2668e-02,  2.5603e-02,
            2.8612e-02,  2.4059e-02,  1.6240e-02,  9.9841e-03,  1.0391e-02,
           -4.0668e-03,  4.7988e-03],
          [-4.3226e-03,  1.5181e-03,  1.2001e-02,  1.0134e-02,  1.7865e-02,
            2.2224e-02,  1.7700e-02,  2.3806e-02,  3.8015e-02,  9.4844e-03,
            7.8799e-03,  9.0432e-03, -2.8770e-03, -4.9939e-03, -8.3662e-03,
           -1.2638e-02, -2.2938e-03,  1.1141e-03,  1.2592e-02,  6.9660e-03,
            2.9239e-03,  9.5243e-03,  7.3036e-03,  2.2038e-02,  3.4282e-02,
            8.3223e-03,  3.6350e-03,  8.5619e-03,  2.3484e-02,  8.8921e-03,
            1.3000e-02,  1.5730e-02],
          [-3.9659e-04,  1.3805e-02,  8.8421e-03,  1.7283e-02,  1.4246e-02,
            9.8759e-03,  1.1558e-02,  1.7044e-02,  1.1038e-02,  2.3624e-02,
            2.4616e-02,  8.7210e-03,  4.9489e-03,  7.5596e-03, -1.2538e-02,
           -1.4446e-02, -1.3150e-02, -3.6161e-03,  2.3079e-02,  2.5662e-03,
            2.4903e-03,  1.7056e-02,  1.5348e-02,  1.6028e-02,  3.4778e-02,
            1.9188e-02,  3.8230e-02,  2.3929e-02,  1.7320e-02,  8.3458e-03,
            7.7250e-03,  1.5564e-02],
          [-4.7155e-04,  2.0444e-02,  1.0515e-02,  1.8980e-02,  5.6629e-03,
           -1.8474e-03,  5.9412e-03,  7.2197e-03,  4.6806e-03,  2.7253e-02,
            6.8673e-03, -1.1722e-02, -3.3123e-03, -9.5616e-03,  5.5509e-03,
            2.9257e-03, -1.6476e-02, -6.6953e-03,  1.9942e-02,  1.1022e-02,
            2.8534e-02,  7.7550e-03,  3.6071e-02,  3.1432e-02,  2.6574e-02,
            2.6990e-02,  1.9022e-02,  3.4688e-02,  1.4262e-02,  2.2606e-02,
            1.1307e-02, -4.5718e-03],
          [-3.4974e-03,  6.4043e-03, -4.4411e-04,  1.3011e-02,  1.5210e-02,
           -1.9998e-03,  1.4331e-02,  1.2701e-02,  1.4820e-02,  1.4694e-02,
            9.2587e-03, -2.2981e-03, -1.8616e-03,  6.5461e-03,  2.8348e-02,
            4.9089e-03, -3.0048e-03,  3.7902e-03,  1.6658e-02,  3.1979e-02,
            3.3382e-02,  3.6868e-02,  4.0273e-02,  3.7921e-02,  2.4138e-02,
            2.9063e-02,  2.5454e-02,  2.3569e-02,  1.8312e-02,  1.6786e-02,
            1.0922e-02,  7.5501e-03],
          [ 6.7466e-03,  7.2056e-03,  1.6328e-02,  1.0412e-02,  8.2623e-03,
            1.7627e-02,  7.7814e-03,  1.8234e-03,  2.8061e-02,  3.7908e-02,
            5.7534e-02,  6.4232e-02,  1.0648e-01,  1.1529e-01,  1.1734e-01,
            9.3964e-02,  5.0826e-02,  4.0486e-02,  4.3294e-02,  5.1517e-02,
            6.9353e-03,  2.7628e-02,  4.0337e-02,  3.7065e-02,  2.9566e-02,
            1.6499e-02,  2.1988e-02,  1.3276e-02,  1.7610e-02, -5.4112e-03,
            1.2604e-02,  1.6842e-02],
          [ 3.4252e-03,  8.1683e-03,  2.2750e-03,  5.6398e-03,  1.0015e-02,
           -3.2959e-03,  3.5162e-03,  2.6158e-02,  2.6572e-02,  1.2351e-01,
            2.5908e-01,  3.1156e-01,  3.0466e-01,  2.9812e-01,  3.0503e-01,
            2.7005e-01,  1.9083e-01,  1.4513e-01,  1.4795e-01,  1.3542e-01,
            1.6389e-02,  1.1351e-02,  5.3170e-03,  1.7231e-02,  6.7181e-03,
           -4.3187e-04,  6.7771e-03,  2.5989e-02, -1.6705e-03,  5.9748e-03,
            2.3677e-03, -1.2156e-03],
          [-5.7057e-03,  8.0743e-03,  2.8869e-02,  4.2624e-02,  3.7683e-02,
            1.4989e-02,  2.0176e-02,  6.5062e-03,  1.1143e-01,  3.4785e-01,
            3.7230e-01,  3.4847e-01,  3.2129e-01,  3.0285e-01,  3.6141e-01,
            3.6271e-01,  2.7730e-01,  2.5609e-01,  2.6247e-01,  1.9165e-01,
            2.6256e-02,  2.6218e-03,  3.1104e-03, -1.2404e-03,  9.7063e-03,
            1.4950e-02, -8.7069e-03,  1.2780e-02,  1.8358e-02,  1.5568e-02,
            1.1928e-03,  3.7244e-03],
          [-2.6735e-03,  4.2690e-02,  3.8429e-02,  7.8247e-02,  6.8183e-02,
            2.7336e-02,  2.2177e-02,  2.7489e-02,  2.3693e-01,  2.8059e-01,
            2.0453e-01,  1.8819e-01,  2.3832e-01,  3.2465e-01,  3.8728e-01,
            3.8991e-01,  3.1758e-01,  2.7258e-01,  2.8476e-01,  2.3431e-01,
            5.8720e-02,  1.7433e-02,  1.4861e-02,  2.8732e-02,  3.1434e-02,
            1.1747e-02,  5.5784e-03, -4.2818e-03,  1.5085e-02,  1.0522e-02,
            1.5988e-02,  1.0859e-02],
          [ 3.8851e-02,  6.7150e-02,  7.4040e-02,  1.3895e-01,  1.3796e-01,
            2.1462e-02,  1.6840e-02,  9.4862e-02,  2.8346e-01,  2.2666e-01,
            2.2788e-01,  2.5631e-01,  3.8656e-01,  5.0526e-01,  4.4270e-01,
            3.8972e-01,  3.5716e-01,  2.6083e-01,  2.9764e-01,  2.2730e-01,
            9.0091e-02,  6.8504e-02,  8.4944e-02,  4.6670e-02,  3.0158e-02,
            2.1697e-02,  1.4971e-02,  1.7606e-02,  1.0628e-02,  3.9174e-03,
            5.5666e-03, -3.0413e-02],
          [ 2.7725e-02,  2.6969e-02,  5.6718e-02,  2.2105e-01,  1.6625e-01,
            6.7491e-03,  2.5713e-02,  2.0094e-01,  3.1910e-01,  3.0643e-01,
            3.2785e-01,  3.8321e-01,  4.3347e-01,  4.4493e-01,  4.1724e-01,
            3.7331e-01,  2.9598e-01,  2.6828e-01,  2.8817e-01,  2.2501e-01,
            1.1457e-01,  1.4557e-01,  2.1149e-01,  7.5603e-02,  2.9785e-02,
            2.0264e-02,  2.5856e-02,  3.2187e-03,  9.2056e-03,  1.3479e-02,
            2.0123e-02,  1.9375e-03],
          [-1.6667e-02, -5.1119e-03,  3.5820e-02,  2.9967e-01,  1.7709e-01,
           -9.8687e-03,  8.3630e-02,  2.8522e-01,  3.3863e-01,  3.2823e-01,
            3.6826e-01,  4.1917e-01,  4.1493e-01,  4.3446e-01,  4.1858e-01,
            3.4755e-01,  2.8018e-01,  2.9029e-01,  3.0332e-01,  2.4604e-01,
            1.2468e-01,  1.4697e-01,  2.3080e-01,  6.4716e-02,  2.2404e-02,
            2.4882e-02,  1.6311e-02,  1.3747e-02,  9.4389e-04,  2.7056e-03,
            5.4136e-02,  4.8879e-02],
          [-6.4739e-03, -2.3338e-02,  2.6933e-02,  2.3017e-01,  1.6963e-01,
           -2.8510e-02,  1.4194e-01,  3.5007e-01,  3.9958e-01,  3.6195e-01,
            4.0560e-01,  5.0601e-01,  4.9894e-01,  3.2922e-01,  1.8108e-01,
            2.7645e-01,  3.5817e-01,  3.0127e-01,  3.0943e-01,  2.7835e-01,
            1.4727e-01,  8.4213e-02,  1.2184e-01,  6.5908e-02,  4.9842e-02,
            1.0298e-02,  7.8074e-03, -5.6340e-04,  9.1827e-04,  4.6979e-03,
            4.2672e-02,  4.4420e-02],
          [-1.3930e-02,  2.4808e-02,  4.9977e-02,  9.3861e-02,  1.4285e-01,
            4.7418e-03,  9.2774e-02,  3.0060e-01,  3.8360e-01,  3.9141e-01,
            4.3890e-01,  5.1547e-01,  5.1566e-01,  3.0578e-01,  1.9432e-01,
            2.8070e-01,  4.7079e-01,  4.2061e-01,  4.0805e-01,  4.1991e-01,
            1.8753e-01,  6.0790e-02,  5.3480e-02,  6.3154e-02,  4.0250e-02,
            2.6529e-02,  2.2177e-02,  7.6094e-03,  4.3941e-03,  1.1471e-02,
            3.8095e-02,  2.6957e-02],
          [ 1.1015e-01,  1.4684e-01,  1.9845e-01,  2.2002e-01,  1.5628e-01,
            3.8665e-02,  1.0082e-01,  1.9557e-01,  2.8486e-01,  3.8112e-01,
            4.6267e-01,  4.9278e-01,  5.3749e-01,  4.1544e-01,  4.8089e-01,
            5.7527e-01,  6.0129e-01,  5.2251e-01,  5.1443e-01,  4.5625e-01,
            3.2913e-01,  1.7840e-01,  1.1501e-01,  9.2244e-02,  5.1927e-02,
            4.1954e-02,  1.7464e-02,  2.1389e-02,  1.7293e-02,  3.9955e-02,
            2.4839e-02,  1.8120e-02],
          [ 1.8211e-01,  2.0646e-01,  2.2070e-01,  2.8346e-01,  2.0508e-01,
            1.0362e-01,  1.2358e-01,  1.3009e-01,  2.3718e-01,  4.0172e-01,
            4.8278e-01,  4.4917e-01,  5.3512e-01,  6.0856e-01,  6.8238e-01,
            6.6794e-01,  6.2312e-01,  6.1894e-01,  5.2457e-01,  5.0240e-01,
            4.5340e-01,  3.3845e-01,  1.8882e-01,  1.2540e-01,  5.6770e-02,
            2.6089e-02,  3.2645e-02,  3.7947e-02,  3.2294e-02,  4.2623e-02,
            2.8725e-02,  2.5636e-03],
          [ 2.1735e-01,  2.4861e-01,  2.1998e-01,  2.5062e-01,  2.5168e-01,
            2.0160e-01,  1.4708e-01,  9.4779e-02,  1.9237e-01,  3.7980e-01,
            4.9836e-01,  4.6519e-01,  4.9517e-01,  6.7482e-01,  6.8278e-01,
            6.3987e-01,  6.4421e-01,  6.8528e-01,  5.6197e-01,  5.4006e-01,
            5.0249e-01,  3.6880e-01,  1.8002e-01,  1.1960e-01,  5.6062e-02,
            2.6446e-02,  2.5219e-02,  3.4062e-02,  7.4740e-02,  3.4636e-02,
            1.0682e-02,  4.9570e-03],
          [ 3.1292e-01,  3.1374e-01,  2.7148e-01,  2.6861e-01,  2.5789e-01,
            2.1916e-01,  1.4861e-01,  8.3631e-02,  1.5758e-01,  3.2454e-01,
            4.5453e-01,  5.6800e-01,  5.5773e-01,  6.0707e-01,  6.4726e-01,
            7.1791e-01,  6.7902e-01,  6.8768e-01,  6.2682e-01,  5.7359e-01,
            5.2529e-01,  3.5689e-01,  1.2832e-01,  6.5885e-02,  4.0559e-02,
            1.3978e-02,  1.6571e-02,  4.9968e-02,  9.0452e-02,  4.1932e-02,
            2.1933e-02,  2.3008e-02],
          [ 3.3812e-01,  3.6479e-01,  3.2779e-01,  3.0185e-01,  3.1355e-01,
            2.4007e-01,  1.2725e-01,  9.7101e-02,  1.0560e-01,  2.7571e-01,
            4.5703e-01,  6.0825e-01,  6.1034e-01,  5.1797e-01,  6.3348e-01,
            7.0733e-01,  6.8271e-01,  6.8303e-01,  6.3590e-01,  5.7226e-01,
            5.0439e-01,  2.9265e-01,  7.9911e-02,  3.0036e-02,  3.5492e-02,
            1.9072e-03,  2.2675e-02,  6.9332e-02,  7.6316e-02,  6.6364e-02,
            5.5769e-02,  4.2900e-02],
          [ 2.8718e-01,  3.8558e-01,  3.6100e-01,  2.9325e-01,  3.0685e-01,
            2.4640e-01,  1.7366e-01,  1.2676e-01,  1.2211e-01,  2.5801e-01,
            4.7930e-01,  5.9937e-01,  6.0463e-01,  4.9622e-01,  5.8796e-01,
            6.4859e-01,  6.4419e-01,  6.6059e-01,  6.2916e-01,  5.4243e-01,
            5.0488e-01,  2.8541e-01,  6.3777e-02,  3.2026e-02,  2.2048e-02,
            4.7845e-02,  4.1451e-02,  4.2027e-02,  6.8948e-02,  5.2737e-02,
            5.0391e-02,  5.8591e-02],
          [ 3.0974e-01,  5.7044e-01,  4.7975e-01,  3.0820e-01,  2.5833e-01,
            2.2017e-01,  1.7033e-01,  1.3490e-01,  1.2727e-01,  2.7842e-01,
            4.6795e-01,  5.7294e-01,  5.7388e-01,  5.1676e-01,  5.4519e-01,
            6.1540e-01,  6.2800e-01,  6.1778e-01,  5.9066e-01,  5.2628e-01,
            4.9078e-01,  2.2934e-01,  4.7904e-02,  3.0802e-02,  6.4081e-03,
            6.4954e-02,  7.1371e-02,  9.7164e-02,  9.1278e-02,  6.2904e-02,
            6.8350e-02,  4.6953e-02],
          [ 4.8252e-01,  6.5638e-01,  5.2976e-01,  4.2634e-01,  4.0300e-01,
            2.7162e-01,  1.9609e-01,  1.2671e-01,  1.5027e-01,  2.4324e-01,
            2.9780e-01,  4.6228e-01,  5.1484e-01,  5.0763e-01,  5.0538e-01,
            5.9281e-01,  6.1942e-01,  6.0421e-01,  5.8723e-01,  5.3852e-01,
            4.7071e-01,  1.9305e-01,  7.2830e-02,  7.8936e-03,  2.2587e-02,
            7.6192e-02,  1.1216e-01,  9.3228e-02,  8.2458e-02,  6.5834e-02,
            2.7118e-02,  4.9266e-02],
          [ 5.5924e-01,  5.7333e-01,  4.3969e-01,  3.6436e-01,  3.8285e-01,
            3.5164e-01,  2.7871e-01,  1.1853e-01,  1.1266e-01,  1.0595e-01,
            1.3540e-01,  2.5044e-01,  3.8179e-01,  4.4255e-01,  4.6035e-01,
            5.0390e-01,  5.6849e-01,  5.9950e-01,  5.6588e-01,  4.8459e-01,
            3.8915e-01,  1.3847e-01,  7.7828e-02, -1.6952e-02,  2.2981e-02,
            7.6331e-02,  1.3400e-01,  1.2266e-01,  8.8573e-02,  6.4648e-02,
            2.9920e-02,  3.0600e-02],
          [ 6.1953e-01,  4.6910e-01,  3.2074e-01,  2.7257e-01,  2.9858e-01,
            3.3743e-01,  2.5847e-01,  1.0916e-01,  5.3257e-02,  5.1346e-02,
            8.3930e-02,  1.2872e-01,  1.8647e-01,  3.1355e-01,  3.9644e-01,
            4.7205e-01,  4.7708e-01,  5.3242e-01,  5.6154e-01,  3.8119e-01,
            2.3131e-01,  7.5500e-02,  2.5942e-02,  2.9415e-02,  3.3529e-02,
            6.5571e-02,  1.0259e-01,  9.0462e-02,  5.7484e-02,  4.1549e-02,
            1.9318e-02,  2.3168e-02],
          [ 6.2159e-01,  4.0304e-01,  2.7566e-01,  2.4761e-01,  3.3412e-01,
            3.0171e-01,  2.1179e-01,  1.6091e-01,  1.6821e-01,  1.7312e-01,
            1.1278e-01,  8.8043e-02,  8.9499e-02,  1.5496e-01,  2.5427e-01,
            3.3153e-01,  3.9530e-01,  4.4069e-01,  4.7154e-01,  2.5935e-01,
            9.7976e-02,  3.5279e-02,  3.9105e-02,  9.8711e-02,  1.5307e-01,
            1.8942e-01,  1.7939e-01,  1.3598e-01,  1.0635e-01,  9.6168e-02,
            1.1420e-01,  1.4111e-01],
          [ 5.2175e-01,  3.5703e-01,  2.6851e-01,  2.4659e-01,  3.4096e-01,
            3.4084e-01,  3.5729e-01,  3.0650e-01,  2.3275e-01,  1.7052e-01,
            1.2234e-01,  1.1039e-01,  1.5453e-01,  1.5005e-01,  1.8083e-01,
            2.5497e-01,  2.3829e-01,  2.3854e-01,  2.4244e-01,  1.2706e-01,
            5.4664e-02,  4.7349e-02,  8.3797e-02,  1.9756e-01,  3.4881e-01,
            3.8220e-01,  3.1068e-01,  2.3669e-01,  2.2921e-01,  2.7445e-01,
            2.8082e-01,  3.1390e-01],
          [ 4.4829e-01,  3.0682e-01,  2.7052e-01,  2.9605e-01,  3.5055e-01,
            3.4338e-01,  3.2074e-01,  2.7684e-01,  2.3138e-01,  2.0952e-01,
            2.0577e-01,  2.3279e-01,  2.3050e-01,  1.9182e-01,  1.7865e-01,
            1.2359e-01,  7.7558e-02,  5.8058e-02,  7.2938e-02,  5.8553e-02,
            3.4515e-02,  4.2188e-02,  5.7377e-02,  2.0793e-01,  4.1862e-01,
            3.8203e-01,  3.3587e-01,  3.7708e-01,  3.8187e-01,  3.5794e-01,
            3.1935e-01,  3.2898e-01],
          [ 3.9647e-01,  2.9809e-01,  2.7895e-01,  3.2509e-01,  4.0758e-01,
            4.5699e-01,  4.2223e-01,  3.4310e-01,  3.3769e-01,  3.2855e-01,
            3.1572e-01,  1.7440e-01,  8.5228e-02,  7.8794e-02,  7.3500e-02,
            4.9404e-02,  4.8211e-02,  4.2452e-02,  3.7754e-02,  1.4513e-02,
            1.8801e-02,  2.9780e-02,  4.9367e-02,  1.5854e-01,  2.9936e-01,
            3.5838e-01,  4.3647e-01,  5.2338e-01,  5.1481e-01,  3.8113e-01,
            3.1209e-01,  2.8397e-01],
          [ 4.3544e-01,  3.4796e-01,  3.8263e-01,  3.9244e-01,  4.8261e-01,
            5.3091e-01,  4.3481e-01,  3.7894e-01,  3.4345e-01,  2.2621e-01,
            2.7739e-01,  2.7837e-01,  1.2187e-01,  3.6175e-02,  2.3517e-02,
            1.0654e-02,  1.0600e-01,  1.0012e-01,  9.3071e-03,  1.3608e-02,
            2.2114e-02,  2.8947e-02,  7.7372e-02,  1.2398e-01,  2.2012e-01,
            3.9816e-01,  5.3644e-01,  5.7510e-01,  5.3111e-01,  4.2779e-01,
            4.0622e-01,  3.8019e-01],
          [ 2.4870e-01,  2.3457e-01,  3.8781e-01,  4.9727e-01,  5.3163e-01,
            6.1400e-01,  5.2833e-01,  4.9759e-01,  5.4306e-01,  3.2646e-01,
            1.3052e-01,  2.0659e-01,  2.1041e-01,  6.2715e-02,  8.5092e-02,
            8.3925e-02,  1.4162e-01,  9.2514e-02,  1.6504e-02,  2.5648e-02,
            2.5236e-02,  3.7951e-02,  5.3902e-02,  7.2200e-02,  2.5429e-01,
            4.3932e-01,  5.1117e-01,  5.6937e-01,  5.3732e-01,  4.3925e-01,
            4.2371e-01,  4.4396e-01],
          [ 1.0767e-01,  1.2456e-01,  2.1322e-01,  4.1760e-01,  5.6032e-01,
            6.3311e-01,  6.2715e-01,  6.1619e-01,  6.4873e-01,  5.2734e-01,
            3.1871e-01,  2.6529e-01,  2.6134e-01,  2.5063e-01,  3.0451e-01,
            2.7128e-01,  2.5495e-01,  1.3179e-01,  2.8514e-02,  2.6815e-02,
            1.8917e-02,  2.9332e-02,  5.2872e-02,  1.0065e-01,  2.6641e-01,
            5.0123e-01,  5.6435e-01,  5.5735e-01,  5.5247e-01,  5.1852e-01,
            5.0754e-01,  4.5803e-01],
          [ 2.2117e-01,  2.2223e-01,  2.3659e-01,  3.0713e-01,  4.8500e-01,
            5.9110e-01,  6.4929e-01,  6.5126e-01,  6.6177e-01,  5.7081e-01,
            4.7788e-01,  3.8578e-01,  3.8531e-01,  4.5162e-01,  4.7765e-01,
            4.2319e-01,  3.3158e-01,  2.0180e-01,  8.6743e-02,  3.7661e-02,
            3.5070e-02,  3.9846e-02,  5.3023e-02,  8.9885e-02,  1.8300e-01,
            4.1730e-01,  5.7947e-01,  5.7832e-01,  5.7304e-01,  5.6232e-01,
            5.6657e-01,  5.1621e-01]],

         [[ 1.0244e-02,  3.7112e-02,  1.3362e-02,  1.1678e-02,  9.0647e-03,
            1.7836e-02,  2.1425e-02,  2.7640e-02,  3.8013e-02,  3.0800e-02,
            1.9696e-02,  1.5824e-02,  6.9608e-03, -2.0248e-03,  1.0518e-02,
            1.4996e-02,  1.1618e-02,  8.6073e-03,  8.7253e-03,  2.0291e-02,
            1.7939e-02,  2.6022e-02,  3.0717e-02,  2.4690e-02,  2.9576e-02,
            1.2773e-02,  1.6172e-02,  2.5719e-02,  3.2291e-02,  2.1920e-02,
            2.4298e-02,  1.3475e-02],
          [ 2.0276e-02,  3.1987e-02,  2.3046e-02,  1.7857e-02,  5.0673e-03,
            7.1535e-03,  1.4281e-02,  1.8099e-02,  3.5079e-02,  2.5039e-02,
            8.2378e-03,  1.0360e-02,  1.5088e-02, -3.3849e-03, -6.0667e-03,
            7.3147e-03,  9.6565e-03,  1.5272e-02,  1.2815e-02,  2.1802e-02,
            3.6706e-02,  1.4678e-02,  3.9619e-02,  3.3789e-02,  3.0296e-02,
            1.0066e-02,  2.7131e-02,  2.9320e-02,  2.9882e-02,  1.1924e-02,
            2.5972e-02,  1.4881e-02],
          [ 1.8404e-02,  2.9983e-02,  1.7336e-02,  8.0116e-03,  4.1135e-03,
            1.9575e-03,  1.4195e-02,  1.4046e-02,  2.2649e-02,  2.1286e-02,
            7.7095e-03,  7.2619e-03,  1.0646e-03, -5.0397e-03, -1.0611e-02,
           -7.4926e-03,  7.2387e-03,  8.2776e-03,  7.7620e-03,  3.7331e-02,
            1.8296e-02,  3.9655e-02,  3.9505e-02,  3.9059e-02,  2.1369e-02,
            2.3783e-02,  7.3983e-03,  3.4283e-02,  2.4399e-02,  2.3254e-02,
            3.1913e-02,  6.4575e-03],
          [ 1.6688e-02,  1.6953e-02,  1.6656e-02,  1.6273e-02, -5.4714e-03,
            6.2767e-03,  6.3229e-03,  8.2795e-04, -4.3466e-03,  3.0183e-02,
            2.2920e-02,  2.9951e-02,  1.0781e-02,  9.6921e-03, -9.3440e-04,
            5.5978e-04, -9.4976e-03,  3.9602e-03,  2.9493e-02,  3.1715e-02,
            3.2434e-02,  4.1162e-02,  3.6220e-02,  2.7952e-02,  2.2482e-02,
            1.6717e-02,  9.7709e-03,  1.5850e-02,  4.2674e-02,  2.5150e-02,
            2.1189e-02,  3.7839e-03],
          [ 1.1719e-02,  2.6371e-02,  1.4635e-02,  1.9184e-02,  8.1811e-03,
           -2.6559e-05,  1.4705e-02,  5.8666e-03,  3.0525e-02,  3.2067e-02,
            4.1135e-02,  1.9903e-02,  2.7536e-02,  1.5711e-02,  2.2639e-02,
            9.4264e-03,  3.9032e-03,  8.1387e-03,  3.3134e-02,  4.6639e-02,
            5.4848e-02,  3.1776e-02,  2.4998e-02,  3.6254e-02,  1.7822e-02,
            2.5723e-02,  3.1346e-02,  2.6230e-02,  2.3096e-02,  2.7910e-02,
            1.1506e-02,  3.7914e-03],
          [ 1.2978e-02,  2.5488e-02,  2.5057e-02,  1.2970e-02,  1.0427e-02,
            1.0204e-02,  1.5701e-02,  1.6005e-02,  2.0978e-02,  3.5903e-02,
            3.2851e-02,  4.0681e-02,  6.5762e-02,  6.1826e-02,  4.5243e-02,
            6.7993e-02,  5.4118e-02,  3.2632e-02,  3.9444e-02,  6.1002e-02,
            2.5800e-02,  2.0274e-02,  1.4492e-02,  2.8162e-02,  3.7164e-02,
            2.3384e-02,  3.0788e-02,  3.4879e-02,  2.8372e-02,  3.4004e-02,
            7.4744e-03,  1.2070e-02],
          [ 1.8408e-02,  1.8942e-02,  2.0159e-02,  6.9684e-03,  2.3538e-03,
            5.4960e-04,  2.9626e-02,  1.6299e-02,  1.8363e-02,  8.0455e-02,
            2.0524e-01,  2.5117e-01,  2.2279e-01,  2.2143e-01,  2.3398e-01,
            2.2449e-01,  1.5262e-01,  1.0449e-01,  1.1888e-01,  1.1313e-01,
            2.4130e-02,  1.0862e-02,  2.9497e-02,  2.7176e-02,  4.1464e-02,
            1.6707e-02,  3.3564e-02,  6.0486e-02,  4.0224e-02,  3.1192e-02,
            1.9532e-02, -2.9886e-03],
          [ 1.5505e-02,  2.3865e-02,  1.6222e-02,  2.6994e-02,  2.9813e-02,
            1.5419e-03,  2.9448e-02,  3.3715e-03,  9.6439e-02,  3.0266e-01,
            3.1325e-01,  2.9691e-01,  2.6558e-01,  2.7312e-01,  3.1932e-01,
            3.1002e-01,  2.2862e-01,  1.7868e-01,  2.0649e-01,  1.7591e-01,
            1.6577e-02,  7.0591e-03,  2.9142e-02,  2.8241e-02,  3.9024e-02,
            3.7084e-02,  3.5162e-02,  3.7698e-02,  4.3560e-02,  3.4665e-02,
            4.2362e-02,  2.9929e-02],
          [ 1.4996e-02,  2.6406e-02,  3.2471e-02,  4.5796e-02,  4.8756e-02,
            2.2125e-02,  3.0620e-02,  3.1103e-02,  2.0817e-01,  2.3975e-01,
            1.7303e-01,  1.5987e-01,  2.2250e-01,  3.4017e-01,  3.6049e-01,
            3.8061e-01,  3.0326e-01,  2.0076e-01,  2.2386e-01,  1.8292e-01,
            4.0562e-02,  3.3206e-02,  1.1083e-02,  1.9831e-02,  2.7021e-02,
            3.0786e-02,  3.8764e-02,  3.3241e-02,  3.3788e-02,  4.8392e-02,
            4.0908e-02,  3.6328e-02],
          [ 5.4060e-02,  7.5706e-02,  4.4598e-02,  1.1095e-01,  1.0780e-01,
            1.4614e-02,  4.3002e-02,  9.1556e-02,  2.4443e-01,  1.7105e-01,
            1.8556e-01,  2.0060e-01,  3.3716e-01,  4.8391e-01,  3.8949e-01,
            3.4888e-01,  3.2280e-01,  2.1804e-01,  2.2691e-01,  1.8486e-01,
            6.2244e-02,  6.5994e-03,  1.4238e-02,  1.0158e-02,  2.7775e-02,
            3.0178e-02,  4.5942e-02,  3.1950e-02,  3.3285e-02,  4.8231e-02,
            3.8136e-02, -1.3093e-03],
          [ 3.9077e-02,  4.7269e-02,  4.0457e-02,  1.9400e-01,  1.2129e-01,
            2.0379e-02,  4.7875e-02,  1.8907e-01,  2.4149e-01,  2.1499e-01,
            2.2546e-01,  2.8000e-01,  3.4347e-01,  3.9504e-01,  3.4811e-01,
            3.1260e-01,  2.6688e-01,  2.2212e-01,  2.3967e-01,  2.0868e-01,
            9.5615e-02,  6.2321e-02,  7.5327e-02,  5.6983e-03,  1.9908e-02,
            3.0167e-02,  2.7899e-02,  3.4596e-02,  4.7349e-02,  4.6358e-02,
            4.9464e-02,  2.3851e-02],
          [ 2.3788e-02,  3.2309e-02,  5.7708e-02,  2.9234e-01,  1.4391e-01,
            1.5705e-02,  8.8241e-02,  2.3788e-01,  2.5767e-01,  2.4799e-01,
            2.7445e-01,  3.2764e-01,  3.2435e-01,  3.6282e-01,  3.5466e-01,
            2.8887e-01,  2.5074e-01,  2.2725e-01,  2.3893e-01,  1.8108e-01,
            8.4375e-02,  7.8794e-02,  1.1506e-01,  3.6239e-02,  1.9995e-02,
            2.8750e-02,  2.7580e-02,  2.8188e-02,  3.7132e-02,  5.0474e-02,
            9.1568e-02,  7.0764e-02],
          [ 2.5287e-02,  1.9502e-02,  3.3826e-02,  2.0940e-01,  1.4925e-01,
           -2.8184e-04,  1.2972e-01,  2.7033e-01,  3.0783e-01,  2.6511e-01,
            3.1685e-01,  4.0267e-01,  4.2266e-01,  2.5555e-01,  1.0845e-01,
            2.2780e-01,  3.0209e-01,  2.1791e-01,  2.0181e-01,  1.9468e-01,
            1.0290e-01,  5.8709e-02,  8.6771e-02,  5.0285e-02,  3.0275e-02,
            2.8855e-02,  1.8246e-02,  2.0544e-02,  2.3095e-02,  5.3965e-02,
            1.0297e-01,  7.6838e-02],
          [ 3.0620e-02,  6.9986e-02,  8.4568e-02,  1.0227e-01,  1.2081e-01,
            3.9957e-02,  1.3037e-01,  2.2159e-01,  2.6077e-01,  2.9785e-01,
            3.2391e-01,  4.2396e-01,  4.2400e-01,  2.2141e-01,  1.2685e-01,
            2.4535e-01,  4.2245e-01,  3.2202e-01,  3.0752e-01,  3.0898e-01,
            1.3716e-01,  6.0454e-02,  4.7049e-02,  9.1313e-02,  5.0149e-02,
            2.3902e-02,  2.7797e-02,  1.3830e-02,  3.0332e-02,  6.0053e-02,
            1.1066e-01,  8.2079e-02],
          [ 1.6922e-01,  2.1933e-01,  2.8373e-01,  2.5160e-01,  1.5332e-01,
            8.3381e-02,  1.2813e-01,  1.4234e-01,  1.6468e-01,  2.8738e-01,
            3.6125e-01,  3.9762e-01,  4.7645e-01,  3.3413e-01,  3.8803e-01,
            4.9453e-01,  5.2685e-01,  4.3630e-01,  4.0062e-01,  3.4536e-01,
            2.0926e-01,  1.4090e-01,  9.4218e-02,  9.4732e-02,  5.3609e-02,
            1.9587e-02,  2.4234e-02,  3.2597e-02,  2.8383e-02,  5.4928e-02,
            1.1117e-01,  7.4492e-02],
          [ 2.9335e-01,  2.9252e-01,  3.3074e-01,  3.8417e-01,  2.6160e-01,
            1.3795e-01,  1.4385e-01,  9.0728e-02,  1.7688e-01,  3.2516e-01,
            3.8461e-01,  3.6514e-01,  4.5377e-01,  5.1759e-01,  5.6533e-01,
            5.6714e-01,  5.2499e-01,  5.2450e-01,  4.2040e-01,  3.3408e-01,
            2.8796e-01,  2.5221e-01,  1.5865e-01,  1.2148e-01,  4.6658e-02,
            2.2629e-02,  2.5790e-02,  3.9866e-02,  4.4065e-02,  7.6575e-02,
            8.3491e-02,  6.7387e-02],
          [ 3.4055e-01,  3.2176e-01,  3.1447e-01,  3.3244e-01,  3.0842e-01,
            2.1306e-01,  1.5244e-01,  5.0213e-02,  1.4920e-01,  3.3018e-01,
            4.2540e-01,  3.7620e-01,  4.1634e-01,  5.8743e-01,  5.9013e-01,
            5.4582e-01,  5.4361e-01,  5.8196e-01,  4.4529e-01,  3.8665e-01,
            3.6420e-01,  3.0241e-01,  1.7533e-01,  1.1156e-01,  5.7131e-02,
            2.4073e-02,  1.4402e-02,  1.3760e-02,  6.8311e-02,  7.7403e-02,
            6.8515e-02,  6.6415e-02],
          [ 3.8400e-01,  3.7457e-01,  3.5139e-01,  3.5814e-01,  3.0933e-01,
            2.2942e-01,  1.3647e-01,  7.0429e-02,  1.2469e-01,  2.9777e-01,
            4.1245e-01,  4.6871e-01,  4.7558e-01,  5.2157e-01,  5.7382e-01,
            6.2675e-01,  5.8678e-01,  5.9058e-01,  5.3091e-01,  4.3617e-01,
            3.9616e-01,  2.9252e-01,  1.3236e-01,  6.6736e-02,  4.7122e-02,
            2.0838e-02,  1.6913e-02,  3.9399e-02,  9.9363e-02,  8.7355e-02,
            8.4286e-02,  8.1547e-02],
          [ 3.7644e-01,  4.1729e-01,  4.1702e-01,  3.9279e-01,  3.6236e-01,
            2.4466e-01,  1.0515e-01,  7.5326e-02,  8.6147e-02,  2.2906e-01,
            4.0136e-01,  5.2915e-01,  5.1730e-01,  4.3910e-01,  5.5772e-01,
            6.2440e-01,  5.8093e-01,  5.8417e-01,  5.4150e-01,  4.5367e-01,
            3.9567e-01,  2.4411e-01,  9.0856e-02,  2.9980e-02,  2.1972e-02,
            1.6814e-02, -1.7968e-03,  6.1757e-02,  8.6461e-02,  8.5535e-02,
            1.0523e-01,  1.0904e-01],
          [ 3.1296e-01,  4.2557e-01,  3.9466e-01,  3.5570e-01,  3.5301e-01,
            2.2019e-01,  1.4793e-01,  1.1319e-01,  1.2150e-01,  2.1992e-01,
            4.1356e-01,  5.2795e-01,  5.1682e-01,  4.2705e-01,  5.0192e-01,
            5.7394e-01,  5.2842e-01,  5.5756e-01,  5.1809e-01,  4.3545e-01,
            3.8069e-01,  2.3351e-01,  6.4231e-02,  2.5248e-02,  7.3395e-03,
            1.4640e-02,  2.1489e-02,  2.9374e-02,  5.1541e-02,  7.3527e-02,
            1.1262e-01,  1.4649e-01],
          [ 3.5778e-01,  5.8983e-01,  4.6594e-01,  3.0942e-01,  2.8407e-01,
            1.9439e-01,  1.4653e-01,  1.2744e-01,  1.0202e-01,  2.5685e-01,
            4.0355e-01,  5.0079e-01,  4.8530e-01,  4.4115e-01,  4.7078e-01,
            5.2040e-01,  5.2119e-01,  5.0864e-01,  4.8730e-01,  4.3180e-01,
            3.8214e-01,  1.7223e-01,  2.2262e-02,  4.3755e-03,  2.1534e-03,
            5.5988e-02,  6.1877e-02,  6.2081e-02,  3.8976e-02,  3.1324e-02,
            1.0209e-01,  1.2481e-01],
          [ 4.8626e-01,  6.5138e-01,  5.6815e-01,  4.4116e-01,  4.2033e-01,
            2.8198e-01,  1.8276e-01,  1.2817e-01,  1.0617e-01,  2.0335e-01,
            2.6063e-01,  3.8793e-01,  4.4051e-01,  4.2808e-01,  4.1164e-01,
            4.7085e-01,  4.9090e-01,  4.9960e-01,  4.7530e-01,  4.4136e-01,
            3.9704e-01,  1.4454e-01,  6.0646e-02,  1.0236e-02, -3.4143e-03,
            5.5018e-02,  1.1017e-01,  8.1999e-02,  5.4384e-02,  3.2506e-02,
            8.4912e-02,  1.1842e-01],
          [ 5.3040e-01,  5.4953e-01,  4.5594e-01,  4.0474e-01,  4.1555e-01,
            3.5702e-01,  2.9361e-01,  1.1979e-01,  9.5517e-02,  9.8939e-02,
            1.1211e-01,  2.2488e-01,  3.2918e-01,  3.8183e-01,  3.7449e-01,
            4.0614e-01,  4.6444e-01,  5.1945e-01,  4.9247e-01,  4.0599e-01,
            3.3320e-01,  1.0968e-01,  6.9568e-02,  3.0680e-02,  2.6021e-02,
            7.8205e-02,  1.3913e-01,  1.3168e-01,  8.9723e-02,  4.4323e-02,
            8.2025e-02,  1.0368e-01],
          [ 5.8024e-01,  4.8159e-01,  3.3115e-01,  2.9799e-01,  3.4171e-01,
            3.7666e-01,  2.9217e-01,  1.4728e-01,  5.7967e-02,  7.9626e-02,
            9.7681e-02,  1.1543e-01,  1.6078e-01,  2.9335e-01,  3.5163e-01,
            3.6756e-01,  4.0261e-01,  4.6748e-01,  4.7500e-01,  3.4084e-01,
            1.9763e-01,  5.2904e-02,  2.0946e-02,  4.8773e-02,  7.4475e-02,
            7.9885e-02,  9.3782e-02,  1.0836e-01,  9.7086e-02,  6.3517e-02,
            7.5596e-02,  9.9935e-02],
          [ 6.2309e-01,  4.4150e-01,  2.9824e-01,  2.7092e-01,  3.6023e-01,
            3.3296e-01,  2.1775e-01,  1.5743e-01,  1.5300e-01,  1.5817e-01,
            1.1982e-01,  8.4308e-02,  9.1544e-02,  1.4261e-01,  2.0562e-01,
            2.6081e-01,  3.3925e-01,  3.8046e-01,  3.9157e-01,  2.1410e-01,
            7.4607e-02,  3.7375e-02,  4.8023e-02,  1.2095e-01,  1.9380e-01,
            2.0424e-01,  1.8138e-01,  1.3537e-01,  1.2633e-01,  1.5689e-01,
            1.7993e-01,  2.1778e-01],
          [ 5.6120e-01,  3.9450e-01,  2.8764e-01,  2.6873e-01,  3.4071e-01,
            3.3010e-01,  3.1221e-01,  2.6884e-01,  1.9920e-01,  1.0869e-01,
            8.2492e-02,  8.3031e-02,  1.2232e-01,  1.4692e-01,  1.9438e-01,
            2.6778e-01,  2.2650e-01,  1.8395e-01,  1.8175e-01,  8.1725e-02,
            3.7758e-02,  3.2554e-02,  9.4695e-02,  2.5407e-01,  3.6617e-01,
            3.4279e-01,  2.4340e-01,  2.1101e-01,  2.2847e-01,  2.7720e-01,
            3.3717e-01,  3.7605e-01],
          [ 4.9167e-01,  3.2036e-01,  2.7744e-01,  3.1520e-01,  3.7321e-01,
            3.6238e-01,  3.3347e-01,  2.9670e-01,  2.4717e-01,  1.9674e-01,
            1.4866e-01,  1.9719e-01,  2.4892e-01,  2.3840e-01,  2.1263e-01,
            1.6612e-01,  9.9785e-02,  7.0247e-02,  5.7967e-02,  4.4117e-02,
            3.5589e-02,  5.3821e-02,  9.8045e-02,  2.5620e-01,  3.8684e-01,
            3.1722e-01,  3.0473e-01,  3.4677e-01,  3.8558e-01,  3.5730e-01,
            3.6999e-01,  4.3404e-01],
          [ 4.3935e-01,  3.2027e-01,  2.9743e-01,  3.5974e-01,  4.5358e-01,
            4.9026e-01,  4.3921e-01,  3.9021e-01,  3.6877e-01,  3.4067e-01,
            3.0559e-01,  1.6459e-01,  8.5172e-02,  1.1470e-01,  8.5299e-02,
            9.5110e-02,  6.2412e-02,  6.5864e-02,  5.8476e-02,  4.8412e-02,
            5.5015e-02,  4.7076e-02,  8.2244e-02,  1.8418e-01,  2.6668e-01,
            3.2664e-01,  4.6240e-01,  5.6660e-01,  5.6934e-01,  4.3638e-01,
            3.9470e-01,  3.8974e-01],
          [ 4.6484e-01,  3.6521e-01,  3.8862e-01,  4.2007e-01,  4.8095e-01,
            5.2967e-01,  4.5398e-01,  4.2793e-01,  3.4671e-01,  2.2762e-01,
            3.3211e-01,  3.1734e-01,  1.1285e-01,  3.9203e-02,  3.2847e-02,
            2.4115e-02,  1.1003e-01,  1.3450e-01,  6.3210e-02,  4.9552e-02,
            4.8520e-02,  6.9048e-02,  7.3423e-02,  1.1797e-01,  1.8383e-01,
            3.8526e-01,  5.4454e-01,  5.8057e-01,  5.5304e-01,  4.4591e-01,
            4.3384e-01,  4.3164e-01],
          [ 2.6693e-01,  2.7098e-01,  4.2027e-01,  4.9625e-01,  5.0102e-01,
            5.8277e-01,  5.2836e-01,  5.1098e-01,  4.9605e-01,  2.5114e-01,
            1.5361e-01,  2.5029e-01,  2.3011e-01,  7.2234e-02,  8.5518e-02,
            1.0950e-01,  1.4981e-01,  1.3388e-01,  4.7185e-02,  5.0390e-02,
            4.5892e-02,  6.8218e-02,  7.0605e-02,  7.1434e-02,  2.2427e-01,
            4.2542e-01,  5.0976e-01,  5.8826e-01,  5.5753e-01,  4.5850e-01,
            4.5876e-01,  4.7196e-01],
          [ 1.6230e-01,  1.6998e-01,  2.6892e-01,  4.2903e-01,  5.2220e-01,
            5.9581e-01,  6.4335e-01,  6.0190e-01,  6.0612e-01,  4.7591e-01,
            3.0522e-01,  2.6516e-01,  2.7120e-01,  2.6315e-01,  2.9942e-01,
            2.7682e-01,  2.8323e-01,  1.6891e-01,  6.8047e-02,  4.9147e-02,
            4.7228e-02,  5.9403e-02,  4.7721e-02,  1.0878e-01,  2.4432e-01,
            4.6619e-01,  5.4312e-01,  5.6752e-01,  5.8495e-01,  5.4197e-01,
            5.1584e-01,  4.9914e-01],
          [ 3.3972e-01,  2.9890e-01,  2.7694e-01,  3.2895e-01,  4.3507e-01,
            5.5205e-01,  6.3417e-01,  6.3123e-01,  6.0442e-01,  4.8951e-01,
            4.1240e-01,  3.5711e-01,  3.5919e-01,  4.6466e-01,  4.8160e-01,
            4.3296e-01,  3.6754e-01,  2.4985e-01,  1.3464e-01,  8.9815e-02,
            6.7613e-02,  6.1477e-02,  6.5247e-02,  9.3916e-02,  1.5548e-01,
            3.9506e-01,  5.5854e-01,  6.0505e-01,  5.7620e-01,  5.6340e-01,
            5.8717e-01,  5.5322e-01]]],


        [[[ 1.5572e-01,  1.3890e-01,  1.7067e-01,  2.1536e-01,  3.2542e-01,
            3.4508e-01,  3.3538e-01,  2.8781e-01,  2.7480e-01,  2.6373e-01,
            2.3621e-01,  2.3724e-01,  2.5624e-01,  2.5651e-01,  2.3493e-01,
            2.2873e-01,  2.2187e-01,  2.4255e-01,  2.5399e-01,  2.7705e-01,
            2.9857e-01,  3.1925e-01,  3.2461e-01,  3.2257e-01,  2.7738e-01,
            3.2846e-01,  3.7772e-01,  3.9503e-01,  3.1049e-01,  2.8559e-01,
            3.2444e-01,  3.2473e-01],
          [ 1.7701e-01,  1.6433e-01,  1.7353e-01,  2.5995e-01,  3.4489e-01,
            3.2674e-01,  3.0930e-01,  2.9404e-01,  3.0310e-01,  2.5708e-01,
            2.3951e-01,  2.5926e-01,  2.7763e-01,  2.6134e-01,  2.0465e-01,
            2.3114e-01,  2.4471e-01,  2.6020e-01,  2.7740e-01,  2.8330e-01,
            3.1226e-01,  3.0245e-01,  2.8906e-01,  3.1654e-01,  3.2543e-01,
            3.6295e-01,  4.1930e-01,  3.9460e-01,  3.3673e-01,  3.1184e-01,
            3.3755e-01,  3.5022e-01],
          [ 1.5985e-01,  1.6777e-01,  2.0090e-01,  2.7671e-01,  3.6415e-01,
            3.3178e-01,  2.7458e-01,  2.5333e-01,  2.7955e-01,  2.5745e-01,
            2.7817e-01,  2.8427e-01,  2.7953e-01,  2.4631e-01,  2.1237e-01,
            2.3998e-01,  2.8044e-01,  2.7605e-01,  3.0754e-01,  3.1480e-01,
            3.2002e-01,  2.8048e-01,  3.0301e-01,  2.8322e-01,  3.0851e-01,
            3.9450e-01,  4.3201e-01,  3.8485e-01,  3.5451e-01,  3.3210e-01,
            3.7123e-01,  3.4689e-01],
          [ 1.5084e-01,  1.4874e-01,  1.7445e-01,  2.5980e-01,  3.6212e-01,
            3.1915e-01,  2.6645e-01,  2.6639e-01,  2.4997e-01,  2.7230e-01,
            2.9554e-01,  3.0162e-01,  2.9550e-01,  2.5066e-01,  2.1152e-01,
            2.4473e-01,  3.2787e-01,  2.9539e-01,  3.0405e-01,  3.0407e-01,
            3.1373e-01,  2.7158e-01,  2.6521e-01,  2.6953e-01,  3.3142e-01,
            4.3627e-01,  4.0238e-01,  3.7825e-01,  3.1023e-01,  3.3226e-01,
            3.2925e-01,  3.2260e-01],
          [ 1.5697e-01,  1.4217e-01,  1.8460e-01,  2.7694e-01,  3.6310e-01,
            3.3369e-01,  2.9299e-01,  2.8063e-01,  2.7568e-01,  2.7379e-01,
            2.9347e-01,  2.7780e-01,  2.5429e-01,  2.3969e-01,  2.3399e-01,
            2.8214e-01,  3.4203e-01,  3.2119e-01,  2.8488e-01,  2.7453e-01,
            2.9562e-01,  2.8677e-01,  2.4891e-01,  2.6054e-01,  3.3704e-01,
            4.2179e-01,  4.0916e-01,  3.1015e-01,  2.9487e-01,  3.0723e-01,
            2.8443e-01,  2.9695e-01],
          [ 1.5757e-01,  1.6112e-01,  1.7697e-01,  2.8844e-01,  3.8294e-01,
            3.7038e-01,  2.9845e-01,  2.7937e-01,  2.9810e-01,  2.7627e-01,
            2.8421e-01,  2.7833e-01,  2.8286e-01,  2.9006e-01,  2.9156e-01,
            3.0270e-01,  3.4282e-01,  3.1074e-01,  2.6935e-01,  2.8187e-01,
            2.9425e-01,  3.9632e-01,  3.9720e-01,  3.1957e-01,  3.6353e-01,
            4.0332e-01,  4.1185e-01,  3.2163e-01,  3.1979e-01,  2.8571e-01,
            2.8368e-01,  2.9051e-01],
          [ 1.7044e-01,  1.5586e-01,  1.7091e-01,  2.7516e-01,  3.7780e-01,
            4.4885e-01,  3.6329e-01,  2.9606e-01,  3.0135e-01,  2.7029e-01,
            2.7797e-01,  2.7924e-01,  2.9607e-01,  3.4631e-01,  3.4060e-01,
            3.0690e-01,  3.3353e-01,  3.3924e-01,  3.0370e-01,  3.6264e-01,
            4.1014e-01,  4.6376e-01,  4.6168e-01,  4.1034e-01,  3.8115e-01,
            4.3173e-01,  4.2782e-01,  3.4257e-01,  3.1244e-01,  2.8652e-01,
            2.8336e-01,  3.1239e-01],
          [ 1.8081e-01,  2.4384e-01,  2.6137e-01,  3.2567e-01,  3.4582e-01,
            3.9894e-01,  3.9146e-01,  3.5006e-01,  3.1173e-01,  2.9481e-01,
            2.6115e-01,  2.7982e-01,  3.0237e-01,  3.3681e-01,  3.9277e-01,
            3.6614e-01,  3.8377e-01,  3.6566e-01,  3.8766e-01,  4.2980e-01,
            5.0885e-01,  5.3529e-01,  4.2032e-01,  4.0306e-01,  3.6276e-01,
            4.2990e-01,  3.7321e-01,  2.8188e-01,  3.0239e-01,  2.9852e-01,
            3.0273e-01,  3.5082e-01],
          [ 1.6789e-01,  2.8113e-01,  3.0944e-01,  3.3904e-01,  3.7025e-01,
            3.6925e-01,  3.5999e-01,  3.5713e-01,  3.1943e-01,  3.2155e-01,
            2.8795e-01,  3.2429e-01,  3.4987e-01,  3.5924e-01,  4.1806e-01,
            3.9810e-01,  3.8390e-01,  3.3207e-01,  3.6701e-01,  3.8578e-01,
            4.8626e-01,  5.3546e-01,  4.1446e-01,  3.7133e-01,  3.8543e-01,
            3.6137e-01,  2.9988e-01,  2.8269e-01,  2.9079e-01,  2.9424e-01,
            3.2254e-01,  3.7454e-01],
          [ 2.1014e-01,  2.3672e-01,  2.7722e-01,  2.9612e-01,  3.4269e-01,
            3.5885e-01,  3.5351e-01,  3.3507e-01,  3.2608e-01,  3.4459e-01,
            3.3897e-01,  3.5085e-01,  3.6967e-01,  3.8862e-01,  4.1347e-01,
            3.7913e-01,  3.7240e-01,  4.3795e-01,  4.2880e-01,  3.8776e-01,
            4.6003e-01,  4.6754e-01,  3.9287e-01,  4.0574e-01,  3.6056e-01,
            3.3923e-01,  2.9663e-01,  2.6922e-01,  2.8303e-01,  2.9299e-01,
            3.2061e-01,  3.6126e-01],
          [ 2.6025e-01,  2.6053e-01,  2.2610e-01,  1.9838e-01,  2.8050e-01,
            3.2895e-01,  3.5531e-01,  3.5756e-01,  3.4204e-01,  3.5829e-01,
            3.7024e-01,  3.9745e-01,  4.0630e-01,  3.8572e-01,  3.9130e-01,
            3.8624e-01,  4.0117e-01,  4.0200e-01,  3.7749e-01,  3.9505e-01,
            4.7501e-01,  3.5799e-01,  2.9087e-01,  3.4464e-01,  3.4366e-01,
            3.8166e-01,  2.3361e-01,  2.1738e-01,  2.4002e-01,  3.0896e-01,
            3.2055e-01,  3.2962e-01],
          [ 2.3438e-01,  2.5564e-01,  2.3100e-01,  2.0886e-01,  2.4003e-01,
            3.3381e-01,  3.8103e-01,  3.7889e-01,  3.6440e-01,  3.9373e-01,
            4.2822e-01,  4.1723e-01,  4.5883e-01,  4.0625e-01,  3.7932e-01,
            3.8139e-01,  3.8756e-01,  4.0093e-01,  3.9341e-01,  4.6993e-01,
            5.2465e-01,  4.2236e-01,  3.3205e-01,  3.4488e-01,  3.9133e-01,
            3.5772e-01,  1.7819e-01,  1.9508e-01,  2.4125e-01,  2.9964e-01,
            3.3725e-01,  3.0411e-01],
          [ 1.9681e-01,  2.4789e-01,  2.9504e-01,  2.1438e-01,  2.1028e-01,
            3.2045e-01,  3.9408e-01,  3.7944e-01,  3.9158e-01,  4.4892e-01,
            4.7758e-01,  4.2428e-01,  4.1082e-01,  4.1476e-01,  4.3122e-01,
            4.3801e-01,  4.4291e-01,  4.2873e-01,  5.0166e-01,  4.8214e-01,
            5.3667e-01,  5.8591e-01,  5.3717e-01,  5.2342e-01,  5.5996e-01,
            3.8984e-01,  2.0123e-01,  2.0721e-01,  2.3277e-01,  2.4865e-01,
            3.0946e-01,  2.8271e-01],
          [ 2.1447e-01,  2.6670e-01,  3.4599e-01,  2.6533e-01,  1.7172e-01,
            2.5900e-01,  3.4030e-01,  3.7054e-01,  4.1633e-01,  4.9810e-01,
            4.7315e-01,  4.0265e-01,  4.0001e-01,  4.4850e-01,  4.7476e-01,
            4.6458e-01,  4.7121e-01,  4.9414e-01,  5.3852e-01,  5.3955e-01,
            6.0397e-01,  6.4588e-01,  6.3717e-01,  5.8589e-01,  6.5765e-01,
            3.7625e-01,  2.1729e-01,  2.2623e-01,  2.3079e-01,  2.1599e-01,
            2.3699e-01,  2.7182e-01],
          [ 2.4169e-01,  3.6024e-01,  3.9675e-01,  3.2278e-01,  2.1542e-01,
            2.3775e-01,  3.3864e-01,  3.2638e-01,  4.7923e-01,  5.3705e-01,
            4.1171e-01,  3.8816e-01,  3.9564e-01,  4.2635e-01,  4.9329e-01,
            4.6483e-01,  4.2732e-01,  4.7744e-01,  5.2191e-01,  5.6696e-01,
            4.8656e-01,  4.2332e-01,  6.1039e-01,  6.1725e-01,  3.8649e-01,
            2.1347e-01,  2.4220e-01,  2.3133e-01,  2.2415e-01,  2.0363e-01,
            1.9278e-01,  2.2079e-01],
          [ 3.1012e-01,  3.7690e-01,  4.0739e-01,  4.4942e-01,  2.8038e-01,
            2.3442e-01,  3.2833e-01,  3.4846e-01,  5.0772e-01,  4.9545e-01,
            4.0007e-01,  4.0221e-01,  3.8650e-01,  4.1102e-01,  4.8649e-01,
            4.9896e-01,  3.7638e-01,  3.5969e-01,  4.2462e-01,  3.7164e-01,
            2.4576e-01,  2.5716e-01,  5.2328e-01,  4.8664e-01,  1.1351e-01,
            2.1360e-01,  3.3173e-01,  2.5653e-01,  2.2253e-01,  1.8948e-01,
            2.1792e-01,  2.6228e-01],
          [ 2.8224e-01,  3.1262e-01,  3.7726e-01,  5.1277e-01,  3.2314e-01,
            2.3251e-01,  3.2886e-01,  4.3037e-01,  4.6015e-01,  4.2597e-01,
            3.7298e-01,  3.4527e-01,  3.4127e-01,  3.7918e-01,  4.7146e-01,
            4.9175e-01,  3.8193e-01,  2.9852e-01,  2.7971e-01,  1.8768e-01,
            2.0900e-01,  2.8741e-01,  3.6388e-01,  3.1696e-01,  1.3197e-01,
            3.2318e-01,  4.0348e-01,  2.9510e-01,  2.5816e-01,  2.4428e-01,
            2.6417e-01,  3.0603e-01],
          [ 2.1870e-01,  2.5456e-01,  3.0300e-01,  4.7662e-01,  2.9938e-01,
            1.9329e-01,  3.6018e-01,  4.3632e-01,  4.0033e-01,  3.8523e-01,
            3.4960e-01,  3.2951e-01,  3.4231e-01,  4.0684e-01,  4.6479e-01,
            5.0478e-01,  3.6898e-01,  1.8866e-01,  1.6473e-01,  1.3568e-01,
            1.8621e-01,  2.1063e-01,  2.5143e-01,  2.2247e-01,  1.5847e-01,
            3.0210e-01,  4.1322e-01,  3.5487e-01,  3.5417e-01,  3.3163e-01,
            3.3627e-01,  3.2426e-01],
          [ 2.0840e-01,  2.4966e-01,  2.9466e-01,  4.3391e-01,  2.5116e-01,
            2.0905e-01,  4.4122e-01,  4.4214e-01,  4.4729e-01,  4.3319e-01,
            4.0322e-01,  3.3284e-01,  3.6169e-01,  3.9875e-01,  5.1971e-01,
            5.0384e-01,  2.4414e-01,  1.3687e-01,  1.5430e-01,  1.9290e-01,
            1.7764e-01,  1.5508e-01,  1.9174e-01,  1.9142e-01,  1.6958e-01,
            2.4540e-01,  3.4594e-01,  3.6960e-01,  3.5244e-01,  3.6235e-01,
            3.8176e-01,  3.6720e-01],
          [ 2.1014e-01,  2.4969e-01,  3.8323e-01,  4.5074e-01,  2.0775e-01,
            2.5239e-01,  4.9404e-01,  4.5083e-01,  4.5366e-01,  5.0120e-01,
            4.5065e-01,  3.4775e-01,  3.6963e-01,  3.6939e-01,  4.4379e-01,
            4.1047e-01,  2.1702e-01,  1.8258e-01,  1.9729e-01,  1.7377e-01,
            1.8647e-01,  2.1145e-01,  2.1059e-01,  1.9930e-01,  1.9266e-01,
            2.7453e-01,  3.3914e-01,  3.5287e-01,  3.6600e-01,  4.0040e-01,
            4.1682e-01,  4.0659e-01],
          [ 2.0841e-01,  2.6144e-01,  3.8790e-01,  3.7216e-01,  2.3688e-01,
            3.8494e-01,  5.0032e-01,  4.3318e-01,  4.4479e-01,  5.2326e-01,
            5.1017e-01,  4.4701e-01,  3.5357e-01,  3.5425e-01,  3.8993e-01,
            3.5249e-01,  2.7085e-01,  1.9924e-01,  2.1728e-01,  2.2463e-01,
            2.7819e-01,  2.7087e-01,  2.7546e-01,  3.2171e-01,  3.5627e-01,
            3.6875e-01,  3.6708e-01,  3.4124e-01,  3.7470e-01,  3.7441e-01,
            4.0298e-01,  4.1072e-01],
          [ 2.2148e-01,  2.6963e-01,  3.2617e-01,  3.0222e-01,  3.5568e-01,
            5.0493e-01,  5.0879e-01,  4.5589e-01,  4.7479e-01,  5.0053e-01,
            5.1263e-01,  5.6113e-01,  4.8976e-01,  4.2622e-01,  4.2725e-01,
            3.1903e-01,  2.7496e-01,  2.3731e-01,  2.4130e-01,  3.1702e-01,
            4.0870e-01,  3.9733e-01,  3.8068e-01,  4.2842e-01,  4.8006e-01,
            4.4127e-01,  4.3031e-01,  3.7451e-01,  4.1848e-01,  4.0137e-01,
            3.7451e-01,  3.8139e-01],
          [ 2.4648e-01,  2.6245e-01,  3.7275e-01,  3.4214e-01,  4.1962e-01,
            4.7868e-01,  4.9657e-01,  4.7886e-01,  5.1623e-01,  5.4213e-01,
            5.5495e-01,  6.1121e-01,  6.4368e-01,  5.3600e-01,  4.4007e-01,
            3.7920e-01,  3.0584e-01,  3.3138e-01,  3.8881e-01,  4.4251e-01,
            4.2606e-01,  3.5150e-01,  3.7927e-01,  4.1622e-01,  4.5345e-01,
            4.4575e-01,  4.9209e-01,  4.4300e-01,  4.2981e-01,  4.1879e-01,
            4.0794e-01,  3.5829e-01],
          [ 2.4542e-01,  2.9513e-01,  3.8403e-01,  3.9353e-01,  4.1391e-01,
            4.4021e-01,  4.5837e-01,  4.2752e-01,  4.8962e-01,  5.8382e-01,
            5.9143e-01,  5.7842e-01,  5.6889e-01,  5.6000e-01,  4.9498e-01,
            4.4403e-01,  4.7448e-01,  4.6953e-01,  4.7502e-01,  5.0856e-01,
            4.3189e-01,  3.7377e-01,  3.6856e-01,  4.0749e-01,  4.2313e-01,
            4.1622e-01,  5.0063e-01,  4.9195e-01,  4.5441e-01,  4.0818e-01,
            4.3812e-01,  3.9760e-01],
          [ 2.4341e-01,  3.7409e-01,  3.9716e-01,  3.9444e-01,  4.2450e-01,
            4.3375e-01,  4.4922e-01,  4.0635e-01,  4.2168e-01,  5.0910e-01,
            5.5909e-01,  5.4486e-01,  5.0933e-01,  5.4403e-01,  5.2360e-01,
            5.1378e-01,  5.2628e-01,  5.0541e-01,  4.8439e-01,  4.9628e-01,
            4.8770e-01,  4.3064e-01,  3.4536e-01,  3.5055e-01,  3.9209e-01,
            4.0434e-01,  4.7774e-01,  4.7210e-01,  4.0646e-01,  4.0138e-01,
            4.4571e-01,  4.0861e-01],
          [ 2.6718e-01,  4.2178e-01,  3.8534e-01,  4.1912e-01,  4.7055e-01,
            5.0697e-01,  5.1469e-01,  4.1871e-01,  4.7702e-01,  4.5828e-01,
            4.8149e-01,  5.0985e-01,  5.0630e-01,  5.2887e-01,  5.0578e-01,
            5.3648e-01,  5.3532e-01,  5.8972e-01,  5.6685e-01,  5.2383e-01,
            4.8844e-01,  4.3195e-01,  4.0146e-01,  3.7954e-01,  3.8580e-01,
            3.7813e-01,  4.3393e-01,  4.6092e-01,  4.2617e-01,  4.3901e-01,
            4.5350e-01,  3.9508e-01],
          [ 3.1762e-01,  3.4698e-01,  3.0686e-01,  3.9508e-01,  4.5276e-01,
            4.9859e-01,  4.5701e-01,  4.5054e-01,  5.0780e-01,  4.7484e-01,
            4.8157e-01,  4.4384e-01,  4.7843e-01,  4.8942e-01,  4.8934e-01,
            5.1431e-01,  4.9849e-01,  6.2509e-01,  6.5614e-01,  4.9927e-01,
            4.4770e-01,  4.5020e-01,  3.9979e-01,  3.9214e-01,  3.9994e-01,
            3.7408e-01,  3.8689e-01,  4.5463e-01,  4.6252e-01,  4.5546e-01,
            4.5066e-01,  3.9726e-01],
          [ 3.8870e-01,  3.5090e-01,  3.7058e-01,  3.8393e-01,  3.7641e-01,
            4.6761e-01,  4.2875e-01,  4.0523e-01,  4.2151e-01,  4.8225e-01,
            4.5765e-01,  4.1688e-01,  4.3103e-01,  4.2181e-01,  3.7008e-01,
            4.1915e-01,  4.1131e-01,  5.0640e-01,  5.7890e-01,  5.0532e-01,
            4.3219e-01,  4.6466e-01,  4.4535e-01,  3.9661e-01,  3.7316e-01,
            3.5160e-01,  3.4986e-01,  4.2956e-01,  4.5269e-01,  4.2214e-01,
            4.2315e-01,  3.6931e-01],
          [ 4.0117e-01,  3.9343e-01,  3.7502e-01,  4.2223e-01,  3.9955e-01,
            4.2678e-01,  3.9177e-01,  3.8282e-01,  3.9452e-01,  4.5629e-01,
            4.4715e-01,  4.2533e-01,  3.8181e-01,  4.3205e-01,  3.9218e-01,
            3.8545e-01,  4.3955e-01,  4.2447e-01,  4.6282e-01,  4.8058e-01,
            4.7440e-01,  4.8407e-01,  4.7674e-01,  3.8392e-01,  3.5178e-01,
            3.4963e-01,  3.3116e-01,  3.8140e-01,  4.1098e-01,  4.0464e-01,
            3.8145e-01,  3.3799e-01],
          [ 4.1219e-01,  3.6831e-01,  3.2036e-01,  4.2093e-01,  4.1263e-01,
            4.0472e-01,  3.7301e-01,  3.8377e-01,  4.2804e-01,  4.3839e-01,
            4.3999e-01,  3.9801e-01,  3.2529e-01,  3.4532e-01,  3.7403e-01,
            3.8758e-01,  4.1360e-01,  4.2135e-01,  4.3793e-01,  4.3846e-01,
            4.5850e-01,  4.6296e-01,  4.2582e-01,  3.3881e-01,  3.3262e-01,
            3.0867e-01,  3.3089e-01,  3.5479e-01,  3.7339e-01,  3.7113e-01,
            3.6095e-01,  3.1721e-01],
          [ 4.3762e-01,  3.5523e-01,  3.4055e-01,  3.8282e-01,  4.6478e-01,
            4.8197e-01,  4.3685e-01,  3.8031e-01,  4.3044e-01,  4.6194e-01,
            4.4243e-01,  4.1593e-01,  4.1738e-01,  3.9761e-01,  4.1076e-01,
            4.2142e-01,  4.0458e-01,  4.0715e-01,  4.2215e-01,  3.9667e-01,
            3.8046e-01,  4.1957e-01,  3.7253e-01,  3.4158e-01,  3.2183e-01,
            2.9855e-01,  3.1813e-01,  3.2911e-01,  3.3692e-01,  3.2346e-01,
            3.4877e-01,  3.5151e-01],
          [ 4.4013e-01,  3.6991e-01,  4.4231e-01,  4.6862e-01,  5.0422e-01,
            5.2971e-01,  5.1190e-01,  4.7635e-01,  4.4804e-01,  4.7892e-01,
            4.5107e-01,  4.3062e-01,  5.2756e-01,  4.8974e-01,  4.3336e-01,
            4.1674e-01,  4.4152e-01,  4.4318e-01,  4.2930e-01,  4.1517e-01,
            3.7537e-01,  3.9236e-01,  3.5634e-01,  3.1435e-01,  3.1061e-01,
            2.9293e-01,  2.7217e-01,  2.9575e-01,  2.8999e-01,  2.8700e-01,
            3.8623e-01,  4.1237e-01]],

         [[ 9.0195e-02,  6.9606e-02,  7.9276e-02,  1.3304e-01,  2.0537e-01,
            2.5352e-01,  2.4179e-01,  2.0083e-01,  1.6983e-01,  1.8616e-01,
            1.6589e-01,  1.5783e-01,  1.7385e-01,  1.5378e-01,  1.3880e-01,
            1.2488e-01,  1.1777e-01,  1.2675e-01,  1.5081e-01,  1.5669e-01,
            1.7233e-01,  1.6629e-01,  1.8084e-01,  1.9858e-01,  1.6576e-01,
            2.0172e-01,  2.3997e-01,  2.3894e-01,  1.5420e-01,  1.4753e-01,
            1.5383e-01,  1.5631e-01],
          [ 1.2520e-01,  1.0426e-01,  1.1324e-01,  1.5763e-01,  2.4616e-01,
            2.3460e-01,  2.0283e-01,  1.9322e-01,  1.9266e-01,  1.7841e-01,
            1.7631e-01,  1.6094e-01,  1.6409e-01,  1.5300e-01,  1.0003e-01,
            1.3492e-01,  1.3175e-01,  1.4866e-01,  1.6583e-01,  1.4547e-01,
            1.7283e-01,  1.6923e-01,  1.6214e-01,  1.9008e-01,  1.8348e-01,
            2.3314e-01,  2.8054e-01,  2.4602e-01,  1.6489e-01,  1.4523e-01,
            1.7824e-01,  2.0480e-01],
          [ 1.1759e-01,  1.1793e-01,  1.3741e-01,  1.9133e-01,  2.5685e-01,
            2.1901e-01,  1.8712e-01,  1.5901e-01,  1.8016e-01,  1.7341e-01,
            1.7636e-01,  1.9318e-01,  1.7392e-01,  1.4100e-01,  1.0998e-01,
            1.2910e-01,  1.7122e-01,  1.5043e-01,  1.9736e-01,  1.8054e-01,
            1.8780e-01,  1.6145e-01,  1.6185e-01,  1.7246e-01,  1.9045e-01,
            2.6330e-01,  2.8463e-01,  2.2904e-01,  1.8117e-01,  1.7130e-01,
            1.9428e-01,  1.9367e-01],
          [ 8.3939e-02,  9.8113e-02,  1.2009e-01,  1.8749e-01,  2.5585e-01,
            2.2833e-01,  1.6686e-01,  1.7412e-01,  1.5928e-01,  1.6510e-01,
            1.7906e-01,  2.0007e-01,  1.7002e-01,  1.4440e-01,  1.1994e-01,
            1.5052e-01,  2.0252e-01,  1.9073e-01,  2.0436e-01,  1.8717e-01,
            1.8995e-01,  1.7761e-01,  1.3582e-01,  1.4948e-01,  2.1344e-01,
            2.9850e-01,  2.7456e-01,  2.1526e-01,  1.2756e-01,  1.6824e-01,
            1.5695e-01,  1.3709e-01],
          [ 9.2939e-02,  1.0271e-01,  1.0613e-01,  1.8859e-01,  2.5506e-01,
            2.1690e-01,  1.8010e-01,  1.7927e-01,  1.7747e-01,  1.5273e-01,
            1.6800e-01,  1.8141e-01,  1.5907e-01,  1.4640e-01,  1.4268e-01,
            1.6862e-01,  2.3045e-01,  1.9302e-01,  1.6843e-01,  1.5125e-01,
            1.8885e-01,  1.8046e-01,  1.3922e-01,  1.4974e-01,  2.3418e-01,
            3.0190e-01,  2.7888e-01,  1.6230e-01,  1.1820e-01,  1.3333e-01,
            1.2806e-01,  1.3402e-01],
          [ 9.2440e-02,  9.3330e-02,  1.1362e-01,  1.8983e-01,  2.6286e-01,
            2.4616e-01,  1.9830e-01,  1.8737e-01,  1.8402e-01,  1.6094e-01,
            1.5713e-01,  1.5724e-01,  1.7178e-01,  1.8118e-01,  1.7787e-01,
            1.9681e-01,  2.1484e-01,  1.9795e-01,  1.3096e-01,  1.5805e-01,
            1.9176e-01,  2.6866e-01,  2.4473e-01,  1.7801e-01,  2.3927e-01,
            2.8250e-01,  2.7101e-01,  1.7197e-01,  1.5075e-01,  1.1314e-01,
            1.1895e-01,  1.2852e-01],
          [ 9.5400e-02,  9.7319e-02,  1.1145e-01,  1.8271e-01,  2.7503e-01,
            2.9875e-01,  2.5221e-01,  2.0492e-01,  1.9016e-01,  1.4770e-01,
            1.6681e-01,  1.5815e-01,  1.9341e-01,  2.4387e-01,  2.3640e-01,
            2.1145e-01,  2.1577e-01,  2.1621e-01,  1.7831e-01,  2.3698e-01,
            3.0411e-01,  3.4519e-01,  3.1537e-01,  2.5115e-01,  2.4317e-01,
            2.7655e-01,  2.8720e-01,  1.9893e-01,  1.4630e-01,  1.3264e-01,
            1.3044e-01,  1.5523e-01],
          [ 1.0654e-01,  1.5724e-01,  1.7483e-01,  2.0059e-01,  2.2985e-01,
            2.6082e-01,  2.4897e-01,  2.3120e-01,  2.1664e-01,  1.9348e-01,
            1.6890e-01,  1.8370e-01,  1.9434e-01,  2.2483e-01,  2.7353e-01,
            2.5303e-01,  2.7424e-01,  2.4589e-01,  2.8566e-01,  3.0510e-01,
            4.1989e-01,  4.1954e-01,  2.7286e-01,  2.6296e-01,  2.3469e-01,
            2.7984e-01,  2.5567e-01,  1.3669e-01,  1.5442e-01,  1.3895e-01,
            1.4874e-01,  2.0010e-01],
          [ 6.3073e-02,  1.9739e-01,  2.3061e-01,  2.2957e-01,  2.3190e-01,
            2.2735e-01,  2.2518e-01,  2.3308e-01,  2.0533e-01,  2.0495e-01,
            1.7664e-01,  2.1727e-01,  2.4179e-01,  2.3713e-01,  2.9752e-01,
            2.7273e-01,  2.6193e-01,  2.4186e-01,  2.4477e-01,  2.6640e-01,
            3.8717e-01,  4.2020e-01,  2.6146e-01,  2.4567e-01,  2.4540e-01,
            2.1173e-01,  1.6462e-01,  1.4580e-01,  1.5126e-01,  1.4552e-01,
            1.4560e-01,  2.0067e-01],
          [ 8.6303e-02,  1.5366e-01,  1.8308e-01,  1.9103e-01,  2.3135e-01,
            2.4150e-01,  2.3724e-01,  2.2543e-01,  2.0905e-01,  2.4039e-01,
            2.5136e-01,  2.6276e-01,  2.8108e-01,  2.8944e-01,  2.9717e-01,
            2.7097e-01,  2.8170e-01,  3.3457e-01,  2.9585e-01,  2.7210e-01,
            3.7350e-01,  3.5644e-01,  2.4900e-01,  2.8383e-01,  2.3721e-01,
            2.2010e-01,  1.5643e-01,  1.5190e-01,  1.4287e-01,  1.5608e-01,
            1.6060e-01,  2.0464e-01],
          [ 1.5620e-01,  1.5484e-01,  1.2747e-01,  1.1242e-01,  1.6793e-01,
            2.1487e-01,  2.4350e-01,  2.4767e-01,  2.3824e-01,  2.6974e-01,
            2.7664e-01,  3.0318e-01,  3.2045e-01,  3.0136e-01,  2.9085e-01,
            2.7517e-01,  3.0491e-01,  3.0906e-01,  2.8625e-01,  3.0713e-01,
            3.9927e-01,  2.5707e-01,  1.6537e-01,  2.5132e-01,  2.4712e-01,
            2.6872e-01,  1.4634e-01,  1.0750e-01,  1.2302e-01,  1.5922e-01,
            1.7744e-01,  1.8027e-01],
          [ 1.1944e-01,  1.4971e-01,  1.5034e-01,  1.4520e-01,  1.5016e-01,
            2.3427e-01,  2.9101e-01,  2.7430e-01,  2.6202e-01,  2.9840e-01,
            3.4895e-01,  3.5199e-01,  3.5441e-01,  3.1786e-01,  2.9651e-01,
            3.0176e-01,  3.0835e-01,  3.1010e-01,  3.0998e-01,  3.9066e-01,
            4.3441e-01,  3.2043e-01,  2.2179e-01,  2.4254e-01,  3.0480e-01,
            2.8319e-01,  8.4028e-02,  1.0288e-01,  1.1529e-01,  1.6626e-01,
            1.9623e-01,  1.5042e-01],
          [ 8.2630e-02,  1.4719e-01,  2.0957e-01,  1.4551e-01,  1.2511e-01,
            2.3886e-01,  2.8303e-01,  2.8271e-01,  2.7682e-01,  3.3340e-01,
            3.9598e-01,  3.5809e-01,  3.3090e-01,  3.3229e-01,  3.2270e-01,
            3.5289e-01,  3.5596e-01,  3.3653e-01,  4.0337e-01,  3.9221e-01,
            4.3152e-01,  4.9775e-01,  4.4284e-01,  4.1053e-01,  4.7771e-01,
            3.1549e-01,  1.1746e-01,  1.1470e-01,  1.3871e-01,  1.5250e-01,
            1.8613e-01,  1.8008e-01],
          [ 1.1922e-01,  1.8159e-01,  2.4724e-01,  1.8611e-01,  1.0568e-01,
            1.6986e-01,  2.7174e-01,  2.4470e-01,  3.1103e-01,  4.3068e-01,
            3.8992e-01,  3.3840e-01,  3.3182e-01,  3.5537e-01,  3.9194e-01,
            3.6858e-01,  3.7666e-01,  4.1204e-01,  4.5012e-01,  4.4431e-01,
            4.9471e-01,  5.4816e-01,  5.4086e-01,  5.0096e-01,  5.6865e-01,
            3.0135e-01,  1.4568e-01,  1.4342e-01,  1.3982e-01,  1.3578e-01,
            1.2593e-01,  1.5146e-01],
          [ 1.4390e-01,  2.6099e-01,  3.0331e-01,  2.3876e-01,  1.3666e-01,
            1.6517e-01,  2.6362e-01,  2.2421e-01,  3.6740e-01,  4.2855e-01,
            3.1158e-01,  3.2402e-01,  2.9655e-01,  3.2882e-01,  3.8708e-01,
            3.5455e-01,  3.4707e-01,  3.9101e-01,  4.4316e-01,  4.7248e-01,
            4.2320e-01,  3.4264e-01,  5.2938e-01,  5.5537e-01,  2.9234e-01,
            1.2576e-01,  1.5921e-01,  1.3127e-01,  1.2900e-01,  1.0944e-01,
            1.0450e-01,  1.3812e-01],
          [ 2.0154e-01,  2.6945e-01,  2.9314e-01,  3.6252e-01,  1.9438e-01,
            1.4706e-01,  2.4659e-01,  2.5121e-01,  3.8024e-01,  3.6876e-01,
            2.8147e-01,  3.0358e-01,  2.7191e-01,  2.9760e-01,  3.7572e-01,
            3.8727e-01,  2.8855e-01,  3.0747e-01,  3.4598e-01,  2.8031e-01,
            1.6373e-01,  1.7190e-01,  4.3148e-01,  3.8966e-01,  1.9331e-02,
            1.3336e-01,  2.5197e-01,  1.3018e-01,  1.1307e-01,  1.1722e-01,
            1.3101e-01,  1.5829e-01],
          [ 2.1982e-01,  2.1459e-01,  2.7283e-01,  3.9341e-01,  2.2275e-01,
            1.4263e-01,  2.4700e-01,  3.3418e-01,  3.4477e-01,  3.0948e-01,
            2.7402e-01,  2.3411e-01,  2.4698e-01,  2.7473e-01,  3.5824e-01,
            3.8835e-01,  3.1064e-01,  2.2402e-01,  2.0605e-01,  1.0445e-01,
            1.3899e-01,  1.9674e-01,  2.7008e-01,  2.1885e-01,  5.9575e-02,
            2.6160e-01,  3.1443e-01,  1.8840e-01,  1.5088e-01,  1.4768e-01,
            1.4980e-01,  1.8244e-01],
          [ 1.5879e-01,  1.5751e-01,  2.0429e-01,  3.6190e-01,  2.0146e-01,
            1.0455e-01,  2.7448e-01,  3.4223e-01,  2.9776e-01,  2.6819e-01,
            2.4663e-01,  2.2255e-01,  2.5442e-01,  2.8585e-01,  3.3816e-01,
            4.1810e-01,  2.6878e-01,  1.2686e-01,  9.2554e-02,  6.5646e-02,
            1.0297e-01,  1.4680e-01,  1.7047e-01,  1.6348e-01,  9.3599e-02,
            2.3966e-01,  3.0810e-01,  2.6134e-01,  2.4141e-01,  1.9991e-01,
            1.9541e-01,  1.9199e-01],
          [ 1.3803e-01,  1.7034e-01,  2.0503e-01,  3.1731e-01,  1.6826e-01,
            1.4326e-01,  3.4979e-01,  3.6173e-01,  3.4312e-01,  3.0607e-01,
            2.8151e-01,  2.2501e-01,  2.6314e-01,  2.9072e-01,  3.9660e-01,
            4.0491e-01,  1.4849e-01,  7.3316e-02,  9.6048e-02,  1.3471e-01,
            1.0918e-01,  8.5497e-02,  1.4354e-01,  1.1429e-01,  1.0022e-01,
            1.7253e-01,  2.4853e-01,  2.4711e-01,  2.2985e-01,  2.3133e-01,
            2.3307e-01,  2.1610e-01],
          [ 1.4091e-01,  1.6766e-01,  2.8416e-01,  3.6206e-01,  1.3818e-01,
            1.8178e-01,  3.9888e-01,  3.3819e-01,  3.3940e-01,  3.5547e-01,
            3.2531e-01,  2.6897e-01,  2.6100e-01,  2.5725e-01,  3.4548e-01,
            3.1004e-01,  1.4834e-01,  1.1319e-01,  1.5260e-01,  1.1334e-01,
            1.2141e-01,  1.5019e-01,  1.2406e-01,  1.3194e-01,  1.3037e-01,
            1.7779e-01,  2.4577e-01,  2.2668e-01,  2.2605e-01,  2.4095e-01,
            2.6266e-01,  2.5924e-01],
          [ 1.3244e-01,  1.7509e-01,  2.8270e-01,  2.5668e-01,  1.5088e-01,
            3.0516e-01,  4.0508e-01,  3.1838e-01,  3.0863e-01,  3.8651e-01,
            3.6719e-01,  3.4330e-01,  2.6443e-01,  2.4465e-01,  2.8666e-01,
            2.3698e-01,  1.9976e-01,  1.5294e-01,  1.6315e-01,  1.6944e-01,
            2.0626e-01,  1.7154e-01,  1.6972e-01,  2.2080e-01,  2.5514e-01,
            2.6750e-01,  2.5903e-01,  2.0016e-01,  2.1424e-01,  2.3376e-01,
            2.4326e-01,  2.4924e-01],
          [ 1.4829e-01,  1.8591e-01,  2.3589e-01,  1.8183e-01,  2.4353e-01,
            4.4068e-01,  4.0263e-01,  3.2515e-01,  3.5069e-01,  3.4789e-01,
            3.4964e-01,  4.2155e-01,  3.6438e-01,  3.1081e-01,  2.9167e-01,
            1.9738e-01,  1.7602e-01,  1.5870e-01,  1.7706e-01,  2.4851e-01,
            2.9998e-01,  2.7653e-01,  2.6167e-01,  2.9851e-01,  3.6389e-01,
            3.0340e-01,  2.8703e-01,  2.2233e-01,  2.5081e-01,  2.6092e-01,
            2.3205e-01,  2.2343e-01],
          [ 1.3774e-01,  1.6304e-01,  2.5311e-01,  2.3143e-01,  3.0709e-01,
            3.8319e-01,  3.6747e-01,  3.4578e-01,  3.9351e-01,  3.7334e-01,
            3.8199e-01,  4.3519e-01,  4.8920e-01,  4.0122e-01,  3.1120e-01,
            2.4414e-01,  2.0002e-01,  2.2450e-01,  2.9046e-01,  3.2443e-01,
            2.8602e-01,  2.1516e-01,  2.4556e-01,  2.9956e-01,  3.2922e-01,
            3.0358e-01,  3.5744e-01,  2.9464e-01,  2.7052e-01,  2.8104e-01,
            2.7327e-01,  2.2005e-01],
          [ 1.5430e-01,  1.8533e-01,  2.5458e-01,  2.8142e-01,  3.0848e-01,
            3.4411e-01,  3.3323e-01,  3.1280e-01,  3.6130e-01,  4.2312e-01,
            4.0152e-01,  3.7380e-01,  3.9775e-01,  4.0735e-01,  3.4804e-01,
            3.1010e-01,  3.0805e-01,  3.3219e-01,  3.2571e-01,  3.3168e-01,
            2.8681e-01,  2.4084e-01,  2.1383e-01,  2.7057e-01,  3.0622e-01,
            2.9035e-01,  3.5425e-01,  3.3784e-01,  2.9039e-01,  2.6870e-01,
            2.7347e-01,  2.4390e-01],
          [ 1.4833e-01,  2.5322e-01,  2.7408e-01,  2.6978e-01,  2.9141e-01,
            2.9545e-01,  3.1584e-01,  3.0832e-01,  2.9518e-01,  3.5720e-01,
            3.6111e-01,  3.5417e-01,  3.4513e-01,  3.8793e-01,  4.0518e-01,
            3.6489e-01,  3.3975e-01,  3.4185e-01,  3.1545e-01,  3.4080e-01,
            3.5376e-01,  2.7826e-01,  1.9056e-01,  2.1863e-01,  2.7669e-01,
            2.9297e-01,  3.1645e-01,  3.2075e-01,  2.5723e-01,  2.6741e-01,
            2.8466e-01,  2.7554e-01],
          [ 1.8634e-01,  2.9234e-01,  2.6238e-01,  2.8587e-01,  3.2910e-01,
            3.6788e-01,  3.9065e-01,  3.1181e-01,  3.6219e-01,  3.4673e-01,
            3.3901e-01,  3.2474e-01,  3.4547e-01,  3.8053e-01,  3.7923e-01,
            3.7995e-01,  3.8585e-01,  3.9702e-01,  3.8765e-01,  3.4510e-01,
            3.1072e-01,  2.6837e-01,  2.6804e-01,  2.5706e-01,  2.7098e-01,
            2.5360e-01,  2.8111e-01,  3.1010e-01,  2.7524e-01,  2.9035e-01,
            2.9943e-01,  2.6709e-01],
          [ 2.3983e-01,  2.5725e-01,  2.1610e-01,  2.6715e-01,  3.1968e-01,
            3.4869e-01,  3.2602e-01,  3.2837e-01,  4.1503e-01,  3.9393e-01,
            3.6518e-01,  3.0604e-01,  3.4280e-01,  3.2631e-01,  3.4729e-01,
            3.8313e-01,  3.5535e-01,  4.5206e-01,  4.8095e-01,  3.3478e-01,
            2.9672e-01,  2.9427e-01,  2.5773e-01,  2.5748e-01,  2.6918e-01,
            2.3602e-01,  2.4864e-01,  3.1494e-01,  3.2776e-01,  2.9552e-01,
            3.0165e-01,  2.6020e-01],
          [ 2.9617e-01,  2.6836e-01,  2.5574e-01,  2.8948e-01,  2.3774e-01,
            3.2670e-01,  3.2095e-01,  2.9375e-01,  2.9733e-01,  3.8512e-01,
            3.4741e-01,  2.8368e-01,  3.1751e-01,  3.0746e-01,  2.4306e-01,
            2.9461e-01,  2.8799e-01,  3.4725e-01,  4.1886e-01,  3.3705e-01,
            2.9040e-01,  3.1930e-01,  3.1177e-01,  2.5972e-01,  2.5005e-01,
            2.3613e-01,  2.4282e-01,  2.9285e-01,  3.1465e-01,  2.7912e-01,
            2.5803e-01,  2.4212e-01],
          [ 3.1944e-01,  3.1439e-01,  2.7551e-01,  3.2365e-01,  2.8286e-01,
            3.0412e-01,  2.8724e-01,  2.7055e-01,  2.6732e-01,  3.3793e-01,
            3.1756e-01,  2.9973e-01,  2.6704e-01,  3.2089e-01,  2.7161e-01,
            2.7233e-01,  3.1960e-01,  2.9919e-01,  3.3019e-01,  3.4217e-01,
            3.4485e-01,  3.5144e-01,  3.3438e-01,  2.4693e-01,  2.2173e-01,
            2.0645e-01,  2.1026e-01,  2.4923e-01,  2.8203e-01,  2.8155e-01,
            2.5342e-01,  2.2806e-01],
          [ 3.2489e-01,  2.7428e-01,  2.2891e-01,  3.0411e-01,  2.8893e-01,
            2.6687e-01,  2.7110e-01,  3.0106e-01,  3.0767e-01,  3.3725e-01,
            3.3238e-01,  2.9732e-01,  2.2484e-01,  2.4532e-01,  2.5393e-01,
            2.8443e-01,  3.1136e-01,  3.0300e-01,  3.2607e-01,  3.1037e-01,
            3.3273e-01,  3.4335e-01,  2.9703e-01,  2.1166e-01,  2.0779e-01,
            1.9604e-01,  1.9935e-01,  2.2443e-01,  2.5250e-01,  2.7635e-01,
            2.2275e-01,  1.8840e-01],
          [ 3.5710e-01,  2.6982e-01,  2.4222e-01,  2.9219e-01,  3.4024e-01,
            3.4765e-01,  3.0232e-01,  2.8612e-01,  3.2380e-01,  3.4878e-01,
            3.5471e-01,  3.1687e-01,  3.1546e-01,  2.8723e-01,  2.8455e-01,
            3.0489e-01,  2.8900e-01,  2.8257e-01,  3.0462e-01,  2.7307e-01,
            2.7377e-01,  3.2166e-01,  2.5892e-01,  2.0482e-01,  1.9629e-01,
            1.8348e-01,  2.1709e-01,  2.2280e-01,  2.1287e-01,  2.1045e-01,
            2.2917e-01,  2.3273e-01],
          [ 3.6107e-01,  2.9928e-01,  3.6771e-01,  3.5648e-01,  3.8292e-01,
            4.0134e-01,  3.8309e-01,  3.7718e-01,  3.2702e-01,  3.6296e-01,
            3.5466e-01,  3.4411e-01,  4.5256e-01,  3.6755e-01,  3.2614e-01,
            3.1077e-01,  3.1721e-01,  3.1505e-01,  3.0771e-01,  2.7076e-01,
            2.6149e-01,  2.8494e-01,  2.7175e-01,  2.0869e-01,  1.8951e-01,
            1.7127e-01,  1.6629e-01,  1.9488e-01,  2.1568e-01,  1.8298e-01,
            2.5674e-01,  2.5887e-01]],

         [[-4.7599e-03, -1.2778e-02,  1.3440e-02,  3.4988e-02,  1.2878e-01,
            1.6517e-01,  1.7123e-01,  1.1395e-01,  9.7179e-02,  1.0241e-01,
            7.9242e-02,  7.1092e-02,  9.6668e-02,  7.8746e-02,  6.5340e-02,
            7.5147e-02,  3.4274e-02,  3.7329e-02,  4.3943e-02,  5.8826e-02,
            6.0385e-02,  7.4261e-02,  1.1291e-01,  8.6774e-02,  6.6135e-02,
            1.2301e-01,  1.3336e-01,  1.4330e-01,  7.6612e-02,  6.9195e-02,
            8.9603e-02,  8.7339e-02],
          [ 2.6966e-02,  3.7670e-02,  3.5398e-02,  6.7038e-02,  1.2149e-01,
            1.3293e-01,  1.1169e-01,  1.0829e-01,  1.2679e-01,  9.6326e-02,
            8.1668e-02,  8.0999e-02,  8.2933e-02,  7.6974e-02,  4.3080e-02,
            6.6895e-02,  4.5265e-02,  5.9123e-02,  6.8163e-02,  4.0067e-02,
            6.0463e-02,  6.1672e-02,  5.6076e-02,  7.5984e-02,  1.0071e-01,
            1.1773e-01,  1.6281e-01,  1.1635e-01,  5.7999e-02,  6.0168e-02,
            8.5881e-02,  1.0277e-01],
          [ 8.6490e-03,  3.8942e-02,  5.0355e-02,  7.3430e-02,  1.2493e-01,
            1.0667e-01,  7.3766e-02,  6.4724e-02,  8.6638e-02,  9.4898e-02,
            9.8518e-02,  9.1681e-02,  7.8405e-02,  7.0862e-02,  3.6743e-02,
            5.3654e-02,  8.2302e-02,  5.6120e-02,  9.4421e-02,  6.1661e-02,
            7.1476e-02,  6.0421e-02,  6.2423e-02,  6.9500e-02,  7.5146e-02,
            1.4138e-01,  1.6531e-01,  1.0920e-01,  6.3098e-02,  4.4519e-02,
            8.9874e-02,  9.1179e-02],
          [ 6.6277e-03,  2.1252e-02,  4.2097e-02,  6.6531e-02,  1.1928e-01,
            9.8490e-02,  7.9983e-02,  7.5126e-02,  7.1581e-02,  8.8600e-02,
            9.4463e-02,  7.9749e-02,  8.8648e-02,  6.3337e-02,  4.4535e-02,
            7.3260e-02,  1.1073e-01,  8.4956e-02,  7.8646e-02,  8.8785e-02,
            6.8299e-02,  5.4705e-02,  6.1785e-02,  3.9759e-02,  9.1078e-02,
            1.7665e-01,  1.3846e-01,  1.0704e-01,  4.4340e-02,  4.9614e-02,
            5.4961e-02,  5.5663e-02],
          [ 2.8726e-02,  2.5597e-03,  1.4900e-02,  9.4119e-02,  1.1247e-01,
            1.1192e-01,  8.6869e-02,  1.0255e-01,  8.6745e-02,  6.8799e-02,
            9.1662e-02,  7.9099e-02,  7.4017e-02,  6.8997e-02,  4.8202e-02,
            8.2269e-02,  1.2771e-01,  9.3721e-02,  6.2110e-02,  6.7848e-02,
            8.9014e-02,  8.4525e-02,  4.8249e-02,  4.6698e-02,  9.7134e-02,
            1.8859e-01,  1.6281e-01,  6.4843e-02,  3.1115e-02,  5.0556e-02,
            3.7904e-02,  4.1754e-02],
          [ 2.0033e-02,  1.9686e-02,  3.2507e-02,  8.0932e-02,  1.2633e-01,
            1.2805e-01,  9.0667e-02,  9.6431e-02,  1.0452e-01,  6.8847e-02,
            7.5643e-02,  7.2436e-02,  7.7183e-02,  8.7644e-02,  8.0682e-02,
            1.2104e-01,  1.2946e-01,  1.1328e-01,  5.7868e-02,  7.8028e-02,
            9.5427e-02,  1.6328e-01,  1.2560e-01,  7.4930e-02,  1.3273e-01,
            1.5907e-01,  1.5405e-01,  9.0943e-02,  7.9988e-02,  4.3111e-02,
            3.7555e-02,  3.1839e-02],
          [ 2.1010e-02,  9.2592e-03,  1.8799e-02,  5.1861e-02,  1.0627e-01,
            1.5225e-01,  1.2173e-01,  7.1651e-02,  8.6630e-02,  6.8133e-02,
            7.5102e-02,  7.6069e-02,  9.8534e-02,  1.3646e-01,  1.2772e-01,
            9.8142e-02,  1.4178e-01,  1.2303e-01,  8.0480e-02,  1.2819e-01,
            1.6993e-01,  2.1240e-01,  1.8098e-01,  1.2995e-01,  1.0542e-01,
            1.3902e-01,  1.7959e-01,  1.1462e-01,  7.8357e-02,  4.9090e-02,
            1.8030e-02,  3.8388e-02],
          [ 4.5683e-02,  4.4795e-02,  4.6529e-02,  3.4862e-02,  8.5868e-02,
            1.0034e-01,  1.1102e-01,  9.2638e-02,  8.6931e-02,  7.9049e-02,
            6.1295e-02,  8.1001e-02,  9.5866e-02,  1.2410e-01,  1.4937e-01,
            1.2340e-01,  1.3843e-01,  1.4820e-01,  1.6693e-01,  2.1472e-01,
            2.8549e-01,  3.0739e-01,  1.7854e-01,  1.3354e-01,  1.1887e-01,
            1.6279e-01,  1.3431e-01,  5.6243e-02,  5.5520e-02,  4.8245e-02,
            3.9829e-02,  4.8202e-02],
          [ 2.7918e-04,  6.1383e-02,  6.8707e-02,  7.9390e-02,  9.0975e-02,
            7.8970e-02,  8.2004e-02,  9.3312e-02,  9.1929e-02,  9.1305e-02,
            8.6771e-02,  1.0510e-01,  1.2176e-01,  1.2743e-01,  1.7175e-01,
            1.3559e-01,  1.4084e-01,  1.1015e-01,  1.1910e-01,  1.4497e-01,
            2.4383e-01,  3.0571e-01,  1.8090e-01,  1.2229e-01,  1.6239e-01,
            1.1084e-01,  8.0808e-02,  6.4356e-02,  6.4055e-02,  4.6433e-02,
            4.5475e-02,  8.3017e-02],
          [ 2.7347e-02,  2.3571e-02,  5.1851e-02,  7.2965e-02,  9.1617e-02,
            8.4424e-02,  6.7899e-02,  6.3345e-02,  7.7332e-02,  1.0386e-01,
            1.1964e-01,  1.4559e-01,  1.7287e-01,  1.5195e-01,  1.6208e-01,
            1.3875e-01,  1.6416e-01,  1.8716e-01,  1.1954e-01,  1.0499e-01,
            1.9735e-01,  2.1980e-01,  1.4293e-01,  1.4434e-01,  1.1185e-01,
            1.1926e-01,  9.1072e-02,  7.2950e-02,  4.6556e-02,  4.9631e-02,
            5.7136e-02,  9.0219e-02],
          [ 5.3857e-02,  5.8907e-02,  2.3995e-02,  1.1378e-02,  8.0789e-02,
            9.5383e-02,  8.0365e-02,  7.8703e-02,  7.5615e-02,  1.1418e-01,
            1.6289e-01,  1.8388e-01,  2.0418e-01,  1.4026e-01,  1.3529e-01,
            1.4504e-01,  1.7161e-01,  1.4985e-01,  1.2258e-01,  1.4900e-01,
            2.0167e-01,  1.1090e-01,  8.8113e-02,  8.8327e-02,  9.7413e-02,
            1.9253e-01,  6.0067e-02,  2.4694e-02,  3.2548e-02,  6.4030e-02,
            6.7475e-02,  8.3915e-02],
          [ 2.4103e-02,  3.0751e-02,  4.7104e-02,  3.1962e-02,  6.1323e-02,
            1.2396e-01,  1.3563e-01,  1.0658e-01,  8.5154e-02,  1.4234e-01,
            2.1223e-01,  2.1784e-01,  2.2490e-01,  1.7046e-01,  1.4169e-01,
            1.3764e-01,  1.5040e-01,  1.6098e-01,  1.6171e-01,  2.4541e-01,
            2.7129e-01,  1.9400e-01,  1.1804e-01,  1.1569e-01,  1.7901e-01,
            1.8299e-01,  3.6334e-02,  4.5029e-02,  3.8688e-02,  8.5753e-02,
            9.2412e-02,  6.1803e-02],
          [-5.2098e-03,  4.4195e-02,  1.0458e-01,  5.8643e-02,  5.6588e-02,
            1.2128e-01,  1.5715e-01,  1.1103e-01,  1.0370e-01,  2.0290e-01,
            2.5725e-01,  2.2785e-01,  1.9808e-01,  1.8877e-01,  1.9415e-01,
            2.1456e-01,  1.8307e-01,  2.0863e-01,  2.8955e-01,  2.8159e-01,
            3.1802e-01,  3.6257e-01,  3.1826e-01,  3.2907e-01,  3.8003e-01,
            2.3306e-01,  5.4578e-02,  6.1698e-02,  4.7005e-02,  6.3756e-02,
            7.5323e-02,  6.8793e-02],
          [ 1.1040e-02,  5.1112e-02,  1.2243e-01,  9.1147e-02,  2.9624e-02,
            1.0377e-01,  1.2559e-01,  1.0530e-01,  1.4507e-01,  2.6539e-01,
            2.3477e-01,  1.9206e-01,  1.9919e-01,  2.2909e-01,  2.4433e-01,
            2.3525e-01,  2.3314e-01,  2.9907e-01,  3.6365e-01,  3.5737e-01,
            3.8159e-01,  4.1458e-01,  3.9456e-01,  3.8330e-01,  4.7249e-01,
            1.9708e-01,  6.2970e-02,  7.4823e-02,  5.9731e-02,  3.2201e-02,
            5.4165e-02,  8.8419e-02],
          [ 4.5087e-02,  1.3925e-01,  1.6085e-01,  1.2542e-01,  4.0985e-02,
            5.6138e-02,  1.2071e-01,  8.9847e-02,  2.1948e-01,  3.0058e-01,
            1.7495e-01,  1.7206e-01,  1.6398e-01,  2.0090e-01,  2.6035e-01,
            2.4917e-01,  2.2618e-01,  2.9770e-01,  3.3916e-01,  3.7401e-01,
            2.9612e-01,  1.8026e-01,  3.8450e-01,  4.2461e-01,  2.1733e-01,
            6.2857e-02,  8.9627e-02,  5.2983e-02,  2.8780e-02,  4.0892e-02,
            4.0622e-02,  6.9296e-02],
          [ 8.3726e-02,  1.2885e-01,  1.6084e-01,  2.4220e-01,  8.5800e-02,
            5.5573e-02,  1.3393e-01,  1.2800e-01,  2.6312e-01,  2.5654e-01,
            1.3022e-01,  1.3925e-01,  1.3371e-01,  1.6136e-01,  2.6069e-01,
            2.8669e-01,  1.8776e-01,  1.9842e-01,  2.3866e-01,  1.7634e-01,
            7.9726e-02,  6.6718e-02,  3.1655e-01,  2.9561e-01, -1.8867e-02,
            6.7473e-02,  1.5853e-01,  6.1934e-02,  1.8748e-02,  4.2061e-02,
            5.2197e-02,  7.5592e-02],
          [ 1.0167e-01,  9.2517e-02,  1.2282e-01,  2.9361e-01,  1.1232e-01,
            4.7154e-02,  1.3193e-01,  1.9944e-01,  2.2153e-01,  1.9523e-01,
            1.0707e-01,  8.1596e-02,  9.8857e-02,  1.3161e-01,  2.3217e-01,
            2.8175e-01,  1.9875e-01,  1.5302e-01,  1.3245e-01,  2.8597e-02,
            7.3153e-02,  1.3557e-01,  2.0558e-01,  1.6369e-01, -2.8767e-02,
            1.3469e-01,  1.9053e-01,  6.7049e-02,  5.1264e-02,  5.9777e-02,
            6.9065e-02,  8.9581e-02],
          [ 5.1029e-02,  6.6685e-02,  7.5791e-02,  2.0583e-01,  9.6619e-02,
            2.8156e-02,  1.5172e-01,  2.1292e-01,  1.6969e-01,  1.1884e-01,
            8.8426e-02,  9.0859e-02,  1.1946e-01,  1.5737e-01,  2.2432e-01,
            2.9208e-01,  1.9658e-01,  6.0767e-02,  3.5756e-02,  2.7116e-02,
            4.4202e-02,  7.8264e-02,  1.0578e-01,  8.6886e-02,  1.4351e-02,
            1.2836e-01,  1.6843e-01,  1.2163e-01,  1.0798e-01,  1.0053e-01,
            1.1357e-01,  9.8263e-02],
          [ 4.7251e-02,  6.8060e-02,  9.3040e-02,  2.0454e-01,  5.1507e-02,
            5.3992e-02,  2.0877e-01,  2.2430e-01,  1.9414e-01,  1.4236e-01,
            1.1149e-01,  9.4790e-02,  1.3137e-01,  1.4813e-01,  2.5344e-01,
            2.7389e-01,  6.2636e-02,  3.9111e-03,  3.9307e-02,  7.9661e-02,
            5.0539e-02,  5.1959e-02,  6.2427e-02,  6.4525e-02,  4.4787e-02,
            8.1835e-02,  1.0620e-01,  9.8793e-02,  8.6226e-02,  9.6043e-02,
            1.0466e-01,  1.1262e-01],
          [ 2.7821e-02,  6.2648e-02,  1.7021e-01,  2.3571e-01,  3.3541e-02,
            9.8466e-02,  2.2707e-01,  1.7408e-01,  2.0108e-01,  2.0023e-01,
            1.4214e-01,  1.2814e-01,  1.4609e-01,  1.3479e-01,  2.0469e-01,
            1.9728e-01,  8.0016e-02,  5.3413e-02,  7.1748e-02,  3.8700e-02,
            5.3059e-02,  9.0403e-02,  5.8934e-02,  7.0870e-02,  6.3874e-02,
            1.1146e-01,  1.1133e-01,  9.5074e-02,  1.0397e-01,  1.1316e-01,
            1.1748e-01,  1.2336e-01],
          [ 2.0725e-02,  5.8416e-02,  1.6810e-01,  1.2027e-01,  3.6366e-02,
            1.8652e-01,  2.4031e-01,  1.5866e-01,  1.7467e-01,  2.3431e-01,
            2.1213e-01,  2.0302e-01,  1.3786e-01,  1.2149e-01,  1.7849e-01,
            1.3438e-01,  9.3638e-02,  6.2561e-02,  8.0871e-02,  7.2601e-02,
            9.9070e-02,  1.0407e-01,  9.7430e-02,  1.5208e-01,  1.8721e-01,
            1.5986e-01,  1.4409e-01,  9.3503e-02,  1.1339e-01,  1.0509e-01,
            1.1832e-01,  1.1466e-01],
          [ 3.3799e-02,  8.0804e-02,  1.0768e-01,  3.5733e-02,  1.1407e-01,
            2.4900e-01,  2.1897e-01,  1.7293e-01,  1.9946e-01,  2.2832e-01,
            2.2825e-01,  2.8644e-01,  2.1700e-01,  1.8554e-01,  1.6452e-01,
            8.8580e-02,  6.4900e-02,  3.9221e-02,  7.3204e-02,  1.1189e-01,
            1.6971e-01,  1.8249e-01,  1.6649e-01,  2.0700e-01,  2.5197e-01,
            2.0165e-01,  1.6390e-01,  1.1881e-01,  1.3413e-01,  1.4120e-01,
            1.3174e-01,  9.8833e-02],
          [ 3.5923e-02,  6.0764e-02,  1.4258e-01,  8.3458e-02,  1.4511e-01,
            1.8599e-01,  2.0317e-01,  1.8975e-01,  2.3974e-01,  2.3169e-01,
            2.4734e-01,  2.9897e-01,  3.2293e-01,  2.4138e-01,  1.6912e-01,
            1.1507e-01,  6.4903e-02,  8.3846e-02,  1.4273e-01,  1.7462e-01,
            1.5547e-01,  1.2210e-01,  1.3719e-01,  1.8819e-01,  1.9911e-01,
            1.9787e-01,  2.3496e-01,  1.7455e-01,  1.3524e-01,  1.5354e-01,
            1.5409e-01,  1.0645e-01],
          [ 6.3790e-02,  8.3358e-02,  1.4218e-01,  1.2919e-01,  1.4856e-01,
            1.5609e-01,  1.7043e-01,  1.3656e-01,  2.1014e-01,  2.8474e-01,
            2.4575e-01,  2.1957e-01,  1.7458e-01,  2.1128e-01,  2.0659e-01,
            1.8365e-01,  1.5007e-01,  1.7750e-01,  1.8117e-01,  2.0998e-01,
            1.7277e-01,  1.2128e-01,  1.0951e-01,  1.4929e-01,  1.7443e-01,
            1.5959e-01,  2.1892e-01,  1.9387e-01,  1.6128e-01,  1.4659e-01,
            1.5793e-01,  1.4662e-01],
          [ 5.8546e-02,  1.5897e-01,  1.4244e-01,  1.1722e-01,  1.5028e-01,
            1.6241e-01,  1.7594e-01,  1.3175e-01,  1.5203e-01,  2.2574e-01,
            2.2469e-01,  1.7305e-01,  1.2836e-01,  1.7276e-01,  2.0336e-01,
            2.1909e-01,  1.9355e-01,  1.9120e-01,  1.7605e-01,  2.1646e-01,
            2.0037e-01,  1.5396e-01,  8.7833e-02,  1.0773e-01,  1.3842e-01,
            1.5839e-01,  1.8693e-01,  1.6553e-01,  1.1996e-01,  1.3414e-01,
            1.6588e-01,  1.6441e-01],
          [ 5.4356e-02,  1.8510e-01,  1.3615e-01,  1.3557e-01,  1.9201e-01,
            2.4011e-01,  2.6087e-01,  1.8074e-01,  2.4554e-01,  2.1687e-01,
            1.9379e-01,  1.7984e-01,  1.7334e-01,  1.6321e-01,  1.9659e-01,
            2.3845e-01,  2.4568e-01,  2.7453e-01,  2.7120e-01,  2.2502e-01,
            1.8788e-01,  1.4815e-01,  1.5347e-01,  1.3834e-01,  1.5534e-01,
            1.4153e-01,  1.6364e-01,  1.7967e-01,  1.5054e-01,  1.3645e-01,
            1.7137e-01,  1.4773e-01],
          [ 1.0256e-01,  1.0495e-01,  5.3904e-02,  1.3510e-01,  1.8123e-01,
            2.0893e-01,  1.7543e-01,  2.0911e-01,  3.1652e-01,  2.8975e-01,
            2.2253e-01,  1.5026e-01,  1.6512e-01,  1.2319e-01,  1.7005e-01,
            2.3227e-01,  2.0155e-01,  3.2406e-01,  3.5733e-01,  2.0371e-01,
            1.5073e-01,  1.6232e-01,  1.2439e-01,  1.4018e-01,  1.7054e-01,
            1.1585e-01,  1.2900e-01,  1.8229e-01,  1.8191e-01,  1.5422e-01,
            1.5430e-01,  1.4351e-01],
          [ 1.5090e-01,  1.1986e-01,  1.1538e-01,  1.4646e-01,  1.2477e-01,
            1.9867e-01,  1.4227e-01,  1.8074e-01,  2.2075e-01,  2.8081e-01,
            2.1628e-01,  1.3848e-01,  1.4105e-01,  1.1940e-01,  9.2003e-02,
            1.5077e-01,  1.2667e-01,  2.0855e-01,  2.9099e-01,  2.2123e-01,
            1.4730e-01,  1.7239e-01,  1.7319e-01,  1.4074e-01,  1.3992e-01,
            1.2730e-01,  1.2124e-01,  1.7001e-01,  1.8711e-01,  1.5101e-01,
            1.4024e-01,  1.2444e-01],
          [ 1.7134e-01,  1.7604e-01,  1.3093e-01,  1.4771e-01,  1.2828e-01,
            1.5307e-01,  1.4297e-01,  1.6752e-01,  1.8506e-01,  2.3746e-01,
            2.1601e-01,  2.0147e-01,  1.5435e-01,  1.7917e-01,  1.4873e-01,
            1.4327e-01,  1.7336e-01,  1.4538e-01,  2.0427e-01,  2.2699e-01,
            2.0832e-01,  2.4014e-01,  2.2217e-01,  1.2278e-01,  1.2143e-01,
            1.2528e-01,  1.1470e-01,  1.3630e-01,  1.6889e-01,  1.5977e-01,
            1.3782e-01,  9.7060e-02],
          [ 1.6953e-01,  1.4400e-01,  8.4569e-02,  1.2943e-01,  1.3525e-01,
            1.1673e-01,  1.1124e-01,  1.6483e-01,  2.1933e-01,  2.2655e-01,
            2.4382e-01,  2.0402e-01,  1.3659e-01,  1.1770e-01,  1.3239e-01,
            1.3308e-01,  1.5270e-01,  1.7385e-01,  1.8865e-01,  1.7360e-01,
            1.9853e-01,  2.1085e-01,  1.5500e-01,  9.5321e-02,  9.8577e-02,
            9.1853e-02,  1.0580e-01,  1.4176e-01,  1.5146e-01,  1.4434e-01,
            1.0562e-01,  4.0881e-02],
          [ 1.9592e-01,  1.4369e-01,  9.2820e-02,  1.3323e-01,  1.7704e-01,
            1.8405e-01,  1.4060e-01,  1.5427e-01,  2.1163e-01,  2.4465e-01,
            2.4468e-01,  2.1637e-01,  1.9624e-01,  1.5853e-01,  1.6913e-01,
            1.6527e-01,  1.5696e-01,  1.7495e-01,  1.7613e-01,  1.4579e-01,
            1.3595e-01,  1.9690e-01,  1.2723e-01,  1.1046e-01,  9.7229e-02,
            8.4080e-02,  1.2623e-01,  1.2399e-01,  1.1885e-01,  9.9529e-02,
            8.9658e-02,  6.8890e-02],
          [ 2.0673e-01,  1.7484e-01,  2.2745e-01,  2.0041e-01,  2.3041e-01,
            2.4935e-01,  2.1762e-01,  2.4193e-01,  2.2636e-01,  2.5377e-01,
            2.4372e-01,  2.3162e-01,  3.1729e-01,  2.3125e-01,  1.9821e-01,
            1.8317e-01,  1.9040e-01,  1.8516e-01,  1.8449e-01,  1.5475e-01,
            1.3331e-01,  1.4452e-01,  1.4900e-01,  1.0231e-01,  9.1381e-02,
            9.0827e-02,  7.9075e-02,  1.0396e-01,  1.1145e-01,  6.7933e-02,
            1.0350e-01,  1.0502e-01]]],


        [[[ 1.2940e-01,  1.4846e-01,  1.2829e-01,  1.2564e-01,  1.2446e-01,
            1.2090e-01,  1.3058e-01,  1.1104e-01,  8.6849e-02,  7.8676e-02,
            6.5562e-02,  5.7930e-02,  4.9606e-02,  8.9742e-02,  1.4955e-01,
            1.8609e-01,  1.7776e-01,  1.3222e-01,  7.7888e-02,  9.4552e-02,
            1.1167e-01,  1.1948e-01,  1.8323e-01,  1.9164e-01,  1.8978e-01,
            1.3685e-01,  9.4622e-02,  7.1004e-02,  7.8257e-02,  1.0996e-01,
            8.3011e-02,  1.0649e-01],
          [ 1.0128e-01,  1.2414e-01,  1.1924e-01,  1.3171e-01,  1.1227e-01,
            1.0278e-01,  8.0591e-02,  7.5236e-02,  8.2291e-02,  6.0210e-02,
            5.9146e-02,  5.3796e-02,  4.5669e-02,  8.6518e-02,  1.3804e-01,
            1.9660e-01,  1.7001e-01,  1.1147e-01,  8.3915e-02,  8.2842e-02,
            9.2520e-02,  1.2270e-01,  1.6905e-01,  1.2884e-01,  9.7317e-02,
            7.7386e-02,  8.7434e-02,  8.1906e-02,  7.1653e-02,  9.8461e-02,
            7.4410e-02,  9.0839e-02],
          [ 1.0537e-01,  8.7664e-02,  1.2276e-01,  1.3327e-01,  1.0720e-01,
            9.8348e-02,  6.4103e-02,  7.6150e-02,  6.9254e-02,  6.2615e-02,
            5.6930e-02,  4.9661e-02,  5.1587e-02,  5.3270e-02,  6.6978e-02,
            1.0080e-01,  1.2181e-01,  9.4141e-02,  7.6234e-02,  9.9064e-02,
            8.5471e-02,  9.4288e-02,  1.0563e-01,  8.7667e-02,  9.3342e-02,
            8.7857e-02,  9.5618e-02,  9.2827e-02,  8.8772e-02,  9.7943e-02,
            1.1216e-01,  7.9283e-02],
          [ 1.0715e-01,  1.2062e-01,  1.1116e-01,  9.7503e-02,  9.5285e-02,
            7.6448e-02,  6.2701e-02,  4.8632e-02,  5.8347e-02,  5.9531e-02,
            5.2922e-02,  4.8150e-02,  5.5692e-02,  5.8200e-02,  4.5831e-02,
            8.2437e-02,  1.1165e-01,  9.5201e-02,  7.6719e-02,  9.3996e-02,
            7.5673e-02,  1.0636e-01,  1.0217e-01,  9.6099e-02,  8.9276e-02,
            1.0194e-01,  9.5904e-02,  8.4391e-02,  1.0346e-01,  9.9397e-02,
            6.8643e-02,  4.5508e-02],
          [ 1.0329e-01,  9.6501e-02,  9.0031e-02,  9.4152e-02,  7.7283e-02,
            5.8876e-02,  4.0171e-02,  4.6108e-02,  6.0283e-02,  5.1438e-02,
            3.7178e-02,  4.5669e-02,  4.6525e-02,  6.7092e-02,  6.6147e-02,
            6.7190e-02,  8.4554e-02,  9.1457e-02,  8.7234e-02,  7.6655e-02,
            9.8387e-02,  8.5005e-02,  8.8261e-02,  7.1273e-02,  8.0853e-02,
            9.2815e-02,  1.2854e-01,  1.3037e-01,  1.8196e-01,  6.5988e-02,
            4.1486e-02,  4.8457e-02],
          [ 8.9371e-02,  8.6856e-02,  7.3809e-02,  7.2002e-02,  7.9531e-02,
            7.2387e-02,  4.3566e-02,  6.3412e-02,  1.1159e-01,  1.0074e-01,
            1.2792e-01,  1.3371e-01,  1.9276e-01,  2.1240e-01,  2.4130e-01,
            1.8597e-01,  1.3707e-01,  9.0903e-02,  9.8180e-02,  1.0047e-01,
            1.0559e-01,  1.0098e-01,  9.1941e-02,  1.0637e-01,  1.3242e-01,
            1.0762e-01,  1.2980e-01,  1.3811e-01,  1.4534e-01,  7.7347e-02,
            5.5692e-02,  2.9240e-02],
          [ 8.4415e-02,  8.3999e-02,  8.4436e-02,  7.4776e-02,  7.2384e-02,
            7.2152e-02,  6.9704e-02,  9.5921e-02,  1.8076e-01,  3.5039e-01,
            4.0883e-01,  4.2951e-01,  4.8409e-01,  5.1029e-01,  5.4065e-01,
            5.1908e-01,  4.8306e-01,  3.5428e-01,  2.5455e-01,  2.8419e-01,
            3.6477e-01,  3.7761e-01,  3.3769e-01,  3.4769e-01,  3.6894e-01,
            2.6502e-01,  1.9586e-01,  1.7898e-01,  2.0970e-01,  9.4922e-02,
            5.5865e-02,  4.1277e-02],
          [ 1.5014e-01,  1.6717e-01,  1.7327e-01,  1.6771e-01,  1.7774e-01,
            2.0642e-01,  2.1435e-01,  2.3169e-01,  3.2147e-01,  3.9405e-01,
            3.6793e-01,  3.8104e-01,  4.4100e-01,  5.0306e-01,  5.6039e-01,
            6.3373e-01,  6.2427e-01,  6.1711e-01,  5.6169e-01,  5.8474e-01,
            6.1721e-01,  6.0969e-01,  5.7919e-01,  5.3403e-01,  5.2735e-01,
            5.2110e-01,  4.5488e-01,  3.6420e-01,  4.3704e-01,  2.0699e-01,
            4.9041e-02,  8.5332e-02],
          [ 3.1624e-01,  2.9890e-01,  3.0107e-01,  3.0453e-01,  3.1102e-01,
            3.4163e-01,  3.4689e-01,  3.5654e-01,  3.4741e-01,  3.4011e-01,
            3.2678e-01,  3.1360e-01,  3.4835e-01,  4.3825e-01,  5.1848e-01,
            5.7898e-01,  5.5646e-01,  6.0634e-01,  5.8514e-01,  5.4825e-01,
            5.7567e-01,  5.9348e-01,  5.5912e-01,  5.0368e-01,  5.1482e-01,
            5.6437e-01,  4.9865e-01,  3.4321e-01,  3.5933e-01,  2.1717e-01,
            9.7592e-02,  1.1555e-01],
          [ 4.2190e-01,  3.7588e-01,  3.6040e-01,  3.5694e-01,  3.6686e-01,
            3.8213e-01,  3.5403e-01,  2.7807e-01,  2.4865e-01,  2.8985e-01,
            3.1169e-01,  3.1937e-01,  3.5923e-01,  4.0358e-01,  4.7654e-01,
            5.8190e-01,  5.4896e-01,  5.5257e-01,  5.3644e-01,  4.6222e-01,
            4.6851e-01,  5.4368e-01,  5.2644e-01,  4.9250e-01,  4.9472e-01,
            5.0315e-01,  4.6441e-01,  3.2571e-01,  2.8282e-01,  2.7111e-01,
            3.4766e-01,  3.5901e-01],
          [ 4.2532e-01,  4.1153e-01,  3.5929e-01,  3.4904e-01,  3.7260e-01,
            2.9918e-01,  2.2007e-01,  1.5565e-01,  1.6100e-01,  2.3959e-01,
            2.8157e-01,  3.1876e-01,  3.3881e-01,  3.7015e-01,  4.3614e-01,
            4.9607e-01,  4.9934e-01,  4.8642e-01,  4.9553e-01,  4.8320e-01,
            4.5063e-01,  4.5178e-01,  4.8262e-01,  4.4949e-01,  4.0852e-01,
            4.5941e-01,  4.5924e-01,  4.7543e-01,  3.8343e-01,  4.1329e-01,
            4.5352e-01,  4.6672e-01],
          [ 3.9489e-01,  3.7049e-01,  3.1079e-01,  3.1856e-01,  3.1463e-01,
            2.4143e-01,  1.8844e-01,  1.7953e-01,  2.1416e-01,  2.5209e-01,
            2.6579e-01,  3.2999e-01,  3.1359e-01,  3.1395e-01,  3.2852e-01,
            3.0633e-01,  3.3626e-01,  3.9982e-01,  4.4616e-01,  4.7772e-01,
            4.3485e-01,  3.7019e-01,  3.1274e-01,  2.7032e-01,  2.2424e-01,
            3.3896e-01,  4.3842e-01,  4.4921e-01,  4.4366e-01,  4.6453e-01,
            4.5618e-01,  4.5968e-01],
          [ 3.5946e-01,  3.1650e-01,  3.0474e-01,  3.0262e-01,  3.0812e-01,
            2.4664e-01,  2.3378e-01,  2.5037e-01,  2.8470e-01,  2.6967e-01,
            2.9263e-01,  3.0382e-01,  2.8928e-01,  3.0206e-01,  2.9102e-01,
            2.7210e-01,  2.9894e-01,  3.7298e-01,  4.1819e-01,  4.2703e-01,
            4.2235e-01,  3.5062e-01,  2.8820e-01,  2.1764e-01,  1.5849e-01,
            1.7951e-01,  3.3908e-01,  4.1194e-01,  3.8945e-01,  4.4368e-01,
            5.0357e-01,  4.6146e-01],
          [ 3.5367e-01,  2.7093e-01,  3.0791e-01,  3.1700e-01,  2.9277e-01,
            2.5689e-01,  2.7167e-01,  2.8759e-01,  2.9365e-01,  2.7083e-01,
            2.5454e-01,  2.7690e-01,  2.9262e-01,  3.0483e-01,  2.6732e-01,
            2.8305e-01,  3.1040e-01,  3.3260e-01,  3.5226e-01,  3.7608e-01,
            3.7463e-01,  3.6929e-01,  2.9094e-01,  2.2098e-01,  1.5733e-01,
            1.1295e-01,  2.1056e-01,  3.6612e-01,  3.7228e-01,  4.1970e-01,
            4.9301e-01,  5.0334e-01],
          [ 3.2186e-01,  2.8357e-01,  2.9764e-01,  3.1110e-01,  2.8075e-01,
            2.6589e-01,  2.7338e-01,  3.3819e-01,  3.2812e-01,  2.4305e-01,
            2.8742e-01,  2.9199e-01,  3.0361e-01,  2.8004e-01,  2.8057e-01,
            2.7107e-01,  3.1582e-01,  3.1544e-01,  3.3414e-01,  3.6737e-01,
            3.4054e-01,  3.2953e-01,  2.5821e-01,  2.0083e-01,  1.4537e-01,
            9.9718e-02,  1.3048e-01,  2.8883e-01,  3.8405e-01,  4.1914e-01,
            4.8363e-01,  5.1034e-01],
          [ 2.9301e-01,  2.8453e-01,  3.0071e-01,  2.9179e-01,  2.8187e-01,
            2.5698e-01,  2.7189e-01,  3.0169e-01,  2.8086e-01,  2.2634e-01,
            2.9297e-01,  2.6675e-01,  2.4737e-01,  2.7992e-01,  2.6886e-01,
            2.6902e-01,  3.2294e-01,  3.1735e-01,  3.8080e-01,  4.1650e-01,
            3.7862e-01,  2.9572e-01,  2.0493e-01,  1.5583e-01,  1.2540e-01,
            9.1708e-02,  9.6017e-02,  2.3912e-01,  3.8208e-01,  4.4915e-01,
            4.8849e-01,  5.0553e-01],
          [ 2.8392e-01,  2.8589e-01,  2.7438e-01,  2.6732e-01,  2.7970e-01,
            2.4163e-01,  2.1287e-01,  2.3350e-01,  2.1547e-01,  2.1821e-01,
            2.9077e-01,  2.6523e-01,  2.6556e-01,  2.3484e-01,  2.0054e-01,
            2.5354e-01,  3.7331e-01,  3.6883e-01,  4.0337e-01,  4.0592e-01,
            3.3429e-01,  2.0290e-01,  1.3177e-01,  1.0219e-01,  1.0776e-01,
            8.0159e-02,  7.4714e-02,  1.8242e-01,  3.4052e-01,  4.4734e-01,
            4.8423e-01,  4.6770e-01],
          [ 2.7059e-01,  2.3924e-01,  2.5401e-01,  2.2647e-01,  2.0701e-01,
            2.1535e-01,  1.9143e-01,  1.8075e-01,  2.0698e-01,  2.8977e-01,
            3.5168e-01,  2.8474e-01,  2.4103e-01,  2.3443e-01,  1.8960e-01,
            2.0670e-01,  2.5817e-01,  2.3767e-01,  2.2433e-01,  2.2896e-01,
            1.6944e-01,  1.5306e-01,  1.4960e-01,  1.5937e-01,  1.5138e-01,
            9.7008e-02,  6.5478e-02,  1.9687e-01,  3.3893e-01,  4.2916e-01,
            4.9833e-01,  4.6491e-01],
          [ 2.9869e-01,  2.7539e-01,  2.8549e-01,  2.3648e-01,  1.9927e-01,
            1.9506e-01,  1.7527e-01,  1.7013e-01,  2.4268e-01,  3.0134e-01,
            2.7270e-01,  2.3543e-01,  2.1537e-01,  1.6754e-01,  1.9858e-01,
            3.7351e-01,  4.3006e-01,  2.7698e-01,  1.4272e-01,  2.2027e-01,
            2.9551e-01,  2.8767e-01,  1.6058e-01,  1.6713e-01,  1.6747e-01,
            9.1647e-02,  8.8342e-02,  1.9412e-01,  3.3972e-01,  4.5541e-01,
            4.8181e-01,  4.9850e-01],
          [ 3.6046e-01,  3.4833e-01,  3.6615e-01,  2.9496e-01,  2.0441e-01,
            1.7736e-01,  1.8786e-01,  2.3087e-01,  2.7547e-01,  2.7209e-01,
            2.4087e-01,  2.0278e-01,  1.7184e-01,  2.1272e-01,  4.3446e-01,
            5.5298e-01,  5.5082e-01,  4.8956e-01,  4.3238e-01,  4.8231e-01,
            5.1186e-01,  3.7374e-01,  2.0405e-01,  1.8112e-01,  2.0576e-01,
            1.3542e-01,  1.1305e-01,  2.0720e-01,  3.6833e-01,  4.6731e-01,
            4.4445e-01,  4.9799e-01],
          [ 3.5455e-01,  3.4329e-01,  3.4314e-01,  3.3125e-01,  2.8202e-01,
            2.4785e-01,  2.5427e-01,  2.9086e-01,  2.7729e-01,  2.5715e-01,
            2.2999e-01,  2.6786e-01,  4.0927e-01,  4.6068e-01,  4.8177e-01,
            4.8443e-01,  5.0663e-01,  5.0577e-01,  5.1470e-01,  5.0055e-01,
            5.2685e-01,  4.5370e-01,  2.9964e-01,  2.1745e-01,  2.9255e-01,
            2.5180e-01,  1.5196e-01,  2.1851e-01,  3.8901e-01,  4.9153e-01,
            4.8718e-01,  5.0746e-01],
          [ 3.2450e-01,  3.1970e-01,  3.3561e-01,  3.3238e-01,  3.2612e-01,
            3.1594e-01,  2.8782e-01,  2.5593e-01,  2.4235e-01,  2.9048e-01,
            3.6632e-01,  4.7736e-01,  4.6393e-01,  4.6307e-01,  4.7545e-01,
            4.9021e-01,  5.1822e-01,  5.2425e-01,  5.2685e-01,  5.2489e-01,
            5.4615e-01,  4.7865e-01,  2.7700e-01,  2.7927e-01,  2.8274e-01,
            2.4396e-01,  1.4084e-01,  2.3120e-01,  4.0140e-01,  4.9523e-01,
            4.7984e-01,  5.2464e-01],
          [ 3.4143e-01,  3.4105e-01,  3.4938e-01,  3.3498e-01,  3.1740e-01,
            2.6374e-01,  2.3598e-01,  3.0519e-01,  3.8539e-01,  4.4691e-01,
            4.6402e-01,  5.1136e-01,  4.6109e-01,  4.5377e-01,  5.0571e-01,
            5.3330e-01,  4.9588e-01,  5.2692e-01,  5.3026e-01,  5.2215e-01,
            5.4981e-01,  4.2788e-01,  2.2041e-01,  2.4404e-01,  1.9026e-01,
            1.7378e-01,  1.0611e-01,  2.6029e-01,  4.4373e-01,  4.4037e-01,
            4.3842e-01,  5.0045e-01],
          [ 3.2790e-01,  3.2926e-01,  3.2847e-01,  3.2047e-01,  2.6339e-01,
            2.2004e-01,  3.3414e-01,  4.9035e-01,  5.2237e-01,  5.0057e-01,
            4.9879e-01,  5.4708e-01,  4.9275e-01,  4.4094e-01,  5.0853e-01,
            5.2604e-01,  5.2490e-01,  5.3156e-01,  5.1783e-01,  5.1994e-01,
            5.3149e-01,  3.4386e-01,  2.1696e-01,  2.9378e-01,  3.1424e-01,
            2.2690e-01,  1.0877e-01,  2.9242e-01,  4.5111e-01,  4.7074e-01,
            4.6968e-01,  4.8863e-01],
          [ 2.8809e-01,  3.0009e-01,  2.8864e-01,  2.5233e-01,  2.5252e-01,
            3.2330e-01,  4.4311e-01,  5.2067e-01,  4.8349e-01,  4.5168e-01,
            5.3369e-01,  5.5812e-01,  5.3552e-01,  4.7152e-01,  5.0519e-01,
            5.2581e-01,  5.2679e-01,  5.2146e-01,  5.1811e-01,  5.2816e-01,
            5.4378e-01,  3.0723e-01,  1.9042e-01,  3.0391e-01,  4.3552e-01,
            3.5035e-01,  1.0850e-01,  3.0383e-01,  4.8101e-01,  4.7739e-01,
            4.7296e-01,  4.6413e-01],
          [ 2.5451e-01,  2.4869e-01,  2.2783e-01,  2.7120e-01,  3.5460e-01,
            4.4703e-01,  4.7954e-01,  4.7194e-01,  4.4379e-01,  5.1026e-01,
            5.8827e-01,  5.7339e-01,  5.6370e-01,  4.9553e-01,  4.7389e-01,
            4.9950e-01,  5.1303e-01,  4.9964e-01,  5.2178e-01,  5.2265e-01,
            5.2872e-01,  3.2210e-01,  1.2754e-01,  3.1277e-01,  4.8318e-01,
            4.4151e-01,  1.4703e-01,  2.8778e-01,  5.2386e-01,  5.0373e-01,
            4.9803e-01,  5.2381e-01],
          [ 2.6741e-01,  2.9492e-01,  3.5325e-01,  3.9840e-01,  4.5591e-01,
            4.8688e-01,  4.8029e-01,  4.6574e-01,  4.4261e-01,  5.1437e-01,
            5.5131e-01,  5.4807e-01,  5.5910e-01,  5.4940e-01,  4.9301e-01,
            5.0964e-01,  5.3149e-01,  5.2643e-01,  5.2804e-01,  5.3264e-01,
            5.4488e-01,  4.0415e-01,  1.6765e-01,  2.8922e-01,  4.5281e-01,
            4.2900e-01,  1.8729e-01,  2.5484e-01,  5.0235e-01,  5.3771e-01,
            5.7239e-01,  5.7298e-01],
          [ 4.2997e-01,  4.6737e-01,  4.6618e-01,  4.7988e-01,  4.8416e-01,
            4.9231e-01,  4.8756e-01,  4.7851e-01,  4.4156e-01,  4.7228e-01,
            5.0503e-01,  5.2330e-01,  5.5144e-01,  5.6560e-01,  5.2187e-01,
            5.0723e-01,  5.3676e-01,  5.4432e-01,  5.2435e-01,  5.4402e-01,
            5.6191e-01,  5.1140e-01,  2.7662e-01,  2.6455e-01,  4.5924e-01,
            4.7633e-01,  2.4165e-01,  2.2312e-01,  4.9691e-01,  5.5298e-01,
            5.8955e-01,  5.8874e-01],
          [ 4.8367e-01,  4.8018e-01,  4.8964e-01,  4.9785e-01,  4.9619e-01,
            5.1306e-01,  4.9746e-01,  4.9812e-01,  5.0381e-01,  5.2342e-01,
            5.2654e-01,  5.2465e-01,  5.3599e-01,  5.4682e-01,  5.2812e-01,
            5.0946e-01,  5.2781e-01,  5.4676e-01,  5.3937e-01,  5.3632e-01,
            5.5683e-01,  5.2329e-01,  3.2325e-01,  2.4841e-01,  4.9516e-01,
            5.2682e-01,  2.3682e-01,  2.2635e-01,  4.7853e-01,  5.1191e-01,
            5.6058e-01,  5.6680e-01],
          [ 4.9627e-01,  4.8497e-01,  4.9901e-01,  5.1177e-01,  5.0359e-01,
            5.1910e-01,  5.1949e-01,  5.2459e-01,  5.3919e-01,  5.3554e-01,
            5.4204e-01,  5.3858e-01,  5.5928e-01,  5.3172e-01,  5.1939e-01,
            5.3032e-01,  5.4269e-01,  5.4639e-01,  5.4160e-01,  5.4704e-01,
            5.2965e-01,  5.0337e-01,  3.3994e-01,  2.3295e-01,  4.5496e-01,
            4.8488e-01,  2.0929e-01,  2.6181e-01,  4.6787e-01,  4.8921e-01,
            5.1934e-01,  4.8448e-01],
          [ 5.2404e-01,  5.2242e-01,  5.1976e-01,  5.2635e-01,  5.2398e-01,
            5.3920e-01,  5.4920e-01,  5.4903e-01,  5.6141e-01,  5.5051e-01,
            5.6862e-01,  5.6568e-01,  5.5015e-01,  5.4701e-01,  5.3591e-01,
            5.4384e-01,  5.7076e-01,  5.6078e-01,  5.4672e-01,  5.5658e-01,
            5.3676e-01,  5.0535e-01,  4.1637e-01,  2.5324e-01,  4.0033e-01,
            4.7834e-01,  2.4971e-01,  2.7681e-01,  4.0053e-01,  4.2738e-01,
            4.2657e-01,  3.4914e-01],
          [ 5.4521e-01,  5.3990e-01,  5.1850e-01,  5.0294e-01,  5.1686e-01,
            5.4119e-01,  5.5144e-01,  5.5567e-01,  5.7278e-01,  5.6710e-01,
            5.6475e-01,  5.5283e-01,  5.2843e-01,  5.2266e-01,  5.2850e-01,
            5.4508e-01,  5.3879e-01,  5.5168e-01,  5.4767e-01,  5.3960e-01,
            5.3730e-01,  5.0135e-01,  4.1253e-01,  3.1873e-01,  3.3793e-01,
            4.5204e-01,  3.0404e-01,  2.9240e-01,  3.1238e-01,  3.1338e-01,
            2.6767e-01,  1.9531e-01]],

         [[ 1.2843e-01,  1.2813e-01,  1.1994e-01,  1.1608e-01,  9.6612e-02,
            1.0127e-01,  1.3163e-01,  1.0398e-01,  8.7444e-02,  7.7782e-02,
            6.5821e-02,  7.3866e-02,  5.5393e-02,  6.6396e-02,  9.2414e-02,
            1.2739e-01,  1.3455e-01,  8.7695e-02,  6.2088e-02,  6.1132e-02,
            8.9272e-02,  9.6196e-02,  1.5741e-01,  1.8390e-01,  1.6535e-01,
            1.4006e-01,  1.0200e-01,  6.6887e-02,  6.9598e-02,  8.8175e-02,
            9.4673e-02,  1.0455e-01],
          [ 1.1221e-01,  1.1696e-01,  1.1679e-01,  1.4400e-01,  1.4064e-01,
            1.1055e-01,  8.9646e-02,  1.0537e-01,  8.0232e-02,  5.7768e-02,
            5.1996e-02,  4.8992e-02,  6.7088e-02,  7.6912e-02,  1.2475e-01,
            1.5240e-01,  1.1990e-01,  7.5663e-02,  4.7969e-02,  3.7539e-02,
            6.9044e-02,  1.0868e-01,  1.2767e-01,  1.0512e-01,  8.6500e-02,
            6.7680e-02,  7.2377e-02,  8.1410e-02,  8.2151e-02,  7.0237e-02,
            6.9382e-02,  9.6263e-02],
          [ 1.1746e-01,  1.0324e-01,  1.2994e-01,  1.3519e-01,  1.1852e-01,
            1.0075e-01,  8.5231e-02,  8.5920e-02,  7.3069e-02,  6.1447e-02,
            6.6102e-02,  6.1574e-02,  7.4785e-02,  6.4508e-02,  7.3376e-02,
            9.0278e-02,  7.7922e-02,  6.2828e-02,  4.0882e-02,  6.3368e-02,
            5.6690e-02,  8.8032e-02,  8.9779e-02,  7.9881e-02,  6.4340e-02,
            8.4393e-02,  7.6649e-02,  6.7410e-02,  5.6688e-02,  8.1132e-02,
            1.0993e-01,  6.6856e-02],
          [ 1.2275e-01,  1.1061e-01,  1.1382e-01,  1.1533e-01,  9.7049e-02,
            8.6989e-02,  6.4853e-02,  7.4903e-02,  5.0952e-02,  6.6822e-02,
            4.6968e-02,  5.3238e-02,  6.4452e-02,  5.8994e-02,  6.5221e-02,
            7.7380e-02,  6.2523e-02,  6.0175e-02,  6.0341e-02,  5.8330e-02,
            7.0758e-02,  6.9632e-02,  9.0261e-02,  8.6195e-02,  7.5054e-02,
            6.8824e-02,  4.9123e-02,  4.7703e-02,  7.9107e-02,  6.7389e-02,
            6.7713e-02,  5.5279e-02],
          [ 1.0889e-01,  1.0064e-01,  9.8410e-02,  8.9818e-02,  6.9801e-02,
            6.3288e-02,  6.5486e-02,  5.1308e-02,  6.1945e-02,  6.0873e-02,
            3.9033e-02,  6.7684e-02,  6.1900e-02,  6.8932e-02,  5.7134e-02,
            4.3261e-02,  6.3581e-02,  6.2049e-02,  5.1529e-02,  6.3858e-02,
            7.5196e-02,  7.6974e-02,  6.6581e-02,  5.4520e-02,  4.6060e-02,
            6.9832e-02,  8.5860e-02,  8.1080e-02,  1.5137e-01,  5.2526e-02,
            5.5908e-02,  3.1114e-02],
          [ 8.8613e-02,  8.3130e-02,  7.2781e-02,  6.2982e-02,  6.5486e-02,
            6.2578e-02,  4.5601e-02,  5.1240e-02,  8.6677e-02,  9.8496e-02,
            1.0600e-01,  1.3272e-01,  1.6379e-01,  1.8604e-01,  1.9531e-01,
            1.4922e-01,  1.0037e-01,  9.5489e-02,  6.4751e-02,  7.3485e-02,
            9.0124e-02,  6.9685e-02,  8.1423e-02,  7.8842e-02,  1.1111e-01,
            7.2471e-02,  9.1218e-02,  9.3327e-02,  1.0019e-01,  5.8017e-02,
            4.6532e-02,  2.6483e-02],
          [ 7.8376e-02,  7.4830e-02,  7.3736e-02,  7.1323e-02,  5.8056e-02,
            6.9823e-02,  7.7954e-02,  9.8526e-02,  1.6203e-01,  3.0537e-01,
            3.3608e-01,  3.2446e-01,  3.6478e-01,  4.0322e-01,  4.4166e-01,
            4.4612e-01,  4.1798e-01,  3.3110e-01,  2.4662e-01,  2.6489e-01,
            3.5517e-01,  3.4142e-01,  3.2100e-01,  3.3891e-01,  3.3494e-01,
            2.1707e-01,  1.5226e-01,  1.5218e-01,  1.6998e-01,  7.6428e-02,
            3.4418e-02,  3.8869e-02],
          [ 1.2820e-01,  1.2580e-01,  1.2771e-01,  1.3384e-01,  1.3978e-01,
            1.5798e-01,  1.6720e-01,  1.8685e-01,  2.5068e-01,  3.3036e-01,
            2.6894e-01,  2.6348e-01,  3.0332e-01,  3.6623e-01,  4.6994e-01,
            4.9965e-01,  5.2013e-01,  5.5258e-01,  5.3390e-01,  5.4122e-01,
            5.6977e-01,  5.6923e-01,  5.3934e-01,  5.0227e-01,  4.9262e-01,
            4.6826e-01,  4.0927e-01,  3.3601e-01,  3.8757e-01,  1.6719e-01,
            3.4217e-02,  5.8053e-02],
          [ 2.4088e-01,  2.3642e-01,  2.3197e-01,  2.2543e-01,  2.4061e-01,
            2.4590e-01,  2.5519e-01,  2.5779e-01,  2.5739e-01,  2.4346e-01,
            2.0741e-01,  1.9626e-01,  2.6229e-01,  3.3387e-01,  4.0873e-01,
            4.2661e-01,  4.3759e-01,  5.4559e-01,  5.4569e-01,  5.0333e-01,
            5.3128e-01,  5.5454e-01,  5.0039e-01,  4.5999e-01,  4.8198e-01,
            5.1276e-01,  4.5118e-01,  2.8559e-01,  3.4193e-01,  1.8269e-01,
            5.7554e-02,  8.1156e-02],
          [ 2.9601e-01,  2.8888e-01,  2.8228e-01,  2.8526e-01,  2.8273e-01,
            2.8553e-01,  2.4514e-01,  2.1376e-01,  1.8751e-01,  1.8884e-01,
            2.1214e-01,  2.1902e-01,  2.4773e-01,  2.7753e-01,  3.5994e-01,
            4.5085e-01,  4.4014e-01,  4.8288e-01,  4.9072e-01,  4.0658e-01,
            4.1551e-01,  4.7152e-01,  4.6606e-01,  4.5877e-01,  4.5019e-01,
            4.5934e-01,  3.9541e-01,  2.7798e-01,  2.9135e-01,  2.6958e-01,
            3.1045e-01,  3.0863e-01],
          [ 3.2209e-01,  3.0378e-01,  2.7115e-01,  2.7063e-01,  2.8059e-01,
            2.1672e-01,  1.4189e-01,  1.1044e-01,  1.4002e-01,  1.8236e-01,
            2.0579e-01,  2.2837e-01,  2.3656e-01,  2.6734e-01,  3.2483e-01,
            4.0922e-01,  4.1381e-01,  4.1908e-01,  4.5171e-01,  4.4534e-01,
            3.8029e-01,  3.9410e-01,  4.2063e-01,  4.0152e-01,  3.4437e-01,
            3.8774e-01,  4.0130e-01,  3.9427e-01,  3.8111e-01,  4.0858e-01,
            4.2870e-01,  4.2449e-01],
          [ 2.8009e-01,  2.6057e-01,  2.0532e-01,  2.0116e-01,  2.1359e-01,
            1.3652e-01,  1.2236e-01,  1.4276e-01,  1.6430e-01,  1.9133e-01,
            1.8855e-01,  2.2089e-01,  2.1559e-01,  2.1924e-01,  2.3284e-01,
            2.2279e-01,  2.4141e-01,  3.0751e-01,  3.9057e-01,  4.2885e-01,
            3.8177e-01,  3.0688e-01,  2.6459e-01,  2.0485e-01,  1.5966e-01,
            2.4639e-01,  3.6384e-01,  4.0134e-01,  4.0777e-01,  4.4416e-01,
            4.3250e-01,  4.2698e-01],
          [ 2.2672e-01,  1.9429e-01,  1.7779e-01,  1.6961e-01,  1.7327e-01,
            1.3445e-01,  1.4507e-01,  1.7099e-01,  1.8691e-01,  1.6982e-01,
            1.8337e-01,  2.1615e-01,  1.7690e-01,  2.1265e-01,  2.0187e-01,
            1.8171e-01,  2.0469e-01,  2.6295e-01,  3.2786e-01,  3.5211e-01,
            3.7034e-01,  2.8619e-01,  2.2441e-01,  1.6825e-01,  9.6145e-02,
            1.1764e-01,  2.4896e-01,  3.4596e-01,  3.2695e-01,  4.0264e-01,
            4.5940e-01,  4.1869e-01],
          [ 2.2233e-01,  1.6944e-01,  1.8205e-01,  1.9354e-01,  1.5717e-01,
            1.3101e-01,  1.5119e-01,  1.7028e-01,  1.6689e-01,  1.5669e-01,
            1.5123e-01,  1.8594e-01,  1.8559e-01,  1.8358e-01,  1.7442e-01,
            1.9408e-01,  2.0435e-01,  2.2565e-01,  2.6472e-01,  2.8613e-01,
            2.9989e-01,  2.7847e-01,  2.1125e-01,  1.6684e-01,  9.2073e-02,
            4.9955e-02,  1.2531e-01,  2.8435e-01,  3.2296e-01,  3.4724e-01,
            4.3249e-01,  4.6084e-01],
          [ 2.1232e-01,  1.4704e-01,  1.8521e-01,  1.8642e-01,  1.6206e-01,
            1.2790e-01,  1.4537e-01,  1.8976e-01,  1.6707e-01,  1.2173e-01,
            1.8409e-01,  1.8554e-01,  1.7410e-01,  1.7088e-01,  1.7125e-01,
            1.8341e-01,  2.1860e-01,  2.3035e-01,  2.5722e-01,  2.7636e-01,
            2.6166e-01,  2.4871e-01,  1.8359e-01,  1.2679e-01,  9.2189e-02,
            3.6609e-02,  4.7173e-02,  1.9809e-01,  3.1103e-01,  3.5625e-01,
            4.2905e-01,  4.5098e-01],
          [ 1.5694e-01,  1.7032e-01,  1.8286e-01,  1.8626e-01,  1.6648e-01,
            1.3644e-01,  1.3378e-01,  1.7565e-01,  1.6167e-01,  1.2306e-01,
            1.9735e-01,  1.5900e-01,  1.3558e-01,  1.4860e-01,  1.6293e-01,
            1.7268e-01,  2.2427e-01,  2.3624e-01,  3.0194e-01,  3.1349e-01,
            2.8676e-01,  2.1202e-01,  1.2389e-01,  8.2848e-02,  6.8743e-02,
            2.9176e-02,  4.8378e-02,  1.7760e-01,  3.1082e-01,  3.6820e-01,
            4.1664e-01,  4.2537e-01],
          [ 1.5142e-01,  1.5594e-01,  1.5241e-01,  1.8019e-01,  1.5625e-01,
            1.3151e-01,  1.2437e-01,  1.2372e-01,  1.2013e-01,  1.3985e-01,
            1.5964e-01,  1.4488e-01,  1.3131e-01,  9.4920e-02,  9.6367e-02,
            1.2052e-01,  2.7694e-01,  2.7586e-01,  3.1274e-01,  3.2004e-01,
            2.5330e-01,  1.4113e-01,  6.0339e-02,  5.2355e-02,  5.8199e-02,
            2.2610e-02,  1.5008e-02,  1.5070e-01,  2.8638e-01,  3.8116e-01,
            4.2194e-01,  4.0305e-01],
          [ 1.3358e-01,  1.3235e-01,  1.5064e-01,  1.2895e-01,  1.1669e-01,
            1.3259e-01,  1.1771e-01,  1.1688e-01,  1.2912e-01,  1.9104e-01,
            2.1320e-01,  1.4421e-01,  1.0509e-01,  9.6213e-02,  6.2451e-02,
            8.9983e-02,  2.0761e-01,  1.7766e-01,  1.5192e-01,  1.5886e-01,
            1.1871e-01,  8.6029e-02,  9.1846e-02,  8.7063e-02,  9.6832e-02,
            5.1146e-02,  8.9313e-03,  1.4100e-01,  2.7633e-01,  3.6073e-01,
            4.5134e-01,  4.2070e-01],
          [ 1.7207e-01,  1.8264e-01,  1.9008e-01,  1.3879e-01,  1.1897e-01,
            1.1140e-01,  8.8935e-02,  8.9553e-02,  1.4979e-01,  1.8182e-01,
            1.6150e-01,  1.3042e-01,  9.9279e-02,  6.6690e-02,  1.4835e-01,
            3.2180e-01,  3.9923e-01,  2.2830e-01,  1.1892e-01,  1.5998e-01,
            2.1458e-01,  2.0620e-01,  7.8386e-02,  9.1671e-02,  1.1438e-01,
            4.1497e-02,  2.1557e-02,  1.2684e-01,  2.7746e-01,  3.9118e-01,
            4.2549e-01,  4.2713e-01],
          [ 2.2944e-01,  2.3899e-01,  2.5504e-01,  2.1950e-01,  1.2208e-01,
            9.8360e-02,  9.7690e-02,  1.3304e-01,  1.7969e-01,  1.5744e-01,
            1.4879e-01,  1.0158e-01,  7.0372e-02,  1.4910e-01,  3.8378e-01,
            4.7819e-01,  4.2932e-01,  3.8695e-01,  3.6764e-01,  3.8658e-01,
            4.0398e-01,  2.8371e-01,  8.1836e-02,  8.5265e-02,  1.5038e-01,
            8.0201e-02,  3.1552e-02,  1.3926e-01,  3.0803e-01,  3.9251e-01,
            4.0197e-01,  4.3716e-01],
          [ 2.2968e-01,  2.3804e-01,  2.4324e-01,  2.3953e-01,  1.8204e-01,
            1.3433e-01,  1.5866e-01,  1.8308e-01,  1.5721e-01,  1.4717e-01,
            1.3513e-01,  1.4826e-01,  3.1918e-01,  3.8254e-01,  3.9571e-01,
            3.9203e-01,  3.8482e-01,  4.0927e-01,  4.2977e-01,  4.1134e-01,
            3.9574e-01,  3.4606e-01,  1.7942e-01,  1.1520e-01,  1.8558e-01,
            1.6003e-01,  3.6905e-02,  1.2677e-01,  3.2455e-01,  4.4506e-01,
            4.2855e-01,  4.4316e-01],
          [ 2.3544e-01,  2.3155e-01,  2.3540e-01,  2.2983e-01,  2.0867e-01,
            1.9985e-01,  1.8354e-01,  1.2618e-01,  1.4363e-01,  1.7062e-01,
            2.5707e-01,  3.3859e-01,  3.6650e-01,  3.5188e-01,  3.7239e-01,
            3.9149e-01,  4.1235e-01,  4.3345e-01,  4.2407e-01,  4.3119e-01,
            4.4266e-01,  3.7444e-01,  1.4002e-01,  1.6421e-01,  1.5464e-01,
            1.1995e-01,  2.8883e-02,  1.3140e-01,  3.6327e-01,  4.4528e-01,
            4.1603e-01,  4.5099e-01],
          [ 2.4423e-01,  2.4789e-01,  2.3844e-01,  2.2218e-01,  2.1797e-01,
            1.7191e-01,  1.1877e-01,  1.9509e-01,  2.8013e-01,  3.5203e-01,
            3.7773e-01,  4.1260e-01,  3.6938e-01,  3.4065e-01,  4.0848e-01,
            4.2354e-01,  4.0178e-01,  4.3312e-01,  4.3040e-01,  4.3956e-01,
            4.6364e-01,  3.1831e-01,  9.0120e-02,  1.0718e-01,  8.7562e-02,
            6.0443e-02,  8.2826e-03,  1.7365e-01,  3.9427e-01,  4.0024e-01,
            3.9350e-01,  4.4760e-01],
          [ 2.3484e-01,  2.2328e-01,  2.2963e-01,  2.1395e-01,  1.8132e-01,
            1.2722e-01,  2.4192e-01,  3.7743e-01,  4.1801e-01,  3.8461e-01,
            3.8694e-01,  4.4052e-01,  3.8939e-01,  3.4184e-01,  4.0576e-01,
            4.3035e-01,  4.2704e-01,  4.4134e-01,  4.2622e-01,  4.3604e-01,
            4.3447e-01,  2.5460e-01,  1.1166e-01,  1.7272e-01,  1.9904e-01,
            1.2271e-01,  2.6678e-02,  1.9806e-01,  3.9937e-01,  4.0742e-01,
            4.1526e-01,  4.3395e-01],
          [ 2.0767e-01,  2.1440e-01,  2.0289e-01,  1.5904e-01,  1.7725e-01,
            2.4103e-01,  3.5172e-01,  4.0518e-01,  3.8048e-01,  3.6437e-01,
            4.2276e-01,  4.5999e-01,  4.4466e-01,  3.9160e-01,  4.0263e-01,
            4.4087e-01,  4.3690e-01,  4.3369e-01,  4.3854e-01,  4.2613e-01,
            4.6023e-01,  2.3682e-01,  1.0139e-01,  2.3231e-01,  3.4446e-01,
            2.9257e-01,  4.2000e-02,  2.2176e-01,  4.1979e-01,  4.2348e-01,
            4.0220e-01,  4.0429e-01],
          [ 1.5914e-01,  1.5218e-01,  1.2024e-01,  1.6995e-01,  2.5799e-01,
            3.5092e-01,  3.7740e-01,  3.7625e-01,  3.3802e-01,  4.1918e-01,
            4.8539e-01,  4.7247e-01,  4.7213e-01,  4.2259e-01,  3.9684e-01,
            4.0532e-01,  4.2301e-01,  4.2108e-01,  4.3664e-01,  4.4634e-01,
            4.5283e-01,  2.4796e-01,  3.7351e-02,  2.3395e-01,  4.1143e-01,
            4.0322e-01,  1.0631e-01,  2.1676e-01,  4.6425e-01,  4.5331e-01,
            4.6722e-01,  4.7170e-01],
          [ 1.7006e-01,  2.0313e-01,  2.5290e-01,  2.9827e-01,  3.5262e-01,
            3.7316e-01,  3.6682e-01,  3.7652e-01,  3.3606e-01,  4.1004e-01,
            4.6419e-01,  4.5784e-01,  4.7305e-01,  4.6707e-01,  4.1363e-01,
            4.2622e-01,  4.3933e-01,  4.4550e-01,  4.5008e-01,  4.5653e-01,
            4.5213e-01,  3.4941e-01,  1.0387e-01,  2.1988e-01,  3.7861e-01,
            3.5788e-01,  1.3000e-01,  1.8064e-01,  4.4828e-01,  4.8024e-01,
            5.2480e-01,  5.2896e-01],
          [ 3.1513e-01,  3.5353e-01,  3.7306e-01,  3.6112e-01,  3.7569e-01,
            3.7658e-01,  3.7978e-01,  3.7093e-01,  3.4649e-01,  3.9472e-01,
            4.2688e-01,  4.5175e-01,  4.7016e-01,  4.7099e-01,  4.3679e-01,
            4.2106e-01,  4.6942e-01,  4.6186e-01,  4.5125e-01,  4.5851e-01,
            4.8252e-01,  4.4098e-01,  2.2042e-01,  2.2772e-01,  3.8117e-01,
            4.1858e-01,  1.7909e-01,  1.7786e-01,  4.4202e-01,  4.8584e-01,
            5.1839e-01,  5.2569e-01],
          [ 3.6175e-01,  3.7070e-01,  3.7658e-01,  3.7493e-01,  3.9500e-01,
            3.9991e-01,  3.9470e-01,  3.9734e-01,  4.2308e-01,  4.3584e-01,
            4.4483e-01,  4.4008e-01,  4.5471e-01,  4.5574e-01,  4.4835e-01,
            4.3412e-01,  4.5517e-01,  4.5494e-01,  4.6483e-01,  4.7666e-01,
            4.8440e-01,  4.5369e-01,  2.6042e-01,  1.9814e-01,  4.2604e-01,
            4.4922e-01,  2.1024e-01,  1.6425e-01,  4.1509e-01,  4.3382e-01,
            4.8976e-01,  4.9345e-01],
          [ 3.8398e-01,  3.7539e-01,  3.7227e-01,  4.0162e-01,  3.9514e-01,
            4.2034e-01,  4.1621e-01,  4.5420e-01,  4.4718e-01,  4.5856e-01,
            4.5230e-01,  4.7222e-01,  4.6040e-01,  4.6232e-01,  4.5407e-01,
            4.4365e-01,  4.6990e-01,  4.6888e-01,  4.6663e-01,  4.7899e-01,
            4.6712e-01,  4.3483e-01,  2.8547e-01,  1.7722e-01,  4.0959e-01,
            4.1899e-01,  1.5818e-01,  1.9878e-01,  3.9943e-01,  4.1486e-01,
            4.4136e-01,  4.1550e-01],
          [ 4.0223e-01,  3.9888e-01,  4.0218e-01,  4.1975e-01,  4.2422e-01,
            4.3669e-01,  4.4951e-01,  4.6075e-01,  4.7923e-01,  4.8624e-01,
            4.7245e-01,  4.8558e-01,  4.8314e-01,  4.6577e-01,  4.5978e-01,
            4.6806e-01,  4.8018e-01,  4.8840e-01,  4.6609e-01,  4.8613e-01,
            4.6107e-01,  4.3535e-01,  3.2490e-01,  2.2669e-01,  3.4406e-01,
            4.3002e-01,  1.9799e-01,  2.3464e-01,  3.4198e-01,  3.7242e-01,
            3.8547e-01,  3.0077e-01],
          [ 4.0162e-01,  3.9293e-01,  4.0443e-01,  3.8723e-01,  3.9558e-01,
            4.2360e-01,  4.5654e-01,  4.6238e-01,  4.8989e-01,  4.7667e-01,
            4.7288e-01,  4.6603e-01,  4.5993e-01,  4.4443e-01,  4.4168e-01,
            4.6645e-01,  4.7413e-01,  4.5751e-01,  4.5744e-01,  4.7203e-01,
            4.6343e-01,  4.1721e-01,  3.4209e-01,  2.5146e-01,  2.9033e-01,
            3.9044e-01,  2.5930e-01,  2.3120e-01,  2.5383e-01,  2.4628e-01,
            2.1208e-01,  1.5843e-01]],

         [[ 1.3239e-01,  1.3029e-01,  1.0075e-01,  8.2216e-02,  8.9194e-02,
            6.6723e-02,  8.0802e-02,  5.1221e-02,  4.0208e-02,  3.8477e-02,
            4.3927e-02,  3.5472e-02,  2.7184e-02,  4.2489e-02,  8.4380e-02,
            1.0300e-01,  1.1022e-01,  7.1576e-02,  4.7671e-02,  3.4269e-02,
            5.2134e-02,  4.9353e-02,  1.4184e-01,  1.4317e-01,  1.0322e-01,
            6.6904e-02,  5.8401e-02,  2.8770e-02,  6.2213e-02,  6.5830e-02,
            7.4486e-02,  6.2074e-02],
          [ 1.1428e-01,  1.1473e-01,  1.0739e-01,  1.2126e-01,  1.0337e-01,
            8.3909e-02,  4.7231e-02,  5.1129e-02,  5.0061e-02,  3.7666e-02,
            2.9702e-02,  4.5791e-02,  3.4375e-02,  4.4147e-02,  8.9454e-02,
            1.1735e-01,  1.0975e-01,  6.0088e-02,  4.1425e-02,  3.6173e-02,
            3.5914e-02,  7.2088e-02,  9.7557e-02,  7.5696e-02,  4.9880e-02,
            7.3096e-03,  3.7379e-02,  4.6924e-02,  5.1168e-02,  5.4421e-02,
            5.1017e-02,  5.5644e-02],
          [ 1.1508e-01,  1.0922e-01,  1.1789e-01,  1.2134e-01,  1.0371e-01,
            6.2552e-02,  4.6728e-02,  4.7164e-02,  5.1602e-02,  5.3352e-02,
            5.4615e-02,  5.3133e-02,  4.2329e-02,  2.1009e-02,  4.0979e-02,
            5.9612e-02,  6.9098e-02,  4.0548e-02,  2.5445e-02,  4.1990e-02,
            3.8563e-02,  5.5577e-02,  5.4616e-02,  3.1451e-02,  2.3621e-02,
            3.1576e-02,  4.4675e-02,  5.8355e-02,  6.3010e-02,  7.5613e-02,
            7.8202e-02,  5.3431e-02],
          [ 1.0161e-01,  1.0005e-01,  1.0136e-01,  9.9660e-02,  7.3762e-02,
            6.6818e-02,  5.3932e-02,  3.2831e-02,  4.6224e-02,  3.8860e-02,
            3.0783e-02,  3.3743e-02,  3.7773e-02,  4.2765e-02,  2.9019e-02,
            4.1029e-02,  4.3175e-02,  5.2343e-02,  3.5797e-02,  5.0483e-02,
            4.0618e-02,  3.6382e-02,  5.6482e-02,  2.9230e-02,  3.1258e-02,
            1.4096e-02,  5.5883e-02,  7.9164e-02,  9.9919e-02,  8.7552e-02,
            8.5753e-02,  5.2494e-02],
          [ 8.5458e-02,  9.5532e-02,  8.4597e-02,  6.9367e-02,  5.3497e-02,
            3.7341e-02,  4.0281e-02,  4.8987e-02,  4.9586e-02,  3.5652e-02,
            2.4392e-02,  2.5756e-02,  4.0300e-02,  2.2426e-02,  3.4047e-02,
            3.6670e-02,  4.7153e-02,  7.0176e-02,  4.2755e-02,  2.9500e-02,
            7.4317e-02,  5.2843e-02,  5.2982e-02,  4.9287e-02,  3.9466e-02,
            3.9384e-02,  6.8122e-02,  1.0550e-01,  1.8707e-01,  6.6799e-02,
            6.1186e-02,  5.2190e-02],
          [ 6.1302e-02,  6.6114e-02,  5.6943e-02,  6.4748e-02,  6.7922e-02,
            5.9908e-02,  3.4478e-02,  4.0993e-02,  8.4148e-02,  7.2126e-02,
            6.4608e-02,  1.0145e-01,  1.3773e-01,  1.6827e-01,  1.7125e-01,
            1.4064e-01,  1.0472e-01,  6.3875e-02,  5.9456e-02,  7.1700e-02,
            8.8583e-02,  8.7123e-02,  8.7727e-02,  1.0041e-01,  1.0915e-01,
            5.9498e-02,  7.0202e-02,  1.0539e-01,  1.3684e-01,  7.9965e-02,
            4.6679e-02,  3.4637e-02],
          [ 7.4323e-02,  6.2306e-02,  5.7059e-02,  6.1822e-02,  5.3122e-02,
            5.8145e-02,  7.3695e-02,  6.7039e-02,  1.2909e-01,  2.8306e-01,
            3.2098e-01,  3.2328e-01,  3.6487e-01,  3.9394e-01,  4.4063e-01,
            4.5784e-01,  4.0317e-01,  3.3293e-01,  2.4365e-01,  2.6250e-01,
            3.5386e-01,  3.7260e-01,  3.4549e-01,  3.6577e-01,  3.3509e-01,
            2.1688e-01,  1.3243e-01,  1.5634e-01,  1.9410e-01,  9.5774e-02,
            3.9687e-02,  3.6592e-02],
          [ 7.9843e-02,  8.8419e-02,  8.5225e-02,  8.8737e-02,  1.0125e-01,
            1.1381e-01,  1.1889e-01,  1.3301e-01,  2.0226e-01,  2.8851e-01,
            2.4402e-01,  2.4947e-01,  3.1455e-01,  3.8071e-01,  4.3433e-01,
            5.0812e-01,  5.3449e-01,  5.5722e-01,  5.4118e-01,  5.7937e-01,
            6.1674e-01,  5.9861e-01,  5.6098e-01,  5.4060e-01,  5.2471e-01,
            4.8913e-01,  4.1101e-01,  3.4613e-01,  4.1728e-01,  1.9329e-01,
            6.1331e-02,  7.1781e-02],
          [ 1.7859e-01,  1.6587e-01,  1.7132e-01,  1.4422e-01,  1.6186e-01,
            1.8365e-01,  1.9094e-01,  1.9832e-01,  2.0181e-01,  2.0400e-01,
            2.0948e-01,  1.8890e-01,  2.5196e-01,  3.2680e-01,  3.7628e-01,
            4.3303e-01,  4.4082e-01,  5.4217e-01,  5.5383e-01,  5.0716e-01,
            5.6264e-01,  5.6706e-01,  5.2558e-01,  4.9163e-01,  5.1428e-01,
            5.6716e-01,  4.7376e-01,  3.6343e-01,  3.9053e-01,  2.0399e-01,
            8.8744e-02,  8.4215e-02],
          [ 2.4298e-01,  2.1184e-01,  2.0894e-01,  2.0662e-01,  2.1450e-01,
            2.3205e-01,  1.9057e-01,  1.5221e-01,  1.4194e-01,  2.0195e-01,
            1.9964e-01,  1.7974e-01,  2.2393e-01,  2.7088e-01,  3.3981e-01,
            4.4255e-01,  4.3407e-01,  4.8121e-01,  5.1147e-01,  4.3033e-01,
            4.3613e-01,  4.7715e-01,  4.7212e-01,  4.8311e-01,  4.7922e-01,
            4.9020e-01,  4.4612e-01,  3.8796e-01,  3.8864e-01,  2.6531e-01,
            2.9890e-01,  3.1437e-01],
          [ 2.4524e-01,  2.4487e-01,  2.0913e-01,  2.2177e-01,  2.2645e-01,
            1.8931e-01,  1.1199e-01,  1.0713e-01,  1.2114e-01,  1.7286e-01,
            1.8275e-01,  1.8503e-01,  2.0821e-01,  2.6011e-01,  3.2527e-01,
            4.1769e-01,  4.1990e-01,  4.1832e-01,  4.8586e-01,  4.5651e-01,
            3.9730e-01,  4.1594e-01,  4.1867e-01,  4.1535e-01,  3.6181e-01,
            4.1855e-01,  4.0821e-01,  4.8778e-01,  4.5164e-01,  3.7386e-01,
            3.8025e-01,  4.0466e-01],
          [ 2.2373e-01,  2.0944e-01,  1.5192e-01,  1.7525e-01,  1.8204e-01,
            1.2427e-01,  1.4035e-01,  1.6062e-01,  1.8596e-01,  1.7014e-01,
            1.6993e-01,  2.0292e-01,  2.2255e-01,  2.5500e-01,  2.6625e-01,
            2.4125e-01,  2.3843e-01,  2.9970e-01,  3.8933e-01,  4.3655e-01,
            4.0193e-01,  3.1140e-01,  2.5489e-01,  2.1483e-01,  1.6224e-01,
            2.4978e-01,  3.6299e-01,  4.4530e-01,  4.6844e-01,  4.2791e-01,
            3.5259e-01,  3.6124e-01],
          [ 1.7792e-01,  1.3250e-01,  1.3137e-01,  1.3524e-01,  1.6242e-01,
            1.2236e-01,  1.3604e-01,  1.6641e-01,  1.9083e-01,  1.8949e-01,
            1.8991e-01,  2.0006e-01,  2.0980e-01,  2.5721e-01,  2.2728e-01,
            1.9222e-01,  1.8058e-01,  2.1528e-01,  3.0740e-01,  3.5984e-01,
            3.7348e-01,  2.8646e-01,  2.1799e-01,  1.7762e-01,  9.0441e-02,
            8.6508e-02,  2.5490e-01,  3.7323e-01,  3.7873e-01,  4.1076e-01,
            3.9125e-01,  3.6931e-01],
          [ 1.7345e-01,  1.0177e-01,  1.3766e-01,  1.5680e-01,  1.4858e-01,
            1.2762e-01,  1.2113e-01,  1.2792e-01,  1.4134e-01,  1.6285e-01,
            1.5510e-01,  1.7436e-01,  2.0198e-01,  2.3301e-01,  2.0566e-01,
            1.6783e-01,  1.7620e-01,  2.1620e-01,  2.4673e-01,  3.0253e-01,
            2.7580e-01,  2.8094e-01,  2.0866e-01,  1.6436e-01,  1.0709e-01,
            4.5366e-02,  1.0967e-01,  2.9151e-01,  3.1728e-01,  3.4308e-01,
            3.7770e-01,  4.1140e-01],
          [ 1.6255e-01,  1.0622e-01,  1.3793e-01,  1.2679e-01,  1.2627e-01,
            1.2949e-01,  1.2855e-01,  1.6501e-01,  1.4626e-01,  1.0774e-01,
            1.7979e-01,  1.6265e-01,  1.8749e-01,  2.0508e-01,  1.9121e-01,
            1.5399e-01,  1.9338e-01,  2.3414e-01,  2.5667e-01,  2.8473e-01,
            2.4895e-01,  2.4547e-01,  1.7688e-01,  1.1766e-01,  8.7983e-02,
            2.8088e-02,  4.9978e-02,  1.9345e-01,  3.0513e-01,  3.2134e-01,
            3.9212e-01,  4.1339e-01],
          [ 1.2013e-01,  1.0015e-01,  1.4085e-01,  1.4312e-01,  1.2708e-01,
            1.0198e-01,  1.0868e-01,  1.4002e-01,  1.3050e-01,  1.0488e-01,
            1.6619e-01,  1.3013e-01,  1.2018e-01,  1.6340e-01,  1.6252e-01,
            1.3998e-01,  2.1715e-01,  2.5510e-01,  3.1888e-01,  3.2389e-01,
            2.8869e-01,  2.2462e-01,  1.2428e-01,  7.6334e-02,  7.3072e-02,
            2.9122e-02,  4.3485e-02,  1.5840e-01,  2.9748e-01,  3.4693e-01,
            3.9526e-01,  3.9769e-01],
          [ 1.3458e-01,  1.2175e-01,  1.2413e-01,  1.3102e-01,  1.1647e-01,
            8.7409e-02,  8.2293e-02,  1.0871e-01,  8.3585e-02,  1.0431e-01,
            1.3647e-01,  1.1498e-01,  1.1052e-01,  9.6771e-02,  6.4901e-02,
            1.0141e-01,  2.7576e-01,  2.9106e-01,  3.2033e-01,  3.2683e-01,
            2.4489e-01,  1.2644e-01,  4.5490e-02,  3.9253e-02,  5.3968e-02,
            4.0638e-02,  1.7450e-02,  1.2492e-01,  2.6573e-01,  3.4241e-01,
            3.6678e-01,  3.6033e-01],
          [ 1.0658e-01,  1.0706e-01,  1.1341e-01,  1.2304e-01,  1.0542e-01,
            9.2940e-02,  9.4710e-02,  9.8334e-02,  1.0750e-01,  1.7413e-01,
            1.7194e-01,  1.0111e-01,  8.5334e-02,  7.8170e-02,  3.7909e-02,
            6.3893e-02,  1.7743e-01,  1.9940e-01,  1.6605e-01,  1.6151e-01,
            9.8496e-02,  6.3907e-02,  7.1871e-02,  8.3058e-02,  9.4795e-02,
            5.2323e-02, -7.3257e-03,  1.0386e-01,  2.4300e-01,  3.1134e-01,
            3.7344e-01,  3.5582e-01],
          [ 1.3858e-01,  1.4278e-01,  1.6133e-01,  1.2808e-01,  1.0004e-01,
            1.0432e-01,  9.3029e-02,  8.7105e-02,  1.1794e-01,  1.5629e-01,
            1.3817e-01,  9.4155e-02,  7.4497e-02,  4.1061e-02,  1.0279e-01,
            2.7092e-01,  3.4371e-01,  1.7809e-01,  8.3993e-02,  1.5722e-01,
            2.1433e-01,  1.9350e-01,  7.3584e-02,  9.4813e-02,  1.3700e-01,
            4.8850e-02, -1.3818e-02,  9.7484e-02,  2.4000e-01,  3.4640e-01,
            3.6299e-01,  3.6286e-01],
          [ 1.9314e-01,  1.9372e-01,  2.3523e-01,  1.9701e-01,  1.0760e-01,
            7.5641e-02,  8.9659e-02,  1.1490e-01,  1.6273e-01,  1.4705e-01,
            1.2409e-01,  8.4141e-02,  7.0942e-02,  1.1689e-01,  3.3980e-01,
            4.3190e-01,  3.7552e-01,  3.5803e-01,  3.4004e-01,  3.6714e-01,
            3.8132e-01,  2.7258e-01,  7.8194e-02,  7.2827e-02,  1.3168e-01,
            5.2214e-02, -5.6376e-03,  8.3793e-02,  2.6533e-01,  3.6498e-01,
            3.5476e-01,  3.7933e-01],
          [ 1.9171e-01,  2.0301e-01,  2.0834e-01,  2.1182e-01,  1.5823e-01,
            1.2942e-01,  1.5347e-01,  1.7592e-01,  1.4342e-01,  1.2517e-01,
            1.0511e-01,  1.5173e-01,  2.8591e-01,  3.4237e-01,  3.4821e-01,
            3.3112e-01,  3.2694e-01,  3.6421e-01,  3.9049e-01,  3.7339e-01,
            3.4950e-01,  2.9853e-01,  1.4822e-01,  8.7778e-02,  1.5790e-01,
            1.2560e-01, -7.0188e-03,  7.9693e-02,  2.7000e-01,  4.0442e-01,
            3.9185e-01,  3.8481e-01],
          [ 1.8123e-01,  1.9292e-01,  2.0144e-01,  2.0059e-01,  1.8437e-01,
            1.8835e-01,  1.5760e-01,  1.2566e-01,  9.8881e-02,  1.3177e-01,
            2.1425e-01,  3.0232e-01,  3.1557e-01,  2.9769e-01,  3.0446e-01,
            3.3279e-01,  3.4095e-01,  3.7832e-01,  3.5793e-01,  3.5657e-01,
            3.8242e-01,  3.1804e-01,  8.7072e-02,  1.1409e-01,  1.3929e-01,
            7.4695e-02, -1.3764e-02,  8.0038e-02,  2.9893e-01,  3.8957e-01,
            3.8589e-01,  4.1983e-01],
          [ 1.9915e-01,  2.1649e-01,  1.9342e-01,  1.9368e-01,  1.8611e-01,
            1.4603e-01,  9.5657e-02,  1.7190e-01,  2.3077e-01,  2.9460e-01,
            3.0635e-01,  3.6336e-01,  2.9231e-01,  2.7719e-01,  3.4230e-01,
            3.6036e-01,  3.5495e-01,  3.6088e-01,  3.7134e-01,  3.6752e-01,
            3.7398e-01,  2.5058e-01,  3.5106e-02,  6.7744e-02,  5.1262e-02,
            2.4396e-02, -2.8166e-02,  1.2510e-01,  3.4598e-01,  3.6110e-01,
            3.6564e-01,  4.2636e-01],
          [ 2.0465e-01,  1.9745e-01,  1.8863e-01,  1.8038e-01,  1.5633e-01,
            1.0598e-01,  1.7843e-01,  3.3845e-01,  3.4801e-01,  3.3490e-01,
            3.4711e-01,  3.6848e-01,  3.1091e-01,  2.9819e-01,  3.6159e-01,
            3.7773e-01,  3.7062e-01,  3.9146e-01,  3.6684e-01,  3.5946e-01,
            3.8991e-01,  2.0208e-01,  5.3038e-02,  1.4253e-01,  1.7357e-01,
            1.1311e-01, -2.7200e-02,  1.6522e-01,  3.7797e-01,  3.9312e-01,
            3.9216e-01,  3.8851e-01],
          [ 1.6918e-01,  1.6935e-01,  1.7638e-01,  1.1504e-01,  1.3234e-01,
            1.8618e-01,  3.0854e-01,  3.3400e-01,  3.1094e-01,  2.8317e-01,
            3.5459e-01,  4.0325e-01,  3.8930e-01,  3.4084e-01,  3.6298e-01,
            3.7806e-01,  3.8587e-01,  3.7543e-01,  3.6684e-01,  3.7346e-01,
            4.0490e-01,  1.8691e-01,  4.2264e-02,  1.8476e-01,  3.4677e-01,
            2.9254e-01,  1.0440e-02,  1.8314e-01,  3.7241e-01,  3.9952e-01,
            3.8090e-01,  3.7132e-01],
          [ 1.3900e-01,  1.1879e-01,  1.0949e-01,  1.2578e-01,  2.1366e-01,
            2.8776e-01,  3.2661e-01,  3.1030e-01,  2.6733e-01,  3.5954e-01,
            4.2148e-01,  4.2968e-01,  4.1173e-01,  3.5610e-01,  3.4473e-01,
            3.8988e-01,  3.7344e-01,  3.5109e-01,  3.6579e-01,  3.7328e-01,
            3.9382e-01,  1.9817e-01,  3.1302e-03,  2.0273e-01,  4.2507e-01,
            4.1963e-01,  7.6909e-02,  1.7375e-01,  4.1212e-01,  4.2570e-01,
            4.3268e-01,  4.3733e-01],
          [ 1.5117e-01,  1.7046e-01,  1.8883e-01,  2.4333e-01,  2.8316e-01,
            3.2397e-01,  3.1612e-01,  3.0107e-01,  2.7663e-01,  3.6369e-01,
            4.1140e-01,  4.0565e-01,  3.9670e-01,  3.9832e-01,  3.8764e-01,
            3.6723e-01,  3.8884e-01,  3.7550e-01,  3.8658e-01,  3.9712e-01,
            4.0306e-01,  2.9873e-01,  6.3673e-02,  1.9189e-01,  3.6829e-01,
            3.6796e-01,  1.1123e-01,  1.6255e-01,  4.1338e-01,  4.5830e-01,
            5.0232e-01,  5.1502e-01],
          [ 2.6842e-01,  3.1439e-01,  3.1584e-01,  2.9939e-01,  3.1229e-01,
            3.1967e-01,  3.3113e-01,  3.0420e-01,  3.0443e-01,  3.5808e-01,
            3.7441e-01,  3.9561e-01,  4.1653e-01,  4.2279e-01,  3.9304e-01,
            3.9775e-01,  4.0691e-01,  4.0123e-01,  3.9062e-01,  4.0270e-01,
            4.4241e-01,  3.9441e-01,  1.7968e-01,  1.9401e-01,  3.6429e-01,
            3.8365e-01,  1.5711e-01,  1.4615e-01,  4.1365e-01,  4.5836e-01,
            4.9552e-01,  5.0655e-01],
          [ 3.0733e-01,  3.2167e-01,  3.2689e-01,  3.0568e-01,  3.2796e-01,
            3.4462e-01,  3.6273e-01,  3.6420e-01,  3.8223e-01,  3.7747e-01,
            3.9465e-01,  3.9169e-01,  4.0619e-01,  4.1284e-01,  3.7839e-01,
            3.8222e-01,  4.0280e-01,  4.0174e-01,  4.0134e-01,  3.9880e-01,
            4.1933e-01,  4.2829e-01,  2.1916e-01,  1.6725e-01,  3.8132e-01,
            3.9324e-01,  1.5979e-01,  1.3772e-01,  3.9616e-01,  4.1530e-01,
            4.7031e-01,  4.6846e-01],
          [ 3.1632e-01,  3.1602e-01,  3.2378e-01,  3.4485e-01,  3.4363e-01,
            3.5473e-01,  3.8376e-01,  4.0648e-01,  4.1389e-01,  4.1555e-01,
            4.1796e-01,  4.0815e-01,  4.1767e-01,  4.1330e-01,  4.1878e-01,
            4.0785e-01,  4.2208e-01,  4.2239e-01,  4.0815e-01,  4.2719e-01,
            4.4079e-01,  3.9533e-01,  2.3278e-01,  1.4201e-01,  3.5659e-01,
            3.7760e-01,  1.4107e-01,  1.7929e-01,  3.7766e-01,  3.9459e-01,
            4.2278e-01,  3.8630e-01],
          [ 3.3206e-01,  3.4392e-01,  3.3751e-01,  3.6470e-01,  3.7036e-01,
            3.9863e-01,  4.1068e-01,  4.3595e-01,  4.3378e-01,  4.4120e-01,
            4.3678e-01,  4.4362e-01,  4.3077e-01,  4.2026e-01,  4.1842e-01,
            4.2193e-01,  4.3011e-01,  4.3688e-01,  4.2601e-01,  4.2508e-01,
            4.1110e-01,  3.9443e-01,  2.9003e-01,  1.6657e-01,  2.9644e-01,
            3.6768e-01,  1.7573e-01,  2.0746e-01,  3.1885e-01,  3.4519e-01,
            3.4733e-01,  2.5957e-01],
          [ 3.5050e-01,  3.4078e-01,  3.4034e-01,  3.5020e-01,  3.7121e-01,
            3.9093e-01,  4.2052e-01,  4.4133e-01,  4.5291e-01,  4.3755e-01,
            4.2922e-01,  4.2923e-01,  4.2618e-01,  4.1363e-01,  4.1016e-01,
            4.1724e-01,  4.0984e-01,  4.1419e-01,  4.2464e-01,  4.0807e-01,
            4.2214e-01,  3.8511e-01,  3.1739e-01,  1.9421e-01,  2.1940e-01,
            3.5554e-01,  2.4641e-01,  2.1116e-01,  2.3702e-01,  2.0638e-01,
            1.6347e-01,  1.1786e-01]]]])

In [9]:
from torchmetrics.image.inception import InceptionScore
import torch
from torchmetrics.image.fid import FrechetInceptionDistance



pure_noise_is = InceptionScore()
cifar10_is = InceptionScore()
generated_is = InceptionScore()

for batch_idx, (x, class_) in tqdm(enumerate(train_loader), total=len(train_loader)):
    cifar10_is.update(v2.ToDtype(torch.uint8, scale=True)(x))
    generated_is.update(v2.ToDtype(torch.uint8, scale=True)(generated_images))
    noise = torch.randint(0, 255, (64, 3, 32, 32), dtype=torch.uint8)
    pure_noise_is.update(noise)
    print(f'{pure_noise_is.compute()=} {generated_is.compute()=} {cifar10_is.compute()=}')


/Users/lihenan/miniconda3/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028
  8%|▊         | 1/13 [00:17<03:25, 17.09s/it]

pure_noise_is.compute()=(tensor(1.1652), tensor(0.0774)) generated_is.compute()=(tensor(1.), tensor(6.7585e-08)) cifar10_is.compute()=(tensor(3.3538), tensor(0.9758))


 15%|█▌        | 2/13 [00:33<03:02, 16.62s/it]

pure_noise_is.compute()=(tensor(1.1880), tensor(0.0450)) generated_is.compute()=(tensor(1.4910), tensor(0.2396)) cifar10_is.compute()=(tensor(5.0562), tensor(0.9769))


 23%|██▎       | 3/13 [00:49<02:45, 16.59s/it]

...

pure_noise_is.compute()=(tensor(1.2077), tensor(0.0226)) generated_is.compute()=(tensor(2.5101), tensor(0.3153)) cifar10_is.compute()=(tensor(8.4093), tensor(1.1371))


 92%|█████████▏| 12/13 [03:21<00:16, 16.61s/it]

pure_noise_is.compute()=(tensor(1.2061), tensor(0.0126)) generated_is.compute()=(tensor(2.6435), tensor(0.1593)) cifar10_is.compute()=(tensor(8.6291), tensor(1.0532))


100%|██████████| 13/13 [03:34<00:00, 16.52s/it]

pure_noise_is.compute()=(tensor(1.2072), tensor(0.0168)) generated_is.compute()=(tensor(2.5212), tensor(0.4067)) cifar10_is.compute()=(tensor(8.6610), tensor(0.8709))


In [14]:
fid = FrechetInceptionDistance(feature=64, normalize=True)




# generated_imgs = v2.ToDtype(torch.uint8, scale=True)(xt) # scaled to [0, 255]

for test_img, test_class in test_loader:
    fid.update(test_img, real=True)
    fid.update(generated_images, real=False)
    print(fid.compute())
    


tensor(1.9892)
tensor(2.3824)
tensor(2.6352)
tensor(2.7110)
tensor(2.6948)
tensor(2.6769)
tensor(2.6967)
tensor(2.7222)
tensor(2.6953)
tensor(2.6714)
tensor(2.5898)
tensor(2.5860)
tensor(2.5845)
tensor(2.6332)
tensor(2.5906)
tensor(2.5952)
tensor(2.5887)
tensor(2.5952)
tensor(2.6143)


KeyboardInterrupt: 